# Chart Analysis Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB


################################################################################
## Chart Stuff
################################################################################
from artistIgnores import getArtistIgnores
from billboardData import billboardData
from top40Data import top40Data
from spotifyData import spotifyData
from chartArtistAlbumData import chartArtistAlbumData
from chartUtils import *
from extraArtists import extraKnownArtists


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2021-02-23 09:27:29.010565


# Rename Input Data

In [ ]:
updateMultiDB(2)

In [3]:
def updateMultiDB(step=1):
    if step == 1:
        tmpmanDB = masterArtistNameDB("multi", init=True)
        tmpmanDB.forceReload(getFile("multi.yaml"))
        tmpmanDB.checkForRecursives()
        
    if step == 2:
        tmpmanDB = masterArtistNameDB("multi", init=True)
        tmpmanDB.forceReload(getFile("multi.yaml"))
        tmpmanDB.save()
        
        multimanDB = masterArtistNameDB("multi", init=False)
        saveFile(idata=multimanDB.getRenames(), ifile="multi.yaml")

def updateManDB(step=1):
    if step == 1:
        tmpmanDB = masterArtistNameDB("main", init=True)
        tmpmanDB.forceReload(getFile("main.yaml"))
        tmpmanDB.checkForRecursives()

    if step == 2:
        tmpmanDB = masterArtistNameDB("main", init=True)
        tmpmanDB.forceReload(getFile("main.yaml"))
        tmpmanDB.save()
        
        manDB = masterArtistNameDB("main", init=False)
        saveFile(idata=manDB.getRenames(), ifile="main.yaml")
        
    if step == 3:
        tmpmanDB = masterArtistNameDB("main", init=False)
        tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
        tmpmanDB.checkForRecursives()
        
    if step == 4:
        tmpmanDB = masterArtistNameDB("main", init=False)
        tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
        tmpmanDB.save()
        
        manDB = masterArtistNameDB("main", init=False)
        saveFile(idata=manDB.getRenames(), ifile="main.yaml")
        
    
    _, _ = clock("Last Ran")
    
    
def loadMainDB(recreate=False):
    recreate = True
    if recreate:
        maindb = mainDB(mdb=None, create=True, debug=True)
        maindb.setDBArtists(recreate=False)
        maindb = mainDB(mdb=None, create=False, debug=False)
        maindb.loadDBDataMap()
    else:
        maindb = mainDB(mdb=None, create=False, debug=False)
        maindb.loadDBDataMap()    
        
    return maindb
    _, _ = clock("Last Run")
    
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)
maindb = loadMainDB(False)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 8003 artist keys.
  There are currently 9338 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 246 artist keys.
  There are currently 252 renamed artist keys.
Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rateyourmusic   =========================
=========================   deezer   =========================
=========================   albumoftheyear   =========================
========

  --> This file is 2.0MB.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 8003 artist keys.
  There are currently 9338 renamed artist keys.
Adding 1 known multi delim artists.
Current Time is Tue Feb 23, 2021 09:28:36 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 8328 Artists
	Shape --> (8328, 1)
	Shape --> (8328, 2)
  Finding Real Artist Name
	Shape --> (8328, 4)
  Removing None Artist
	Shape --> (8328, 4)
  Finding Disc Artist Name
	Shape --> (8328, 5)
DataFrame Shape is (8328, 7)
Current Time is Tue Feb 23, 2021 09:28:37 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 201.0 mill

Summary Statistics For DB: KWorbiTunes
    Using Known Artists: False
    Found 7542 ID -> Name entries
    Found 7521 Name -> ID entries
    Found 0 Albums


In [ ]:
updateMultiDB(2)

# Reverse Chart Lookup

In [ ]:
updateMultiDB(1)

In [ ]:
updateMultiDB(2)

In [ ]:
updateManDB(1)

In [ ]:
updateManDB(2)

In [ ]:
if False:
    from multiArtist import multiartist
    mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
    mularts.setKnownMultiDelimArtists(getFile("../multiartist/knownMultiArtists.yaml"))
    knownMultiArtists = getFile("../multiartist/knownMultiArtists.yaml")
    print(len(knownMultiArtists),"Known Artists")

    ignoresComp = ['Soundtrack', 'Various Artists', 'Original Broadway Cast Recording', 'Read-Along','Veggietales', 'Pokemon', 
                   'Barney', 'Walt Disney Read-Along', 'The Powerpuff Girls',
                   'Superstar Kidz', 'Original Cast', 'Original Cast Recording']
    ignores = ignoresComp    

    def matchArtist(artistName):
        subNames = mularts.getArtistNames(artistName)
        result = {subName: mdbmap.isKnownByName(subName) for subName in subNames}
        for subName in result.keys():
            if subName in ignores:
                result[subName] = True
        return result


    for chartName in chartDFs.keys():
        try:
            chartDFs[chartName]['DB'] = chartDFs[chartName]['RenamedArtist'].apply(matchArtist)
        except:
            print("Error with {0}".format(chartName))

In [ ]:
vals = {}
for chartName in chartDFs.keys():
    try:
        vals[chartName] = chartDFs[chartName][~chartDFs[chartName]["DB"].apply(lambda x: all(x.values()))].shape[0]
    except:
        pass
from pandas import Series

In [ ]:
Series(vals).sort_values(ascending=False)

In [ ]:
mdbmap.isKnownByName("Salt-N-Pepa")

In [ ]:
chartDF = chartDFs["vinyl-albums"]
def getMissingArtists(chartDF):
    missing = chartDF[~chartDF["DB"].apply(lambda x: all(x.values()))]
    return list(set(getFlatList(missing["DB"].apply(lambda x: [k for k,v in x.items() if v is False]))))
getMissingArtists(chartDF)

In [ ]:
def getMissingArtists(chartDF):
    ichartDF[~chartDF["DB"].apply(lambda x: all(x.values()))]

In [ ]:
chartName = "independent-albums"
chartDFs[chartName][~chartDFs[chartName]["DB"].apply(lambda x: all(x.values()))]

In [ ]:
bMap = {}
for chartTitle,chartName in initMap.items():
    newName = newMap.get(chartTitle)
    if newName is not None:
        bMap[newName] = chartName


In [ ]:
saveFile(ifile="billboardMapping.p", idata=bMap)

In [ ]:
bMap = getFile("billboardMapping.p")

# Top40 Data

In [ ]:
%load_ext autoreload
%autoreload
from top40Data import top40Data
td = top40Data(minYear=1, maxYear=2021)
td.setChartUsage(rank=[0,1,2,3,4,5,6])
td.setDBRenames(manDB)
td.setMultiDBRenames(multimanDB)
td.setFullChartData()
td.setArtistAlbumData()
td.saveArtistAlbumData()
td.saveFullChartData()

_, _ = clock("Last Run")

# Billboard Data

In [ ]:
%load_ext autoreload
%autoreload
from billboardData import billboardData
bd = billboardData(minYear=1, maxYear=2021)
bd.setChartUsage(rank=[0,1,2,3,4,5,6,7,8])
bd.setDBRenames(manDB)
bd.setMultiDBRenames(multimanDB)
bd.setFullChartData()
bd.setArtistAlbumData()

bd.saveArtistAlbumData()
bd.saveFullChartData()

_, _ = clock("Last Run")

# BillboardYE Data

In [ ]:
%load_ext autoreload
%autoreload
from billboardYE import billboardYE
bYE = billboardYE(minYear=1, maxYear=2021)
bYE.setChartUsage(rank=[0,1,2,3,4,5,6,7]) #,6,7])
bYE.setDBRenames(manDB)
bYE.setFullChartData()
bYE.setArtistAlbumData()

bYE.saveFullChartData()
bYE.saveArtistAlbumData()

_, _ = clock("Last Run")

# MusicVF

In [31]:
%load_ext autoreload
%autoreload
from musicVFData import musicVFData
mvf = musicVFData(minYear=1, maxYear=1995)
mvf.setDBRenames(manDB)
mvf.setMultiDBRenames(multimanDB)
mvf.setChartUsage(rank=[0])
mvf.setFullChartData()
mvf.setArtistAlbumData()
mvf.saveArtistAlbumData()
mvf.saveFullChartData()
_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using Charts For Rank 0
    Categories: ['artists']
	Chart: artists
  Using 1 Charts
Using 1 Charts For Rank 0
=== ChartUsage ===
  Using Charts (Rank=[0]): ['Artists']
Found 7 summary files
  Using Artists
Renamed 947 single artists
Renamed 129 multi artists
Saving 3221 Artist Album Data to currentmusicvfArtistAlbumData.p
Saving data to currentmusicvfArtistAlbumData.p
  --> This file is 281.8kB.
Saved data to currentmusicvfArtistAlbumData.p
  --> This file is 281.8kB.
Saving 3221 Full Artist Data
Saving data to currentmusicvfFullChartArtistAlbumData.p
  --> This file is 439.1kB.
Saved data to currentmusicvfFullChartArtistAlbumData.p
  --> This file is 439.1kB.
Current Time is Tue Feb 23, 2021 09:47:02 for Last Run


# Spotify Code

## Parse

In [ ]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
sd = spotifyData(minYear=1, maxYear=2021)
sd.parse()

## Create

In [ ]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
sd = spotifyData(minYear=1, maxYear=2021)
sd.setDBRenames(manDB)
sd.setMultiDBRenames(multimanDB)
sd.setChartUsage(rank=[0,1,2,3,4,5,6,7,8,9,10,11,12,13]) #,1,2,3,4,5,7,8,9,10,11,12])
sd.setFullChartData()
sd.setArtistAlbumData()
sd.saveArtistAlbumData()
sd.saveFullChartData()
_, _ = clock("Last Run")

# UDiscoverMusic

In [ ]:
%load_ext autoreload
%autoreload
from uDiscoveryMusic import uDiscoverMusicData
ud = uDiscoverMusicData(minYear=1, maxYear=2021)
ud.setDBRenames(manDB)
ud.setMultiDBRenames(multimanDB)
ud.setChartUsage(rank=[0]) #,1,2,3,4,5,7,8,9,10,11,12])
ud.setFullChartData()
ud.setArtistAlbumData()
ud.saveArtistAlbumData()
ud.saveFullChartData()
_, _ = clock("Last Run")

# Africa

In [ ]:
%load_ext autoreload
%autoreload
from Africa import africaData
africa = africaData()
africa.setDBRenames(manDB)
africa.setMultiDBRenames(multimanDB)
africa.setChartUsage(rank=[0])
africa.setFullChartData()
africa.setArtistAlbumData()
africa.saveArtistAlbumData()
africa.saveFullChartData()
_, _ = clock("Last Run")

# Analyze Data

In [32]:
## Basic stuff
%load_ext autoreload
%autoreload
from chartArtistAlbumData import chartArtistAlbumData


singleArtistAlbumData = {}
manyArtistAlbumData   = {}

chartType = "MusicVF"
#chartType = "Spotify"
mType     = "Full"
cad = chartArtistAlbumData(chartType, ignoreMultiNames=False)
cad.createIndivArtistAlbumData()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
chartArtistAlbumData(MusicVF)
Found 62 files.


ValueError: There is an error with masking artist Mel & Kim

# Create Master Match Class & Ignores List

In [ ]:
def showMatchedStatus(chartType, mdbmaps, mdbmc):
    print("Matched {0} / {1} Artists for {2} ChartType".format(mdbmaps[chartType].getSize(), len(mdbmc.matchData[chartType]), chartType))
    
def showRemainingMatches(chartType, mdbmc):
    print('='*100)
    print("="*10,"Artists That Were Not Tested (Low Albums)",'='*10)
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=True, ignores=ignoresList)
    print("="*10,"Previously Tested Artists Without A Match",'='*10)
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
    print('='*100)    
    
def analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=0.1):
    dbRenames = {}
    toget   = {}
    togetID = {}
    for primaryKey,artistName in mdbmaps[chartType].getArtists().items():
        primaryArtistName = artistName
        artistData = mdbmaps[chartType].getArtistDataByKey(primaryKey)
        for db,dbID in artistData.getDict().items():
            if db in ["DatPiff", "MetalStorm", "CDandLP", "AceBootlegs", "RockCorner", "MusicStack"]:
                continue
            if dbID is not None:
                secondaryArtistName = maindb.getArtistDBNameFromID(db, dbID)
                if secondaryArtistName is None:
                    if toget.get(db) is None:
                        toget[db] = []
                        togetID[db] = {}
                    toget[db].append(primaryArtistName)
                    togetID[db][dbID] = primaryArtistName
                    continue
                s = SequenceMatcher(None, primaryArtistName, secondaryArtistName)
                ratio = s.ratio()
                maxRatio = maxVal
                if ratio >= maxRatio:
                    continue

                if ratio >= maxRatio-diffVal:
                    if dbRenames.get(secondaryArtistName) is not None:
                        if dbRenames[secondaryArtistName] == primaryArtistName:
                            continue
                        else:
                            pass
                    dbRenames[secondaryArtistName] = primaryArtistName
                    print("[{0: <30} {1: <4} {2: >30}] \t --> ({3}) {4} / {5} ({6})".format(primaryArtistName,round(ratio,2),secondaryArtistName,chartType,primaryKey,dbID,db))


    print("Found {0} artists to get.".format(len(toget)))
    print("Found {0} artists to rename.".format(len(dbRenames)))
    return toget, togetID, dbRenames    
    
def removeDBRenames(mdbmaps, chartType, dbRenames):
    for k,v in dbRenames.items():
        kKey = mdbmaps[chartType].getPrimaryKey(artistName=k, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)

        kKey = mdbmaps[chartType].getPrimaryKey(artistName=v, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)
    mdbmaps[chartType].save()
    
    
def analyzeRenames(manDB, dbRenames):
    redos = {}
    dels = []
    if len(dbRenames) == 0:
        print("Nothing to process.")
        return
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys() and bestname not in manDB.artistNameDB.keys():
            print(oldername,bestname)

            redos[bestname] = oldername
            dels.append(oldername)
            print('\t',oldername,'\t',bestname)

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))
    for oldername in dels:
        del dbRenames[oldername]

    print("# Redos: {0}".format(len(redos)))
    dbRenames.update(redos)
    print("# Renames: {0}".format(len(dbRenames)))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")







    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")
    #print(len(dbRenames))
    
    
        
    
def manualAppends(toMatch, chartType, minAlbums=0, add=True, dbs=["AllMusic", "Deezer"], showSongs=True, pattern=None):
    if len(toMatch[chartType]) == 0:
        print("Nothing to append...")
        return
    
    toMatchDF = DataFrame({item[0]: {"ArtistName": item[1]["ArtistName"], "ArtistAlbums": item[1]["ArtistAlbums"]} for item in toMatch[chartType]}).T
    toMatchDF["NumAlbums"] = toMatchDF["ArtistAlbums"].apply(len)
    toMatchDF = toMatchDF.sort_values(by="NumAlbums", ascending=False)
    
    for idx,row in toMatchDF.iterrows():
        if add:
            artistName = row["ArtistName"]
            albumNames = row["ArtistAlbums"]
            nAlbums    = row["NumAlbums"]
            
            if pattern is not None:
                if artistName.find(pattern) == -1:
                    continue
            
            if nAlbums < minAlbums:
                continue
            for db in dbs:
                print("add{0}(mdbmaps, {1}, {2}, {3})   ### [{4}]".format(db, "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "\"{0}\"".format(""), artistName))
            if showSongs:
                print("### {0}".format("  ,  ".join(albumNames)))            
                for albumName in albumNames:
                    print("### {0}  ---> [{1}]".format(albumName, cad.getAlbumsData(albumName)))
            print("\n")
        else:
            print("ignores.append(\"{0}\")".format(artistName))
            if " & " in artistName:
                continue
                print("ignores.append(\"{0}\")".format(artistName))

    print("mdbmaps[{0}].save()".format("\"{0}\"".format(chartType)))    
    





        
def copyMapData(mdbmap):
    return
    mdbmap.saveCopy()

def saveMapData(mdbmap, result_list):
    for item in result_list:
        primaryKey = item[0] 
        artistName = item[1]
        artistID   = item[2]
        mcs        = item[3]
        mdbmap.addArtistByKey(primaryKey, artistName=artistName, artistID=artistID)
        for db,mc in mcs.items():
            matchID    = mc.matchID
            matchScore = mc.matchScore
            if matchID is not None:
                mdbmap.addArtistDataByKey(primaryKey, db, matchID)

    mdbmap.save()
    

def matchItAll(mdbmaps, thresholds, useAlbums=True, mdbmc=None, db=None, toMatch=None):
    if toMatch is None:
        toMatch   = mdbmc.getDataToMatch(db, maxValues=10000, maxAlbums=50000)

    if len(toMatch) == 0:
        return

    num_processes = 3
    if useAlbums is True:
        func = matchDBArtistWithAlbums
    else:
        func = matchDBArtistWithoutAlbums
    pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
    #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
    dbName = list(toMatch.keys())[0]
    copyMapData(mdbmaps[dbName])

    argument_list = toMatch[dbName]
    if len(argument_list) == 0:
        return
    print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
    result_list = multiProc(func=pfunc, argument_list=argument_list,
                                           num_processes=num_processes)


    start, cmt = clock("Saving...")
    saveMapData(mdbmaps[dbName], result_list)
    elapsed(start, cmt)
    print("\nSleeping for 10 seconds...\n")
    #sleep(10)
    
    #mdbmc.matchMutualMaps()

    
def matchDBArtistWithoutAlbums(item, *args, **kwargs):
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = None
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'],
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score']),
    mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def matchDBArtistWithAlbums(item, *args, **kwargs):    
    #time.sleep(0.0025)

    
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = artistData["ArtistAlbums"]
    
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score'])
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def multiProc(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)
    result_list_tqdm = []
    for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
        result_list_tqdm.append(result)
    return result_list_tqdm

# Master Ignore List

In [ ]:
ignoresList = ["chartIgnores.yaml"]
ignoresList = []

ignoresList = []
ignoresList.append("Artists Stand Up To Cancer")
ignoresList.append("Cast Of Rent")
ignoresList.append("2006 Broadway Cast Recording")
ignoresList.append("Original Broadway Cast Of Dear Evan Hansen")
ignoresList.append("Various")
ignoresList.append("Various Artists")
ignoresList.append("Read-Along")
ignoresList.append("Varios Artistas")
ignoresList.append("Artists Against Bullying")
ignoresList.append("Varios")
ignoresList.append("Sleep Music Lullabies")
ignoresList.append("Yoga Meditation Tribe")
ignoresList.append("The Cast Of Stomp")
ignoresList.append("Quincy Jones Feat. The Cast Of Stomp/The Yes/No Productions")
ignoresList.append("Walt Disney Read-Along")
ignoresList.append("Classical Lullabies For Babies Academy")
ignoresList.append("Diverse")
ignoresList.append("Blandade Artister")
saveFile(idata=ignoresList, ifile="chartIgnoreArtists.yaml")

unknownList = []
unknownList.append("Amil-lion")
unknownList.append("Yung Lito")
unknownList.append("Idols")
unknownList.append("X Factor Finalists")
unknownList.append("R")
unknownList.append("Malena")
unknownList.append("Neales")
unknownList.append("Incognito")
unknownList.append('"Patches" Stewart')
unknownList.append("X Factor NZ Top 12")
saveFile(idata=unknownList, ifile="chartUnknownArtists.yaml")

internationalList = []
internationalList.append("3robiu")
internationalList.append("Multi-interpretes")
internationalList.append("Mormon Tabernacle Choir/Orch. At Temple Square/Yoncheva/Mumford/Villazon/Terfel (Wilberg)")
internationalList.append("Villazon")
internationalList.append("Mumford")
internationalList.append("Vienna Philharmonic (Dudamel)")
internationalList.append("Louisville Orchestra (Abrams)")
internationalList += ['Eight Ranger', 'Yi Yang Qian Xi', "ONE N' ONLY", "MoLa", "Sus Invitados"]
internationalList += ['Juanma Y Su Tuna Para Todo El Ano', 'NYC', 'Nogizaka 48', 'Sho Kiryuin from Golden Bomber']
internationalList += ['The MONSTERS', 'ZERO-G', 'Huang Lige', 'Renzo', 'Bodan Shonen Dan', 'ST RISH', 'King Cream Soda']
internationalList += ['Good Morning America', 'Hotta Ke BAND', 'Feng Jianyu', 'Stanley Most']
internationalList += ['Cardiff City Fc', 'Leeds United Team', 'Monkey Hangerz']
internationalList += ['Good Child Foundation', 'Neon Brotherhood', 'Rogue Souljahz']
internationalList += ['GMC', 'Krossfade', 'Dave Howley', 'Toko', 'Y-not', 'Afro Music']
internationalList += ['Husvenne', 'Maca Boy', 'Skyndeep', 'TwitterXmassingle', 'Julieanne Dineen', 'Julie-Anne Dineen', 'Anybodies']
internationalList += ["Miro"] # Bulgaria
internationalList += ["Lama", 'Vintaj', 'D-flow', 'U. Dumanska', 'Leonard', 'MARY', 'Naomi', 'IKE', 'FIESTA!', 'Francisco Loría']
internationalList += ['SANARI', 'KIRA', 'Deep Ng', 'BOYZ', 'SEUNGRI', 'TLDK', 'Della', 'Dadado Huang', 'WONFU', 'DEN', 'PORTRAIT', '2T FLOW', 'GULF', 'VANGOE', 'DA 阿達']
internationalList += ['Renan Almendarez Coello', 'Alfredo Ramirez Corral']
internationalList += ['K-One', 'K.One', 'Miss Murphy', 'Bandits', 'Pop', 'Industry', 'Bigg Face', 'Atlas', 'Chico', 'Pictures', 'Jay-Jay Harvey']
internationalList += ['Paw Justice', 'Jay-Jay Feeney', 'Guy Williams & Scribe', 'Dominic Harvey & Scribe']
internationalList += ['Ole']

saveFile(idata=internationalList, ifile="chartInternationalArtists.yaml")

tonewUnknownList  = ['Lil Adrei', 'The 7 Dimensions', 'HIXTAPE', 'Porky Slim']
tonewUnknownList += ['SL', 'Sidemen', 'Mastermind', 'HOSH', 'Frosty']
saveFile(idata=tonewUnknownList, ifile="chartToNewArtists.yaml")

{'Gabo Parisi':9846490}
{'Rory & The Island': 1219224, 'Sean Og': 537158, "Hoo Leeger": 10908080}
{'The Suspects and Guests': 5061058, "Dennis Reed & Gap": 15080705}
{}

{"Eri Esittajia": 243610}

soundtrackList = ['Jul På Vesterbro', 'Troy & Gabriella']
saveFile(idata=soundtrackList, ifile="chartSoundtrackArtists.yaml")

applemusicList  = ['Dutch','Deno','DHT','Sadbh']
applemusicList += ['DDG', 'Lean Trap', 'MOLO', 'Amina']
applemusicList += ['Rocket Girls 101', "David Miles", "Cyril"]
applemusicList += ["Camur"]
saveFile(idata=applemusicList, ifile="chartAppleMusicArtists.yaml")

spotifyList  = ['ROS', 'Mortal Frame', 'BENGELS', 'ILLSIGHT', 'KOI', 'VLS', 'EDGR', 'Kyle Edwards;DJ Smallz 732', 'Kyle Edwards;Chris Bevrly', 'La Vida Moderna', 'PJAY CZ', 'SK']
spotifyList += ['KING', 'CHILI', 'İntizar', 'Archez', 'buller', "Moov'art"]
spotifyList += ['Gunwest', 'NEEL', 'Cdeep', 'Fomins', 'Crime Sea']
spotifyList += ['Alan 2', 'Adonis 66']
spotifyList += ['DJ Yayo', 'Jason William']
spotifyList += ['hachi', 'STEADY&CO.', 'Srene', 'Dsean', 'MASON', 'VA', 'IT’S', 'ONEONE',
                'yui FLOWER FLOWER', 'ミゾベリョウ odol', 't', 'The GoGo Tuner Family']
saveFile(idata=spotifyList, ifile="chartSpotifyArtists.yaml")

musicvf  = ['The Georgia Minstrel Co.', "The Gilmore's Band", 'J. Aldrich Libbey', 'George Alexander', 'Lew Dockstader', 'Grace Nelson', 'The Columbia Comedy Trio', 'Frederic Rose', 'Ralph Herz', 'Joe Maxwell', 'A.D. Madeira']
musicvf += ['The Lyric Quartet', "The Conway's Band", 'The Columbia Mixed Quartet', 'The Columbia Mixed Double Quartet', 'The Blue & White Marimba Band', 'The Imperial Quartet of Chicago', 'Louis Gravieure', "M.J. O'Connell", 'Harry Ellis', 'French Army Band', 'Eddy Brown', 'the Lyric Quartet', 'the Columbia Stellar Quartet']
musicvf += ['The Broadway Quartet', 'Anthony Urato', 'The Club Royal Orchestra', 'The Virginians', 'The Broadway Dance Orchestra', 'Okeh Laughing Record', 'King Oliver & His Jazz Band', 'Lew Holtz & His Orchestra', 'Ace Brigode & His Virginians', 'Carolyn Thompson', 'The Radiolites', 'The Colonial Club Orchestra', 'Perry Askam',
            'Harold Grayson & His Orchestra', 'Lee Morse', 'Lou Gold', 'Adrian Schubert', 'Arnold Johnson']
musicvf += ["Joe Morrison", "Emil Velasco", "Mark Fisher & His Orchestra", "His Salon Orchestra"]
musicvf += ["George Hamilton", "Obregon's Orchestra", "Herbie Kay", "Frank Dailey", "Martha Stewart", "His Orchestra"]

saveFile(idata=musicvf, ifile="chartMusicVFArtists.yaml")

ignoresList = ["chartIgnoreArtists.yaml", "chartUnknownArtists.yaml", "chartInternationalArtists.yaml", "chartSpotifyArtists.yaml",
               "chartToNewArtists.yaml", "chartAppleMusicArtists.yaml", "chartSoundtrackArtists.yaml", "chartMusicVFArtists.yaml"]


# MDB Maps

In [ ]:
mdbmaps = {}
print("ChartType = {0}".format(chartType))
print("MType     = {0}".format(mType))
mdbmaps[chartType] = musicDBMap(chartType, init=False, copy=False)
mdbmc = masterDBMatchClass(maindb, mdbmaps)

manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)


def reMatch(mdbmaps=mdbmaps, mdbmc=mdbmc, mtype="Single"):
    if mtype == "Single":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getSingleArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
    elif mtype == "Many":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getManyArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
    elif mtype == "Full":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getFullArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
        
reMatch(mtype=mType)

## Match Using Names And Albums

In [ ]:
for minAlbums,minAlbumsData in getIterItems().items():
    print("="*10,minAlbums)
    maxAlbums = minAlbumsData["Max"]
    maxValues = minAlbumsData["Vals"]
    for i in range(10):
        toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=ignoresList)
        if len(toMatch[chartType]) == 0:
            break
        matchItAll(mdbmaps, getThresholds(minAlbums), toMatch=toMatch)
reMatch(mtype=mType)
print("Done.\n\n")

In [ ]:
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)


## Match Only Using Names (Artists Without A Previous Match)

In [ ]:
for i in range(1):
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
    if len(toMatch[chartType]) == 0:
        break
    matchItAll(mdbmaps, getThresholdsWithoutAlbums(1.0), toMatch=toMatch, useAlbums=False)
reMatch(mtype=mType)

In [ ]:
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)

## Try To Match Artists That Were Not Previously Tested (Low Albums)

In [ ]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=True, ignores=ignoresList)
matchItAll(mdbmaps, getThresholdsWithoutAlbums(1.0), toMatch=toMatch, useAlbums=False)
reMatch(mtype=mType)

In [ ]:
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)

## Extra Artists Info

In [30]:
#from extraArtists import extraKnownArtists
#extraKnownArtists(mdbmaps, chartType)
addMusicVFExtras(mdbmaps)
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)

Could not add artist data from PrimaryKey: 642b5ad5ba31c2c034188b2ae18b1bcc
Could not add artist data from PrimaryKey: 25fc449f0de6e4dbead30d2f006036f0
Could not add artist data from PrimaryKey: 97a3e9fd21ff48f082ce2a5779a489b1
Could not add artist data from PrimaryKey: da6a972bc80cc500f459cb2a577f006a
Could not add artist data from PrimaryKey: 89c08013c817a7d57d629c92612b49e1
Could not add artist data from PrimaryKey: 08fde96818c1dde60ab642e32b4276c2
Could not add artist data from PrimaryKey: 6cd63ada8bd0c670a3e0ad1232a6e5c8
Could not add artist data from PrimaryKey: 6047ae45fdcbb1a61452be74c2e09458
Could not add artist data from PrimaryKey: 6047ae45fdcbb1a61452be74c2e09458
Could not add artist data from PrimaryKey: 6047ae45fdcbb1a61452be74c2e09458
Could not add artist data from PrimaryKey: 38a08457915c65ef3cb1ff0b8a2db435
Could not add artist data from PrimaryKey: 38a08457915c65ef3cb1ff0b8a2db435
Could not add artist data from PrimaryKey: 38a08457915c65ef3cb1ff0b8a2db435
Could not ad

  --> This file is 712.9kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbMusicVFMap.p
  --> This file is 712.9kB.
Matched 2975 / 3036 Artists for MusicVF ChartType
========== Artists That Were Not Tested (Low Albums) ==========
Total                -> 3036
After Ignores        -> 2983
After Known          -> 8
After MaxAlbums      -> 8
After MinAlbums      -> 8
After MaxValues      -> 8
========== Previously Tested Artists Without A Match ==========
Total                -> 3036
After Ignores        -> 2983
After DB Matches     -> 15
After MaxAlbums      -> 15
After MinAlbums      -> 15
After MaxValues      -> 15


#### Unmatched Artists

In [17]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, 
                                 minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
len(toMatch[chartType])

Total                -> 3036
After Ignores        -> 2983
After DB Matches     -> 38
After MaxAlbums      -> 38
After MinAlbums      -> 38
After MaxValues      -> 38


38

# Manual Appends

In [19]:
#manualAppends(toMatch, chartType, minAlbums=10, add=True, dbs=["Discogs", "Deezer", "AllMusic"], showSongs=True)
#manualAppends(toMatch, chartType, minAlbums=0, add=True, dbs=["Discogs"], showSongs=False, pattern="Orchestra")

manualAppends(toMatch, chartType, minAlbums=1, add=True, dbs=["Deezer"], showSongs=True)

addDeezer(mdbmaps, "MusicVF", "642b5ad5ba31c2c034188b2ae18b1bcc", "")   ### [Poison [hard rock band]]
### Talk Dirty to Me  ,  Look What the Cat Dragged In  ,  Nothin' but a Good Time  ,  Open Up and Say...Ahh!  ,  Fallen Angel  ,  Every Rose Has Its Thorn  ,  Unskinny Bop  ,  Flesh & Blood  ,  Something to Believe In
### Talk Dirty to Me  ---> [['Poison [hard rock band]']]
### Look What the Cat Dragged In  ---> [['Poison [hard rock band]']]
### Nothin' but a Good Time  ---> [['Poison [hard rock band]']]
### Open Up and Say...Ahh!  ---> [['Poison [hard rock band]']]
### Fallen Angel  ---> [['Poison [hard rock band]']]
### Every Rose Has Its Thorn  ---> [['Poison [hard rock band]']]
### Unskinny Bop  ---> [['Poison [hard rock band]']]
### Flesh & Blood  ---> [['Poison [hard rock band]']]
### Something to Believe In  ---> [['Poison [hard rock band]']]


addDeezer(mdbmaps, "MusicVF", "4d2f2c8592f9de661b92955e93adf88f", "")   ### [His Clouds Of Joy]
### Christopher Columbus  ,  Until the R

In [ ]:
#updateManDB(3)
updateMultiDB(2)

In [ ]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

## MusicVF Extras

In [22]:
def addMusicVFExtras(mdbmaps):
    addDeezer(mdbmaps, "MusicVF", "642b5ad5ba31c2c034188b2ae18b1bcc", "5733")   ### [Poison [hard rock band]]
    addDeezer(mdbmaps, "MusicVF", "25fc449f0de6e4dbead30d2f006036f0", "7448")   ### [Vanessa Williams [Vanessa Lynn Williams]]
    addDeezer(mdbmaps, "MusicVF", "97a3e9fd21ff48f082ce2a5779a489b1", "415")   ### [Nirvana [American band]]
    addDeezer(mdbmaps, "MusicVF", "da6a972bc80cc500f459cb2a577f006a", "3526")   ### [The Jets [pop band]]
    addDeezer(mdbmaps, "MusicVF", "2285045d649fd03859d0a0006a97cbd0", "57267")   ### [Jade [American band]]
    addDeezer(mdbmaps, "MusicVF", "89c08013c817a7d57d629c92612b49e1", "2402")   ### [Enigma [german electronic music project]]
    addAllMusic(mdbmaps, "MusicVF", "407c6f5950271ed58c15fdb29c2c651e", "mn0000771838")   ### [IMx]
    addDeezer(mdbmaps, "MusicVF", "08fde96818c1dde60ab642e32b4276c2", "2667")   ### [Mr. Big [US band]]
    addDeezer(mdbmaps, "MusicVF", "9c830dfc527627333c6ebca6cfe31c76", "1808031")   ### [4 P.M.]
    addDeezer(mdbmaps, "MusicVF", "880abe0df6030ac2d8c20328de6ccb2f", "7549")   ### [Domino [American rapper]]
    addDeezer(mdbmaps, "MusicVF", "8e1a97fbdbbabc33fa3bbb6efd0c383c", "13604")   ### [Altern-8]
    addDeezer(mdbmaps, "MusicVF", "4906a255a6c377ca0702d42ecd4649b4", "111817")   ### [Candyman]
    addDeezer(mdbmaps, "MusicVF", "6cd63ada8bd0c670a3e0ad1232a6e5c8", "7365")   ### [Mel and Kim]
    addDeezer(mdbmaps, "MusicVF", "7cd09b4f40631db866403fa556bdaf2b", "367290")   ### [Partners in Kryme]
    addDeezer(mdbmaps, "MusicVF", "0930095900cc7de51f1a7d0b3a25e09b", "169264")   ### [Sweet Sensation [American band]]
    addDeezer(mdbmaps, "MusicVF", "88cecf53871bf08f5ad9687f63842c7c", "4016289")   ### [The Adventures of Stevie V]
    addDeezer(mdbmaps, "MusicVF", "b0d7a98d5833146a7acc0c013a56a526", "16321")   ### [Sonia [Sonia Evans]]
    addDeezer(mdbmaps, "MusicVF", "b76e723753b2ac17b303eebfbe64b88e", "289897")   ### [D:Mob]
    addDeezer(mdbmaps, "MusicVF", "61e900e6362defe3136e19d6753b96be", "2363")   ### [Johnny Kemp]
    addDeezer(mdbmaps, "MusicVF", "db2597d4c66bab23e2ed811a5383648b", "1502417")   ### [Doctor and the Medics]
    addDeezer(mdbmaps, "MusicVF", "e14a693cfb27201cbd404751156696dd", "3792")   ### [Don Johnson [actor]]
    addDeezer(mdbmaps, "MusicVF", "f050c4d82861ee7870fc4ebcacb095d3", "9916")   ### [Dave Stewart [David Allan Stewart]]
    addDiscogs(mdbmaps, "MusicVF", "8ed3b973a02c78a2a17862a5179b2af6", "1224507")   ### [Synch]
    
    addDiscogs(mdbmaps, "MusicVF", "6047ae45fdcbb1a61452be74c2e09458", "102404")   ### [Stars on 45/Starsound]
    addDeezer(mdbmaps, "MusicVF", "6047ae45fdcbb1a61452be74c2e09458", "451347")   ### [Stars on 45/Starsound]
    addAllMusic(mdbmaps, "MusicVF", "6047ae45fdcbb1a61452be74c2e09458", "mn0000019073")   ### [Stars on 45/Starsound]    
    
    addDiscogs(mdbmaps, "MusicVF", "92def4f0947b2b3a2b6deb83b698e0dd", "177974")   ### [Toyah]
    addDeezer(mdbmaps, "MusicVF", "92def4f0947b2b3a2b6deb83b698e0dd", "125542")   ### [Toyah]
    addAllMusic(mdbmaps, "MusicVF", "92def4f0947b2b3a2b6deb83b698e0dd", "mn0000012059")   ### [Toyah]
    addDiscogs(mdbmaps, "MusicVF", "928153b2815c2f2ee8f03e3b7d30f31e", "88740")   ### [Junior]
    addDeezer(mdbmaps, "MusicVF", "928153b2815c2f2ee8f03e3b7d30f31e", "64303492")   ### [Junior]
    addAllMusic(mdbmaps, "MusicVF", "928153b2815c2f2ee8f03e3b7d30f31e", "mn0000249552")   ### [Junior]
    addDiscogs(mdbmaps, "MusicVF", "38a08457915c65ef3cb1ff0b8a2db435", "31726")   ### [Ollie and Jerry]
    addDeezer(mdbmaps, "MusicVF", "38a08457915c65ef3cb1ff0b8a2db435", "5517")   ### [Ollie and Jerry]
    addAllMusic(mdbmaps, "MusicVF", "38a08457915c65ef3cb1ff0b8a2db435", "mn0000889962")   ### [Ollie and Jerry]
    addDiscogs(mdbmaps, "MusicVF", "06f8252aff2ada19a262b651bc610b42", "65957")   ### [Arcadia]
    addDeezer(mdbmaps, "MusicVF", "06f8252aff2ada19a262b651bc610b42", "211291")   ### [Arcadia]
    addAllMusic(mdbmaps, "MusicVF", "06f8252aff2ada19a262b651bc610b42", "mn0000501561")   ### [Arcadia]
    addDiscogs(mdbmaps, "MusicVF", "7ae3fb2747978b2d93a22910fb1bb293", "150891")   ### [Dave Stewart [David Lloyd Stewart]]
    addAllMusic(mdbmaps, "MusicVF", "7ae3fb2747978b2d93a22910fb1bb293", "mn0001737671")   ### [Dave Stewart [David Lloyd Stewart]]
    addDiscogs(mdbmaps, "MusicVF", "97845c5ac2a1eda6bc7a40a52550f6cf", "397333")   ### [Scandal [rock band]]
    addDeezer(mdbmaps, "MusicVF", "97845c5ac2a1eda6bc7a40a52550f6cf", "7943564")   ### [Scandal [rock band]]
    addAllMusic(mdbmaps, "MusicVF", "97845c5ac2a1eda6bc7a40a52550f6cf", "mn0000315142")   ### [Scandal [rock band]]

    addDiscogs(mdbmaps, "MusicVF", "e4b3177fc3046cf9d6bef9f8de838668", "152707")   ### [Mel Tormé and His Mel-Tones]
    addDeezer(mdbmaps, "MusicVF", "e4b3177fc3046cf9d6bef9f8de838668", "5036455")   ### [Mel Tormé and His Mel-Tones]    
    addDiscogs(mdbmaps, "MusicVF", "51bdddcad9ec3f5a5e37350c4fd05bf7", "319856")   ### [Keith]
    addDeezer(mdbmaps, "MusicVF", "51bdddcad9ec3f5a5e37350c4fd05bf7", "177616")   ### [Keith]    
    addDiscogs(mdbmaps, "MusicVF", "e1d965d3065c3be9e7bd57c63a6e20e5", "31208")   ### [Focus [1°]]
    addDiscogs(mdbmaps, "MusicVF", "c35939027e0e7ec06e8320dd46e790ef", "273394")   ### [John Williams [composer]]
    addDiscogs(mdbmaps, "MusicVF", "5329dea1b8ecda6841c5ab6bdab61a43", "384584")   ### [Rick Dees & His Cast of Idiots]
    addDeezer(mdbmaps, "MusicVF", "5329dea1b8ecda6841c5ab6bdab61a43", "183782")   ### [Rick Dees & His Cast of Idiots]
    addDiscogs(mdbmaps, "MusicVF", "f42685ea4f66c85a4635a8d3f95366a6", "283288")   ### [Delaney, Bonnie & Friends]
    addDiscogs(mdbmaps, "MusicVF", "dece6e0c0c1c1cad29e0e0f625d4018c", "319103")   ### [Gallery]
    addDiscogs(mdbmaps, "MusicVF", "47f792069a03acd651a6ecbde65895a0", "511467")   ### [Climax [US band]]
    addDiscogs(mdbmaps, "MusicVF", "22f9b012c07c6890e1bed160182226b3", "115131")   ### [The Dramatics [1°]]
    addDiscogs(mdbmaps, "MusicVF", "2c947f7a889c2d431cd00e5e85913df6", "3452")   ### [M.F.S.B.]
    addDiscogs(mdbmaps, "MusicVF", "b26d4ae19a5b1cf2ddb07b32371e69d9", "139260")   ### [Eruption]
    addDiscogs(mdbmaps, "MusicVF", "e6972ff5d00f1d66b88746d2fc81260a", "86248")   ### [The Emotions [female R&B trio]]
    addDiscogs(mdbmaps, "MusicVF", "085f10fce6a33c786f967fc1729f2042", "303048")   ### [Brian and Michael]
    addDiscogs(mdbmaps, "MusicVF", "b1014004f336ff817d2c905413665d71", "223235")   ### [Jigsaw [English group]]
    addDiscogs(mdbmaps, "MusicVF", "c79c6b1627a8e4bc4027a3fcd8617b72", "166278")   ### [Dave and Ansil Collins]
    addDiscogs(mdbmaps, "MusicVF", "f14ae50f772d6187b5e821f5a1547a4a", "325408")   ### [Sweet Sensation [english band]]
    addDiscogs(mdbmaps, "MusicVF", "5ff0fb159078d09e3d20d23c36e36a70", "234332")   ### [The Free Movement]    
    
    addDiscogs(mdbmaps, "MusicVF", "7cc6279cab5f001ecbf8b848727bb272", "324860")   ### [The Scaffold]
    addDiscogs(mdbmaps, "MusicVF", "a863ec7ba236ab6614b59e37a8bd69e6", "458405")   ### [The Outsiders [1°]]
    addDiscogs(mdbmaps, "MusicVF", "672a7e5a27ae7c8ea520df12753fb5d7", "630848")   ### [Staff Sgt. Barry Sadler]
    addDiscogs(mdbmaps, "MusicVF", "d6952fda07639466bf3efbc89b46e63b", "372980")   ### [Unit 4 + 2]
    addDiscogs(mdbmaps, "MusicVF", "0b68011a45365670d3c914265302fa59", "62500")   ### [The Larks [Los Angeles band]]
    addDiscogs(mdbmaps, "MusicVF", "000e2d4b0fff826c1aef2bfe7634590f", "277351")   ### [The Four Pennies [English pop group]]
    addDiscogs(mdbmaps, "MusicVF", "7ed47eccfad0f0538664bf35989eab92", "368275")   ### [The Angels [US girl group]]
    addDiscogs(mdbmaps, "MusicVF", "29c31ab9aa41caa9389c02d6fec4461d", "318675")   ### [B. Bumble and the Stingers]
    addDiscogs(mdbmaps, "MusicVF", "3f59c400561a7b838e9ac29a06e49c04", "886357")   ### [Joe Henderson [American R&B singer]]
    addDiscogs(mdbmaps, "MusicVF", "a8c211fe0d027c386fe16e2ddb15753f", "337529")   ### [Ned Miller]
    addDiscogs(mdbmaps, "MusicVF", "210876931ba27271afa8197663ccc915", "1888215")   ### [Little Caesar & the Romans]
    addDiscogs(mdbmaps, "MusicVF", "2da77bc7d525ec9d446c3055bd4a22c2", "1214698")   ### [Kokomo [Jimmy Wisner]]
    addDiscogs(mdbmaps, "MusicVF", "6685fc6e664ad472deb9461525248eb8", "593039")   ### [The Sims Twins]
    addDiscogs(mdbmaps, "MusicVF", "81fb128ce380557b0464045d4bbd5713", "294706")   ### [Bobby Lewis]
    addDeezer(mdbmaps, "MusicVF", "81fb128ce380557b0464045d4bbd5713", "104995")   ### [Bobby Lewis]
    addAllMusic(mdbmaps, "MusicVF", "81fb128ce380557b0464045d4bbd5713", "mn0000071409")   ### [Bobby Lewis]
    addDiscogs(mdbmaps, "MusicVF", "dbe12b0fd1f39d4baa1b03990374f660", "275791")   ### [Them]
    addDeezer(mdbmaps, "MusicVF", "dbe12b0fd1f39d4baa1b03990374f660", "2463")   ### [Them]
    addAllMusic(mdbmaps, "MusicVF", "dbe12b0fd1f39d4baa1b03990374f660", "mn0000925181")   ### [Them]
    addDiscogs(mdbmaps, "MusicVF", "fd64625ef0ce3f6a1d6d91b8b932cd94", "2082135")   ### [The Cuff Links]
    addDiscogs(mdbmaps, "MusicVF", "f427959753a9b354845e51fdb7ab8dda", "4278444")   ### [Every Mother's Son]
    addDiscogs(mdbmaps, "MusicVF", "1b9252d1f463dd59a80a09a9a6049afa", "319856")   ### [Keith [American pop singer]]
    addDiscogs(mdbmaps, "MusicVF", "afceff565d12dc99275867186100b959", "336642")   ### [Ruby & the Romantics]
    addDiscogs(mdbmaps, "MusicVF", "2f4f399b889d3e70513490344c3fc647", "731174")   ### [Dick and Dee Dee]

    addDiscogs(mdbmaps, "MusicVF", "a7efe1bba5bb56d1f0ade2a48c82c260", "677033")   ### [The Highwaymen [folk group]]
    addDeezer(mdbmaps, "MusicVF", "a7efe1bba5bb56d1f0ade2a48c82c260", "112328")   ### [The Highwaymen [folk group]]
    addAllMusic(mdbmaps, "MusicVF", "a7efe1bba5bb56d1f0ade2a48c82c260", "mn0000075801")   ### [The Highwaymen [folk group]]

    addDiscogs(mdbmaps, "MusicVF", "585e0b8fbd3e1e6109ccdd0b924a15ed", "337625")   ### [Freddie and the Dreamers]
    addDeezer(mdbmaps, "MusicVF", "585e0b8fbd3e1e6109ccdd0b924a15ed", "61723")   ### [Freddie and the Dreamers]
    addAllMusic(mdbmaps, "MusicVF", "585e0b8fbd3e1e6109ccdd0b924a15ed", "mn0000177558")   ### [Freddie and the Dreamers]    
    
    addDeezer(mdbmaps, "MusicVF", "7f3f30dd052dda98c4d387a3522b107e", "14998")   ### [Sonny and Cher]
    addAllMusic(mdbmaps, "MusicVF", "7f3f30dd052dda98c4d387a3522b107e", "mn0000042993")   ### [Sonny and Cher]    
    
    addDiscogs(mdbmaps, "MusicVF", "688def85e032254c234d8441f307033f", "292376")   ### [Martha and the Vandellas]
    addDeezer(mdbmaps, "MusicVF", "688def85e032254c234d8441f307033f", "272982")   ### [Martha and the Vandellas]
    addAllMusic(mdbmaps, "MusicVF", "688def85e032254c234d8441f307033f", "mn0001199135")   ### [Martha and the Vandellas]    
    
    addDiscogs(mdbmaps, "MusicVF", "7a97c74c2b75a68121523083e4170d4e", "3761848")   ### [Harold Stern & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1f7b5453ba1804f413bf127e230278ad", "4892300")   ### [The Castillian Troubadors]
    addAllMusic(mdbmaps, "MusicVF", "9c31b906e799c6be454297e9368d9aa0", "mn0003550643")   ### [The Castillians]

    addDiscogs(mdbmaps, "MusicVF", "0f50f13de71c3fe2c345ddf05b969181", "3547291")   ### [Tex Grande and His Range Riders]
    addDiscogs(mdbmaps, "MusicVF", "c8ced3e0213ee278030a5010d9c99cf3", "317903")   ### [Bennie Moten's Kansas City Orch.]
    addDiscogs(mdbmaps, "MusicVF", "959f839dd111bbb6bd2a099d8b8438d8", "1836420")   ### [Fred Hillebrand]
    addDiscogs(mdbmaps, "MusicVF", "0f74724d746e8e4bb2e38c48d99d18d0", "5562397")   ### [Dick Barrie]
    addDiscogs(mdbmaps, "MusicVF", "8b75992b1dd9207bbc405a144f70fbcf", "7369644")   ### [Hod Williams]
    addDiscogs(mdbmaps, "MusicVF", "a9bd1784af29d4a358ac6994aeae8294", "3338780")   ### [Red Jessup]
    addDiscogs(mdbmaps, "MusicVF", "36e533d418ac3f05e29bfa4a1390c4f4", "258518")   ### [the Mills Brothers]
    addDiscogs(mdbmaps, "MusicVF", "fc0b440ba7ef6104661b11a433a85acc", "1514960")   ### [Joe Sanders]
    addDiscogs(mdbmaps, "MusicVF", "f256e9c891c17308d9dfc8ca8844bf61", "1417290")   ### [King Garcia]
    addDiscogs(mdbmaps, "MusicVF", "8e141aaf19712fdc32189ddfeb80430e", "781771")   ### [Bob Howard]
    addDiscogs(mdbmaps, "MusicVF", "50d3706b2605b07b8b8a9b9961bea6ba", "1335225")   ### [Raymond Paige]
    addDiscogs(mdbmaps, "MusicVF", "ef9bf813076f90b5f8eaf2da0d244357", "899557")   ### [Rhythm Wreckers]
    addDiscogs(mdbmaps, "MusicVF", "0b9550156cafd474b09fe8778504d2ad", "905976")   ### [Rose Marie]
    addDiscogs(mdbmaps, "MusicVF", "bc1d9ac4203fedd720c413be4fa0a8f4", "383328")   ### [The Ravens]
    addDiscogs(mdbmaps, "MusicVF", "fd6d53a7d15d0844f39f16da39a5b360", "847610")   ### [Slim Gaillard Trio]
    addDiscogs(mdbmaps, "MusicVF", "7294bddc35185cd867b705196438ea18", "332327")   ### [Roosevelt Sykes and His Piano]
    addDiscogs(mdbmaps, "MusicVF", "9ed340e7c406c35d62e51ecb4c47ffb0", "320605")   ### [Pvt. Cecil Gant]
    addDiscogs(mdbmaps, "MusicVF", "e17c9d6b12e51038367a15bc40d84107", "337637")   ### [Don Reid]

    addDiscogs(mdbmaps, "MusicVF", "4f7f88f7e39b56a37e1f6cfcab989684", "4109908")   ### [The Havana Novelty Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "8517343b6aafcd5373bd348b2940f107", "864317")   ### [The Royal Teens]
    addDiscogs(mdbmaps, "MusicVF", "6c7dda5c7af0d386d430c3d096eacc75", "229181")   ### [The Big Bopper]
    addDiscogs(mdbmaps, "MusicVF", "3bf6a6337a24644243ccede70fd1d4e0", "635174")   ### [Charles McDevitt Skiffle Group]
    addDiscogs(mdbmaps, "MusicVF", "4e9789b4dbc9c9ff3c21edafeea91691", "1434057")   ### [Johnny Brandon]
    addDiscogs(mdbmaps, "MusicVF", "ae9f6d8935c41a1f5d78bf5650dec443", "1405959")   ### [Ray Burns]
    addDiscogs(mdbmaps, "MusicVF", "e04a5e161d882c31c39a80f6dfbe3efb", "656752")   ### [Somethin' Smith & the Redheads]
    addDiscogs(mdbmaps, "MusicVF", "d6f83baaad3622cf3bbcfea3dcf7db72", "661725")   ### [David Carroll]
    addDiscogs(mdbmaps, "MusicVF", "a7fb814f78370c1709f8d71d89cdbbb7", "907304")   ### [Norman Petty Trio]
    addDiscogs(mdbmaps, "MusicVF", "d1fcd1acf91f461760f5db89d6371243", "1554497")   ### [José Ferrer]
    addDiscogs(mdbmaps, "MusicVF", "7fe410b9139c911fce973d3214c59b7e", "3727380")   ### [Frank Weir & His Saxophone, Chorus & Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c596cadb17d03d7e0a1a100fd6de5f3b", "1262946")   ### [The DeCastro Sisters]
    addDiscogs(mdbmaps, "MusicVF", "4796166bde6ef44bc2ce9eff6552fd8f", "1388910")   ### [Mariners]
    addDiscogs(mdbmaps, "MusicVF", "8cf7b312ec780e576720661724408a48", "417192")   ### [Les Compagnons de la chanson]
    addDiscogs(mdbmaps, "MusicVF", "7ebc2c92dbda26a5f35ec6609d9fec6e", "3598682")   ### [The Harry Grove Trio]
    addDiscogs(mdbmaps, "MusicVF", "2aa2c3d7a02d9717285ce61414a40520", "2504095")   ### [Eddie Wilcox Orch.]
    addDiscogs(mdbmaps, "MusicVF", "724e838678ee9e6c035146a1061a5d78", "8567655")   ### [The Wesson Brothers]
    addDiscogs(mdbmaps, "MusicVF", "b9639af619090eaddce0680eb25f93ea", "317205")   ### [Stardusters]
    addDiscogs(mdbmaps, "MusicVF", "7437e24d08744ee791f60c08f71913ab", "938654")   ### [Dan Russo & His Oriole Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f019a54512ffd89d0308b1aef5166ea0", "4917055")   ### [Vaughn Horton and His Polka Debs]
    addDiscogs(mdbmaps, "MusicVF", "f962c6513c6bd518c2ff896bf46d2b79", "3888608")   ### [Jack Lathrop and the Drugstore Cowboys]
    addDiscogs(mdbmaps, "MusicVF", "ae29b7c1d80d4e045d5390a50bb10d70", "1667729")   ### [Brother Bones and His Shadows]
    addDiscogs(mdbmaps, "MusicVF", "d9451d357d4099416723ea60d0ed6c8f", "1558243")   ### [Anne Vincent]
    addDiscogs(mdbmaps, "MusicVF", "96ab13605e07ad3b97b398b307da8eb7", "1469965")   ### [The Poni-Tails]
    addDiscogs(mdbmaps, "MusicVF", "8095bcff1d8303d5575f1b4f937d0622", "383343")   ### [The Monotones]
    addDiscogs(mdbmaps, "MusicVF", "5c5707abdb7762182cbab8090143a83b", "5102087")   ### [Dick Two Ton Baker and His Music Makers]
    addDiscogs(mdbmaps, "MusicVF", "c90554db46d2a16fa43a47fecef71127", "395324")   ### [Hugo Winterhalter And His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e1641cc8a8376ec4d76d74693b3390f4", "1968385")   ### [Billie Anthony]
    addDiscogs(mdbmaps, "MusicVF", "9c8b8d608c91561c6b88cee25361bf34", "367959")   ### [Mantovani And His Orchestra And Chorus]
    addDiscogs(mdbmaps, "MusicVF", "c9b54441216933c30501b0c5be4ddc72", "3587388")   ### [Grady Martin And His Slew Foot Five]
    addDiscogs(mdbmaps, "MusicVF", "3fb860cf933027fb20c35ae3fcfd6e67", "2547731")   ### [Laurie Anders]
    addDiscogs(mdbmaps, "MusicVF", "8328d465bb4eb143b2f8fba5d81eea1b", "1476286")   ### [Gloria DeHaven]
    addDiscogs(mdbmaps, "MusicVF", "3488e88f070db7ceb2400a560c01fc5f", "1514912")   ### [Victor Young And His Orchestra And Chorus]
    addDiscogs(mdbmaps, "MusicVF", "76eac162552ff5d6ac53e0a287fa453f", "1455378")   ### [The Keynotes]
    addDiscogs(mdbmaps, "MusicVF", "3f95e818916a3728097d2d5746d35d13", "1474769")   ### [Ernie Fields]
    addDiscogs(mdbmaps, "MusicVF", "980c84a5a3ac32aec4734996632fd25e", "272302")   ### [Mel Blanc and His Original Woody Woodpecker Voice]
    addDiscogs(mdbmaps, "MusicVF", "098bd0895351657a096917d224d630ef", "350223")   ### [Bob Crosby And His Bob Cats]
    addDiscogs(mdbmaps, "MusicVF", "c348f499cd1aae2a1d4da55954a5a514", "987396")   ### [Harry Cool]
    addDiscogs(mdbmaps, "MusicVF", "9c815acfd163b9034f3e09bc433acd0b", "2049936")   ### [Sally Foster]
    addDiscogs(mdbmaps, "MusicVF", "1593fbca70245e0ef0cb9d6ed375e4f8", "3155878")   ### [Muriel Lane]
    addDiscogs(mdbmaps, "MusicVF", "4ba957ba66bf53e0bb07c4da6147d24b", "573278")   ### [Gracie Allen]
    addDiscogs(mdbmaps, "MusicVF", "d4f29b4108645f92a1409b30a4ace170", "161916")   ### [Lolita]
    addDiscogs(mdbmaps, "MusicVF", "5650a29f417ad387c6921cc87b082300", "379734")   ### [Rosie]
    addDiscogs(mdbmaps, "MusicVF", "60de3b382798b1cf4bb63ce5fa675723", "901244")   ### [Miss Toni Fisher]
    addDiscogs(mdbmaps, "MusicVF", "c7c9f3935857a0bbc1ed2341040be9f4", "3111328")   ### [Billy Williams and the Pecos River Rogues]
    addDiscogs(mdbmaps, "MusicVF", "853ab232e08af792c2d65bbaa6340989", "2168178")   ### [The Uptown String Band]
    addDiscogs(mdbmaps, "MusicVF", "e3a376e87ef622aa85d8df23c574b64a", "317867")   ### [Nellie Lutcher and Her Rhythm]
    addDiscogs(mdbmaps, "MusicVF", "dd73a219cf98b7cfc16fe040b7738c6e", "3447620")   ### [Boone's Jumpin' Jacks]
    addDiscogs(mdbmaps, "MusicVF", "2a51cf04bd16e24cd0868639441d68ba", "4981989")   ### [Jimmy Lytell and His All Star Seven]
    addDiscogs(mdbmaps, "MusicVF", "ab56a6296bb617d0fe456852c9bdcae0", "3583401")   ### [The Royal Harmony Quartet]
    addDiscogs(mdbmaps, "MusicVF", "c0f175962521dbfe76a78c87e17fe44f", "1771795")   ### [Joan Merrill]
    addDiscogs(mdbmaps, "MusicVF", "b09f4a642a189c908803c168f8b32f2f", "557958")   ### [The Cats and the Fiddle]
    addDiscogs(mdbmaps, "MusicVF", "ae1f3d85e98fd09f235bcad1826ab703", "6990602")   ### [Al Kavelin and His Cascading Chords]
    addDiscogs(mdbmaps, "MusicVF", "6465abd321796236f977274ff6d6cb1f", "1678804")   ### [The Three Peppers]
    addDiscogs(mdbmaps, "MusicVF", "c024192437df576ff5cae3c7b409ec94", "374160")   ### [Jack McVea and His All Stars]
    addDiscogs(mdbmaps, "MusicVF", "02318adbba0b059a10e90b8b5a3bbc84", "1514914")   ### [Patricia Norman]
    addDiscogs(mdbmaps, "MusicVF", "a5006bb89a57d13ed942a00e1732e814", "3680685")   ### [The Milt Herth Trio]
    addDiscogs(mdbmaps, "MusicVF", "a59011ebcf56caedfe2145b6f5dad40b", "3838547")   ### [The Top Hatters]
    addDiscogs(mdbmaps, "MusicVF", "29c108cd41f644fd4253d32065929887", "1249695")   ### [The Sweet Violet Boys]
    addDiscogs(mdbmaps, "MusicVF", "b9b0d5327aa46f043b5cb674e8987d9d", "301370")   ### [Adrian Rollini & the Gang]
    addDiscogs(mdbmaps, "MusicVF", "a24ae2f1563085e1bbca2e213dd315f7", "1388909")   ### [Frank Parker]
    addDiscogs(mdbmaps, "MusicVF", "0975b95c438e2749aababfa0b9ed60b2", "1087275")   ### [Walter O'Keefe]
    addDiscogs(mdbmaps, "MusicVF", "8ea487d42f8bc4ac90c61eb7b1d72b87", "3778825")   ### [Conrad Thibault]
    addDiscogs(mdbmaps, "MusicVF", "82052cbfc3245267698272e4a3b4cfe4", "3444018")   ### [The Rondoliers & Piano Pals]
    addDiscogs(mdbmaps, "MusicVF", "1a0b3b68bbc0b6cccacfe7a463c27167", "651412")   ### [Six Hits and a Miss]
    addDiscogs(mdbmaps, "MusicVF", "3484f84ebdfee149b9b7d5638a863c54", "552073")   ### [Edward Byrnes]
    addDiscogs(mdbmaps, "MusicVF", "eb01b709e14365bf842b348bf75f5ee9", "7205117")   ### [Beverly White and Her Blues Chasers]
    addDiscogs(mdbmaps, "MusicVF", "39cf6983b7a23fb216da3bbd965539c8", "3235012")   ### [Phil Moore Four]
    addDiscogs(mdbmaps, "MusicVF", "27b50128a8ed15dfa9e696c0572f4abd", "2847852")   ### [5 Red Caps]
    addDiscogs(mdbmaps, "MusicVF", "a8ecbc08426611afc0348e1af183a49e", "1950077")   ### [Helen Carroll]
    addDiscogs(mdbmaps, "MusicVF", "55953e4539e4e5fd1b8ae761bd191f97", "895462")   ### [Dick Thomas]
    addDiscogs(mdbmaps, "MusicVF", "aef328dd8453ebcd3554224d8f28b7e5", "2081299")   ### [The Three Flames]
    addDiscogs(mdbmaps, "MusicVF", "bc78373a0bc71ca596adc70aa5dffe25", "2547592")   ### [Jerry Jerome and His Cats and Jammers]
    addDiscogs(mdbmaps, "MusicVF", "e589a798adf5ef25c8b7422f40684307", "4204858")   ### [Fred Waring and His Concert Vochestra]
    addDiscogs(mdbmaps, "MusicVF", "594076385af78f70120acafd064ce702", "4178021")   ### [Phil Hanna]

    addDiscogs(mdbmaps, "MusicVF", "5dc881691a067fa0a6cd6fcad2bba762", "759128")   ### [John Zacherle the Cool Ghoul]
    addDiscogs(mdbmaps, "MusicVF", "370d38be0e589813ad46b01a602902d2", "368604")   ### [King Brothers]
    addDiscogs(mdbmaps, "MusicVF", "4fb6ab67d1cbad34c3fc92490d43b309", "190250")   ### [Buchanan and Goodman]
    addDiscogs(mdbmaps, "MusicVF", "253643bcac6f68d5bc9f5bbb71ce0bb3", "1988086")   ### [Johnny Duncan [bluegrass musician]]
    addDiscogs(mdbmaps, "MusicVF", "eb347699e9e10d06721cfd0b48f94491", "297492")   ### [The Chords [1950s american doo wop group]]
    addDiscogs(mdbmaps, "MusicVF", "a5fd7caf3c0e9a1f04ace99d10b64836", "253855")   ### [Tommy Dorsey Ork.]
    addDiscogs(mdbmaps, "MusicVF", "5ba5fc8aad13de7cd1b6dd1d551d2b87", "1775011")   ### [the Lancers]

    addDiscogs(mdbmaps, "MusicVF", "fd724efa6b00d22d253709bb192e7941", "3387486")   ### [Frank Froeba & His Swing Band]
    addDiscogs(mdbmaps, "MusicVF", "03bdc7069772b8b97585ea378a19d88e", "312090")   ### [Otis Williams & The Charms]
    addDiscogs(mdbmaps, "MusicVF", "3463f172d0c20c18c7be1751781e2a12", "2350031")   ### [Tiny Hill]
    addDiscogs(mdbmaps, "MusicVF", "15f1ad3a8340840bfdd05ddcba0ffdf9", "1239010")   ### [Benny Strong And His Orchestraa]
    addDiscogs(mdbmaps, "MusicVF", "0a79439ac5d35709853504af6ecaa7f0", "5914209")   ### [Tom Coakley & His Palace Hotel Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "168ce0f17cb46c73cb685e5348a61583", "893226")   ### [Billy Williams]
    addDiscogs(mdbmaps, "MusicVF", "144244e3229bbe989c6db9efe6ee77f5", "418972")   ### [Delta Rhythm Boys]
    addDiscogs(mdbmaps, "MusicVF", "e7a09af3f02cd73a9731f5bdcc3a9fa1", "798934")   ### [Joe Liggins and His Honeydrippers]
    addDiscogs(mdbmaps, "MusicVF", "68c11950a8674db0ae8eb317a8bf7580", "758541")   ### [Sportsmen]
    addDiscogs(mdbmaps, "MusicVF", "f3b33db42d0205998dfcace184c74035", "312090")   ### [The Charms]
    addDiscogs(mdbmaps, "MusicVF", "950195f930ffac627fd60389448103b8", "2822201")   ### [Art and Dotty Todd]
    addDiscogs(mdbmaps, "MusicVF", "f1fdde3f9b04a07031409a4a5b919660", "281075")   ### [Jerry Wayne]
    addDiscogs(mdbmaps, "MusicVF", "d2e7f7df29489827a769427189944546", "307305")   ### [Al Morgan]
    addDiscogs(mdbmaps, "MusicVF", "8f0ddc32fccfc5ba30204410455c2c5c", "302684")   ### [Joe Fingers Carr]
    addAllMusic(mdbmaps, "MusicVF", "c5a32420d3a2fb6dfa25b74b009fe1ef", "mn0001768318")   ### [Pinetoppers]
    addDiscogs(mdbmaps, "MusicVF", "71c6249285dd6b7dad774ec4ae0e1c8b", "984446")   ### [Song Spinners]
    addDiscogs(mdbmaps, "MusicVF", "4186632a3d8f4b415cfac0b89d8e46da", "1777288")   ### [Bobby Wayne]
    addDiscogs(mdbmaps, "MusicVF", "69ceb43a1076ede530403cddd2fbe5c7", "638171")   ### [Pinky Tomlin]
    addDiscogs(mdbmaps, "MusicVF", "909d4adb2d2c30ea0f78fab4b4e0098c", "1604825")   ### [Louise Massey and the Westerners]
    addDiscogs(mdbmaps, "MusicVF", "22c802e6c5da0ed138e510b634a0110f", "585772")   ### [Dick Powell]
    addDiscogs(mdbmaps, "MusicVF", "acbceebafa88f32801748a144efc5e29", "1169164")   ### [Barry Wood]
    addDiscogs(mdbmaps, "MusicVF", "327e660219fe8294f8622cadd72892d5", "3766687")   ### [The Hotel Commodore Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e8c09762848a72497df7757fafbd9e2e", "2755752")   ### [Jerry Blaine & His Streamline Rhythm]

    addDiscogs(mdbmaps, "MusicVF", "c5cd9627f130cf4cae0c8c7a0f5ad51d", "447537")   ### [The Jesters]
    addDiscogs(mdbmaps, "MusicVF", "cf20fd9520d3f64be55bfefde5740907", "1239803")   ### [The Stargazers]
    addDiscogs(mdbmaps, "MusicVF", "b2a52870d6d0959c214eafda098d765a", "1089961")   ### [Gray Gordon]
    addDiscogs(mdbmaps, "MusicVF", "86cc9cc2430255441266f086719425ec", "506604")   ### [Julia Lee and Her Boy Friends]
    addDiscogs(mdbmaps, "MusicVF", "48f18a34d00fc9d64dd304b1c3ace7d7", "889650")   ### [Donald Novis]
    addDiscogs(mdbmaps, "MusicVF", "25db75ac34ad19ff373c25ec27d7c0cf", "1320571")   ### [Gertrude Niesen]
    addDiscogs(mdbmaps, "MusicVF", "54ad04919641707c535b9d0e0cb78fbc", "1188913")   ### [Johnny Maddox]
    addDiscogs(mdbmaps, "MusicVF", "99a4c19c1d091eadde2361fee561dcd3", "1912920")   ### [Mitchell Ayres and His Fashions in Music]
    addDiscogs(mdbmaps, "MusicVF", "b8e5b236ab73bf0ee6f0c223eafac251", "2891073")   ### [Tempo King & His Kings of Tempo]
    addDiscogs(mdbmaps, "MusicVF", "74bcd2926d57a2bac2f56b3d2c2b62a2", "709833")   ### [Jimmie Rodgers [pop singer]]
    addDiscogs(mdbmaps, "MusicVF", "c294c15e746828bd4f3ef842856d3d9a", "1239803")   ### [The Stargazers [British vocal group]]

    addDiscogs(mdbmaps, "MusicVF", "1311428767d81e19393dca9e26fd5392", "548019")   ### [Jack Smith]
    addDiscogs(mdbmaps, "MusicVF", "78b263751efbd75720f901ab0d78c1d9", "3164458")   ### [Tommy Tucker Time]
    addDiscogs(mdbmaps, "MusicVF", "1acc9f3642a9a2ba179d9945019b8aea", "4532644")   ### [Johnnie Johnston]
    addDiscogs(mdbmaps, "MusicVF", "3ff2d787edff99136e77f7864cc8c75a", "320797")   ### [The King Cole Trio]
    addDiscogs(mdbmaps, "MusicVF", "96c6305f37ffe8e7a86f4b49f3c23851", "915906")   ### [Swing And Sway With Sammy Kaye]

    addDiscogs(mdbmaps, "MusicVF", "303c0a0d96229c9581426b1fd33bd9f9", "307369")   ### [Connee Boswell]
    addDiscogs(mdbmaps, "MusicVF", "9dd3afd3c1e8a0f00392f3a1adee1d50", "2725656")   ### [Art Kassel and His Kassels-in-the-Air]
    addDiscogs(mdbmaps, "MusicVF", "b45d6d5f8c69607588df29d0748a89df", "2891172")   ### [Dolly Dawn & Her Dawn Patrol]
    addDiscogs(mdbmaps, "MusicVF", "6d891605cca4053b7956de9ddda8bd73", "335595")   ### [Andy Kirk and His Clouds of Joy]
    addDiscogs(mdbmaps, "MusicVF", "df2027d9e5b2df8691f950d59cc5a322", "349509")   ### [Eddy Duchin and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b52dbc06b774edfd2d32ef664958397b", "349494")   ### [Kay Kyser and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7c833833f5a7019308daa45c0497ddff", "1348205")   ### [Ted Wallace & His Campus Boys]
    addDiscogs(mdbmaps, "MusicVF", "b04bf1b01cc83657ce2033def0b9fb1f", "1700393")   ### [Moran & Mack]
    addDiscogs(mdbmaps, "MusicVF", "c3a8e594c568d01b41ec85f6e572c1fc", "1117996")   ### [Don Azpiazu & His Havana Casino Orchestra]

    addDiscogs(mdbmaps, "MusicVF", "0cbf330d8edc6626ab13340a54c281a7", "4515599")   ### [Kitty O'Connor]
    addDiscogs(mdbmaps, "MusicVF", "3a4d9b2db9828197f6014e586219d6ab", "1637594")   ### [The Charleston Chasers]
    addDiscogs(mdbmaps, "MusicVF", "e8b8acba3d3e770d65131602dd149ede", "969363")   ### [The Cotton Pickers]
    addDiscogs(mdbmaps, "MusicVF", "dff50fd358f481f7700224a34fe7d237", "486224")   ### [The Duncan Sisters [1°]]
    addDiscogs(mdbmaps, "MusicVF", "282a55383b3b2531e2af708dcc6de186", "516930")   ### [Henry Whittier]
    addDiscogs(mdbmaps, "MusicVF", "2a3c936e4d859c36f18fe32013f52989", "3115628")   ### [Warner's Seven Aces]
    addDiscogs(mdbmaps, "MusicVF", "183b68d77047d8fc95dce60b1adfa03b", "3617667")   ### [Dennis King]
    addDiscogs(mdbmaps, "MusicVF", "849e543e93f5ecebd77ed894f71f9894", "1528216")   ### [Willie Howard]

    addDiscogs(mdbmaps, "MusicVF", "be1f16c73817b80c1dcd8a40809bb7e9", "3199669")   ### [High Hatters]
    addDiscogs(mdbmaps, "MusicVF", "536cdeb9f15951002e0b2bd4e784d1e3", "2524169")   ### [Irving Aaronson & His Commanders]
    addDiscogs(mdbmaps, "MusicVF", "1b95f8d64ddaf4ef32b4ee53c6d80c73", "1753513")   ### [Art Gillham]
    addDiscogs(mdbmaps, "MusicVF", "039a4ed21c34d2b6691ee89a55aab12d", "3219276")   ### [Jane Green]
    addDiscogs(mdbmaps, "MusicVF", "fd65e5da58193dfbef86bbc2d6309e14", "2979161")   ### [Marcia Freer]
    addDiscogs(mdbmaps, "MusicVF", "e83a5984d733718d4b9f23323aa8022a", "1157658")   ### [Scrappy Lambert]
    addDiscogs(mdbmaps, "MusicVF", "ccd4999de700cf53032bb55bdb7e6e68", "2370466")   ### [The Clicquot Club Eskimos]
    addDiscogs(mdbmaps, "MusicVF", "a9fcddd5a046d89fca079eadafd0538c", "1534535")   ### [The Regent Club Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "59a8f074247f3df46dd7429e4e12cd3a", "4442232")   ### [The Beverly Hill Billies]
    addDiscogs(mdbmaps, "MusicVF", "7d3f3000f025191d1746f1053b8c878b", "3370337")   ### [Arthur Gibbs & His Gang]
    addDiscogs(mdbmaps, "MusicVF", "cf2d49073c56b8d5edc714d54cc69618", "4643811")   ### [The Associated Glee Clubs of America]

    addDiscogs(mdbmaps, "MusicVF", "637a650477c0567a4b6bca7ad49333cc", "826064")   ### [Vaughn De Leath]
    addDiscogs(mdbmaps, "MusicVF", "0cde569a9f694c8814c68dbb02cf71d1", "708256")   ### [The California Ramblers]
    addDiscogs(mdbmaps, "MusicVF", "1ae7eba65460ecfed11cb1af7c069eb3", "1313374")   ### [Ford & Glenn]
    addDiscogs(mdbmaps, "MusicVF", "3b1c44282812439bd5e48c39fa0f214b", "2903429")   ### [Edwin Dale]
    addDiscogs(mdbmaps, "MusicVF", "bfa0c0324e7acc417df47e2bda8bf364", "1533274")   ### [The Brox Sisters]
    addDiscogs(mdbmaps, "MusicVF", "0f7952d54f822eb899ba72ccd8419e18", "1261670")   ### [Billy Jones]
    addDiscogs(mdbmaps, "MusicVF", "8141e4ed4b735c19d9b566ea802d9fd9", "1048195")   ### [George Olsen and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "74a0461718b14d24860a657cb45b837f", "1538599")   ### [Weber & Fields]

    addDiscogs(mdbmaps, "MusicVF", "7a0ff86e52a88b8c87fd86505d36417d", "866093")   ### [Harry Fox]
    addDiscogs(mdbmaps, "MusicVF", "06fb4b0901135a954a5f40abeecf0efa", "1700332")   ### [Julian Rose]
    addDiscogs(mdbmaps, "MusicVF", "c2658529dbd5c96ca5d9bca9ae5606ba", "2829163")   ### [Belle Baker]
    addDiscogs(mdbmaps, "MusicVF", "9a20e6e56cd6c9e4a2c21ec0ef761677", "3374285")   ### [The Farber Sisters]
    addDiscogs(mdbmaps, "MusicVF", "e38faa8c21fd102aca30e25eec8fbeff", "1835752")   ### [Percy Hemus]
    addDiscogs(mdbmaps, "MusicVF", "0c4556e72b5f9bb1f9eb4a89d33945ee", "2898914")   ### [Hipolito Lazaro]
    addDiscogs(mdbmaps, "MusicVF", "776940af81a4e2e2ecb5bce9f6aa2d9d", "1768625")   ### [The Knickerbocker Quartet]
    addDiscogs(mdbmaps, "MusicVF", "5325038a2866515f8e86e141d6af5422", "5416455")   ### [Reginald McAll]
    addDiscogs(mdbmaps, "MusicVF", "4ada244fd430d4585f6b365ad44dd3dc", "1272386")   ### [Elsie Janis]
    addDiscogs(mdbmaps, "MusicVF", "1947751d3ff5603dd35d78a85a5e0bf3", "2708821")   ### [Kathleen Parlow]
    addDiscogs(mdbmaps, "MusicVF", "d20f2f573dc7c6c9111eb9b572b42d1c", "740344")   ### [Ernest Ball]
    addDiscogs(mdbmaps, "MusicVF", "a77d02cd9ee77fcb218744773d1e0b62", "5018453")   ### [Frank Tinney]
    addDiscogs(mdbmaps, "MusicVF", "5d2d580dd960072f06ee44bb08f1b283", "2813306")   ### [The Venetian Trio]
    addDiscogs(mdbmaps, "MusicVF", "351858939ae70d17e2bcc028aa365f8a", "2827022")   ### [Charles Kellogg]
    addDiscogs(mdbmaps, "MusicVF", "67daff7cb8f6771546b907c9cdf17960", "2667409")   ### [The Victor Mixed Chorus]
    addDiscogs(mdbmaps, "MusicVF", "44bdc5fa077e6dba2d29cadd396f19a6", "3349119")   ### [Owen McCormack]
    addDiscogs(mdbmaps, "MusicVF", "ce2af862047fe0b5117f96511b84b27b", "3529099")   ### [Virginia Asher]
    addDiscogs(mdbmaps, "MusicVF", "2eab96f5dbf0362ce569d26e04dc8fa6", "2314053")   ### [the Orpheus Quartet]
    addDiscogs(mdbmaps, "MusicVF", "3354a2245997515a40e67c1787273375", "2093757")   ### [Happy Six]
    addDiscogs(mdbmaps, "MusicVF", "b8888888649cfa8a577b012afc1fb1f6", "3160570")   ### [George Meader]
    addDiscogs(mdbmaps, "MusicVF", "bb4060e0a54ced1e5ca012a109ee716b", "1340791")   ### [the Victor Light Opera Co.]
    addDiscogs(mdbmaps, "MusicVF", "437f31cb2645204f6feed226d01b5fc0", "2567602")   ### [William Robyn]
    addDiscogs(mdbmaps, "MusicVF", "c7b4791e72f894810860eaa98d155263", "3805757")   ### [Mary Carson]
    addDiscogs(mdbmaps, "MusicVF", "ea8df109f91a975dbccbffe463f58695", "1534545")   ### [the Sterling Trio]
    addDiscogs(mdbmaps, "MusicVF", "34513adc4b08819e420b0e439e6e1ea3", "1700445")   ### [Helen Louise]
    addDiscogs(mdbmaps, "MusicVF", "15e33f0cb8ed4f76362f517d0b0b74ce", "2637024")   ### [James Harrod]
    addDiscogs(mdbmaps, "MusicVF", "7f56ae62f696a63c2f98aa6ce3c13ed5", "3681942")   ### [Frank Coombs]
    addDiscogs(mdbmaps, "MusicVF", "f1610c7f9a40df7ed45a84483b92df86", "1775471")   ### [Maurice Burkhardt]
    addDiscogs(mdbmaps, "MusicVF", "16abf00a321c0f93e4ed13e754735196", "1835757")   ### [The Neapolitan Trio]
    addDiscogs(mdbmaps, "MusicVF", "9bab68feb111f335f38311810b1baf46", "3836929")   ### [Gene Rodemich & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "0b26d0b7c70fdd67a011721cab0aff52", "2954135")   ### [The Kentucky Serenaders]
    addDiscogs(mdbmaps, "MusicVF", "5b757ced53ae65b7c226f8b84d7b8926", "2093852")   ### [Guido Deiro]
    addDiscogs(mdbmaps, "MusicVF", "e7fd8511eeda2ec20f82cfa67908c984", "925226")   ### [The Victor Concert Orchestra]

    addDiscogs(mdbmaps, "MusicVF", "f06c3716e8ae24f46ef55d2e6a961619", "2838233")   ### [Anna Wheaton]
    addDiscogs(mdbmaps, "MusicVF", "c97096206968151f344102098844e1b0", "4720328")   ### [Horace Wright]
    addDiscogs(mdbmaps, "MusicVF", "23eab09f00750b8f07697cd62ebdca11", "1233624")   ### [Lucy Gates]
    addDiscogs(mdbmaps, "MusicVF", "07ba21344cee2e24b5ecd8f3037615de", "5414420")   ### [Monroe Jockers]
    addDiscogs(mdbmaps, "MusicVF", "081ff0ea0dcea3f5fb20361ff59f42d6", "2581194")   ### [The Columbia Saxophone Sextette]
    addDiscogs(mdbmaps, "MusicVF", "e0846a5616b5118f48f91c9690bc1dd7", "1582538")   ### [Lillian Rosedale]
    addDiscogs(mdbmaps, "MusicVF", "27bc0cd46b7644dc741f18fc3b921665", "1700076")   ### [the Shannon Four]
    addDiscogs(mdbmaps, "MusicVF", "abf3f05d50e0177fa2e44c3f4fa0942e", "3785106")   ### [Gene Greene]
    addDiscogs(mdbmaps, "MusicVF", "e6112bbf0b4e22e17f368c8e9b397860", "2383029")   ### [William J. Halley]
    addDiscogs(mdbmaps, "MusicVF", "3e97fa98c40deb08a0bda3ca1dfcb8be", "6843673")   ### [Joseph Cawthorn]
    addDiscogs(mdbmaps, "MusicVF", "e61f1c29df82b036f88e56e0b0838584", "6262062")   ### [Joe Hayman]
    addDiscogs(mdbmaps, "MusicVF", "cd5cb502bc05b3c5099979df5d5ef918", "3333738")   ### [The Paul Biese Trio]
    addDiscogs(mdbmaps, "MusicVF", "dcc7cf3c7be6954fe83dd4f8519b4d67", "2918251")   ### [George Wilton Ballard]
    addDiscogs(mdbmaps, "MusicVF", "e921f75d1a98bdea5f4c01b1a44cc4b8", "2581047")   ### [George O'Connor]
    addDiscogs(mdbmaps, "MusicVF", "6d608dda152f2234a10874a7222fa061", "3621101")   ### [Elliot Shaw]
    addDiscogs(mdbmaps, "MusicVF", "2711769219ff792132a716dbb70654e8", "2309206")   ### [Dolly Connolly]
    addDiscogs(mdbmaps, "MusicVF", "94603bd68729be64bb5fbbc32d2e94fc", "4052246")   ### [Anna Chandler]
    addDiscogs(mdbmaps, "MusicVF", "4501e2cb614920d78a3402b9f885acc5", "1638060")   ### [George MacFarlane]
    addDiscogs(mdbmaps, "MusicVF", "412c8688cbfe290bd5c291d360d5ab41", "655644")   ### [Chauncey Olcott]
    addDiscogs(mdbmaps, "MusicVF", "71f6eb4ef827b0185c4d6f6bb739c47a", "3388558")   ### [Marguerite Farrell]
    addDiscogs(mdbmaps, "MusicVF", "91c249d08b5f4504141ed26e91e93ff2", "6028413")   ### [Will Robbins]
    addDiscogs(mdbmaps, "MusicVF", "3325f5e5535bbc4a6fc5f61a4aff429d", "3992295")   ### [Andrea Sarto]
    addDiscogs(mdbmaps, "MusicVF", "2879f8d7080effe00067a2d4fb2b0008", "2637033")   ### [Taylor Trio]
    addDiscogs(mdbmaps, "MusicVF", "6804ed1c99d9a717000db6c7ad970bbd", "1534556")   ### [The Avon Comedy Four]
    addDiscogs(mdbmaps, "MusicVF", "dea0214ca7949a4aa1eb6f66c47923dc", "2174340")   ### [Charles Hart]
    addDiscogs(mdbmaps, "MusicVF", "6c7c6aabcb657383b24b5929b7a2819d", "1534545")   ### [The Sterling Trio]
    addDiscogs(mdbmaps, "MusicVF", "b15bf8ba743e67310ecdb628f024d1b5", "1534360")   ### [John Steel]
    addDiscogs(mdbmaps, "MusicVF", "ac24cb5f7ea5ca9ebca46d55863e1d40", "1775463")   ### [The Heidelberg Quintet]
    addDiscogs(mdbmaps, "MusicVF", "4b2640d5a6fee565f16dad701a6d0d0d", "1700076")   ### [The Shannon Four]
    addDiscogs(mdbmaps, "MusicVF", "1aef553674f63531f0676521eb0affc0", "2577868")   ### [The Columbia Stellar Quartet]
    addDiscogs(mdbmaps, "MusicVF", "87b2ed88b0d4233172cfb30042a6b602", "6069200")   ### [Ada Jones & Walter Van Brunt]
    addDiscogs(mdbmaps, "MusicVF", "156cef596c313b8bf47c7e2be7597e86", "3247552")   ### [Marguerite Dunlap]
    addDiscogs(mdbmaps, "MusicVF", "55c203544f7f87cb2ce4e5cd8891f7bb", "1313719")   ### [the Peerless Quartet]
    addDiscogs(mdbmaps, "MusicVF", "8bbc5188169d6feabd282c2801b70d9a", "2314053")   ### [The Orpheus Quartet]
    addDiscogs(mdbmaps, "MusicVF", "124194725e14cb8c2ee774e14c5b7a39", "5524645")   ### [Elizabeth Brice]
    addDiscogs(mdbmaps, "MusicVF", "4fc68f4e1c4b66b614e5879f76dfc3e1", "413940")   ### [Charles King]
    addDiscogs(mdbmaps, "MusicVF", "2d1d7c9d09fa004dff58e99be8383626", "1334935")   ### [The All-Star Trio]
    addDiscogs(mdbmaps, "MusicVF", "687f2f78766b4afb3db125e47991f64f", "2570607")   ### [John Barnes Wells]
    addDiscogs(mdbmaps, "MusicVF", "0b6cb5335a14477392a57d8332866d62", "6104768")   ### [Florodora Girls]
    addDiscogs(mdbmaps, "MusicVF", "1d98f1125ac4b7fc3d9cb6e8eddb5abf", "307346")   ### [The Fisk University Jubilee Quartet]
    addDiscogs(mdbmaps, "MusicVF", "86744afb227c06b3ebe5c211f2d051e2", "3068289")   ### [Kitty Cheatham]
    addDiscogs(mdbmaps, "MusicVF", "800742427bca8a3bc53b1e90813a8d7f", "4196180")   ### [Paul Southe]
    addDiscogs(mdbmaps, "MusicVF", "46cb836b572875c6f1f897e2af03d75e", "2705492")   ### [Gina Viafora]
    addDiscogs(mdbmaps, "MusicVF", "8f5104cd2c3c40e2f3e360b5bbdf8727", "4080215")   ### [Harold Jarvis]
    addDiscogs(mdbmaps, "MusicVF", "7f23916f0099bc98609cd31175c85048", "3068292")   ### [May Irwin]
    addDiscogs(mdbmaps, "MusicVF", "bf7387ba2045dda4d45bfe6a09cc44ab", "6701051")   ### [Helene Noldi]
    addDiscogs(mdbmaps, "MusicVF", "6b893f72ffbcd4e203ec16862a9e93ac", "562820")   ### [James McCool]
    addDiscogs(mdbmaps, "MusicVF", "0ba7e39c46e0c125312e1acb245c4fb6", "1517508")   ### [DeWolf Hopper]
    addDiscogs(mdbmaps, "MusicVF", "6b94b70722ce20ac987d0039deac7c36", "495702")   ### [The Columbia Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "16aa31b2a6d648944f0d8bdb1511f4f9", "3445561")   ### [William H. Thompson]
    addDiscogs(mdbmaps, "MusicVF", "983d919e863d0234fe414427f0051cdf", "3776429")   ### [William Redmond]
    addDiscogs(mdbmaps, "MusicVF", "89ab8ecd02d67b4f5033616146a3016e", "1337794")   ### [the American Quartet]

    addDiscogs(mdbmaps, "MusicVF", "044f611b92098846fb92fbc73fada281", "847619")   ### [George J. Gaskin]
    addDiscogs(mdbmaps, "MusicVF", "14f2e63b3a16d0d8b3befafee0abc84a", "1534417")   ### [Alan Turner]
    addDiscogs(mdbmaps, "MusicVF", "79fcadfb9c285562c09025ba9db0e58c", "3253082")   ### [Arthur Clough]
    addAllMusic(mdbmaps, "MusicVF", "3249ec1b23042ab6b80f9a8822101189", "mn0000318081")   ### [Jere Mahoney]
    addDiscogs(mdbmaps, "MusicVF", "d9ff09bf53dbb7e72dc5ea0b6b9d4c70", "4887325")   ### [Silas Leachman]
    addDiscogs(mdbmaps, "MusicVF", "2d777270139f2209e47b6f9b14cebcd7", "1202354")   ### [Jules Levy]
    addDiscogs(mdbmaps, "MusicVF", "37c6c9812cf67fcbb68f932897435408", "4812494")   ### [Grace Spencer]
    addDiscogs(mdbmaps, "MusicVF", "d78a0aa58ab85928515788aa76459b1e", "1487761")   ### [Nat Wills]
    addDiscogs(mdbmaps, "MusicVF", "1e0178d363cd693f6b22baf3b403fed5", "3570381")   ### [Will Denny

    addDiscogs(mdbmaps, "MusicVF", "f2ba906e2c895afa7cc274f84a8a64f5", "3854419")   ### [S. H. Dudley]
    addDiscogs(mdbmaps, "MusicVF", "2659712a0675f0121db7f4dd1dea1495", "3113189")   ### [John Bieling]
    addDiscogs(mdbmaps, "MusicVF", "e85d85cf298584efbbd5b83f0db7ee82", "4557912")   ### [Richard Jose]
    addDiscogs(mdbmaps, "MusicVF", "8dee48bab325476580ec7f96e94f286f", "8005580")   ### [Mina Hickman]
    addDiscogs(mdbmaps, "MusicVF", "77d495aa4446778d87d0606a28ecd595", "847633")   ### [Dan Quinn]
    addDiscogs(mdbmaps, "MusicVF", "b71835d352d8570c70b3e5faa0a0b4df", "2905804")   ### [Harry Tally]

    addDiscogs(mdbmaps, "MusicVF", "40da9295fedc06007599b5aed7ed23c4", "915906")   ### [Swing and Sway with Sammy Kaye]
    addDiscogs(mdbmaps, "MusicVF", "424a198fbe9cdb4b9824e0dab2d02b8d", "2577546")   ### [Frank Stanley]

    addDiscogs(mdbmaps, "MusicVF", "4a0e76a3f7d60126def2ad9eb4fc797d", "833291")   ### [The Green Brothers Novelty Band]
    addDiscogs(mdbmaps, "MusicVF", "7ae4466bc08021ac9bb000a08b4ec534", "1725437")   ### [the Watts 103rd Street Rhythm Band]
    addDiscogs(mdbmaps, "MusicVF", "10401ed282fb35b566c27c2ff3ac5325", "1435727")   ### [The Eli Young Band]
    addDiscogs(mdbmaps, "MusicVF", "78897201543b498aacf1c88ea26530eb", "518900")   ### [The Debroy Somers Band]
    addDiscogs(mdbmaps, "MusicVF", "62c5ee996268ddf0959c4900f7a3c565", "3678154")   ### [Metronome All Star Band]
    addDiscogs(mdbmaps, "MusicVF", "8a8d9f24b674503156f61945d2bfeb17", "1319719")   ### [The New York Military Band]
    addDiscogs(mdbmaps, "MusicVF", "b6f449a458feffa45d1733690050b813", "6620157")   ### [U.S. Naval Academy Band]
    addDiscogs(mdbmaps, "MusicVF", "89ee96133f18678875fb588a1804534d", "4846155")   ### [The Vernon Country Club Band]
    addDiscogs(mdbmaps, "MusicVF", "b214de55c5016792e857b4e1cea9d547", "631489")   ### [Ferko String Band]
    addDiscogs(mdbmaps, "MusicVF", "59a1f622916a78cb86f486057fe398a1", "341400")   ### [Jimmy Dorsey and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5e91727a080c9c2d39e24fda912f1e97", "750213")   ### [Isham Jones & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1162e06ab796e03f2569674315c517cb", "323993")   ### [Freddy Martin and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d16f4b2bc7dc28c9ca1a391910d269b", "342802")   ### [Leo Reisman and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "90eaa040177c2811146f9bd9596f0e9a", "348903")   ### [Hal Kemp and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "364a3b80de20af42c4d91b29ea9fb24f", "1277416")   ### [Glen Gray and the Casa Loma Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "663d60521dc70e807c1597f6b4699085", "3453989")   ### [Nat Shilkret & the Victor Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "afce6faff59a28d344434f9a1b0695ca", "349510")   ### [Ray Noble and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "df05c6f55851e6578477ca388660dc56", "1253756")   ### [Jan Garber and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "02ab9faaa95a329ed8dd672e5904c0a5", "1424900")   ### [Ted Weems and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d7bcbfee811d2f50c06f801c79dcf98", "341410")   ### [Larry Clinton and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "dc4a1e1f2b07b534353aa2d67fc18a17", "1089954")   ### [Ozzie Nelson and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1b945945f43c0420f9d20ccd59c83d88", "1369041")   ### [Vincent Lopez & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "68876fa095048150249bafa168d67f69", "342361")   ### [Teddy Wilson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7143b00323c00bd261ad17f645d7ab64", "2426367")   ### [The Benson Orchestra of Chicago]
    addDiscogs(mdbmaps, "MusicVF", "2241f0ef784c8ac08ea7a47a48255f35", "339917")   ### [Roger Wolfe Kahn & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a8e52909def4852558fe2437e06f43b1", "2610502")   ### [Arden-Ohman Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3aa0308fe7e68b05fd8934e39e3cd390", "349522")   ### [Orrin Tucker and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7777c383c810e40a3eadfce4d1b91a6c", "708255")   ### [Gus Arnheim & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9048dc7c26905c398e086c8c67d4b508", "3197262")   ### [Paul Specht & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "21c1f7f1d5e459accd5b35621978b191", "1916128")   ### [Xavier Cugat and His Waldorf-Astoria Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "93976a04e70e7d73ac764770c54072b3", "2509255")   ### [Ted Fio Rito & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "85453843117a5eff96a9f102eff93666", "708247")   ### [Ray Miller & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "13ba349d06b706dd6f11f1f26bc1fac8", "2053208")   ### [Benny Krueger and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1aa9688887d53658cc41a1db9518301c", "1514912")   ### [Victor Young & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5d848b88a6e1b36b110abf3b61014b41", "341414")   ### [Tony Pastor and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7a27db8bf0450bb84b58eb8ec5de9735", "338450")   ### [Jean Goldkette & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "07e3044e4b2eebfd6a89411bddf15019", "376962")   ### [Paul Weston and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "03c254c24017c8cb8aaf73ea2cf5be8b", "1909656")   ### [Dick Robertson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9fc8fbd76fd9d408157fa351c801f62e", "393014")   ### [Red Norvo & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "71fc57fe60a582a0ca3973f88711ddf1", "349507")   ### [Frankie Carle and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "86841c06a640a2717210bf54841c7a00", "3778766")   ### [Richard Himber & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d3eb01fde0fb4a6de0df0228cc9184be", "1089948")   ### [Alvino Rey and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "2fd2396c6f988059397b0d6c36096fb5", "1270963")   ### [Sam Donahue and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "91f0224dc363f481debfdb3190f7ec21", "340734")   ### [Erskine Hawkins and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c2d52ce3f15eb769eaab0be4a12e9486", "2705133")   ### [Henry King and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "21ab9c4037c261145865b119786c877e", "778418")   ### [Johnny Green & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "8739692acbb74242899157014478c777", "3349535")   ### [The Coon-Sanders Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "06126254a90f73d2a3ad4b8adc4c332a", "317889")   ### [Frankie Trumbauer & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7c4f29dd44fb3859a09fe764e44aa2c9", "2765469")   ### [Al Donahue and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "4b7739a3b430a7ebafdcff9a908fc537", "540268")   ### [Jacques Renard and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "f00e5a681822186c70c9c022c85cc2cb", "mn0001664426")   ### [Victor Herbert & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9fa16a83d97e89319c28125903ce9a55", "1922766")   ### [Johnny Long and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "079674fb465a2b67123724e55b3bb32c", "357493")   ### [Ben Pollack & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1b3dec386442752641ec283ff3f684cd", "2584575")   ### [Bert Lown and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "647c7ba1a93ac1ea8bc71c5225085e99", "843797")   ### [Freddie Slack and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5fde257835593d3b4a6e17c07e946db2", "349512")   ### [Henry Busse & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e805bb06ad2e064554515ad6f44b3a9b", "2990501")   ### [Johnny Hamp & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1a49b43d30bb4dbfe0440149cd07e8d8", "3463207")   ### [Smith Ballew & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "efa149b82b5e9d0b8f0e16588436acf6", "349525")   ### [Will Bradley and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1a1a29a71d2a9768f13fee5ec9397f13", "8051598")   ### [Earl Fuller's Novelty Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d22e5ffe8e7ba9346455332509216ba5", "3194253")   ### [Jack Denny & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e7e918c211119950c538271fe7c140c2", "708258")   ### [Fred Rich & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ead3f3d7aa3c0975da0db2cb703be35c", "429644")   ### [Jimmie Grier & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b4d9d721d37807d83145744914e57e9b", "7902851")   ### [Tom Gerun & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "53202b03bb71bd095ddab26432935d16", "349492")   ### [Clyde McCoy & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "49d5ea8f8e052f6b95fd68036b924e97", "2499512")   ### [Harry Reser's Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ba2374b01c2c8a238f3da50946e6e04c", "446654")   ### [Don Redman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e1fe7d8022f0c55e7199d3ba780efa12", "1909635")   ### [Ruby Newman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "46dea3fde8edeae6b2675d7e4736b630", "1527229")   ### [Phil Spitalny & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "18d6881a09e9ebea3713ce08534df60c", "1386796")   ### [Jan Savitt and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "8d7ee1984988bf4cc92ffdc3372b7106", "349501")   ### [Frankie Masters and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "399579d73e82eea2403c0e968aac3e74", "4317302")   ### [Buddy Rogers & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c7dc152e3675b6c89a854edaf86e8001", "3330524")   ### [Ted Black & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9fba994839e0e20edf5f74a478dea5ea", "3113509")   ### [Art Landry & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "09662762e6b1222572e121db07d3d397", "5189304")   ### [Lud Gluskin & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "fd6f7dbd728e896265fa836c2fb9c206", "432780")   ### [Henri René and His Musette Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d67f1d65c8d4b80acfb2be380db375a3", "830957")   ### [Joe Venuti & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "4f26c243c4fb5dd9974009f128fabf8f", "2469413")   ### [Yerkes Jazarimba Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d6a525fc93bafc6654c3b633e5ba917a", "917501")   ### [The Waldorf-Astoria Dance Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7034518e9f13f25730f0220d0990fd1e", "1089955")   ### [Bob Chester and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "71753da0e75b7620e5a5d253d77e0e3b", "4699749")   ### [Paul Pendarvis & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "991d13db85dc8c149a3e262395f987f6", "2884360")   ### [Joe Green Novelty Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c0dbd0a98ee1f2dbc5f821e4c7069ad0", "4370332")   ### [Al Trace and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9cebc11d4538d5b8e5841a30001ab0ea", "341397")   ### [Raymond Scott and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "cbc2144c31d44c14b213c41095bd5730", "419475")   ### [Jack Jackson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c81486bf147b732e66ae95374e6c43cf", "1807615")   ### [The Hudson-DeLange Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "44dc319f4a15889d4a227e2697f6d4ce", "7342653")   ### [Reginald Foresythe & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5464a3bac47d402ec094fdf37cb01326", "1612846")   ### [Art Jarrett and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9053fe6b892fde478309af2e1316247b", "2773668")   ### [Mal Hallett & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "504b4f75a95d06335a1e3cb5dac95315", "3476694")   ### [Nat Brandwynne & His Stork Club Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "18cebbf9b4cfb8bda1ca957e50a2c992", "349502")   ### [Hal McIntyre and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "16a6048c3ef87052e05da99ab5dad8f2", "3087019")   ### [Anson Weeks & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "0f3c6b721541e1c741ff87fdc85d62a3", "1201611")   ### [Henry Allen & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "16f75b62f43e6501109d499b164706b3", "5675216")   ### [Peter van Steeden & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7bd2b23381955fe428d95261ab6e9e1e", "1201977")   ### [The Victor Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "0fa3a0a5620ad16a47643b256005a830", "1541489")   ### [The International Novelty Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f695a12ad76d87fc9f224ced8bf2cf0f", "3978868")   ### [Seger Ellis & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "15d524fcf25c0fe5e277ea476d4283d5", "3247982")   ### [Meyer Davis & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1b5dca776f0ef2fd1205107cd9556ef7", "368180")   ### [Billy Butterfield and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "493e9274729503ebd82138eb04ee09a5", "1239010")   ### [Benny Strong and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "43cd3c56c657b29dc1d2adbc2d8e0f91", "1089157")   ### [Pee Wee Hunt and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "07649958fd7cd4f1035de4ebfd157771", "368184")   ### [Benny Carter and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6386ec97b809d7afa589c33227c0dd95", "869942")   ### [Francis Craig and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1e6336eeedc3f9436cb166b74b2db142", "349504")   ### [Elliot Lawrence and His Orchestra
    addDiscogs(mdbmaps, "MusicVF", "9ca5dd24ed8df06840865fc9021a0933", "341971")   ### [Ray Martin Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "872966b95880bc6fb65eb39f4828b111", "3325840")   ### [Carmen Cavallaro and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "975e3ed288a38b89797172a17ac201a2", "200966")   ### [Ted Heath Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "eb59c6b4e73764bca0d99e00acafb757", "271871")   ### [Frank Chacksfield and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "66ebcf967b25b1a72cfb576fefca80a1", "1403626")   ### [Will Osborne & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "011453b5586099bc69971661d18d8b0f", "3211715")   ### [Willie Kelly and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5d450e2ef51fcdcd50e47aae756e0012", "2571353")   ### [Eddie Elkins & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "aaf6f171b47aa462dd6fdb8fe6531a4f", "3101048")   ### [W.C. Handy's Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "db7fac12df5fe8f9afdabf8bd48a0ef2", "1733554")   ### [Hilo Hawaiian Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "33208f2c0f69046675d175be287066f4", "925226")   ### [The Victor Symphony Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "df089b055fbe08f588b219e1e2e3032b", "5503818")   ### [Roy Ingraham & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "4d9fd245aedc6712a02f9f8bec978fd0", "886413")   ### [Johnny Johnson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a2d457fb1677103212bdc008fd531cfe", "6175352")   ### [Frank Black & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "763ea2bd44882528710c405e3a70c80b", "7125528")   ### [Jack Shilkret & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7855f2793904d80de446edcc71252207", "3174998")   ### [Ernie Golden & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "10f0ac04981d8f7b1ebfe46dfbd18ad2", "3836928")   ### [Harry Archer & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "44b045bcf9e9d424f36716838e514a51", "3444587")   ### [Ross Gorman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "59f5fc538500a11839f9b6f539bae568", "2884387")   ### [The Jaudas' Society Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9a92789023d11881fd968cd8a73f090b", "1667817")   ### [McKee's Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f772619fd4153349fb47f9ccda97a310", "3444582")   ### [Don Voorhees & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "778429d8a9821e88445969e3dc18b89a", "4299574")   ### [Phil Ohman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b44f3520e071957b8503ecced88767a8", "2891082")   ### [Paul Tremaine & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3cdafda9a575a60eef2885004d65bc06", "312154")   ### [Stanley Black and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "740d289bb972ffc20b965a5317cef883", "1534133")   ### [Freddie Fisher & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5a056efffa4ce2f92cf8b0550ec393fb", "1981924")   ### [Johnny Messner & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "0ef1e7b623c27d1f250229532484bb9d", "2064042")   ### [Bobby Byrne and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "fc34aae20cab9df2ae938b00a8d0efbf", "341409")   ### [Teddy Powell and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a237529600f01e769b1d9cea9cc654f7", "341406")   ### [Claude Thornhill and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e0f7b00f6c4950bde5da78645f0b1569", "367945")   ### [Jack Hylton & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "973a6531397e89bb78d2de6bcb702bc7", "3973861")   ### [Hal Goodman and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "76db57e97f4f99db7db7d2625e634123", "340731")   ### [Jack Teagarden & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7523b97dfeb2ac3209e47d93180622dd", "5688823")   ### [Joe Moss & His Society Dance Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "8c4a58fa2eeffdeee25d3773bbb264a6", "2873355")   ### [Chick Bullock & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "18b04a7c320c287bfd63d7d1c48c8d9e", "5321173")   ### [Primo Scala's Banjo and Accordion Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7cb82c5a2e57a82389be69200a4f44f0", "395324")   ### [Hugo Winterhalter's Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ab86beec7d0969d43d784a6cedc8405d", "1968386")   ### [Eric Jupp and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "bdc6a1f249f43e4555ef2a4150f37611", "6197004")   ### [Jerry Johnson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "eb56f846b0ab36eb3d63221181e29308", "3108182")   ### [Jolly Coburn & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6d9be0f6509804108684fdaf84017ef4", "5100070")   ### [Ferde Grofé & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "310edf853dbd21626e09ecb9832a7b8c", "1658630")   ### [Arthur Lally & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9859472117697df7d6862943e443bc2e", "1514952")   ### [Slatz Randall & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6eca07976f58c020d2e54bafa2961e58", "1668971")   ### [The Riley-Farley Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ab12a4348377acb624df0dbbf4906cc8", "1020569")   ### [The Roy Fox Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "4ae5cd21c231b532ab007a0091e6bc50", "2236664")   ### [Art Kahn & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "34302d19c358666521a535411af530bd", "1206210")   ### [The RKO Studio Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c83cb0386f904d73f9755c0df8d9dc86", "1598356")   ### [Lew Stone & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "45d72ae61b67005d696a79a88fce60e1", "5283673")   ### [Charles Wolcott and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5a216494307b7ec3f2ace9e110a93d9b", "4693278")   ### [Sonny Boy Williams and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "bb84eb39cf49cd24a54065551d655304", "2130269")   ### [Ceelle Burke and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "465897d2140b35204079a686166ba7aa", "3108653")   ### [Allen Miller and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d4c22b725181e370189f32dd098bc8ff", "950513")   ### [Clyde Lucas and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f225c614925408fc8e100d356a5f54de", "2685342")   ### [Terry Shand and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "10aaf6361bd752ce70c5407eec78b3d4", "3252185")   ### [Andy Iona & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "2e21c0926346a9be0990e62880deb9ea", "1089946")   ### [Skinnay Ennis & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "964b316d1ce04bb3ab186d6349ec1423", "1113839")   ### [Emery Deutsch & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d8eda72f4e03acbd0db2805843bf73c", "2534757")   ### [Louis Levy & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "cfc2b7abcc61165ce9af15dcdca80001", "5115585")   ### [Orville Knapp & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b555c01db44d2054f8b5291707d66f9a", "2076665")   ### [Willie Bryant & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "60107fd180a7085a8a0b662fc8d93ab6", "8599059")   ### [Frank Westphal & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a7f8728d0464ba2cdb280d60cf73e757", "3424996")   ### [Nathan Glantz & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d068924c00e547bbca5bfab17298b21", "1699772")   ### [Zez Confrey & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "39a1a3ec713ed1a4a4b30a3cca2617bf", "388185")   ### [The Philharmonic Orchestra of New York]
    addDiscogs(mdbmaps, "MusicVF", "c87626c18a8ef0fc98373b827f1a39c2", "834227")   ### [The Cincinnati Symphony Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7a5b2a00960a6fcc7906e44241a40c6f", "383340")   ### [Leroy Holmes and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6814c6d3e5509475cc7d82f96106c966", "1717539")   ### [Morris Stoloff and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "da127666e61c0670709952d3b5ba6f95", "1911338")   ### [Jerry Wald and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "60fe62c5faa8e16170d8349a687db147", "2048947")   ### [Herbie Fields and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "86e56ff8c3825915d78bcc4dfbd36cd5", "700576")   ### [Ray Bloch and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3bffe9403c94e4303d4c50b720187264", "951382")   ### [Al Sack and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6a5ef6dd4e7ccb755866d76edc3e9d11", "3476192")   ### [Gray Rains and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "da2331602a1eeb3b191638bce3ebd15c", "843805")   ### [Johnny Hodges and His Orchestra]

    addDiscogs(mdbmaps, "MusicVF", "b70f953700e5d0088335c84a223cce94", "6269664")   ### [Billy Jones & Ernest Hare]
    addDiscogs(mdbmaps, "MusicVF", "5de22f3396df806061d6fb1178d55495", "1541488")   ### [Carl Fenton and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "68876fa095048150249bafa168d67f69", "342361")   ### [Teddy Wilson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1b945945f43c0420f9d20ccd59c83d88", "1369041")   ### [Vincent Lopez & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "70020d3d9c374709e82a2a49e304f074", "1089971")   ### [Abe Lyman and His Californians]
    addDiscogs(mdbmaps, "MusicVF", "73337b116f577d25dceb545c9819c436", "1340791")   ### [The Victor Light Opera Co.]
    addDiscogs(mdbmaps, "MusicVF", "e02b9b1454c7e363bad165d512bc20df", "170357")   ### [The Carpenters]
    addDiscogs(mdbmaps, "MusicVF", "46abd922d7c4fdcfe654a39f6920e088", "1534354")   ### [Elsie Baker]
    addDiscogs(mdbmaps, "MusicVF", "b5cffbaa3ef63048522c90adb10af06a", "621943")   ### [Bob Roberts]
    addDiscogs(mdbmaps, "MusicVF", "fe178b6d86ceada3c50c26af07d55185", "1534185")   ### [Van & Schenck]
    addDiscogs(mdbmaps, "MusicVF", "d70e60bc1664eef96d37b0d8ee9369f3", "3704749")   ### [Earl Burtnett & His Los Angeles Biltmore Hotel Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6a3aa27e6c3914c63ff83ac4b9231982", "317885")   ### [Red Nichols & His Five Pennies]
    addDiscogs(mdbmaps, "MusicVF", "7143b00323c00bd261ad17f645d7ab64", "2426367")   ### [The Benson Orchestra of Chicago]
    addDiscogs(mdbmaps, "MusicVF", "31e1781613731885e10f9204b566dfc2", "732768")   ### [Vess Ossman]
    addDiscogs(mdbmaps, "MusicVF", "cc654ba68ea4a88eefbe969cbc9438b6", "424111")   ### [the Haydn Quartet]
    addDiscogs(mdbmaps, "MusicVF", "58c56f36fb19ccbb45f390f0ab01a01a", "307205")   ### [Boswell Sisters]
    addDiscogs(mdbmaps, "MusicVF", "2241f0ef784c8ac08ea7a47a48255f35", "339917")   ### [Roger Wolfe Kahn & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "336d8488cbe2d09af93b92728a1bcd2d", "3251382")   ### [Corrine Morgan]
    addDiscogs(mdbmaps, "MusicVF", "a8e52909def4852558fe2437e06f43b1", "2610502")   ### [Arden-Ohman Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3557232a1330590e9e500f6af7ce566c", "1551026")   ### [Don Bestor & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3aa0308fe7e68b05fd8934e39e3cd390", "349522")   ### [Orrin Tucker and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a98b66aaebcf635f7a8eb6c4c07bfd30", "1667953")   ### [The Glee Cast]
    addDiscogs(mdbmaps, "MusicVF", "330315f48c9ba8187fe75e7c1a8e2f6b", "82735")   ### [The Spinners [US band]]
    addDiscogs(mdbmaps, "MusicVF", "db72c76196f0469fbf1cbd901b8b6646", "1089953")   ### [Enric Madriguera and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "618d1ac69d767a02600e17fd21ad75d0", "1639763")   ### [Art Hickman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ec7168be46b5f1b985ee45c04e79cfe2", "341398")   ### [Charlie Spivak and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b3ec2cbefce16cdcfb717302b4949560", "1186529")   ### [Dick Jurgens and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7777c383c810e40a3eadfce4d1b91a6c", "708255")   ### [Gus Arnheim & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "364a3b80de20af42c4d91b29ea9fb24f", "1277416")   ### [Glen Gray and the Casa Loma Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "663d60521dc70e807c1597f6b4699085", "3453989")   ### [Nat Shilkret & the Victor Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "56a7191be8bbfc305156ca789bf5f31d", "1369825")   ### [Ben Bernie & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "afce6faff59a28d344434f9a1b0695ca", "349510")   ### [Ray Noble and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "599389f3b040e49d82e6a8385e17bb1a", "424111")   ### [The Haydn Quartet]
    addDiscogs(mdbmaps, "MusicVF", "df05c6f55851e6578477ca388660dc56", "1253756")   ### [Jan Garber and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ada1f66f40ecb5b1f068b2ee780d1f3f", "1534207")   ### [Charles Harrison]
    addDiscogs(mdbmaps, "MusicVF", "02ab9faaa95a329ed8dd672e5904c0a5", "1424900")   ### [Ted Weems and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "dc4a1e1f2b07b534353aa2d67fc18a17", "1089954")   ### [Ozzie Nelson and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d7bcbfee811d2f50c06f801c79dcf98", "341410")   ### [Larry Clinton and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "78954f5f199505d1100853141aa92c92", "893947")   ### [Shep Fields and His Rippling Rhythm]
    addDiscogs(mdbmaps, "MusicVF", "9d16f4b2bc7dc28c9ca1a391910d269b", "342802")   ### [Leo Reisman and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "90eaa040177c2811146f9bd9596f0e9a", "348903")   ### [Hal Kemp and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1206cc734327d41b89b7b21b614b24c1", "388298")   ### [Fred Waring's Pennsylvanians]
    addDiscogs(mdbmaps, "MusicVF", "a0da070b4d4b8481e517575e25da0d0f", "1337794")   ### [The American Quartet]
    addDiscogs(mdbmaps, "MusicVF", "8141e4ed4b735c19d9b566ea802d9fd9", "1048195")   ### [George Olsen and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "572af4b5e5c28b35d0c01bc32f35969c", "mn0000032614")   ### [Paul Whiteman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "572af4b5e5c28b35d0c01bc32f35969c", "82656")    ### [Paul Whiteman & His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "73318e203d3d01d134b78d76fdea6ede", "mn0000028948")   ### [Ted Lewis & His Band]
    addDiscogs(mdbmaps, "MusicVF", "73318e203d3d01d134b78d76fdea6ede", "344672")    ### [Ted Lewis & His Band]
    addAllMusic(mdbmaps, "MusicVF", "e73b5eb55c8d9cbd6c4e94ef34e857b4", "mn0000252794")   ### [The Peerless Quartet]
    addDiscogs(mdbmaps, "MusicVF", "e73b5eb55c8d9cbd6c4e94ef34e857b4", "459440")    ### [The Peerless Quartet]
    addAllMusic(mdbmaps, "MusicVF", "cfd29fb73a1df61969cc4e2c0d528889", "mn0000108905")   ### [Arthur Collins & Byron Harlan]
    addDiscogs(mdbmaps, "MusicVF", "cfd29fb73a1df61969cc4e2c0d528889", "1102536")    ### [Arthur Collins & Byron Harlan]
    addAllMusic(mdbmaps, "MusicVF", "7e48771184a11d7a02b2809bbeae73c6", "mn0000855695")   ### [Rudy Vallée & His Connecticut Yankees]
    addDiscogs(mdbmaps, "MusicVF", "7e48771184a11d7a02b2809bbeae73c6", "1309989")    ### [Rudy Vallée & His Connecticut Yankees]
    addAllMusic(mdbmaps, "MusicVF", "59a1f622916a78cb86f486057fe398a1", "mn0000358248")   ### [Jimmy Dorsey and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "59a1f622916a78cb86f486057fe398a1", "147557")    ### [Jimmy Dorsey and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "5e91727a080c9c2d39e24fda912f1e97", "mn0000773745")   ### [Isham Jones & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5e91727a080c9c2d39e24fda912f1e97", "296197")    ### [Isham Jones & His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "1162e06ab796e03f2569674315c517cb", "mn0000186373")   ### [Freddy Martin and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1162e06ab796e03f2569674315c517cb", "333548")    ### [Freddy Martin and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "b52dbc06b774edfd2d32ef664958397b", "mn0001712608")   ### [Kay Kyser and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b52dbc06b774edfd2d32ef664958397b", "159370")    ### [Kay Kyser and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "df2027d9e5b2df8691f950d59cc5a322", "mn0000145904")   ### [Eddy Duchin and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "df2027d9e5b2df8691f950d59cc5a322", "428029")    ### [Eddy Duchin and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "40da9295fedc06007599b5aed7ed23c4", "mn0000045220")   ### [Swing and Sway with Sammy Kaye]
    addDiscogs(mdbmaps, "MusicVF", "40da9295fedc06007599b5aed7ed23c4", "273876")    ### [Swing and Sway with Sammy Kaye]

    mdbmaps["MusicVF"].save()
    
addMusicVFExtras(mdbmaps)

Could not add artist data from PrimaryKey: 6047ae45fdcbb1a61452be74c2e09458
Could not add artist data from PrimaryKey: 6047ae45fdcbb1a61452be74c2e09458
Could not add artist data from PrimaryKey: 6047ae45fdcbb1a61452be74c2e09458
Could not add artist data from PrimaryKey: 38a08457915c65ef3cb1ff0b8a2db435
Could not add artist data from PrimaryKey: 38a08457915c65ef3cb1ff0b8a2db435
Could not add artist data from PrimaryKey: 38a08457915c65ef3cb1ff0b8a2db435
Could not add artist data from PrimaryKey: 7ae3fb2747978b2d93a22910fb1bb293
Could not add artist data from PrimaryKey: 7ae3fb2747978b2d93a22910fb1bb293
Could not add artist data from PrimaryKey: 97845c5ac2a1eda6bc7a40a52550f6cf
Could not add artist data from PrimaryKey: 97845c5ac2a1eda6bc7a40a52550f6cf
Could not add artist data from PrimaryKey: 97845c5ac2a1eda6bc7a40a52550f6cf
Could not add artist data from PrimaryKey: e4b3177fc3046cf9d6bef9f8de838668
Could not add artist data from PrimaryKey: e4b3177fc3046cf9d6bef9f8de838668
Could not ad

  --> This file is 714.6kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbMusicVFMap.p
  --> This file is 714.6kB.


In [ ]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

## Spotify

In [ ]:
addAllMusic(mdbmaps, "Spotify", "2c7704262fe420489299bef485e21f10", "mn0003130756")   ### [VIC MENSA]
addDeezer(mdbmaps, "Spotify", "2c7704262fe420489299bef485e21f10", "1044777")    ### [VIC MENSA]
addDeezer(mdbmaps, "Spotify", "920c63b4fcad01b381d646d6b8b365bc", "557185")    ### [NEW CITY]
addDeezer(mdbmaps, "Spotify", "26154d0d5295a4f04f64f0b7f1c0e067", "1521325")    ### [DiRTY RADiO]
addDeezer(mdbmaps, "Spotify", "845f615d3f2c32f4335fe39bef730d05", "566999")    ### [Rv]
addAllMusic(mdbmaps, "Spotify", "ae62113cf8a3f76960b374a1aefffc3d", "mn0000591459")   ### [TiÃ«sto]
addDeezer(mdbmaps, "Spotify", "ae62113cf8a3f76960b374a1aefffc3d", "434")    ### [TiÃ«sto]
addDeezer(mdbmaps, "Spotify", "a28cc660d82570830e3d7a652b4dda50", "5355839")    ### [DJ JEEZY]
addDeezer(mdbmaps, "Spotify", "802f1e688970ffdae3036900c02a48d4", "46853482")    ### [ToTheMoon]
addDeezer(mdbmaps, "Spotify", "bf1a1a76b86020560d80d25939467283", "1345313")    ### [LIBA]
addDeezer(mdbmaps, "Spotify", "172b31efbafca7f93222e89535823c71", "6713359")    ### [ValsBezig]
addDeezer(mdbmaps, "Spotify", "5c7f68b5c9d98dc02f75b7a052c655d6", "14251")    ### [BangBros]
addDeezer(mdbmaps, "Spotify", "efa40ffa68a7c9b3d9b44b335959f5be", "10011140")    ### [Wikw]
addDeezer(mdbmaps, "Spotify", "3cc604c59cfb71a740983a330b874cae", "10807780")    ### [DYSTINCT]
addDeezer(mdbmaps, "Spotify", "84a5e716bdce0b761cdd9b3995565cfe", "7639780")    ### [RUGGED]
addDeezer(mdbmaps, "Spotify", "8dda0d0fa1201255d30cddd370b4527a", "59466262")    ### [LUNA KI]
addDeezer(mdbmaps, "Spotify", "6deb82dda4cd26bec6b94e8e7e970b7f", "14364799")    ### [FlowZeta]
addDeezer(mdbmaps, "Spotify", "c4596c33a8f464e29b07bbb0f1c7c0ff", "10342534")    ### [Poorchoice]
addDeezer(mdbmaps, "Spotify", "841b83ac5259a4b2bd769695adf387f8", "325115")    ### [Nosi]
addDeezer(mdbmaps, "Spotify", "cc6f52fa7b2368269985744be5214b1b", "1517421")    ### [DENZO]
addDeezer(mdbmaps, "Spotify", "28d0487969b481c095a44464af37a9a9", "67450512")    ### [GS]
addDeezer(mdbmaps, "Spotify", "fc2b87972f8a8ec07b311e59825be184", "122381472")    ### [Dioscures]
addDeezer(mdbmaps, "Spotify", "b253d9e0f63cfe594ad59c1573c14b64", "5794446")    ### [AXEC]
addDeezer(mdbmaps, "Spotify", "8d3d126619a7a4757f4f2c248ade9aa6", "61471392")    ### [SKiNNY BARBER]
addDeezer(mdbmaps, "Spotify", "9e153fd9c556ac76075c23a4e2401272", "15644")    ### [Dan Barta]
addDeezer(mdbmaps, "Spotify", "3347bd679e845d9cb60181daf815d879", "1442177")    ### [LYSA]
addDeezer(mdbmaps, "Spotify", "75074c0c9d0202a88c21f866c9debe9d", "4646560")    ### [Apu]
addDeezer(mdbmaps, "Spotify", "8efaa0f6bcfb4d9d2bf43d29e47ca19d", "7637470")    ### [VALMAR]
addDeezer(mdbmaps, "Spotify", "4645b07014e7de28b3eb4fdc2b09ece4", "117840852")    ### [SHE]
addDeezer(mdbmaps, "Spotify", "e68cb9c121fd6da74857936c67f0f06f", "1354350")    ### [Kid1]
addDeezer(mdbmaps, "Spotify", "68496911494cdd8e63039cccbfac733d", "4560405")    ### [JORI SJÖROOS]
addDeezer(mdbmaps, "Spotify", "7027fdc8d392b9a06c57a4566b144445", "96643")    ### [ODE]
addDeezer(mdbmaps, "Spotify", "d3555bd07f974448684672f85aabef02", "63712282")    ### [ZÖE]
addDeezer(mdbmaps, "Spotify", "6d4ca17986eee42444e86dbfc7e0b137", "134065")    ### [ZENA]
addDeezer(mdbmaps, "Spotify", "c17577a0af8e0ba087f802001b600b9b", "77554172")    ### [fuse.]
addDeezer(mdbmaps, "Spotify", "9fa8323b0c2feb08be976c7e6b97d37c", "85045172")    ### [VICTORIA]
addDeezer(mdbmaps, "Spotify", "78da7b843a50d1bed3ddfe556eb661a7", "98175162")    ### [Bruno x spacc]
addAllMusic(mdbmaps, "Spotify", "3498367b46c2f47cb70bdb9fb75a2cd2", "mn0003597150")   ### [Zävodi]
addDeezer(mdbmaps, "Spotify", "5a54b188dfb176fe6d47912e30c39e84", "12790253")    ### [Felix & Dean]
addDeezer(mdbmaps, "Spotify", "103972014ba658abe4b40a4fa9b74495", "507314")    ### [4Korners]
addDeezer(mdbmaps, "Spotify", "6d61811c24db9279f28e1e1921619aff", "208250")    ### [HAYES]
addDeezer(mdbmaps, "Spotify", "0a9d2e9a751ac074778eefd4fe0449ca", "113731272")    ### [EMIR]
addDeezer(mdbmaps, "Spotify", "2f351c66d2fa77578e1ce385cccf6578", "13609323")    ### [LayLay]
addDeezer(mdbmaps, "Spotify", "966bac2b2a33212f907f0ead939b7f9d", "3713431")    ### [SunCity]
addDeezer(mdbmaps, "Spotify", "d80c0934559a8ccfb558f1cbd8f7f361", "72022")    ### [EMMY]
addDeezer(mdbmaps, "Spotify", "890fdf97a97559703a6b0751e2d15048", "114020712")    ### [IBO]
addDeezer(mdbmaps, "Spotify", "90e4f38de02fe89733974a1e1538f4a0", "1112997")    ### [NALO]
addDeezer(mdbmaps, "Spotify", "195e988c3cd6f12d1906aaeb5111d36c", "12089130")    ### [CASPR]
addDeezer(mdbmaps, "Spotify", "ef407ddceb97d8e618394a0e7b2947ae", "12188928")    ### [Vkation]
addDeezer(mdbmaps, "Spotify", "a728fbfe6dc4de2bd87407d3d3e23e76", "5653407")    ### [Meso]
addDeezer(mdbmaps, "Spotify", "6a1eff593643f6b231caf35224485374", "11478356")    ### [Mad.S]
addDeezer(mdbmaps, "Spotify", "ea00bcea92149aec2ba35641d5e25789", "9719800")    ### [Solguden & Mannen]
addDeezer(mdbmaps, "Spotify", "4d68a565a29117574836e3a7f1760d22", "327360")    ### [Madd]
addDeezer(mdbmaps, "Spotify", "33d7726b3779bb6007b25e5608bd8bcc", "70773932")    ### [Afroto]
addDeezer(mdbmaps, "Spotify", "422213e84cc37e39b83ce21c3e94137d", "342125")    ### [ISSAM]
addDeezer(mdbmaps, "Spotify", "b46e5c7b15b7427c94a1903b3a9f8c83", "11419694")    ### [Tflow]
addDeezer(mdbmaps, "Spotify", "9b3766afd03eca31d6b33959b5e6b54b", "117480")    ### [Majid Almohandis]
addDeezer(mdbmaps, "Spotify", "9e62594b9185e8fe3e2f4ee7c2380b6c", "13272315")    ### [L7or]
addDeezer(mdbmaps, "Spotify", "4fd56f6c7826769bf01a0687ca01ca46", "4341385")    ### [Mohamed AlSalim]
addDeezer(mdbmaps, "Spotify", "62aafefe147d236674ea7b6c31e064d9", "2469601")    ### [Fnaire]
addDeezer(mdbmaps, "Spotify", "dd6022f7bc11f42cfe10f37db3ab3805", "14943131")    ### [ARNOLD]
addDeezer(mdbmaps, "Spotify", "c5de08852f51c6d4b64443c6ff28b064", "14999849")    ### [micem]
addDeezer(mdbmaps, "Spotify", "44ca8fe2e5b74e945d51c4be314345f0", "7107933")    ### [Lil Eazy]
addDeezer(mdbmaps, "Spotify", "25ebd6fe7a146bbeba83a9430a18d22e", "83499152")    ### [DJ LYAN]
addDeezer(mdbmaps, "Spotify", "402f040af4a7667a87491c3f81fc51dd", "13900015")    ### [FEYI]
addDeezer(mdbmaps, "Spotify", "64fb5161ab6bf6bd9685ec461218f27d", "5414687")    ### [TYTE]
addDeezer(mdbmaps, "Spotify", "0d6f38579b8d7a6d3b39abe2cc673918", "6051522")    ### [Ebx Music]
addDeezer(mdbmaps, "Spotify", "b0de7a947f0ec3f64ea69422d7737a09", "114617632")    ### [Barry]
addDeezer(mdbmaps, "Spotify", "e74c665fab1655c7214d088b1cfcc631", "59338402")    ### [Gher]
addDeezer(mdbmaps, "Spotify", "4472f33a7bcc305660f2459c40ee56b0", "2478401")    ### [ansis]
addDeezer(mdbmaps, "Spotify", "e0c4033e2f7913a914c7920d564c9e2b", "125477")    ### [Livi]
addDeezer(mdbmaps, "Spotify", "09b13dabe4b1cc54728627b6958a35d2", "157690")    ### [säm]
addDeezer(mdbmaps, "Spotify", "71ad6ba0221a38b61929597170f931b8", "14003")    ### [Ania Dabrowska]
addDeezer(mdbmaps, "Spotify", "a3765e586e042254b4a369b16cdd6dc4", "10956326")    ### [rolands če]
addDeezer(mdbmaps, "Spotify", "d8ee0c11cb37dbb97dbe4589a4545282", "89313432")    ### [LUISA]
addDeezer(mdbmaps, "Spotify", "fdf70d482adf5f6fb9fe3691b66fd666", "10864428")    ### [BAZOOKA]
addDeezer(mdbmaps, "Spotify", "c63d668bad2daac009d38d69b06709a9", "181827")    ### [STEFAN]
addDeezer(mdbmaps, "Spotify", "3e51c2a492a3208d44ed4b68c5c0de11", "4683785")    ### [INGER]
addDeezer(mdbmaps, "Spotify", "d0ea789adb64bde45d31ed2b957f04af", "68813")    ### [LIDA]
addDeezer(mdbmaps, "Spotify", "efdcc76555c18b9a47da12a095d5f4da", "4660822")    ### [KHANS]
addDeezer(mdbmaps, "Spotify", "f15e7437521872b6aa54287b4c1d3ddc", "1136614")    ### [kaw]
addDeezer(mdbmaps, "Spotify", "1e3271d5849a140d3f6e5698d9804764", "4686396")    ### [Exiled]
addDeezer(mdbmaps, "Spotify", "fbea3070535d01a829acf5c0580fdf32", "106388592")    ### [payton]
addDeezer(mdbmaps, "Spotify", "87c7f6875d65a79bc2f8a97dfc3bae7a", "8954502")    ### [Kiol]
addDeezer(mdbmaps, "Spotify", "0e74d4e6e94af3ef261e8f83483483c4", "804945")    ### [SAINTS]
addDeezer(mdbmaps, "Spotify", "a2c82bc5baadb0c156927c3aaf2e577c", "165664")    ### [Mariño]
addDeezer(mdbmaps, "Spotify", "e1c94b24ad823c50979510eb5df5eb29", "1085396")    ### [Stix]
addDeezer(mdbmaps, "Spotify", "1bd82622eeaad5c700241013ffd4a39a", "4220098")    ### [VIŅA]
addDeezer(mdbmaps, "Spotify", "783518b05ced6ee19c334d3954f71fe2", "12741165")    ### [Young DrA]
addDeezer(mdbmaps, "Spotify", "f6e14b60135b2e9023db60d4e5aef7fe", "53173512")    ### [SEEMEE]
addDeezer(mdbmaps, "Spotify", "a0f7810c25589860ac8973e63238e56c", "5331617")    ### [KOIT TOOME]
addDeezer(mdbmaps, "Spotify", "dd2042c741f536dd19998a749c498899", "15166901")    ### [372kaspar]
addDeezer(mdbmaps, "Spotify", "2f327dedb546e1d6ee6d96e41a41f27e", "54812362")    ### [IsaacMusic]
addDeezer(mdbmaps, "Spotify", "9052f79c5866ca8509bdba6b13d7d4ff", "482837")    ### [KNUX]
addDeezer(mdbmaps, "Spotify", "ff777d857ffd620f66c9677034ba5137", "1362521")    ### [MacShawn100]
addDeezer(mdbmaps, "Spotify", "16395062a4e4a343c2e4fd6326837d9f", "12438312")    ### [Leol]
addDeezer(mdbmaps, "Spotify", "b311824763f4757dd5557233781b041f", "317194")    ### [andrei]
addDeezer(mdbmaps, "Spotify", "3d65586d18ec29c99342c6f85a7ab9ee", "3767391")    ### [VUNK]
addDeezer(mdbmaps, "Spotify", "a722dccf85df771ce9c061e45b292990", "1664003")    ### [RAVA]
addDeezer(mdbmaps, "Spotify", "212aee750484f97810fe3446092bd8c7", "14649413")    ### [Ali Dolla$]
addDeezer(mdbmaps, "Spotify", "8a7a8cb8e7cad7f49869dabb19aae3a9", "143637")    ### [TRISHES]
addDeezer(mdbmaps, "Spotify", "0500b238f4271c84348b5a4643c8895d", "7143773")    ### [Beau'Joli]
addDeezer(mdbmaps, "Spotify", "9c8fd5547029ceca9134d5f77690d595", "7742848")    ### [Monika Linkyte]
addDeezer(mdbmaps, "Spotify", "845974b91908722810a650988afc5416", "9476028")    ### [MEANDI]
addDeezer(mdbmaps, "Spotify", "75115e81be8211dac1ea58283c8542bf", "53002782")    ### [Radvilas RA]
addDeezer(mdbmaps, "Spotify", "27ae5d527bb68f4d2636a0d4e619f526", "14861157")    ### [be vis]
addDeezer(mdbmaps, "Spotify", "1aaf83a6c22205806a04690f58f982c9", "14014275")    ### [KiROT]
addDeezer(mdbmaps, "Spotify", "6c928d014408cbe348a78f08041e5c85", "8644134")    ### [Plamen & Ivo]
addDeezer(mdbmaps, "Spotify", "95ab8f5a6277c25332cf0f2ae43f86a0", "6137")    ### [Heroes Del Silencio]
addDeezer(mdbmaps, "Spotify", "283e9ad2edc7926f1225d00caa3703dc", "244980")    ### [La Mosca Tse-Tse]
addDeezer(mdbmaps, "Spotify", "15c4a8facf210c3bfc7f0cac28b31a4c", "305771")    ### [Bobo]
addDeezer(mdbmaps, "Spotify", "f817ce364f7768dd727d90324bc8e91e", "1217586")    ### [MOI MUSIC]
addDeezer(mdbmaps, "Spotify", "70a347e39165362ec895f0582ec35e54", "2946291")    ### [BARBEL]
addDeezer(mdbmaps, "Spotify", "4bbaa1fb2632cb64077188609e441aef", "4607825")    ### [Talley]
addDeezer(mdbmaps, "Spotify", "b2f2bf4d5d78ea7e4fce5adec65f860d", "49192912")    ### [JOSEPH CHASE]
addDeezer(mdbmaps, "Spotify", "ae14a4a2e9de5d9e2707d4ed482c1063", "14609075")    ### [His Kings]
addDeezer(mdbmaps, "Spotify", "082d82c2cfa5bd0188d7c978b53bc58c", "9927892")    ### [BONNIE X CLYDE]
addDeezer(mdbmaps, "Spotify", "f17f23e5351e33f30b14584ffa51e80f", "1415466")    ### [DJ Yayo]
addDeezer(mdbmaps, "Spotify", "ae7abd8b47d277b7be0109e91015631a", "14901793")    ### [Claveles de la Cumbia]
addDeezer(mdbmaps, "Spotify", "2ea00dcb2e06802bd4d2a594e26d6159", "12664741")    ### [VICE MENTA]
addDeezer(mdbmaps, "Spotify", "adac75d8af78c47417607eed58ac2b58", "7816600")    ### [Urband]
addDeezer(mdbmaps, "Spotify", "3fdec593956e5d930420d02d2666b24a", "12251454")    ### [Los inquietos del vallenato]
addDeezer(mdbmaps, "Spotify", "5a6eb36d1241dd8991ed77c5c76f7d63", "366177")    ### [MC LEO]
addDeezer(mdbmaps, "Spotify", "283e9ad2edc7926f1225d00caa3703dc", "244980")    ### [La Mosca Tse-Tse]
addDeezer(mdbmaps, "Spotify", "34c0a13db2387965a5ebfaa14296c6eb", "12251454")    ### [Los Inquietos Del Vallenato]
addDeezer(mdbmaps, "Spotify", "c4c92ea1f897b796c91ade25828b2ec6", "9231042")    ### [CHIAKI SATO]
addDeezer(mdbmaps, "Spotify", "2f0133066d975b78c013934554e7eedb", "87908682")    ### [The New Boyz]
addDeezer(mdbmaps, "Spotify", "8de6581b8c7f02485d8c8d4008772a07", "135962")    ### [DOLLA]
addDeezer(mdbmaps, "Spotify", "3326a1459e0eb70a7f0fce751e60316c", "9220944")    ### [XPOSE]
addDeezer(mdbmaps, "Spotify", "2422543147956008f31f94d6574735dd", "53217322")    ### [ABANGSAPAU]
addDeezer(mdbmaps, "Spotify", "da2135c1ad33ea3cc20899ef9db4a0a5", "7349002")    ### [JOSIMAR Y SU YAMBU]
addDeezer(mdbmaps, "Spotify", "fac24962ac25281899e9fb086744e8dc", "1190692")    ### [PETITE]
addDeezer(mdbmaps, "Spotify", "012d1b92264beb5beb5b5d6b9ad107ba", "11107496")    ### [TOSSAKAN]
addDeezer(mdbmaps, "Spotify", "10c8c10c4d508e61d5c0ce9920f4e616", "96924502")    ### [KUN]
addDeezer(mdbmaps, "Spotify", "d21abd9a924824b3591881083f0cdd86", "49098811")    ### [PiA]
addDeezer(mdbmaps, "Spotify", "e2a03f421834df8a3b354c0475ca45d3", "4457870")    ### [JINU]
addDeezer(mdbmaps, "Spotify", "3fb999bcd426e08f451ae381a3e68b38", "8410882")    ### [Mr.Siro]
addDeezer(mdbmaps, "Spotify", "cdda9cbbdb1ef951778b3cfc81c8fd5b", "88017162")    ### [COSMO.]
addDeezer(mdbmaps, "Spotify", "7dc5ad3d308ecaefc3107719bee41b39", "13747253")    ### [3NIGMA]
addDeezer(mdbmaps, "Spotify", "cd22a685e541843314ebbd7f5c60efd8", "12354536")    ### [Kiian]
addDeezer(mdbmaps, "Spotify", "1b8b9edcaf6afb10abab14ba84d06fe5", "12086134")    ### [FAO>dino]
addDeezer(mdbmaps, "Spotify", "12b828a4aec973bd2973546a08bb03b8", "11473350")    ### [Jt el Utility]
addDeezer(mdbmaps, "Spotify", "21da20975b043559ab8b2ceaaf7b6e0a", "372689")    ### [my little airport]
addDeezer(mdbmaps, "Spotify", "79febaab2b17a129e67afa418da8ea35", "14753211")    ### [AYA A.K.A PANDA]
addDeezer(mdbmaps, "Spotify", "673db1aee7862515f4b04d7a8bfdea4d", "5367324")    ### [ANATII]
addDeezer(mdbmaps, "Spotify", "8d7cca977c9983f6784d5942626bdf0a", "15316233")    ### [Dj Obza]
addDeezer(mdbmaps, "Spotify", "f902ea414120d93bf8ca66a75a47f1a0", "13134247")    ### [WUANT]
addDeezer(mdbmaps, "Spotify", "ac0997d0e6975ec8b40a5e9bccb99aa7", "10399494")    ### [OIK]
addDeezer(mdbmaps, "Spotify", "82e0e6e18a81788dacfafccd0a45ffbe", "14478167")    ### [MOBBERS]
addDeezer(mdbmaps, "Spotify", "3c7f465d7af84f658b57dde0203275b6", "8599352")    ### [xtinto]

mdbmaps["Spotify"].save()

# Downloads

In [ ]:
set1  = set(['Joshua Bassett', 'DJ Scheme', 'Pedro CapÃ³', '16yrold', 'Sueco', 'BRELAND', 'Alicia Moffet', 'Skye', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'VIC MENSA', 'NEW CITY', 'Madalen Duke', 'Cassow', 'DiRTY RADiO', 'Bob Pressner', 'Christian Hudson', 'Vanic', 'Elijah Woods', 'Jamie Fine'])
set2  = set(['Mimi Webb', 'Frisco', 'Morrisson', 'MizOrMac', 'Miniminter', 'Fekky', 'Kenny Allstar', 'Rv', 'TiÃ«sto', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'OSH', 'Koomz', 'Amelia Monét', 'Tranell', 'Alex Ross', 'Ralph Felix', 'David Campbell', 'Vidwaan Boon', 'Lea Heart', 'Ché-Fu', 'Lee Mvtthews', 'SACHI', 'Elektriches Weihnachtfest', 'Trill Ryan', 'TEEKS', 'Maimoa', 'J1', 'The Blue Jeans', 'Bob B. Soxx', 'Collectors', 'Henny', 'Zinny', 'AP'])
set3  = set(['DJ JEEZY', 'Maya Berovic', 'Neptunica', 'Poptracker', 'Zappi', 'LZ27', 'Milan Stankovic', 'Standart Skill', 'Butrint Imeri', 'Tanja Savic', 'Monja Mari Achi', 'Die Hollerstauden', 'Claudius Vlasak', 'DJ Selecta', 'Klaus Peter', 'Ellis Adventure', 'Mortal Frame', 'DJ Greg V', 'Mister Claude', 'Little Strong', 'River Tyne', '4 Giants', 'Martina Kaiser', 'Patrick Knight', 'A Stritzi', 'Shlump', 'BENGELS', 'Stefanie Kloß', 'ARTY', 'Nullzweizwei', 'Koushino', '65Goonz', 'Pietro Basile', 'Sadri', '2ara', 'Provinz', 'Laas Unltd.', 'Z', 'HAMZO 500', 'Gewitter im Kopf', 'Erabi', 'Koosen', 'O.G.', 'Ramon Roselly', 'Senna Gammour', 'Maaf', 'Riccardo', 'ToTheMoon', 'Teddy Teclebrhan', 'Sipo', 'Joker Bratan', 'Bato', 'Dafina Zeqiri', 'Oflow', 'Dj Iljano', 'OFFTR3ND', 'F4ilure', 'Jasin Blanco', 'Pronto', 'Rocio Alvarez', 'Swiluu', 'Mimiks', 'LCone', 'Zwei Am Morge', 'Cricket', 'LIBA', 'Manillio', 'SHINSKY', 'Pappa Razzi', 'Carnie', 'Lily'])
set4  = set(['Enula', 'Raffaele Renda', 'Aka 7even', 'Marïna', 'WØODY', 'Niko Pandetta', 'Cancun', 'Milano Ovest', 'Axos', 'Marta Daddato', 'Tosca', 'MARCO SENTIERI', 'Paolo Jannacci', 'Gabriella Martinelli', 'NOËP', 'Chakra', 'Greta Menchi', 'Rosalba', 'Sherol Dos Santos', 'Renza Castelli', 'Zano', 'Giovanni Caccamo', 'Lorenzo Baglioni', 'Ron', 'Mirkoeilcane', 'Renzo Rubino', 'Lorenzo Licitra', 'Andrea Radice', 'ROS', 'Raige', 'Maldestro', 'Francesco Guasti', 'Lesram', 'Bilton', 'joysad', 'Dioscures', 'Kanoé', 'GS', 'ISK', 'Osen', 'T Garcia', 'twinsmatic', 'DENZO', 'Kodes', 'Zikxo', 'Badjer', '404Billy', 'Kopp Johnson', 'Sirius', 'Seezy', 'Jahyanai King', 'Jaded', 'Nassi', 'DJ JEEZY', 'Wahlstedt', 'Don Criminals', 'Novi', '5HOW', 'Wikw', 'Goonew', 'Edo Ferragamo', 'Lost Identities', 'GioBulla', 'Roxx Remora', 'Otira', 'R3LL', 'Brooze', 'DASIC', 'DJ LeanGun', 'Toni der Assi', 'Maale Gars', 'Nevlo', 'Nosi', 'Chasey Negro', 'The FifthGuys', 'KOLIDESCOPES', 'Hanzy', 'thisisNAMASTE', 'Jared Moreno', 'Krozz', 'Boris Ayson', 'Alley', 'BROMYGO', 'Kroshwell', 'Nacho Fano', 'Kyd Knarly', 'EQRIC', '4Korners', 'Helen', 'KLOUD', 'Jason Thurell', 'Heuse', 'Lokyto', 'Clambake', 'Josh Stanley', 'Gazz', 'Brevin Kim', 'Dan Lemaire', 'Makasi', 'Mila Jam', 'Jeff Kush', 'EGalas', 'Uniekgrace', 'Basics', 'Camryn', 'Jay 305', 'NAZAAR', 'Benjin', 'VLS', 'Tom Berx', 'KNYD', 'Mark Melgo', 'EDGR', 'Neighbors', 'Jeyday', 'Flairo', 'FVR AVAY', 'Nicky Vaent', 'Vinni Wonka', 'Michael Wayne', 'Ricky Gazetta', 'Busta K', 'Prince Paris', 'Poorchoice', 'Blanke', 'Cytrax', 'Slue', 'Rex Stax', 'Brianna Mazzola', 'Meresha', 'GLDN', 'Simock', 'Ruben Pol', 'Those 2 Guys', 'Seorsia', 'Lennard Ellis', 'AXEC', 'SHANNI.', 'Sophia Treadway', 'Aysha Monet', 'Mystic Natives', 'Schuz', 'CRZY', 'Kalipzo', 'Dee Gomes', 'Lifetime Member', 'Tritan', 'teamwork.', 'The Unfits', 'The Him', 'Deadrose', 'JJ Hannon', 'Hones-T', 'Kyle Edwards;DJ Smallz 732', 'Kyle Edwards;Chris Bevrly', 'Miquela', 'Ian Erix', 'Chris Tyson', 'Dylan Diamond', 'Cal Scruby', 'Bandana', 'Lérica', 'Hard GZ', 'Aiman JR', 'Marc Seguí', 'Emanuel A', 'Yulien Oviedo', 'REYKO', 'FlowZeta', 'Joan Garrido', 'Damion Frost', 'Dave Zulueta', 'María Escarmiento', 'Pol Granch', 'Chica Sobresalto', 'Yung Sarria', 'Sebastián Cortés', 'TheGrefg', 'Paula Cendejas', 'Anaju', 'Eva B', 'Ankhal', 'Resistiré 2020', 'Samantha', 'Duo Dinamico', 'ARON', 'Javy Ramírez', 'Love Yi', 'Eklectico', 'Locoplaya', 'LUNA KI', 'Dylan Fuentes', 'Soge Culebra', 'Elilluminari', 'Sergio Ramos', 'Selecta', 'Kaze', 'Jirafa Rey', 'Paula Gonu', 'Yuawi', 'La Vida Moderna', 'Angeles', 'Tutto Duran', 'ICE', 'Dior', 'Goldkimono', 'Jeffrey Heesen', 'David Neres', 'Jjakoveertien', 'Rotjoch United', 'Wolfgang', 'Makkie', 'Joey Bravo', 'Jerra', 'Mario Cash', 'ValsBezig', 'Oualid', 'Team Rush Hour', 'DYSTINCT', 'Bigidagoe', 'KNALLER', 'Jordymone9', 'Pierrii', 'Matthy', 'Selma Omari', 'Frozen Fire', 'Akwasi', 'DJ Nazil', 'Pjotr', 'Trobi', 'Kevcody', 'Jayboogz', 'S10', 'The voice of Holland', 'Blow', 'Ares', 'Dv', 'Dylisa', 'Joel Beukers', 'Dana', 'DJ Wale', 'Souff', 'Floor Jansen', 'Paul Sinha', 'Lirical', 'Artistic Raw', 'LU BACH', 'Skiezo', 'Rene Karst', 'Jiri11', 'Nijo', 'MRD', 'Zoë-Jadha', 'Quatro Vision', 'Daniël Busser', 'F.I.', 'SuriBeats', 'Fabiola', 'Dave Roelvink', 'Iliass', 'Memphis Depay', 'Qucee', 'Shock Elias', 'Quessswho', 'RUGGED', 'Vonneke Bonneke', 'Scooby072', 'CJaye', '808milli', 'Bridde Boys', 'Lucass', 'Nesim Najih', 'Gebroeders Rossig', 'Anbu', 'Kid de Blits', 'Mael', 'Gamemeneer', 'Anu-D', 'BangBros', 'Milan Knol', 'Langesjaak', 'Darryl', 'Tavi Castro', 'Johnny Sellah', 'Minitrapper', 'Rabby Racks', 'RONDÉ', 'PLEUN', 'Angelo King', 'J1', 'Dean', 'Kellini', 'Fito', 'Waor', 'Natos', 'Prok', 'El High', 'Dayme', 'Diaz'])
set5  = set(['Whyrun', 'Villi Neto', 'Gummi Tóta', 'Unnsteinn', 'Miquela', 'Ian Erix', 'Chris Tyson', 'Greifarnir', 'Magni', 'Vilhjálmur Vilhjálmss', 'Hreimur', 'Venjulegir Gaurar', 'Sigga Beinteins', 'Edda Heiðrún Backman', 'Eythor Ingi', 'VICTORIA', 'Ferðumst innanhúss', 'Ouse', 'R3SYNTH', 'fuse.', 'Scraby', 'Sharp Man', 'Br0med', 'krassasig', 'Háski', 'Andri Már', 'Shakespeare verður ástfanginn', 'Reynir', 'EVR3stNess', 'Kolassoid', 'Akeckee', 'ZENA', 'zalagasper', 'Eliot', 'Michael Rice', 'S!sters', 'Tulia', 'Srbuk', 'Jurijus', 'Sarah McTernan', 'Flosi V', 'Screamerboys', 'Egill Stolz', 'Helgi B', 'Stefán Karl Stefánsson', 'Bergur Leó', 'Caulfield', 'ZÖE', 'Youngcuzzoo', 'Óðinn Valdimarsson', "J'adora", 'Ieva Zasimauskaitė', 'Lea Sirk', 'DoReDoS', 'SuRie', 'Flosi', '$igmund', 'Snow Dept.', 'Ragga Holm', 'iDubbbz', 'Dihaj', 'Hovig', 'IMRI', 'Jana Burčeska', 'Slavko Kalezić', 'Rúnar Eff', 'Þórdís Birna Borgarsdóttir', 'Erna Mist Pétursdóttir', 'Rakel Pálsdóttir', 'Sara Farell', 'TheoFuego', 'Frans Harju', 'Helena Routa', 'The View', 'Hussa', 'Oskr', 'DJ Massimo', 'Kuohu', 'Henriguez', 'Lxandra', 'Knipi', 'Sara Chafak', 'Raaka-Aine', 'NICHOLAS X', 'KIKI 76', 'Xtojo', 'Mauton', 'VV', 'Tomi Amor', 'Antony Parviainen', 'Toni Kakko', 'Leyton Lake', 'Frances', 'Pajafella', 'ODE', 'Mikaveli', 'Romu', 'SANÉ', 'Tre Kronor', 'Samppa Linna', 'ILLSIGHT', 'JORI SJÖROOS', 'Paul Elias', 'Malka', 'Kauriinmetsästäjät', 'Julia Rautio', 'Nelli Milan', 'Vahtera', 'Blue Street Players', 'Ronja Mariela', 'Planet SAHH', 'Kerava', 'Kid1', 'Deata', 'SHE', 'City Street Jazz Band', 'Møtions', 'MIROxNIKI', 'Epelit', 'Pasa', 'KOI', 'Rácz Gergõ', 'VALMAR', 'Kasza Tibi', 'IstenEst', 'Nks', 'Bereczki Zoltán', 'HRflow', 'Mario Judah', 'Tiah', 'Apu', 'Carson Coma', 'Strongi', 'LMEN PRALA', 'Pamkutya', 'Vanek Andor', 'Misshmusic', 'USNK', 'Senkise', 'Horváth Tamás', 'Pixa', 'Filius Dei', 'Samuel Kim', 'JustVidman', 'Red Bull Pilvaker', 'Dé:Nash', 'Konyha', 'Hundred Sins', 'King Monroe', 'AKC Misi', 'ibbigang', 'New Level Empire', 'Berkes Olivér', 'Szakács Gergő', 'Zséda', 'TNT', 'Sipos Tamás', 'Radics Gigi', 'Soulwave', 'Zävodi', 'HoneyBeast', 'LYSA', 'Opitz Barbara', 'Kollányi Zsuzsi', 'Margaret Island', '30Y', 'Trifecta', 'Kindler Shah', 'Stolen Beat', 'Molnár István', 'Egs', 'Pøla', 'Elefánt', 'Kispál és a Borz', 'Ismerős Arcok', 'Szabó Balázs Bandája', 'Magneoton Sztarok', 'Dana', 'FIZIstyle', 'Grey256', 'Skwor', 'Wesome', 'Milion Plus', 'Refew', 'HILLS 97', 'P T K', 'Adam Ďurica', 'Fobia Kid', 'Dan Barta', 'Skardu', 'ONEMANSHOW Foundation', 'Adam Mišík', 'Reginald', 'Za100let', 'Amco', 'Redzed', 'MikeJePan', 'Johny Machette', 'Janek Ledecky', 'XINDL X', 'Nightwork', 'DJ Rusty', 'Berenika Kohoutová', 'SKiNNY BARBER', 'Maniak', 'Cypis', 'Hard Rico', 'Michal Hruza', 'ATMO Music', 'Porsche Boy', 'SpecialBeatz', 'DJ Kadr', 'Dětský sbor Camerata', 'LIL GRIPPIE', 'Hugo Toxxx', 'DeeThane', 'Martin Matys', 'Naflexx', 'Forgen', 'Vadak', 'FattyPillow', 'TVTwixx', 'Dětský sbor', 'Josh Jips', 'Leoni Jut', 'Tomas Klus', 'Jakub Děkan', 'JJ Hannon', 'Sofian Medjmedj', 'JCKPT', 'Annet X', '5NAPBACK', 'STIKSTOF', 'Sara Gold', 'Sylver', 'Double Pleasure', 'Kenn Colt', 'Liefde Voor Muziek voor Rode Neuzen Dag', 'Jjakoveertien', 'The Starlings', 'DJ Nazil', 'Sean Dhondt', 'Karen Damen', 'Peter Vanlaet', 'Yung Mavu', 'Seedy', 'J Pablo', 'Twin n Twice', 'Trobi', 'Mastachi', 'Sordal', 'Amanda Wittman', '404Billy', 'bvd kult', 'Silvy', 'DJ Licious', 'Curtis Alto', 'Tavi Castro', 'Nassi', 'Natalia', 'Tim Arisu', 'D5R', 'J1', 'Tilhet', 'pajut ja muut', 'Spacc', 'the Serpent', 'spacc', 'Freddie Shuman', 'Halloran', 'Kate', 'Lil G.', 'Jickson', 'Sheen', 'PJAY CZ', 'SK'])
set6  = set(['Carl Knast', 'DIBSET', 'SunCity', 'Boris Laursen', 'Jamie Kamara', 'Børnemusik Dyrene', 'Awave', 'N3LLY', 'HAM fra Syd', 'Merro8', 'IBO', 'Berd', 'LayLay', 'Jesu Bankerz', 'Kristian Kjærlund', 'Alexander Husum', 'AMRO', 'Leprom', 'Karizman', 'Elvira Pitzner', 'VM Holdet', 'KING', 'Vkation', 'Tramper Torben', 'Place On Earth', 'Delahoia', 'Sandra Hussein', 'Soleima', 'Tore', 'Tiril', 'Henrik Blomme', 'Katie Keller', 'Salazar', 'Dj Guldsmed', 'Jytte', 'Lakserytteren', 'CHILI', 'MGP Allstarz 2017', 'Guldpige', 'JB', 'Euroo', 'Marti', 'Malaka', 'buller', 'Oskar Häggström', 'A36', 'Seedy', 'HON', 'SG Lewis', 'Style', 'Erik Lundin', 'Matte Caliste', '2.clock', 'Abidaz', 'Biggie Juke', 'Suzi P', '2M', '10an', 'Linda Olsson', 'Bailey Jehl', 'Four of Diamonds', 'Janice', 'The Mayries', 'Jelassi', 'Wild-Boy', 'Michael Rice', 'zalagasper', 'Herbert Munkhammar', 'Jonathan Johansson', 'Mila Crowell', 'Lionman', 'Leyton Lake', 'Amy Soonyoung', 'Tony Lieberman', 'Hallman', 'Max George', 'Ieva Zasimauskaitė', 'May Raya', 'Thomas Sekelius', 'Total Ape', 'Hovig', 'IMRI', 'Henrik Berggren', 'Les Gordons', 'Galavant', 'Blåsemafian', 'Jorn', 'Halva Priset', 'Ole Hartz', 'EMMY', 'Kristin Husøy', 'Magnus Bokn', 'S1sco', 'HAYES', 'Philip Emilio', 'Oskar Westerlin', 'Daskeladden', 'Kevin Boine', 'Torine', 'Roc Boyz', 'Yasmine San Miguel', 'Martin Heier', 'GoToGuy', 'GABIFUEGO', 'NALO', 'Drikkeleken', 'Simon Kleiveland', 'DMNDS', 'Matilda', 'B-Boy Myhre', 'Ida Celine', 'Kutoppen', 'Jonathan Floyd', 'Petter Northug Jr', 'Konen på torget', 'Bendik Brænne', 'Ulrike Hansen Døvigsen', 'Penthox', 'Bearson', 'Vilde og Anna', 'Dunisco', 'Future Duper', 'ARY', 'Adrees', 'Clairmont', 'Alex Ross', 'Serlina', 'Tunge Ferrari', 'Piateed', 'Olav Haust', 'Chris Haugan', 'Huemist', 'Ezzari', 'Louam', 'Alberto Ciccarini', 'ZadeKing', 'INDIIA', 'Jonas Benyoub', 'Swedum', 'Chad Cooper', 'Temmpo', 'Travelle', 'CASPR', 'Turab', 'Pojkarna', 'Marte Bratberg', 'Sara Farell', 'dePresno', 'DJ Anton', 'August Dahl', 'Atle', 'Maria Celin Strisland', 'EMIR', 'Maybon', 'Ella Lockert', 'Prebz Og Dennis', 'Stein Berge', 'Bendik', 'Lxandra', 'Tayla Parx', 'Yomi', 'Bad Geckos', 'Frank Løke', 'Hanne Mjøen', 'JRL', 'Marthe Wang', 'X Daniel', 'Øystein Hegvik', 'Meso', 'Henrix Borg', 'Kung Vladimir', 'DJ NorgeJodel', 'Mina Lund', 'Sirkel Sag', 'Juur', 'Triana Iglesias', 'INAP', 'ZL-Project', 'Mad.S', 'Ola Nordmann', 'Bernt Hulsker', 'Yung Potato', 'Thea Gudmundsen', 'Petter Pilgaard', 'Snow Boyz', 'Georgia Box', 'Britz', 'Kapteinen', 'NorthKid', 'Olympis', 'J1', 'the Dodos', 'Bea', 'Victoria', 'Dimitri Vangelis', 'Wyman', 'Halloran', 'Kate', 'Edgar', 'Meland', 'Skei', 'PT', 'Mannen', 'Zino', 'Eddy', 'Norway', 'Paradise Hotel'])
set7  = set(['Kamal Khaira', 'Madhu Priya', 'Roll Rida', 'Raghav Chaitanya', 'Simran Kaur Dhadli', 'Khushank Dalal', 'Nimrat Khaira', 'Sickflip', 'Kulbir Jhinjer', 'Nirvair Pannu', 'Bhuvan Bam', 'Korala Maan', 'Tanzeel Khan', 'Sam C.S.', 'The Landers', 'Sanjith Hegde', 'Arun Dev Yadav', 'Sarath Santosh', 'Amruta Fadnavis', 'Happy Raikoti', 'Mickey J. Meyer', 'Rupinn', 'Nitin Mukesh', 'Tamir Bar', 'Orr Amrami Brockman', 'Tay G', 'Jimbo J', 'Teddy Neguse', "100 First Songs' Participants", 'Francis Duran', 'Glen Dean', 'Føx & the Hound', 'Sergio', 'Pako Ramirez', 'Young Wavee', 'דקל וקנין', 'Berg', 'Roy Kafri', 'ZENA', 'Asaf Goren', 'Ishay Levi', 'Avior Malasa', 'Yossi Shitrit', 'Gher', 'Michael Swissa', 'אבי פאנל', 'Katrix and Doron Beaton', 'Liron Amram', 'Omri 69 Segal', 'Buttering', 'Afroto', 'Ahmed Santa', 'Mido Gad', 'Angham', 'Barry', 'Ali Adora', 'Abo Lila', 'Emad Kamal', 'Karim Mohsen', 'Rain Sleepings Sounds', 'Limit 29', 'Rain Sound Studios', 'Alex Ran', 'Barachi', 'Idahams', 'Rc $pitta', 'Majid Almohandis', 'Overlord Bnjmnz', 'T-Krazie', 'Lady Sparkle', 'Dufflebag Cal', '7098', 'Ebx Music', 'ARNOLD', 'Luigui Bleand', 'Josh Milli', 'A-RealBeats', 'Dmvc Blvck', 'Shiah Maisel', 'TYTE', 'Terri Cann', 'damnboy!', 'Luca Testa', 'Assassin', 'FEYI', 'Jorge Pensi', 'Julius', 'Yedinci Ev', 'Lvbel C5', 'Karya Çandar', 'Cem Pilevneli', 'Kodes', 'Ahiyan', 'Maho G', 'Grup M. Şanlıel', 'Irem', 'Halil İbrahim Göker', 'Gökşin Derin', '90 BPM', 'Anıl Durmuş', 'Mansa Obi', 'Timuçin Esen', 'DJ LYAN', 'Medusa', 'İntizar', 'Özlem Akgüneş', 'Sibel Alaş', 'Güler Özince', 'Betül Demir', 'Atiye', 'Pentagram', 'Nigar Muharrem', 'Deniz Gürzumar', 'Asala Maleh', 'Killval', 'Mohamed AlSalim', 'Ahmed El Maslawy', 'Aburob', 'Akihiro Manabe', 'Rap AR Anime', 'Blvxb', 'Mohmood Al Ghaiath', 'Joseph Attieh', 'Humood AlKhudher', 'Real Recognize Rio', 'Lil Eazy', 'Archez', 'WESTSIDE BOOGIE', 'Lindiwe Mkhize', 'Jafar Al Ghazal', 'Haifa Wehbe', 'Konata Small', 'the Serpent'])
set10 = set(['Sergi', 'La Mosca Tse-Tse', 'Marcos Menchaca', 'Gera MX', 'Heroes Del Silencio', 'Yuawi', 'Где Фантом?', 'Kara Kross', 'SEEMEE', 'PALC', 'ElyOtto', 'Dzharakhov', 'LIDA', 'G4OUR', 'CAPTOWN', 'Smoky Mo', 'Pacosp', 'IsaacMusic', 'Gunwest', 'Ямаджи', 'Armando Hernandez', 'Ryan Roy', 'Leebrian', 'DJ Goozo', 'LosPetitFellas', 'Dejota2021', 'Luisa Fernanda W', 'Koffee el Kafetero', 'Kenai', 'Sanluis', 'Martin Elías', 'Leo Mattioli', 'Tambó Tambó', 'Felix Gabriel', 'DJ Yayo', 'La Nueva Luna', 'Wanakin', 'Joleane', 'Joe Hasin', 'Miz Lina', 'Nadia Dan', 'Aida gean', 'Karl Michelson', 'Miles Kenn', 'Kennis Inn', 'Nickl Mons', 'Adam Dank', 'Justin Tomber', 'Edik', 'Dean Loo', 'Vitya AK', 'payton', 'Mitchel', 'Фикс', 'Gocata Kangal', 'VessoU', 'Togasang', 'Havenbled', 'Jamaicataxi7', "Leron Rankin'", 'Gcollectah', 'Crystal Byrd', 'Donika', 'Riko Band', 'LZ27', 'KNUX', 'MacShawn100', 'Nikki Luchese', 'FlameShadow', 'SayReal', 'NEEL', 'Leol', 'CashnOut Esco', 'StzCorp', 'VICTORIA', 'Ran Nishihara', 'Jamiryo', 'Josh Jips', 'Leoni Jut', 'Kindler Shah', 'Venata', 'Iskrata', 'Luca Testa', 'JJ Hannon', 'TIFFANY BYNOE', 'Hones-T', '13 Crowns', 'Exiled', 'iDubbbz', 'Kyle Edwards;Chris Bevrly', 'Kyle Edwards;DJ Smallz 732', 'Angerwolf', 'Miquela', 'Cassow', 'Ian Erix', 'Chris Tyson', 'Kid3rd', 'Dylan Diamond', 'Cal Scruby', 'Plamen', 'Ricegum', 'JRY', 'Fran Rozzano', 'Dowba Montana', 'Yiyo Zarante', 'Deiby Lceja', "Leyenda Barrial Boy'z", 'Jay Menez', 'Alex R.', 'El Chuape', 'Sergio Echenique', 'J Quiles', 'Lr Ley Del Rap', 'La Manta', 'Jonzmen', 'Hey Santana', 'Nico Clinico', 'Danay Suárez', 'El Batallon', "D'Markuz", 'Chystemc', 'La Sonora De Tommy Rey', 'Grupo Alegria', 'Agrupación Marilyn', 'Ak 4:20', 'Flor De Rap', 'joel music', 'Músicxs De Chile', 'Ankhal', 'Bayronfire', 'Conjunto Alerce', 'Daniel Munoz', 'Hector Pavez', 'Los de Ramon', 'Jorge Yanez', 'Julianno Sosa', 'Eyci and Cody', 'La Combo Tortuga', 'Mym', 'Leka el Poeta', 'Jona Camacho', 'Roberto Parra', 'Los Hidalgos', 'Hermanos Morales', 'Yulien Oviedo', 'Fiesta Huasa', 'Ballet Folklórico Alma Chilena', 'Power Peralta', 'Ronald El Killa', 'Marvell Boi', 'Romeo El Santo', 'Marvel B', 'Portavoz', 'Tracy McGrady', 'Paithon', 'Atomic Rose', 'Kazzabe', 'Steven Pineda', 'J Calix', 'Nassir Chahin', 'Nio Nassar', 'JWolfy', 'Juan Oliva', 'Angel Gutiérrez', 'Rodolfo Bueso', 'Jesus Aguilar', 'Macario Mejia', 'Los Bohemios', "What's Up", 'andrei', 'Sapte', 'Mario Fresh', 'Alberto Grasu', 'Iobagg', 'Oscar', 'Roxen', 'VUNK', 'Mario Judah', 'RAVA', 'Florian Rus', 'Super Ed', 'Lino Golden', '5GANG', 'Yakki', 'Dorian Popa', 'SICKOTOY', 'ADDA', 'Stefan Hrusca', 'Deliric', 'Jay Watts', 'Lola Brooke', 'Drew Allen', 'Corey Mekell', 'Mark Stam', 'Aerozen', 'boi the creator', 'Inyeewetrust', 'Son Orioz', 'Mr. Glasses', 'Subcarpaţi', 'Speak', 'Calinacho', 'NMW', 'El Nino', 'Ali Dolla$', 'TRISHES', 'Olivia Sabates', 'Doddy', 'Gloria Prince', 'P.M.', "Beau'Joli", 'Lovel', 'BAZOOKA', 'Chris Bones', 'DOC', 'Real Imaj', 'Knack', 'NxtSkool', 'Omar Courtz', 'Pitizion', 'Bebo Yau', 'Tres Dedos', 'Arevalo', 'Durako', 'Bombotunes', 'La Máquina Camaleón', 'Dennis Fernando', 'Navigator', 'Eshconinco', 'DeStorm', 'Dj Coss', 'Dancehall Word', 'Blaiz Fayah', 'DJ Kendo', 'GIMARIO', 'Alex Badilla', 'Pranz', 'Fátima Pinto', 'MAYKEL', 'SpecialBeatz', 'Porsche Boy', 'Frayer Flexking', 'Momo', 'Grey256', 'Fobia Kid', 'AstralKid22', 'Martin Agh', 'Duch', 'Pavol Habera', 'Lubomír Lipský', 'Dominika Mirgova', 'Janek Ledecky', 'Marie Rottrova', 'Robo Opatovsky', 'Luca Brassi10x', 'Cypis', 'ONEMANSHOW Foundation', 'Čis T', 'Adam Ďurica', 'LUISA', 'Radikal Chef', 'Šorty', 'KHANS', 'Milion Plus', 'Kruel', 'Redzed', 'Tina', 'Loko Loko', 'Falusko', 'Miko Floso', 'Taomi', 'Alan Murin', 'Nicholas Naison', 'Samuel Kim', 'Rida Radar', 'Mairee', 'Marián Kochanský', 'Simon Smith', 'Peter Simon', 'Naflexx', 'TKX', 'Slimso', 'Annet X', 'Ultrazvuk', 'For You Acapella', 'P.A.T.', 'DJ Rusty', 'Supa', 'Trifecta', 'Lukas Adamec', 'LYSA', 'Peter Pann', 'Deyvis Orosco', 'Cielo Torres', 'Maricarmen Marin', 'Armonia 10', 'Claveles de la Cumbia', 'Faraón Love Shady', 'Combinacion De La Habana', 'Temple Sour', 'Orquesta Caña Brava', 'Orquesta Bembé', 'Nicole Favre', 'Anna Carina', 'Susan Ochoa', 'Reggaeton bachata Hit', 'Dani Darcourt', 'VICE MENTA', 'Hnos Yaipen', 'DJ Krlos Berrospi', 'Orquesta Caribeños de Guadalupe', 'Marco Romero', 'DJ Luigi', "Pelo D'Ambrosio", 'Cruz', 'Mario Hart', 'Av. Larco', 'Orquesta Zaperoko del Callao', 'Korina Rivadeneira', 'BCA', 'Dubosky', 'El Codigo Kirkao', 'Robinho', 'Towers Mix', 'Señor Loop', 'Bobo', "At' Fat", 'Yemil', 'Kabliz', 'El Kid', 'Sofía Valdés', 'Chamaco', 'El Zeta', 'MOI MUSIC', 'Mista Bombo', 'Fulo El Yeyo', 'T.O.T', 'Robi Guid', 'El Boza', 'BARBEL', 'Jhon el Diver', 'Amindi', 'Los Topsys', 'Original Fat', 'Azhika', 'Rawayana', 'Real Phantom', 'Saulės Kliošas', '7th Block', 'Šventinis bankuchenas', 'Jessica Shy', 'Furytto', 'Keanu Blunt', 'Angelou', '2Kvėpavimas', 'Velvet', 'GJan', '335d', 'Flying Saucer Gang', 'Zoliukas', 'Solo Ansamblis', 'ilyTOMMY', 'Gebrasy', 'wavytunesz', 'Gabrielius Vagelis', 'Daddy Was a Milkman', 'Evgenya Redko', 'Eyedress', 'Beatrich', "Radistai Dj's", 'Monique', 'QV Trap', 'Shouse', 'Jovani', 'GAZIROVKA', 'Ieva Zasimauskaitė', '8 Kambarys', 'Alekna', 'Saulius Prūsaitis', 'Flash Voyage', 'SADBOI', 'Free Finga', 'Vidas Bareikis', 'Vaidas Baumila', 'Mark Les', 'senjasa', 'AREZRA', 'Kamaniu Silelis', 'jautì', 'Swertas', 'OSVISS', 'Abudu', 'NKSN', 'Kriminell', 'Gabrielė Vilkickytė', 'Monika LIU', 'MITCH RECORDS', 'Tinlicker', 'Monika Linkyte', 'On1Beat', 'Junior A', 'ba.', 'Мэвл', 'Audioiko', 'Rūta Loop', 'Monika Marija', 'MEANDI', '3-ий Январь', 'Radvilas RA', 'MR.Nuclear', 'Golden Parazyth', 'Iglė', 'zalagasper', 'Jurijus', 'Karolina Meschino', 'Carousel', 'ZENA', 'Eliot', 'S!sters', 'Srbuk', 'Michael Rice', 'Sarah McTernan', 'Ester Peony', 'Tulia', 'Garbanotas', 'Dog Collective', 'Lea Sirk', 'SuRie', 'Ieva ZasimauskaitÄ—', 'Laura Rizzotto', 'Eugent Bushpepa', 'DoReDoS', 'Hiperbole', 'Baltasis Kiras', 'Dihaj', 'IMRI', 'Hovig', 'Triana Park', 'Jana Burčeska', 'Timebelle', 'Artsvik', 'Kasia Moś', 'NAVIBAND', 'Tijana Bogicevic', 'Slavko Kalezić', 'Norma John', 'The View', 'Tim Legend', 'Garotihnio', 'Mariano Bermudez', 'Cuatro Pesos de Propina', 'Sonido Caracol', 'Karibe con K', 'Once Tiros', 'Chacho Ramos', 'Los Pikantes', 'Los Negroni', 'Buitres', 'Jotape', 'La Sandonga', 'Valeria Gau', 'Marcos da costa', 'Gian', 'Zeballos', 'Américo Young', 'Zanto', 'Blonda', 'CB Records Music', 'Arrancandonga', 'Miway', 'Edu Lombardo', 'Canario Luna', 'Mart', 'Sonido Cristal', 'Lafame', 'Sonido De La Costa', 'Urband', 'The Panas', 'Dessia el Otro', 'Halconizados de Amor', 'Corta La Bocha', 'Estrategas', 'DeGira', 'Tu Papa', 'Enero', 'RC BAND', 'Mala Tuya', 'La Mano Rancia', 'Los Verduleros', 'Cumbia Juan', 'Chili Fernandez', 'Japiaguar', 'Talento de Barrio', 'Noa - Noa', 'Escucha!', 'Samuray', 'Tropical Caribe De Los Hnos Loera', 'Los inquietos del vallenato', 'Pipa Para Tabaco', 'Tropical Caribe', 'La De Roberto', 'LaNuestra', 'Lenuel', 'Bryant Mayers', 'Joshua Dietrich', 'MC LEO', 'Lucas Cliff', 'Tierra Adentro', 'Afaz Natural', 'BB Nobre', 'Reggaetones', 'Villagrán Bolaños', 'Em Rossi', 'Drew Cole', 'Dj Maya', 'Manifiesto Urbano', 'Carin', 'StephenFlint24', 'ScottHall41', 'JulieHarder67', 'CarolAnderson88', 'MariaBrewer62', 'CecilDowns84', 'IsisWampler68', 'Temry', 'CynthiaGorecki58', 'S I', 'Sean Matsukawa', 'Midnight Office', 'D.K. Smith', 'Destripando la Historia', 'Akil Hamilton', 'Michele 69', 'Alan 1', 'Alan 2', 'Tuan Alto Sax', 'Doris Schmeler', 'Dorothea Berger', 'Adonis 88', 'TINA LTT', 'Adonis 66', 'Lucy Beahan', 'Veronica Effertz', 'Joseph Murray', 'SarahStiltner', 'Eula Glover', 'HenryNorrell', 'JamesPonder', 'TerryFerrell', 'Slender2003', 'The Face 1234', 'Indifferent2527', 'Lucky2135', 'Coldly1989', 'Vuvanhy1985', 'Wistful2010', 'Happy9282', 'Silence2001', 'Fragile1988', 'Comfortable5343', 'TonyJones', 'Magnolia1982', 'Sunset2002', 'Marty McKay', 'CaryLamb', 'RonTurner', 'JeffreyLewis', 'Weldon S. Kesterson', 'EdithFernandez', 'DianeKennedy', 'Michele 99', 'Doris 68', 'Harold 69', 'Harold 68', 'Doris 666', 'Haring 68', 'Dong Hung', 'Aria Ohlsson', 'DorisAlvarez', 'JillReece', 'PaulaSanchez', 'ChadCaver', 'AndreaRuby', 'VeraQuintana', 'GeorgeGoin', 'DanaHuntley', 'BennieNolen', 'LindaCourtney', 'MarySchroeter', 'SusanMiller', 'JanieReaves', 'SusanCarter', 'RickSpencer', 'PennyNakamura', 'LouChamberlain', 'micem', 'lil waii', 'NorsePrez', 'Chika Toro', 'YT', 'Anne Dorko', 'Vashoun Stjon', 'Talley', 'Catswithnicefaces', 'King V', 'Streetza Ria', '$hea Butta', 'Phillydark1', 'Xciryl', 'Legend Russell', 'Trilltypefella', 'Philip Anthony', 'Rockin Chefs', 'Tim Hustle', 'Tiny Tree', 'Tom. G', 'Michael Sterling', 'Flood District', 'Saviour3.0', 'OGK John', 'JOSEPH CHASE', 'His Kings', 'ThereGoJo', 'The ocn', 'SIXN', 'D14', 'Moonlight Young', 'Mizzery', 'Jeremy Voltz', 'Keith Lawsen', 'Thomas Deil', 'Samylon92', 'Melanie Weichl', 'Splash Danero', 'Decayer', 'Inas X', 'Mahkenna', 'Eastside Jody', 'Tripple 2', 'Raja Rose', 'Dragana Kosic', 'Christos Mavridis', 'Duo Guardabarranco', 'Zoragoza', 'Top Mixer Dj', 'Perrozompopo', 'Julio Melgar', 'Tayl G', 'Viento En Contra', 'Aliento', 'Los Abejorros', 'Danny Marin', 'be vis', 'Maciej Zielinski', 'Olszakumpel', 'Julia Rocka', 'Josef Bratan', 'Kamil Bednarek', '2K', 'Ania Dabrowska', 'Piotr Ziola', 'Popkiller Młode Wilki', 'Męskie Granie Orkiestra', 'Mela Koteluk', 'Daj To Głośniej', 'Muniek Staszczyk', 'DMN', 'Mlodyskiny', 'Syberia Tdw', 'Eis', 'Lanberry', 'Szymi Szyms', 'Kacper HTA', 'Jacuś', 'Szczyl', 'Julia Wieniawa', 'TUZZA Globale', 'Donguralesko', 'Fresh N Dope', 'WIP Bros', 'QueQuality', 'Marcin Maciejczak', 'Kacper Blonsky', 'Jetlagz', 'Stanisław Soyka', 'Edyta Gorniak', 'Katarzyna Cerekwicka', 'TGD', 'Roksana Węgiel', 'Young Slash', 'Lipa', 'Kamerzysta', 'Jacek Stachursky', 'Fisz Emade Tworzywo', 'Grzegorz Hyzy', 'Magda Steczkowska', 'Mate.O', 'Roan', 'Pamela Stone', 'Agnieszka Chylinska', 'Dj.Frodo', 'Podulka', 'Xxanaxx', 'Męskie Granie Orkiestra 2017', 'SexMasterka', 'LASERY', 'G-Girls', 'Aniceto Molina', 'Los Hermanos Flores', 'Orquesta Internacional Hermanos Flores', 'Comando Tiburon', 'Cumbias Viejitas', 'La Divaza', 'Planeet', 'Andrei Zevakin', 'Bru-C', 'Jüri Pootsmann', 'Grete Paia', 'Maian', 'NOËP', 'Heleza', 'Genka', 'marp$', 'KiROT', 'Raul Ojamaa', 'Jaagup Tuisk', 'kaw', 'Siimi', '372kaspar', 'WATEVA', 'Terminaator', 'Beatrice', 'Maxtract', 'Cityflash', 'Lemon Fight', 'Arop', 'Vajé', 'Leslie Da Bass', 'Öed', 'Sip€lga 14', 'Doc Ketamer', 'okeiokei', 'DJ Koze J', 'Red Tan', 'Paul Ereck', 'Emy Blu', 'Lumevärv', 'KOIT TOOME', 'silverrari', 'fenkii', 'Merilin Mälk', 'Merlyn Uusküla', 'Jozels', 'säm', 'Eleryn Tiit', 'Mr. Happyman', 'Ines', 'Uku Suviste', 'Cdeep', 'Frankie Animal', 'Egert Milder', 'Anett', 'STEFAN', 'INGER', 'Uudo Sepp', 'PSL', 'I AM THE STORM', 'Signs', 'Göki', 'Asena', 'Kerli Kivilaan', 'Xtra Basic', 'Kadiah', 'Sissi', 'Lacy Jay', 'Johanna Eendra', 'Daniel Levi', 'Indrek Ventmann', 'Metsakutsu', 'Miljardid', 'Karl-Kristjan', 'Iiris', 'Hoax', 'Whogaux', 'Angeelia', 'AZMA', 'Cruz Santa', 'CXB', 'LindaTirado', 'SummerFoster', 'Jason William', 'LauGadoi', 'EmilyZimmer', 'Grandha', 'Juan Solo', 'Mayte Flores', 'Luis Vega', 'Diego Merritt', '$ourJ', 'Singapūras Satīns', 'ansis', 'Zigfrīds Muktupāvels', 'Grafomaans', 'Intars Busulis', 'Prusax', 'rolands če', 'Bermudu Divstūris', 'OLAS', 'GACHO', 'Mesa', 'New Wavy', 'Sudden Lights', 'Aija Andrejeva', 'VIŅA', 'Kreisais Krasts', 'Adler Kotsba', 'Livi', 'Putnu Balle', 'Ingus Ulmanis', 'Bet Bet', 'Musiqq', 'Lauris Reiniks', 'Zvaigžņu koris', 'Projekts “Tev tuvumā”', 'Eināra Verro Vokālā Grupa', 'Otra Puse', 'Tev tuvumā', 'Fomins', 'Arturs Skrastins', 'Kiol', 'MBR', 'Emil', 'Forrest Hill', 'morgxn', 'The Key Rocks', 'Kenedi', 'Post Ironic State', 'Steven Rosvell', 'Dior', 'Perkons', 'Apvedceļš', 'Menuets', 'IGO', 'Jātnieki', 'Bruģis', 'Latgalīšu Reps', 'Ferrum', 'Hospitāļu iela', 'Jumis', "Astro'n'out", 'Autobuss debesis', 'Dālderi', 'Metronoms', 'Reinis Sējāns', 'Ainars Mielavs', 'Mart Inc.', 'Kastrācija', 'Sharlot', 'Samanta Tina', 'Trench Mafia Locco', 'Āris un Saulceris Ziemeļi', 'Luda', 'Lil Karlix', 'Edavārdi', 'Simon Riemann', 'London Richards', 'Matt B', 'Kaio Kane', '7eit', 'Fakts', 'MADARA', 'Markus Riva', 'Symon', 'Young DrA', 'Monsieur Job', 'MaWayy', 'Gotti Mirano', 'Nick Hissom', 'Los Profetas', 'Trashock', '24Heavy', 'Jvnior', 'And Then Came Fall', 'Ladotee', 'OFMB', 'Big Sim', 'Brian Ligon', 'Tinny Mafia', 'SAINTS', 'Danny Blue and the Old Socks', 'Hypothetical', 'BowlLane Slick', 'Najuah', 'DJ Smallz 732', 'Mariño', 'Lexz Pryde', 'Rod Maine', 'Tony Chevy', 'Ashley Lovechild', 'Stix', 'The Ludvig', 'Santa Danelevica', 'Miks Dukurs', 'Toms Kalderauskis', 'L.V. Lauris Valters', 'Crime Sea', 'Miks Galvanovskis', 'Ziemassvētku zvaigžņu koris', 'Su Orquesta', 'Su Apollo Sound', 'R.K.M', 'los Fundamentalistas del Aire Acondicionado', 'Waor', 'Natos', 'SAMMY', 'LESEN', 'Ivo', 'Peeva', 'Yomel', 'Silvia Infantas', 'Suro Solar', 'Camila Recabarren', 'Dash', 'Cangri', 'Urba', 'Don Medardo', 'Sus Players', 'Jickson', 'Sheen', 'Halloran', 'Kate', 'Josimar', 'Su Yambú', 'JOSIMAR', 'SU YAMBU', 'Samy', 'Beissoul', 'Einius', 'G', 'G Sindikatas', 'El Gucci', 'Su Banda', 'Tom', 'Collins', 'El High', 'Dayme', 'Néstor Ló', 'Feer', 'Lees', 'LEILAH SAFKA', 'FREDDY FENDER', 'Rafał Borzozowski', 'Małgorzata Burdzel', 'Sound`N`Grace', 'RL9', 'La Luz Roja De San Marcos', 'Mäx', 'Clicherik', 'Küberünnak', 'Karmo', 'The Sun', 'Andris Erglis', 'I MEAN LOVE', 'chris nov', 'CLYDE', 'Cherry'])
set11 = set(['04 Limited Sazabys', 'SPiCYSOL', 'hachi', 'ACE COLLECTION', '稲村オーケストラ', 'PRODUCE 101 JAPAN', 'ヒプノシスマイク -A.R.B- (Division All Stars)', 'Whiteberry', 'CHIAKI SATO', 'AYA A.K.A PANDA', 'THE ORAL CIGARETTES', '浜田雅功と槇原敬之', 'Hypnosis Mic -D.R.B.- (MAD TRIGGER CREW)', 'Kinokoteikoku', "The Urashima's", 'Fumiya Fujii', 'Mone Kamishiraishi', 'Takehara Pistol', 'Ovall', 'Straightener', 'DOBERMAN INFINITY', '吳林峰', 'Panther Chan', 'Cath Wong', 'Yan Ting', 'Edan 呂爵安', 'my little airport', '石山街', 'Adason Lo', 'Thomas DGX YHL', '癲佬 dlo', 'Akano', 'YNW Jordan', 'Floating in Space - Project', 'Big Loso', 'Windy Guai', 'DNA', 'PBE PLUTO', 'Jt el Utility', 'Damián V', 'Sirvan Khosravi', '井谷俊二', 'MariKuroso', 'Mukaybin Mukayze', 'King Gunna', 'Zoee8', 'Bogart Bonales', 'Dennis Fernando', 'Friday Santana', 'Harj Savage', 'bruno livesax', 'JSnake', 'Keyzs-Sean Sparsfa', 'From Under Concrete Kings', 'ED LIIT', 'FAO>dino', 'Buunkin', 'Los Necesarios', 'TG', 'Phantom Caine', 'Molina Molina', 'Thee JAE', 'Kiian', 'DeAnna Stealth', 'Luca Natale', 'Marli Space', 'Torito', 'K Beezy', 'Rachel Mason', 'Abrina', 'LIL BLU£PRINT', 'Kamar Tachio', 'Juke', 'Bradley Mantha', 'Nuski Baby', 'Idahams', 'Duane Flames', 'J.Day', 'Grupo Okixem', 'Srene', 'RealDeal Berns', '3NIGMA', 'King Thateron', 'Said P.', 'ONSC', 'Levi Belle', 'Orphée La Fine', "Niki's Project", 'The Greywolf', 'Vaolight', 'MM', 'Lipford', 'Anzo Weiseil', "YB D'nero", 'Geek-TheHot', 'Cade Eliason', 'Acezz', 'Dario Puleo', 'Musas', 'Frank Briones', 'Moniris', 'Jalayan', 'Ghost Chili', 'JC the Don', 'Mark$man', 'Dsean', 'ViibesFromNetto', 'Fantasy', 'Hanujay', 'Poko', 'Nick Nittoli', 'DON Kilam', 'Michael Hodges', 'Maxim Schunk', 'T. McCrae', 'Preston Hudman', 'MASON', 'Hamah Music', 'zelinen', 'Big K. The Komik', 'Roma Vander', 'Aquarius Jones', 'Lil Trizzyverse', 'Profound Artist', 'Preston Hudman TG', 'SV Stallone', 'Sacal', 'Taylor Cash', 'Lexz Pryde', 'Malik Vadheim', 'Denver Grey', 'Sparksi', 'Hirotoshi Furuya', 'JR', 'Mike Plain', 'Mawat', 'Big Wade', 'Richi Drag', 'John Daniels Band', 'Shannon K', 'BiskBoy', 'Kram', 'Darren Michael Boyd', 'Juandisemo', 'Blohk$r33fa', 'The Notorious Rio', 'No More Vips Allowed', 'Mote Dogg', 'Lil Lee', 'Def Rock', 'Atômico Mc', 'Yady', 'Hk6od6ratt 44', 'Icy Sabbath', 'COSMO.', "Lai'jhon Hawkins", '徐天佑', 'Axel Thesleff', '曾樂彤', 'Datmaniac', 'Oda Kirkwood', 'Mayar Jenisk', 'Missing Seoul', 'WAVYVIBE', 'Chan Se Park', 'Hoang Muoi', 'Quang Long', 'Mr.Siro', 'NHD media', 'Khoi My', 'Hong Nhung', 'Chi Dan', 'Wanbi Tuan Anh', 'Phuong Dung', 'Fay', 'JSOL', 'DatKaa', 'T.R.I', 'Khói', 'Xuân Nghi', 'Phạm Toàn Thắng', 'JINU', 'Thinh Suy', 'Thanh Hung', 'S.T Sơn Thạch', 'Ai Phuong', 'DEAN', 'Linh Cáo', 'JayKii', 'Phạm Anh Duy', 'itsnk', 'Hoà Minzy', 'Hoang Minh Tuan', 'VA', 'Huu Khuong', 'Nhu Quynh', 'Luu Anh Loan', '菲道尔', 'GJ', '卓義峯', '曾博恩', 'Fang Wu', '唐禹哲', '劉以豪', 'TAEMIN', 'Trouze', '颜人中', 'Hua Chen Yu', 'ABAO', '莫宰羊', 'Sharon Kwan', 'Cindy Yen', 'PiA', 'Ruth Kueo', 'KUN', '采子', 'Zooey Wonder', '?te', '鄭興', '鍾明軒', '李佳歡', 'Enno Cheng', 'Barry Chen', '王貳浪', '老莫 ILL MO', 'SEJEONG', '謝博安', 'Trash', '王俊凱', 'Chyi Chin', 'Zhang Zhen Yue', '康樹龍', 'Evan Yo', '路嘉欣', 'AJ Mitchell', '陳雪燃', 'Ella Chen', 'TroutFresh', 'Queen Wei', '邵雨薇', 'Da Zhuang', '黃奕儒Ezu', '187INC', 'Fran', 'Meeia Foo', '安溥 anpu', '呂薔Amuyi', 'Highlight', '陶晶瑩', 'Christine Fan', 'Dongye Song', 'HUSH', '柯智棠', '陳珮茹', 'C.T.O', '楊培安', '肖懿航', 'Anni Hung', 'Crispy脆樂團', 'Sam Lee', 'MXM', '謝金燕', '林生祥', '馬雲', 'Yuzi Qin', 'YOONA', 'Suming', '陳惠婷', 'Deserts Chang', 'Soo Wincci', '小S', 'IT’S', 'Bolly', '老鬼團', 'Xian Zi', 'Twinko', '鄭興琦', 'Rain Lee', '簡弘亦', 'Tien Fu Chen', '嚴藝丹', '殷悅', 'SEOHYUN', 'Danupol Kaewkarn', 'THE RUBE', 'พีธ พีระ', 'DarkFace', 'ลำเพลิน วงศกร', 'VKL', 'Boy Sompob', "Yes'sir Days", 'ตุลานันท์ นรเศรษฐ์พิศาล', 'อ๊อฟ ชัยนนท์', 'ONEONE', 'Gx2', 'Valentina Ploy', 'Jay Phitiwat', 'ALREADY DEADD', 'ปรีชา ปัดภัย', 'Sweet Mullet', 'Crayon', 'Daily Vitamin', 'Napat Injaiuea', 'เด็กเลี้ยงควาย', 'TOSSAKAN', 'Jus2', 'MuchMellow', 'Gyeon', 'GT', 'Catherine', 'Kindler Shah', 'gugudan', 'เทิดศักดิ์ จันทร์ปาน', 'Aeh Syndrome', 'SummerWish', 'Memory Fabricators', 'น้อย พรู', 'T-BIGGEST', 'Yang Da Il', 'นนน กรภัทร์', 'Ruj Suparuj', 'Chaleeda', 'ไอซ์ ปรีชญา', 'The 38 Years Ago', 'PETITE', 'Tanatat Chaiyaat', 'Sukrit Wisetkaew', 'Alfred Sim', 'Robert Philip', '劉瑞琦', 'ATEEZ', '魏奇奇', 'Ashland Are', 'Mega Guar', 'Festi Baal', 'ABANGSAPAU', 'Benjamin Kheng', '曲肖冰', 'TheSmartLocal', 'Ayo97', 'Yung Raja', 'RRILEY', 'BIGDBANGLA', 'Fariz Jabba', 'MYRNE', 'brb.', 'Ffion', 'Rave Republic', 'Katherine Ho', 'Sezairi', 'Venessa Fernandez', 'Haneri', '小潘潘', 'Luqman Podolski', 'Neeta', 'CHA NI (SF9)', 'Astro 全体艺人', 'Harith Zazman', 'Najwa Latif', 'Irfan Haris', 'Tajul', "As'ad Motawh", 'Atmosfera', 'Putera Band', 'Sufi Rashid', 'Zizan Razak', 'Hez Hazmi', 'Dadilia Band', 'Aizat Amdan', 'NYK', 'Hyper Act', 'XPOSE', 'Achey', 'Yazid Izaham', 'Ayda Jebat', 'Aman Aziz', 'Ben Ladin', 'Amylea', 'Wani', 'Benzooloo', 'TreeHill', 'Sheryl Shazwanie', 'Isma Sane', 'Safiey Illias', 'Fieya Julia', 'Azarra Band', 'Eza Edmond', 'Adira Suhaimi', 'ALYPH', 'Talitha.', 'DOLLA', 'Ippo Hafiz', 'Shila Amzah', 'Datuk Sharifah Aini', 'Puan Sri Saloma', "Dato' Sudirman", 'Datuk Ahmad Jais', 'Noorkumalasari', 'Junainah', 'Tan Sri P. Ramlee', 'Aman Shah', 'Nyanyian Ramai', 'Azlina Aziz', 'Fazidah Joned', 'Sanisah Huri', 'Cenderawasih', 'Rosemaria', 'Aishah', 'Ning Baizura', 'GARD WUZGUT', 'The New Boyz', 'De Fam', 'Harry', 'Drama Band', 'STARLYTE', 'Dinda Permata', 'Kmy Kmo', 'Han Byul', 'Midnight Fusic', 'Kuizz', 'lost spaces', 'DAPHNE D調', 'Warner Music Malaysia All Star', 'Alextbh', 'Spuds', 'Julia Duclos', '邱诗凌', 'Fazura', 'Masdo', 'Santesh', 'Cherpen Band', 'Fara Hezel', 'Shaa', 'Fattah Amin', 'Khairil Johari Johar', 'Gamma1', 'Ezad Lazim', 'Harris Baba', 'Laudya Cynthia Bella', 'Black', 'Sakura Band', 'yui FLOWER FLOWER', 'ミゾベリョウ odol', 't', 'H Jungle', 'STEADY', 'CO.', 'The GoGo Tuner Family', 'Mike Mostert', 'Creme', 'Caf', 'E', 'SUPER JUNIOR-D', 'Ellina', 'The Majistret', 'Megat'])
set12 = set(['DJ Stokie', 'Bok Van Blerk', 'Dj Obza', 'Blaqnick', 'Emtee', 'DJ Jaivane', 'Vhudie', 'Mthunzi', 'Loufi', 'ANATII', 'K.O', 'David Scott', 'ILuvJupiter', 'Busiswa', 'GeeSixFive', 'Costa Titch', "Soweto's Finest", 'Yanga Chief', 'Shane Eagle', 'Nija', 'Stillaround', 'Salatiel', 'Gemini Major', 'Midnight Meetings', 'ABS-CBN Music All Star', 'Jana Garcia', 'Brian Pepito', 'Geo Ong', 'Reese Lansangan', 'Amber Leigh Irish', 'Andrea Babierra', 'Kiana V', 'Hatemost', 'ItsOnlySkillz', 'TiÃ«sto', 'Ofili', 'Popayedi', 'Vercetti', 'Natassa Bofiliou', 'Ripen', 'Ethismos', 'Alcatrash', 'Konnie Metaxa', 'Young Tulio$', '12os Pithikos', 'Rio', 'Athliens', 'Kanon', 'Arva', 'Pournaras', 'Eaze', 'Natasha Kay', 'Smuggler', 'Joker Two-Face', 'ZENA', 'Eisvoleas', 'Efta', 'Panos Mouzourakis', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'Kindler Shah', 'Trifecta', 'JJ Hannon', 'Vangelis Kakouriotis', 'LYSA', 'Hovig', 'Dihaj', 'Madstreet', 'Khifnu', 'CHA NI (SF9)', 'Dhika Fawaz', 'YUJU', 'Keisya Levronka', 'Cannabis Club ATL', 'Karina Christy', 'Adrian Khalif', 'Maria Simorangkir', 'Aldo Longa', 'Ustad Jefri Al Buchori', 'Cassandra', 'Syakir Daulay', 'Acha Septriasa', 'Soundwave', 'Arnon', 'Ari Lasso dan Bunga Citra Lestari', 'Cuco', 'Crayon', 'YoWis Ben', 'Ghea Indrawari', 'Sandrina', 'Ramengvrl', 'Armand Maulana', 'Gisel', 'Dash Uciha', 'Aliando Syarief', 'Adam Mitchell', 'Kemal Palevi', 'Salshabilla', 'Inkonnu', 'Snor', 'Tagne', 'L7or', 'ElGrandeToto', 'Abdeelgha4', 'Smallx', 'Foufa Torino', 'Anys', 'Lsan L7or', 'kouz1', 'Tawsen', 'Cravata', 'ISSAM', 'Dada', 'Mok Saib', 'Djalil Palermo', 'Draganov', 'Mocci', 'Lbenj', 'Didine Canon 16', 'LFERDA', 'Hatim Ammor', 'Tiiwtiiw', 'Mehdi Mozayine', 'Madd', '7-Toun', 'Didou Parisien', 'OUENZA', 'Fat Mizzo', 'Dollypran', '21 Tach', 'Furelise', "L'morphine", 'Salma Rachid', 'Adil Assil', 'Moro', 'Mrabet', 'BrownY Original', 'Cheb Wahid', 'Loun', 'La Releve Maghreb', 'Lembawe9', 'Dioscures', 'SecondT3', 'Ayoub Africano', 'Anas', '7liwa', 'Cheb', 'Naar', 'DYSTINCT', 'Zakaria Ghafouli', 'Cheb Bello', 'Cheba Souad', 'Sanfara', 'Nouamane Belaiachi', 'Ihab Amir', 'Shayfeen', 'Qaayel', 'Cyborg', 'DJ Adel', 'MiniByte', 'Bouchra', 'Cheb Houssem', 'Ily', 'Jaylann', 'Aminux', 'Fnaire', 'Hind Ziadi', 'Aezaddy', 'Flenn', 'Shobee', 'A.L.A', 'Faycel Sghir', 'Cheb Akil', 'Liamsi', 'Drizzy', 'Ayoub Hamdaoui', 'Profit Za3im', 'Raste', 'Khtek', 'Yanso', 'Hablo', 'Mohamed Adly', 'Asmae', 'Skaymen', 'Tflow', 'Ali Ssamid', 'JenJoon', 'Barry', 'Zaynab', 'Nizoo', 'Cheb Djalil', 'Flipperachi', 'Master Sina', 'Cheb Bachir', 'Klay BBJ', 'Si Lemhaf', 'Artmasta', 'Cheba Warda', 'Manel Hadli', 'Hamouda', 'Dounia', 'H-Kayne', 'Faycal Mignon', 'Mido Belahbib', 'Psychoqueen', 'Youssef Aoutoul', 'Narco Polo', '7-toun', 'Bilal Sghir', 'Mo Temsamani', 'Anas Guzman', 'Imad Benaomar', 'Bad Flow', 'Ibtissam Tiskat', 'Adel Sweezy', 'Mr. Crazy', 'The S', 'Luigui Bleand', 'Chemsou Freeklane', 'Nizar Idil', 'Barachi', 'Spleux', 'Bilal Africano', 'Limit 29', 'Safae Hbirkou', 'Fayçal Mignon', 'Tortoz', 'Echbiy', 'Ghazir', 'Redone Berhil', 'D14', 'micem', 'lil waii', 'Kobb', 'Mathu', 'Hichem Smati', 'Savage Plug', 'Cheb Abbes', "Moov'art", 'Halloran', 'Kate'])
set13 = set(['Biu do Piseiro', 'Nêgo Jhá', 'OIK', 'Camila Loures', 'DJ Pernambuco', 'Anne', 'Rafael Allmark', 'Maycon e Vinicius', 'Thiago Carvalho', 'Fernanda Salgado', 'Thascya', 'Gabriel Elias', 'Furacao Love', 'Seakret', 'Junior Lord', 'Yuri NR5', 'Greg Ferreira', 'Fumaxa', 'MOBBERS', 'Gama WNTD', 'Yung SpaceGhost', 'WUANT', 'xtinto', 'SYRO', 'Instinto 26', 'Kappa Jotta', 'WiNDOH', 'Aragão', 'Mizzy Miles', 'Gson', 'Timor YSF', 'Achero', "Dino d'Santiago", 'Lhast', 'LON3R JOHNY', 'Irina Barros', 'Minguito 283', 'Dream Boyz', 'Betsy David', 'Waze', 'Valas', 'Domi', 'Mastiksoul', 'João Sousa', 'Miguel Alves', 'Apollo G', 'ÁTOA', 'sora9k', 'Joint One', 'Vado Más Ki Ás', 'Chris Tamayo', 'ARTIGAZZ', 'Jvst Fly', 'GROGNation', 'Mota JR', 'YUZI', 'prettieboy johnson', 'Huzz', 'Ray Polanco', 'Sea', 'Kazzio', 'Ella Nor', 'ANDREZO', 'Mundo Segundo', 'Zara G', 'April Ivy', 'Wuant', 'Carlão', 'HMB', 'Yasmine', 'Ray', 'Maria Cecília', 'Xutos', 'Pontapés'])
setP  = set1.union(set2).union(set3).union(set4).union(set5).union(set6).union(set7).union(set10).union(set11).union(set12).union(set13)
print('set1  = ',len(set1))
print('set2  = ',len(set2))
print('set3  = ',len(set3))
print('set4  = ',len(set4))
print('set5  = ',len(set5))
print('set6  = ',len(set6))
print('set7  = ',len(set7))
print('set10 = ',len(set10))
print('set11 = ',len(set11))
print('set12 = ',len(set12))
print('set13 = ',len(set13))
print('setP  = ',len(setP))

set14=set()

print('set14 = ',len(set14))
print('set14 & setP = ',len(set14.intersection(setP)))
print('set14 - setP = ',len(set14.difference(setP)))
toget=list(set14.difference(setP))
print(toget)

In [ ]:
set1 = set(['Dan Quinn', 'George J. Gaskin', 'Jere Mahoney', 'Vess Ossman', 'The Haydn Quartet', 'S. H. Dudley', 'The Georgia Minstrel Co.', 'Will Denny', 'Mina Hickman', "The Gilmore's Band", 'Silas Leachman', 'Jules Levy', 'William Redmond', 'J. Aldrich Libbey', 'Bob Roberts', 'William H. Thompson', 'George Alexander', 'The Columbia Orchestra', 'Harry Tally', 'Richard Jose', 'The Peerless Quartet', 'Corrine Morgan', 'Frank Stanley', 'Lew Dockstader', 'Grace Nelson', 'DeWolf Hopper', 'James McCool', 'Alan Turner', 'The Victor Orchestra', 'May Irwin', 'The Columbia Comedy Trio', 'Frederic Rose', 'Helene Noldi', 'Nat Wills', 'Harold Jarvis', 'Ralph Herz', 'The Victor Light Opera Co.', 'The American Quartet', 'Arthur Clough', 'Paul Southe', 'Kitty Cheatham', 'Joe Maxwell', 'The Fisk University Jubilee Quartet', 'The Lyric Quartet', 'The Heidelberg Quintet', 'Marguerite Dunlap', 'Guido Deiro', 'Gene Greene', 'Dolly Connolly', 'The New York Military Band', 'Andrea Sarto', 'The Victor Concert Orchestra', 'John Barnes Wells', 'Elsie Baker', "The Conway's Band", 'The Columbia Mixed Quartet', 'Charles Harrison', 'Chauncey Olcott', 'Helen Clark and Walter Van Brunt', 'William J. Halley', 'Joseph Cawthorn', 'Elsie Janis', 'George MacFarlane', 'Joe Hayman', 'Owen McCormack', 'The Columbia Stellar Quartet', 'The Orpheus Quartet', 'Grace Kerns and Reed Miller', "McKee's Orchestra", 'Grace Kerns and John Barnes Wells', 'The Victor Mixed Chorus', 'Charles Kellogg', 'Taylor Trio', 'The Venetian Trio', 'Frank Tinney', "George O'Connor", 'Helen Clark and Joseph Phillips', 'George Wilton Ballard', 'Anna Chandler', 'The Columbia Mixed Double Quartet', 'Marguerite Farrell', 'The Sterling Trio', 'The Imperial Quartet of Chicago', 'Ernest Ball', 'Louis Gravieure', "The Jaudas' Society Orchestra", "M.J. O'Connell", 'Kathleen Parlow', 'Hipolito Lazaro', 'Reginald McAll', 'Horace Wright', "Earl Fuller's Novelty Orchestra", 'The Shannon Four', 'The Knickerbocker Quartet', 'Anna Wheaton', 'Lucy Gates', 'The Neapolitan Trio', 'The Cincinnati Symphony Orchestra', 'The Avon Comedy Four', 'Charles Hart', 'The Farber Sisters', 'Percy Hemus', 'Harry Fox', 'Harry Ellis', 'Julian Rose', "W.C. Handy's Orchestra", 'Helen Clark and Gladys Rice', "Mike Markel's Orchestra", 'The Philharmonic Orchestra of New York', 'The Waldorf-Astoria Dance Orchestra', 'John Steel', 'The Columbia Saxophone Sextette', 'Yerkes Jazarimba Orchestra', 'The All-Star Trio', 'French Army Band', 'Belle Baker', 'Helen Clark and George Wilton Ballard', 'The Green Brothers Novelty Band', 'The Kentucky Serenaders', 'Carl Fenton and His Orchestra', 'The Paul Biese Trio', 'William Robyn', 'Happy Six', 'George Meader', 'Eddy Brown', 'The Benson Orchestra of Chicago', 'Benny Krueger and His Orchestra', 'Leo Reisman and His Orchestra', 'Edwin Dale', 'The Vernon Country Club Band', 'The Coon-Sanders Orchestra', 'The Broadway Quartet', 'Vaughn De Leath', 'U.S. Naval Academy Band', 'Anthony Urato', 'The Club Royal Orchestra', 'The Cotton Pickers', 'The California Ramblers', 'The Brox Sisters', 'Ted Weems and His Orchestra', 'The Virginians', 'The Broadway Dance Orchestra', "Fred Waring's Pennsylvanians", 'Abe Lyman and His Californians', 'Okeh Laughing Record', "Harry Reser's Orchestra", 'The International Novelty Orchestra', 'Marcia Freer', 'The Duncan Sisters [1°]', 'Henry Whittier', 'The Associated Glee Clubs of America', 'Jane Green', 'Art Gillham', 'George Olsen and His Orchestra', "Warner's Seven Aces", 'Arden-Ohman Orchestra', 'Dennis King', 'Willie Howard', 'Carolyn Thompson', 'Jimmie Rodgers [country singer]', 'The Radiolites', 'The Charleston Chasers', 'Jan Garber and His Orchestra', 'Scrappy Lambert', "Kitty O'Connor", 'The Clicquot Club Eskimos', 'Charles King', 'The Victor Symphony Orchestra', 'The Roy Fox Orchestra', 'The Colonial Club Orchestra', 'High Hatters', 'Bert Lown and His Orchestra', 'Hilo Hawaiian Orchestra', 'The Regent Club Orchestra', 'Jacques Renard and His Orchestra', 'The Beverly Hill Billies', 'Ozzie Nelson and His Orchestra', 'Hal Kemp and His Orchestra', 'Perry Askam', 'Hal Goodman and His Orchestra', 'Lee Morse', 'Boswell Sisters', 'Ray Noble and His Orchestra', 'The Havana Novelty Orchestra', 'Glen Gray and the Casa Loma Orchestra', 'Joe Green Novelty Orchestra', 'Bing Crosby and the Mills Brothers', 'Art Kassel and His Kassels-in-the-Air', 'Eddy Duchin and His Orchestra', 'Enric Madriguera and His Orchestra', 'Connee Boswell', 'The Debroy Somers Band', 'Donald Novis', 'Rose Marie', 'The Hotel Commodore Orchestra', 'Henry King and His Orchestra', 'Freddy Martin and His Orchestra', 'Gertrude Niesen', 'Dick Powell', 'Conrad Thibault', 'The Castillian Troubadors', "Walter O'Keefe", 'Frank Parker', 'Emil Velasco', 'The RKO Studio Orchestra', 'Fred Hillebrand', 'The Castillians', 'Xavier Cugat and His Waldorf-Astoria Orchestra', 'The Riley-Farley Orchestra', 'Kay Kyser and His Orchestra', 'Jimmy Dorsey and His Orchestra', 'Joe Morrison', 'Shep Fields and His Rippling Rhythm', 'Andy Kirk and His Clouds of Joy', 'The Hudson-DeLange Orchestra', 'The Sweet Violet Boys', 'Erskine Hawkins and His Orchestra', 'Al Donahue and His Orchestra', 'George Hamilton', 'The Top Hatters', 'Larry Clinton and His Orchestra', 'Bing Crosby and Connee Boswell', 'Raymond Scott and His Orchestra', 'Pinky Tomlin', 'Johnny Hodges and His Orchestra', 'Rhythm Wreckers', 'Bing Crosby and Johnny Mercer', 'Jan Savitt and His Orchestra', 'Gray Gordon', 'The Milt Herth Trio', 'Patricia Norman', "Obregon's Orchestra", 'Orrin Tucker and His Orchestra', 'Dick Jurgens and His Orchestra', 'Frankie Masters and His Orchestra', 'Tiny Hill', 'The Three Peppers', 'Will Bradley and His Orchestra', 'Mitchell Ayres and His Fashions in Music', 'Tommy Tucker Time', 'Bob Chester and His Orchestra', 'Bobby Byrne and His Orchestra', 'Al Kavelin and His Cascading Chords', 'Terry Shand and His Orchestra', 'Louise Massey and the Westerners', 'The Cats and the Fiddle', 'Tony Pastor and His Orchestra', 'Charlie Spivak and His Orchestra', 'Art Jarrett and His Orchestra', 'Bob Wills and His Texas Playboys', 'Teddy Powell and His Orchestra', 'Claude Thornhill and His Orchestra', 'Barry Wood', 'Alvino Rey and His Orchestra', 'Metronome All Star Band', 'Joan Merrill', 'Bing Crosby and Muriel Lane', 'Sam Donahue and His Orchestra', 'Clyde Lucas and His Orchestra', 'The Jesters', 'Freddie Slack and His Orchestra', 'Johnnie Johnston', 'The King Cole Trio', 'The Royal Harmony Quartet', 'Song Spinners', 'Bing Crosby and Trudy Erwin', 'Johnny Long and His Orchestra', 'Frank Sinatra and Harry James', 'Willie Kelly and His Orchestra', 'Billy Butterfield and His Orchestra', 'Allen Miller and His Orchestra', 'Jimmy Lytell and His All Star Seven', 'Six Hits and a Miss', 'Henri René and His Musette Orchestra', 'Ceelle Burke and His Orchestra', "Boone's Jumpin' Jacks", 'Sonny Boy Williams and His Orchestra', 'Beverly White and Her Blues Chasers', 'The Ink Spots and Ella Fitzgerald', 'Dick Haymes and Helen Forrest', 'Kay Armen and the Balladiers', 'Benny Carter and His Orchestra', 'Al Trace and His Orchestra', '5 Red Caps', 'Phil Hanna', 'Charles Wolcott and His Orchestra', 'Al Sack and His Orchestra', 'Fred Waring and His Concert Vochestra', 'Jerry Jerome and His Cats and Jammers', 'Frankie Carle and His Orchestra', 'Tex Grande and His Range Riders', "Bennie Moten's Kansas City Orch.", 'Jerry Wald and His Orchestra', 'Gray Rains and His Orchestra', 'Hal McIntyre and His Orchestra', 'Bing Crosby and Xavier Cugat and His Orchestra', 'Bing Crosby and Carmen Cavallaro', 'Johnny Mercer and Jo Stafford', 'Bing Crosby and Les Paul and His Trio', 'Phil Moore Four', 'Carmen Cavallaro and His Orchestra', 'Bing Crosby and Judy Garland', 'Paul Weston and His Orchestra and Margaret Whiting', 'Hildegarde and Guy Lombardo and His Royal Canadians', 'Ella Fitzgerald and The Delta Rhythm Boys', 'Delta Rhythm Boys', 'Frank Sinatra and the Charioteers', 'Evelyn Knight and the Jesters', 'Dick Thomas', 'Joe Liggins and His Honeydrippers', 'The Milt Herth Trio and the Jesters', 'Martha Stewart', 'Louis Jordan and Bing Crosby', 'Ethel Smith and the Bando Carioca', 'Pvt. Cecil Gant', 'Roosevelt Sykes and His Piano', 'Bing Crosby and the Jesters', 'The Andrews Sisters and Guy Lombardo and His Royal Canadians', 'Louis Jordan and Ella Fitzgerald', 'Dinah Shore and Spade Cooley and His Orchestra', 'Elliot Lawrence and His Orchestra', 'The Andrews Sisters and Les Paul', 'Paul Weston and His Orchestra and Matt Dennis', 'Helen Carroll', 'Ella Fitzgerald and Louis Armstrong', 'Connie Boswell and Russ Morgan and His Orchestra', 'Hoosier Hot Shots and Sally Foster', 'Harry Cool and Mindy Carson', 'The Andrews Sisters and Eddie Heywood and His Orchestra', 'Slim Gaillard Trio', 'Bing Crosby and Mel Tormé and His Mel-Tones', 'Bing Crosby and Jimmy Dorsey and His Orchestra', 'Johnny Mercer and the King Cole Trio', 'The Three Flames', 'Francis Craig and His Orchestra', 'Jack McVea and His All Stars', 'Danny Kaye and the Andrews Sisters', 'Jack Smith', 'Bing Crosby and Fred Waring', 'Bing Crosby and the Calico Kids', 'Kenny Baker and Russ Morgan and His Orchestra', 'Julia Lee and Her Boy Friends', 'Nellie Lutcher and Her Rhythm', 'Ray Bloch and His Orchestra', 'Dick Two Ton Baker and His Music Makers', 'Billy Williams and the Pecos River Rogues', 'Herbie Fields and His Orchestra', 'Dinah Shore and Woody Herman', 'Bing Crosby and Bob Crosby and His Bob Cats', 'Jo Stafford and Gordon MacRae', 'Pee Wee Hunt and His Orchestra', 'Sportsmen', 'The Sportsmen and Mel Blanc and His Original Woody Woodpecker Voice', 'Anne Vincent', 'Buddy Clark and the Charioteers', 'Brother Bones and His Shadows', 'Vaughn Monroe and the Sons of the Pioneers', 'Benny Strong and His Orchestra', 'Jack Lathrop and the Drugstore Cowboys', 'Carmen Miranda and the Andrews Sisters', 'The Ravens', 'Vaughn Horton and His Polka Debs', 'The Uptown String Band', 'Andy Russell and the Pied Pipers', 'Paul Weston and His Orchestra', 'Jerry Wayne', 'Margaret Whiting and Jimmy Wakely', 'Al Morgan', 'Margaret Whiting and Johnny Mercer', 'Dinah Shore and Buddy Clark', 'Stardusters', 'The Andrews Sisters and Russ Morgan and His Orchestra', 'Toni Arden and Hugo Winterhalter and His Orchestra', 'Don Reid', 'Don Cornell and Laura Leslie', 'The Wesson Brothers', 'Patty Andrews and Bob Crosby', 'Frank Sinatra and the Modernaires', 'Gordon Jenkins and His Orchestra and the Weavers', 'Perry Como and the Fontane Sisters', 'Victor Young and His Orchestra and Chorus and Don Cherry', 'Perry Como and Betty Hutton', 'Joe Fingers Carr', 'Peggy Lee and Mel Tormé', 'Dick Haymes and Artie Shaw', 'Jo Stafford and Frankie Laine', 'Pinetoppers', 'Bobby Wayne', 'Harry James and His Orchestra and Doris Day', 'Billy Williams', 'Bing Crosby and Jane Wyman', 'Peggy Lee and Gordon Jenkins and His Orchestra', 'Perry Como and Eddie Fisher', 'Doris Day and Frankie Laine', 'Nat King Cole and Billy May and His Orchestra', 'Doris Day and Johnnie Ray', 'Eddie Wilcox Orch.', 'The Harry Grove Trio', 'Bing Crosby and Grady Martin and His Slew Foot Five', 'Les Compagnons de la chanson', 'Don Cornell and Teresa Brewer', 'Ray Martin Orchestra', 'Louis Armstrong and Gordon Jenkins and His Orchestra', 'Frank Chacksfield and His Orchestra', 'Ted Heath Orchestra', 'Henri René and Hugo Winterhalter', 'The Stargazers [British vocal group]', 'Johnny Maddox', 'Mariners', 'Art and Dotty Todd', 'David Whitfield and Mantovani and His Orchestra and Chorus', 'The DeCastro Sisters', 'The Chords [1950s american doo wop group]', 'Leroy Holmes and His Orchestra', 'José Ferrer', 'Norman Petty Trio', 'The Charms', 'David Carroll', 'Ray Burns', 'Ferko String Band', 'Perry Como and Jaye P. Morgan', 'Johnny Brandon', 'Bing Crosby and Grace Kelly', 'Morris Stoloff and His Orchestra', 'Hugo Winterhalter and Eddie Heywood', 'Buchanan and Goodman', 'Jimmie Rodgers [pop singer]', 'Johnny Duncan [bluegrass musician]', 'King Brothers', 'Charles McDevitt Skiffle Group', 'The Big Bopper', 'The Poni-Tails', 'The Royal Teens', 'The Monotones', 'John Zacherle the Cool Ghoul', 'Ernie Fields', 'Miss Toni Fisher', 'Rosie', 'Lolita', 'The Highwaymen [folk group]', 'Bobby Lewis', 'Dick and Dee Dee', 'The Sims Twins', 'Kokomo [Jimmy Wisner]', 'Ned Miller', 'B. Bumble and the Stingers', 'Martha and the Vandellas', 'The Angels [US girl group]', 'Freddie and the Dreamers', 'The Four Pennies [English pop group]', 'The Larks [Los Angeles band]', 'Sonny and Cher', 'Them', 'Staff Sgt. Barry Sadler', 'The Outsiders [1°]', 'Keith [American pop singer]', "Every Mother's Son", 'The Scaffold', 'The Cuff Links', 'The Meters [1°]', 'The Carpenters', 'Dave and Ansil Collins', 'The Free Movement', 'The Spinners [US band]', 'Gallery', 'Climax [US band]', 'The Dramatics [1°]', 'Focus [1°]', 'M.F.S.B.', 'Sweet Sensation [english band]', 'Jigsaw [English group]', 'Kenny', 'The Goodies', 'John Williams [composer]', 'Eruption', 'Brian and Michael', 'Toyah', 'Dave Stewart [David Lloyd Stewart]', 'Junior', 'Ollie and Jerry', 'Scandal [rock band]', 'Arcadia', 'The Jets [pop band]', 'Patti LaBelle and Michael McDonald', 'Don Johnson [actor]', 'Doctor and the Medics', 'Bill Medley and Jennifer Warnes', 'Mel and Kim', 'Poison [hard rock band]', 'Johnny Kemp', 'D:Mob', 'Vanessa Williams [Vanessa Lynn Williams]', 'Sonia [Sonia Evans]', 'The Adventures of Stevie V', 'Synch', 'Enigma [german electronic music project]', 'Sweet Sensation [American band]', 'Partners in Kryme', 'Candyman', 'Dave Stewart [David Allan Stewart]', 'Mr. Big [US band]', 'Nirvana [American band]', 'Jade [American band]', 'Altern-8', 'Domino [American rapper]', 'IMx', '4 P.M.', 'The Spice Girls', 'Cast', 'Space [english pop band]', '911 [british boy band]', "Somethin' for the People", '5ive', 'Voices of Theory', 'Lyte Funky Ones (LFO)', 'a1', 'Len', 'Blue [2000s band]', 'D-12', 'Jet [Australian band]', '2play', 'The Feeling', 'Cherish', 'Chico [Chico Slimani]', 'Matt Willis', 'High School Musical 2', 'Duffy [Aimee Anne Duffy]', 'September [Swedish dance singer]', 'The Glee Cast', 'Florence and the Machine', 'Dizzee Rascal and Armand Van Helden', 'Agnes [Agnes Carlsson]', 'The Wanted [boy band]', 'Shout for England', 'The Eli Young Band', 'Psy [Korean singer]', 'Hunter Hayes [US country music singer]', 'Future [rapper]', 'Sam Bailey', 'The Vamps [2°]', 'DJ Snake and Lil Jon', 'Lilly Wood & The Prick and Robin Schulz', "Gareth Malone's All-Star Choir", 'NHS Choir', 'Grace [Australian singer]', 'Dave [rapper]', 'Mabel [Mabel McVey]', 'Tyler the Creator', 'A.D. Madeira', 'Grace Spencer', 'Florodora Girls', 'Byron Harlan', 'John Bieling', 'the Haydn Quartet', 'Gina Viafora', 'the American Quartet', 'His Orchestra', 'American Quartet', 'the Victor Light Opera Co.', 'Elizabeth Brice', 'Frank Coombs', 'the Lyric Quartet', 'Weber', 'Fields', 'Maurice Burkhardt', 'Will Robbins', 'Mary Carson', 'Monroe Jockers', 'White Marimba Band', 'Schenck', 'James Harrod', 'Helen Louise', 'the Peerless Quartet', 'the Sterling Trio', 'the Shannon Four', 'Elliot Shaw', 'the Columbia Stellar Quartet', 'Lillian Rosedale', 'the Orpheus Quartet', 'Art Hickman', 'His Band', 'Rachel Grant Gladys Rice', 'Her Jazz Hounds', 'Gene Rodemich', 'Virginia Asher', 'Ray Miller', 'Vincent Lopez', 'Eddie Elkins', 'Arthur Gibbs', 'Frank Westphal', 'His Jazz Band', 'Lew Holtz', 'Art Kahn', 'the Victor Orchestra', 'Nat Shilkret', 'Jack Shilkret', 'His Virginians', 'Ace Brigode', 'Johnny Hamp', 'His Skillet Lickers', 'Mack', 'His Five Pennies', 'Fred Rich', 'Don Voorhees', 'His Los Angeles Biltmore Hotel Orchestra', 'His Commanders', 'Lou Gold', 'His Salon Orchestra', 'Adrian Schubert', 'His Connecticut Yankees', 'Roy Ingraham', 'Jack Denny', 'Tom Gerun', 'Arnold Johnson', 'His Campus Boys', 'Ted Wallace', 'His Havana Casino Orchestra', 'Paul Tremaine', 'Slatz Randall', 'Harold Grayson', 'Peter van Steeden', 'the Boswell Sisters and the Mills Brothers', 'Ted Black', 'Jimmie Grier', 'His Oriole Orchestra', 'Joe Rines', 'Piano Pals', 'The Rondoliers', 'Arthur Lally', 'Harold Stern', 'His Society Dance Orchestra', 'Ted Fio Rito', 'Don Bestor', 'Gracie Allen', 'Joe Haymes', 'His Palace Hotel Orchestra', 'Tom Coakley', 'Lud Gluskin', 'Jolly Coburn', 'Paul Pendarvis', 'Jack Jackson', 'George Hall', 'Reginald Foresythe', 'Raymond Paige', 'Henry Allen', 'the Gang', 'Andy Iona', 'His Stork Club Orchestra', 'His Kings of Tempo', 'His Swing Band', 'Herbie Kay', 'Bob Howard', 'King Garcia', 'Joe Sanders', 'Her Dawn Patrol', 'Dolly Dawn', 'Dick Robertson', 'the Mills Brothers', 'Freddie Fisher', 'His Melody Makers', 'Red Jessup', 'Hod Williams', 'Frank Dailey', 'Slim', 'Jerry Blaine', 'His Streamline Rhythm', 'Dick Barrie', 'Swing and Sway', 'the Keynotes', "Primo Scala's Banjo and Accordion Orchestra", 'Mary Ford', "Hugo Winterhalter's Orchestra", 'the Four Lads', 'His Pops Concert Orchestra', 'Gloria DeHaven', 'His Royal Canadians', 'Laurie Anders', 'Stanley Black and His Orchestra', 'His Comets', 'the Lancers', 'Chorus', 'His Saxophone', 'Billie Anthony', 'Eric Jupp and His Orchestra', 'Dick', 'Jimmy', "Somethin' Smith", 'the Redheads', 'Prudence', 'Patience', 'The Juniors', 'Johnnie', 'Mickey', 'the Hearts', 'Billy', 'Lillie', 'The Imperials', 'Tommy Dorsey Ork.', 'Jan', 'His Quartet', 'Johnny', 'the Hurricanes', 'Edward Byrnes', 'the Checkmates', 'Ferrante', 'Teicher', 'the Zodiacs', 'Frederik', 'His Jazzmen', 'The Limelites', 'the Romans', 'Paul', 'Paula', 'the Crypt-Kickers', 'B singer]', 'Joe Henderson [American R', 'the Blue Jeans', 'the Dakotas', 'the Romantics', 'the Americans', 'the Cavaliers', 'Ronny', 'the Daytonas', 'The Pharaohs', 'Jr. Walker', 'the All-Stars', '2', 'Beaky', 'Dozy', 'the Mysterians', 'Jonathan', 'David', 'Frank', 'Our Gang', 'the Techniques', 'the Drells', 'the Supremes and the Temptations', 'Esther', 'the Watts 103rd Street Rhythm Band', 'Cornelius Brothers', 'Sister Rose', 'Delaney', 'Crofts', 'Seals', 'Hall', 'His Cast of Idiots', 'Lyle', 'McClain', 'Pratt', 'The Emotions [female R', 'B trio]', 'Althea', 'Donna', 'Starsound', 'Franke', 'Godley', 'the Waves', 'Shirlie', 'Pepsi', 'Jive Bunny', 'the Mastermixers', 'the Funky Bunch', 'Robson Green', 'Sweat', 'LSG Levert', 'Divine [R', 'B girl group]', 'Brian Keith', 'Small', 'Coco', 'Mýa and Pink', 'the INC.', 'Claire', 'H', 'JXL', 'Factor Finalists 2008', 'Dr. Dre and 50 Cent', 'D Cup', 'Factor Finalists 2011', 'Ariana Grande and Nicki Minaj', 'Skinner and the Lightning Seeds', 'VickeeLo', 'Kid Laroi'])
set2 = set(['George J. Gaskin', 'Jere Mahoney', 'The Haydn Quartet', 'Dan Quinn', 'Vess Ossman', 'Will Denny', 'S. H. Dudley', 'The Georgia Minstrel Co.', 'Mina Hickman', "The Gilmore's Band", 'Silas Leachman', 'William Redmond', 'J. Aldrich Libbey', 'Jules Levy', 'William H. Thompson', 'Bob Roberts', 'George Alexander', 'The Columbia Orchestra', 'Richard Jose', 'The Peerless Quartet', 'Harry Tally', 'Corrine Morgan', 'Frank Stanley', 'Lew Dockstader', 'Grace Nelson', 'DeWolf Hopper', 'James McCool', 'Alan Turner', 'The Victor Orchestra', 'May Irwin', 'Frederic Rose', 'Helene Noldi', 'Harold Jarvis', 'Ralph Herz', 'Nat Wills', 'The Victor Light Opera Co.', 'The American Quartet', 'Arthur Clough', 'Paul Southe', 'Joe Maxwell', 'Kitty Cheatham', 'The Fisk University Jubilee Quartet', 'The Lyric Quartet', 'The Heidelberg Quintet', 'Guido Deiro', 'Marguerite Dunlap', 'Gene Greene', 'Dolly Connolly', 'The New York Military Band', 'Elsie Baker', "The Conway's Band", 'The Columbia Mixed Quartet', 'Chauncey Olcott', 'Charles Harrison', 'Helen Clark and Walter Van Brunt', 'William J. Halley', 'Joseph Cawthorn', 'Joe Hayman', 'George MacFarlane', 'Owen McCormack', 'John Barnes Wells', 'The Columbia Stellar Quartet', 'Grace Kerns and Reed Miller', 'The Orpheus Quartet', 'Grace Kerns and John Barnes Wells', 'The Victor Mixed Chorus', 'Charles Kellogg', "McKee's Orchestra", 'Taylor Trio', 'The Venetian Trio', 'Frank Tinney', "George O'Connor", 'George Wilton Ballard', 'Helen Clark and Joseph Phillips', 'Marguerite Farrell', 'The Columbia Mixed Double Quartet', 'Anna Chandler', 'The Imperial Quartet of Chicago', 'The Sterling Trio', 'Ernest Ball', 'The Knickerbocker Quartet', 'Horace Wright', 'Anna Wheaton', "M.J. O'Connell", 'Louis Gravieure', 'Andrea Sarto', 'The Shannon Four', 'Lucy Gates', "Earl Fuller's Novelty Orchestra", 'The Neapolitan Trio', 'The Farber Sisters', 'Charles Hart', 'Percy Hemus', 'Harry Fox', 'Harry Ellis', 'Julian Rose', "W.C. Handy's Orchestra", 'Helen Clark and Gladys Rice', 'John Steel', 'The Waldorf-Astoria Dance Orchestra', 'The Columbia Saxophone Sextette', 'Yerkes Jazarimba Orchestra', 'The All-Star Trio', 'French Army Band', 'The Avon Comedy Four', 'Belle Baker', 'The Kentucky Serenaders', 'Carl Fenton and His Orchestra', 'The Paul Biese Trio', 'The Green Brothers Novelty Band', 'William Robyn', 'Happy Six', 'Benny Krueger and His Orchestra', 'Leo Reisman and His Orchestra', 'The Benson Orchestra of Chicago', 'Edwin Dale', 'The Vernon Country Club Band', 'The Club Royal Orchestra', 'The Cotton Pickers', "Fred Waring's Pennsylvanians", 'The Virginians', 'Abe Lyman and His Californians', 'Okeh Laughing Record', 'Ted Weems and His Orchestra', 'The Duncan Sisters [1°]', 'The International Novelty Orchestra', 'Marcia Freer', 'Jane Green', 'The Associated Glee Clubs of America', 'Vaughn De Leath', 'Art Gillham', 'George Olsen and His Orchestra', "Harry Reser's Orchestra", 'Dennis King', 'Arden-Ohman Orchestra', 'Carolyn Thompson', 'Willie Howard', 'Jimmie Rodgers [country singer]', 'The Victor Symphony Orchestra', 'Charles King', 'Hilo Hawaiian Orchestra', 'The Regent Club Orchestra', 'Bert Lown and His Orchestra', 'Jacques Renard and His Orchestra', 'The Colonial Club Orchestra', 'High Hatters', 'The Beverly Hill Billies', 'Ozzie Nelson and His Orchestra', 'Hal Kemp and His Orchestra', 'Boswell Sisters', 'Ray Noble and His Orchestra', 'The California Ramblers', 'The Havana Novelty Orchestra', 'Bing Crosby and the Mills Brothers', 'Art Kassel and His Kassels-in-the-Air', 'Enric Madriguera and His Orchestra', 'Eddy Duchin and His Orchestra', 'The Hotel Commodore Orchestra', 'Joe Green Novelty Orchestra', 'Glen Gray and the Casa Loma Orchestra', 'Freddy Martin and His Orchestra', 'Jan Garber and His Orchestra', 'Henry King and His Orchestra', 'The Castillian Troubadors', "Walter O'Keefe", 'Frank Parker', 'The Riley-Farley Orchestra', 'Xavier Cugat and His Waldorf-Astoria Orchestra', 'Shep Fields and His Rippling Rhythm', 'Jimmy Dorsey and His Orchestra', 'Andy Kirk and His Clouds of Joy', 'Connee Boswell', 'Kay Kyser and His Orchestra', 'Bing Crosby and Connee Boswell', 'Larry Clinton and His Orchestra', 'Raymond Scott and His Orchestra', 'Al Donahue and His Orchestra', 'Bing Crosby and Johnny Mercer', 'Gray Gordon', 'Frankie Masters and His Orchestra', 'Orrin Tucker and His Orchestra', 'Dick Jurgens and His Orchestra', 'Mitchell Ayres and His Fashions in Music', 'Will Bradley and His Orchestra', 'Bob Chester and His Orchestra', 'Jan Savitt and His Orchestra', 'Tommy Tucker Time', 'Erskine Hawkins and His Orchestra', 'Tony Pastor and His Orchestra', 'Charlie Spivak and His Orchestra', 'Barry Wood', 'Bob Wills and His Texas Playboys', 'Alvino Rey and His Orchestra', 'Freddie Slack and His Orchestra', 'Johnnie Johnston', 'Song Spinners', 'Johnny Long and His Orchestra', 'Frank Sinatra and Harry James', 'Bing Crosby and Trudy Erwin', 'Willie Kelly and His Orchestra', 'Billy Butterfield and His Orchestra', 'The King Cole Trio', 'Allen Miller and His Orchestra', 'Jimmy Lytell and His All Star Seven', 'Six Hits and a Miss', 'Henri René and His Musette Orchestra', 'Hal Goodman and His Orchestra', 'The Ink Spots and Ella Fitzgerald', 'Dick Haymes and Helen Forrest', 'Tiny Hill', 'Al Trace and His Orchestra', 'Kay Armen and the Balladiers', '5 Red Caps', 'Benny Carter and His Orchestra', 'Bing Crosby and Carmen Cavallaro', 'Johnny Mercer and Jo Stafford', 'Bing Crosby and Les Paul and His Trio', 'Hal McIntyre and His Orchestra', 'Carmen Cavallaro and His Orchestra', 'Bing Crosby and Xavier Cugat and His Orchestra', 'Phil Moore Four', 'Frankie Carle and His Orchestra', 'Bing Crosby and Judy Garland', 'Paul Weston and His Orchestra and Margaret Whiting', 'Hildegarde and Guy Lombardo and His Royal Canadians', 'Ella Fitzgerald and The Delta Rhythm Boys', 'Delta Rhythm Boys', 'Dinah Shore and Spade Cooley and His Orchestra', 'Bing Crosby and the Jesters', 'The Andrews Sisters and Les Paul', 'Louis Jordan and Ella Fitzgerald', 'Paul Weston and His Orchestra and Matt Dennis', 'Sam Donahue and His Orchestra', 'Helen Carroll', 'The Andrews Sisters and Guy Lombardo and His Royal Canadians', 'The Three Flames', 'Francis Craig and His Orchestra', 'Johnny Mercer and the King Cole Trio', 'Jack McVea and His All Stars', 'Danny Kaye and the Andrews Sisters', 'Elliot Lawrence and His Orchestra', 'Bing Crosby and Fred Waring', 'Bing Crosby and the Calico Kids', 'Pee Wee Hunt and His Orchestra', 'Jo Stafford and Gordon MacRae', 'The Sportsmen and Mel Blanc and His Original Woody Woodpecker Voice', 'Anne Vincent', 'Sportsmen', 'Buddy Clark and the Charioteers', 'Brother Bones and His Shadows', 'Benny Strong and His Orchestra', 'Vaughn Monroe and the Sons of the Pioneers', 'Julia Lee and Her Boy Friends', 'Margaret Whiting and Jimmy Wakely', 'Al Morgan', 'Dinah Shore and Buddy Clark', 'Margaret Whiting and Johnny Mercer', 'Jack Smith', 'Stardusters', 'Jerry Wayne', 'Toni Arden and Hugo Winterhalter and His Orchestra', 'Paul Weston and His Orchestra', 'Gordon Jenkins and His Orchestra and the Weavers', 'Perry Como and the Fontane Sisters', 'Perry Como and Betty Hutton', 'Victor Young and His Orchestra and Chorus and Don Cherry', 'Joe Fingers Carr', 'Pinetoppers', 'Jo Stafford and Frankie Laine', 'Harry James and His Orchestra and Doris Day', 'Bing Crosby and Jane Wyman', 'Doris Day and Frankie Laine', 'Perry Como and Eddie Fisher', 'Peggy Lee and Gordon Jenkins and His Orchestra', 'Bobby Wayne', 'Nat King Cole and Billy May and His Orchestra', 'Doris Day and Johnnie Ray', 'Eddie Wilcox Orch.', 'The Harry Grove Trio', 'Frank Chacksfield and His Orchestra', 'Henri René and Hugo Winterhalter', 'The Stargazers [British vocal group]', 'David Whitfield and Mantovani and His Orchestra and Chorus', 'The DeCastro Sisters', 'The Chords [1950s american doo wop group]', 'Leroy Holmes and His Orchestra', 'José Ferrer', 'The Charms', 'Johnny Maddox', 'David Carroll', 'Bing Crosby and Grace Kelly', 'Morris Stoloff and His Orchestra', 'Hugo Winterhalter and Eddie Heywood', 'Buchanan and Goodman', 'Jimmie Rodgers [pop singer]', 'Billy Williams', 'The Big Bopper', 'The Poni-Tails', 'The Royal Teens', 'The Monotones', 'Art and Dotty Todd', 'John Zacherle the Cool Ghoul', 'Ernie Fields', 'Miss Toni Fisher', 'Rosie', 'Lolita', 'The Highwaymen [folk group]', 'Bobby Lewis', 'Dick and Dee Dee', 'The Sims Twins', 'Kokomo [Jimmy Wisner]', 'Ned Miller', 'B. Bumble and the Stingers', 'The Angels [US girl group]', 'Martha and the Vandellas', 'The Larks [Los Angeles band]', 'Sonny and Cher', 'Freddie and the Dreamers', 'Them', 'Staff Sgt. Barry Sadler', 'Keith [American pop singer]', 'The Blues Magoos', "Every Mother's Son", 'The Scaffold', 'The Cuff Links', 'The Carpenters', 'Dana [Dana Rosemary Scallon]', 'Dave and Ansil Collins', 'The Free Movement', 'The Spinners [US band]', 'Climax [US band]', 'Gallery', 'The Dramatics [1°]', 'Focus [1°]', 'M.F.S.B.', 'Sweet Sensation [english band]', 'Disco-Tex and the Sex-O-Lettes', 'Jigsaw [English group]', 'Wing and a Prayer Fife and Drum Corps', 'Barry De Vorzon and Perry Botkin Jr.', 'Eruption', 'Brian and Michael', 'Michael Zager Band', 'Dave Stewart [David Lloyd Stewart]', 'Junior', 'Nicole [German singer]', 'Ollie and Jerry', 'Scandal [rock band]', 'Arcadia', 'Patti LaBelle and Michael McDonald', 'The Jets [pop band]', 'Don Johnson [actor]', 'Bill Medley and Jennifer Warnes', 'Poison [hard rock band]', 'Johnny Kemp', 'D:Mob', 'Vanessa Williams [Vanessa Lynn Williams]', 'The Adventures of Stevie V', 'Enigma [german electronic music project]', 'Sweet Sensation [American band]', 'Partners in Kryme', 'Dave Stewart [David Allan Stewart]', 'Mr. Big [US band]', 'Nirvana [American band]', 'Jade [American band]', 'Domino [American rapper]', 'IMx', '4 P.M.', 'The Manchester United Football Squad', 'Baby D [dance group]', 'The Spice Girls', "Somethin' for the People", '5ive', 'Voices of Theory', 'Stardust [electronic music trio]', 'Lyte Funky Ones (LFO)', 'Len', 'D-12', 'Rui da Silva', 'Blue [2000s band]', 'LMC vs U2', 'Chico [Chico Slimani]', 'Fedde le Grand', 'Cherish', 'High School Musical 2', 'Duffy [Aimee Anne Duffy]', 'September [Swedish dance singer]', 'H Two O', 'The Glee Cast', 'Dizzee Rascal and Armand Van Helden', 'Vanessa Jenkins and Bryn West', 'Agnes [Agnes Carlsson]', 'Shout for England', 'The Wanted [boy band]', 'Psy [Korean singer]', 'Hunter Hayes [US country music singer]', 'Florence and the Machine', 'Sam Bailey', 'Mike Will Made-It', 'The Vamps [2°]', 'DJ Snake and Lil Jon', 'Lilly Wood & The Prick and Robin Schulz', "Gareth Malone's All-Star Choir", 'NHS Choir', 'Grace [Australian singer]', 'Future [rapper]', 'Dave [rapper]', 'Mabel [Mabel McVey]', 'A.D. Madeira', 'Grace Spencer', 'Florodora Girls', 'Byron Harlan', 'John Bieling', 'the Haydn Quartet', 'Gina Viafora', 'the American Quartet', 'American Quartet', 'the Victor Light Opera Co.', 'His Orchestra', 'Elizabeth Brice', 'Frank Coombs', 'the Lyric Quartet', 'Maurice Burkhardt', 'Will Robbins', 'Mary Carson', 'Monroe Jockers', 'Weber', 'Fields', 'White Marimba Band', 'James Harrod', 'Schenck', 'Helen Louise', 'the Peerless Quartet', 'the Sterling Trio', 'the Shannon Four', 'Elliot Shaw', 'Lillian Rosedale', 'His Band', 'Art Hickman', 'Her Jazz Hounds', 'Gene Rodemich', 'Rachel Grant Gladys Rice', 'Virginia Asher', 'Vincent Lopez', 'Ray Miller', 'Arthur Gibbs', 'Lew Holtz', 'the Victor Orchestra', 'Nat Shilkret', 'Jack Shilkret', 'Johnny Hamp', 'Mack', 'His Five Pennies', 'His Los Angeles Biltmore Hotel Orchestra', 'Don Voorhees', 'His Connecticut Yankees', 'Roy Ingraham', 'His Commanders', 'His Havana Casino Orchestra', 'His Campus Boys', 'Ted Wallace', 'Tom Gerun', 'Peter van Steeden', 'the Boswell Sisters and the Mills Brothers', 'Jack Denny', 'Ted Black', 'Jimmie Grier', 'His Oriole Orchestra', 'Joe Rines', 'Don Bestor', 'Gracie Allen', 'Ted Fio Rito', 'Jolly Coburn', 'Lud Gluskin', 'His Palace Hotel Orchestra', 'Tom Coakley', 'Paul Pendarvis', 'His Society Dance Orchestra', 'Reginald Foresythe', 'His Kings of Tempo', 'George Hall', 'His Stork Club Orchestra', 'His Swing Band', 'Her Dawn Patrol', 'Dolly Dawn', 'Dick Robertson', 'Slim', 'Swing and Sway', 'the Keynotes', "Primo Scala's Banjo and Accordion Orchestra", 'Mary Ford', 'the Four Lads', 'His Pops Concert Orchestra', 'Gloria DeHaven', 'His Royal Canadians', 'Stanley Black and His Orchestra', 'His Comets', 'the Lancers', 'Chorus', 'His Saxophone', "Somethin' Smith", 'the Redheads', 'Prudence', 'Patience', 'The Juniors', 'Johnnie', 'The Imperials', 'Jan', 'Tommy Dorsey Ork.', 'Johnny', 'the Hurricanes', 'Edward Byrnes', 'the Checkmates', 'Ferrante', 'Teicher', 'the Zodiacs', 'His Jazzmen', 'The Limelites', 'Paul', 'Paula', 'the Crypt-Kickers', 'B singer]', 'Joe Henderson [American R', 'the Blue Jeans', 'the Romantics', 'the Dakotas', 'the Cavaliers', 'Ronny', 'the Daytonas', 'the Americans', 'The Pharaohs', 'Jr. Walker', 'the All-Stars', '2', 'the Mysterians', 'Frank', 'the Techniques', 'the Supremes and the Temptations', 'the Drells', 'Esther', 'Cornelius Brothers', 'Sister Rose', 'Crofts', 'Seals', 'Peters', 'His Cast of Idiots', 'Hall', 'McClain', 'Pratt', 'The Emotions [female R', 'B trio]', 'Althea', 'Donna', 'Starsound', 'Franke', 'Garcia', 'Buckner', 'Renato', 'the Waves', 'Jive Bunny', 'the Mastermixers', 'the Funky Bunch', 'Robson Green', 'Skinner and the Lightning Seeds', 'Sweat', 'LSG Levert', 'the Bizz', 'Divine [R', 'B girl group]', 'Brian Keith', 'Coco', 'Mýa and Pink', 'Nicole Kidman', 'the Master of Ceremonies', 'the INC.', 'JXL', 'Kelly', 'Factor Finalists 2008', 'Dr. Dre and 50 Cent', 'D Cup', 'Factor Finalists 2011', 'Ariana Grande and Nicki Minaj'])
set3 = set(['Ned Miller', 'Sandy Stewart [singer]', 'The Castells', 'Dick and Dee Dee', 'Nancy Wilson', 'Robert Maxwell and His Orchestra', 'Staff Sgt. Barry Sadler', 'Senator Everett McKinley Dirksen', 'Groove Holmes', 'Jimmie Rodgers [pop singer]', 'The Gunter Kallmann Chorus', 'The Baja Marimba Band', 'Johnny Mann Singers', 'The Romeos', 'The Cuff Links', 'The Carpenters', 'Sonny and Cher', 'The Free Movement', 'Paul Stookey', 'The Spinners [US band]', 'Climax [US band]', 'Gallery', 'M.F.S.B.', 'Sweet Sensation [english band]', 'Jigsaw [English group]', 'Wing and a Prayer Fife and Drum Corps', 'Barry De Vorzon and Perry Botkin Jr.', 'John Williams [composer]', 'Wilton Place Street Band', 'John Schneider', 'James Taylor and J.D. Souther', 'Sylvia [Sylvia Kirby aka Sylvia Hutton]', 'Dionne Warwick and Johnny Mathis', "Jump 'N the Saddle", 'Band of Gold', 'Patti LaBelle and Michael McDonald', 'The Jets [pop band]', 'Bill Medley and Jennifer Warnes', 'Giant Steps', 'Vanessa Williams [Vanessa Lynn Williams]', 'Synch', 'Sweet Sensation [American band]', 'Dave Stewart [David Allan Stewart]', 'Mr. Big [US band]', 'Chris Walker', 'Enigma [german electronic music project]', '4 P.M.', 'Carnie and Wendy Wilson', 'The Spice Girls', 'Sasha [Sasha Alexander]', 'Chris Emerson', 'Daniel DeBourg', 'Regie Hamm', 'Burke Roney', 'Jet [Australian band]', 'Jeff Timmons', 'Desol', 'The Jones Gang', 'Il Divo and Céline Dion', 'Jade [Canadian singer]', 'Lareau', 'TrueHeart', 'Cnote', 'Greg Medoro', 'Jim Brickman and Adam Crossley', 'The Glee Cast', 'Duffy [Aimee Anne Duffy]', 'Greg London', 'Sam Mizell', 'Glenn Mohr', 'Father Ted Berndt', 'The Wanted [boy band]', 'Margo Rey', 'Dave Barnes', 'Hunter Hayes [US country music singer]', 'Richard Kincaid', 'Whitney Wolanin', 'Allison Park', 'The Villains', 'Grooveatech Orchestra', 'Josh Krajcik', 'Viennie V', 'Kanisha K', 'Kathryn Dean', 'Jane Lynch', 'Corbette Jackson', 'The Alternate Routes', 'John Splithoff', 'Leslie Cours Mather', 'Kaz Bielinski', 'William Prince', 'Charleene Closshey', 'The Rua', 'Matt Hartke', 'King Calaway', 'Danny Griego', 'Ryan Chernin', 'B singer]', 'Joe Henderson [American R', 'Paul', 'Paula', 'the Sunliners', 'Sunny', 'the Romantics', 'the Dakotas', 'Swing and Sway', '2', 'Jonathan', 'David', 'the Allis', 'Frank', 'Our Gang', 'the Americans', 'Ferrante', 'Teicher', 'Delaney', 'Cornelius Brothers', 'Sister Rose', 'Crofts', 'Seals', 'Mouth', 'MacNeal', 'Peters', 'Guys', 'Dolls', 'Hall', 'McClain', 'Pratt', 'Lyle', 'Townsend Band', 'The Sanford', 'LeBlanc', 'Carr', 'Neil', 'Starsound', 'Franke', 'Jon', 'the Waves', 'Godley', 'Scarlett', 'Jive Bunny', 'the Mastermixers', 'Lebo M.', 'Brian Keith', 'Deborah Gibson', 'Randy Jackson', 'Ariana Grande and Nicki Minaj'])
set4 = set(['The King Cole Trio', '5 Red Caps', 'Benny Carter and His Orchestra', 'Texas Jim Lewis', 'Foy Willing and His Riders of the Purple Sage', 'Charles Mitchell', 'Cindy Walker', 'Dick Thomas', 'Walter Shrum', 'Bill Boyd and His Cowboy Ramblers', 'Boyd Heath', 'Hoosier Hot Shots and Sally Foster', 'Wiley and Gene', 'Hoosier Hot Shots and Two Ton Baker', 'Tiny Hill', 'The Carlisles Brothers', 'The Buchanan Brothers', 'Dick Jurgens and His Orchestra', 'Arthur Guitar Boogie Smith', 'Esmereldy', 'Jerry Irby and His Texas Ranchers', 'Eddie Dean', 'Shorty Long [country singer]', 'Jack Rivers', 'Margaret Whiting and Jimmy Wakely', 'Al Morgan', 'Burl Ives and Captain Stubby and the Buccaneers', 'The Andrews Sisters and Ernest Tubb and the Texas Troubadors', 'Bill Nettles', 'Dave Landers', 'Elton Britt and Rosalie Allen', 'Pinetoppers', 'Bing Crosby and Grady Martin and His Slew Foot Five', 'Burl Ives and Grady Martin and His Slew Foot Five', 'Jimmie Dean', 'Jimmy Newman', 'Jimmie Rodgers [country singer]', 'Jimmie Rodgers [pop singer]', 'Jimmy Edwards', 'Frankie Miller [country singer]', 'Luke Gordon', 'Gene Woods', 'Ray Godfrey', 'Bill Leatherwood', 'Alan Riddle', 'Jimmy Smart', 'Laverne Downs', 'Ted Self', 'Bobby Edwards', 'Johnny Hardy', 'Ned Miller', "Coleman O'Neal", 'Gary Buck', 'Carl Butler and Pearl', 'Vernon Stewart', 'Ruby Wright [country singer]', 'Marion Worth and George Morgan', 'Johnny Wright', 'Bobby Bare and Skeeter Davis', 'Staff Sgt. Barry Sadler', 'Senator Everett McKinley Dirksen', 'Bobby Lewis [country singer]', 'Bill Anderson and Jan Howard', 'Waylon Jennings and Anita Carter', 'Buck Owens and Buddy Alan', 'Harlow Wilcox and the Oakies', 'Mel Tillis and Sherry Bryce', 'LaCosta Tucker', 'Bill Anderson and Mary Lou Turner', 'Rod Hart', 'Cody Jameson', 'The Carpenters', 'John Schneider', 'Sylvia [Sylvia Kirby aka Sylvia Hutton]', 'The Highwaymen [country supergroup]', 'Trio [American country supergroup]', 'K. T. Oslin', 'Mike Reid', 'Kevin Sharp', 'The Raybon Brothers', 'Vince Gill and Patty Loveless', 'Jeffrey Steele', 'The Jenkins', 'Rockie Lynne', 'Heartland [2°]', 'Song Trust', 'Rehab', 'The Eli Young Band', 'Melissa Lawson', 'Phil Stacey', 'Adam Gregory', 'The JaneDear Girls', 'James Wesley Prosser', 'Troy Olsen', 'Hunter Hayes [US country music singer]', 'Edens Edge', 'The Farm Inc.', 'Andy Gibson [singer]', 'Bradley Gaskin', 'Greg Bates', 'Keith Urban and Miranda Lambert', 'Charlie Worsham', 'The Henningsens', 'Jake Worthington', 'Sam Palladio', 'Pete Scobell Band', 'Zach Seabaugh', 'Shelby Brown', 'Corey Kent White', 'Josh Gallagher', 'Spensha Baker', 'Kirk Jay', 'Chris Kroeze', 'The Outlaws [US band]', 'Jim Carroll Band', 'James Taylor and J.D. Souther', 'Rainbow [British rock band]', 'Diesel [1°]', 'Scandal [rock band]', 'The Firm [English rock supergroup]', 'Don Johnson [actor]', 'Poison [hard rock band]', 'The Primitives', 'Camouflage [German dance group]', 'Enigma [german electronic music project]', 'Soho', 'The Mission [English band]', 'Mr. Big [US band]', 'Nirvana [American band]', 'Page and Plant', 'Reef', 'Space [english pop band]', 'Mono [British pop duo]', 'Len', 'Lajon', 'Dust for Life', 'TRUSTcompany', 'Earshot', 'Course of Nature', 'Jet [Australian band]', 'Silvertide', 'The Used and My Chemical Romance', 'Angels and Airwaves', 'Operator', 'Pendulum [drum and bass band]', 'Janus', 'Florence and the Machine', 'New Medicine', 'Young Guns', 'Device [metal band]', 'Børns', 'Holy White Hounds', 'COIN', 'MISSIO', 'Leon Seago', 'Mary Ford', 'Johnnie', 'Joe Fingers Carr', 'the Wilburn Brothers', 'Billy Gray', 'Marilyn', 'Doug', 'Reno', 'Jonie Mosby', 'Johnny', 'Norma Jean and Liz Anderson', 'The Glaser Brothers', 'Tompall', 'Cashman', 'Peters', "the Citizen's Band", 'Billy Thunderkloud', 'the Chieftones', 'Squirrely', 'Susan Anton', 'the Doolittle Band', 'Dandy', 'Karen Brooks', 'Overstreet', 'Schuyler', 'Knobloch', 'Baillie', 'the Boys', 'Terry McBride', 'the Ride', 'Brian Keith', 'Victoria Shaw and Bryan White', 'Lisa Hartman Black', 'Mark Willis', 'the RubberBand', 'Ryan Shupe', 'Cole Deggs', 'the Lonesome', 'Connie Britton', 'Rory', 'Zac Brown', 'Copilot Music', 'Sound', 'Hall', 'Franke', 'Doug McKenzie', 'the Waves', 'Godley', 'the Banshees', 'Siouxsie', 'the Melody Makers', 'the On-U Sound System', 'Coverdale', 'John', 'J Roddy Walston'])

setP = set1.union(set2).union(set3).union(set4)
setN = set(['Freddie Slack and His Orchestra', 'Jimmy Dorsey and His Orchestra', 'The King Cole Trio', 'Andy Kirk and His Clouds of Joy', 'Erskine Hawkins and His Orchestra', 'The Royal Harmony Quartet', 'Glen Gray and the Casa Loma Orchestra', 'Frank Sinatra and Harry James', 'Xavier Cugat and His Waldorf-Astoria Orchestra', 'Jacques Renard and His Orchestra', "Boone's Jumpin' Jacks", 'Ceelle Burke and His Orchestra', 'Sonny Boy Williams and His Orchestra', 'Beverly White and Her Blues Chasers', 'The Ink Spots and Ella Fitzgerald', '5 Red Caps', 'Benny Carter and His Orchestra', 'Pvt. Cecil Gant', 'Johnny Hodges and His Orchestra', 'Delta Rhythm Boys', 'Ella Fitzgerald and The Delta Rhythm Boys', 'Joe Liggins and His Honeydrippers', 'Roosevelt Sykes and His Piano', 'Louis Jordan and Ella Fitzgerald', 'Slim Gaillard Trio', "The Cats 'N Jammer Three", 'Julia Lee and Her Boy Friends', 'The Blues Woman', 'Ella Fitzgerald and Louis Armstrong', 'Lennie Lewis and His Orchestra', 'The Three Flames', 'Jack McVea and His All Stars', 'Nellie Lutcher and Her Rhythm', 'Eddie Vinson and His Orchestra', 'Annie Laurie and Paul Gayten and His Orchestra', 'Gene Ammons and His Sextet', 'Floyd Hunt Quartette', 'Hadda Brooks Trio', 'Sonny Boy Williamson [1°]', 'Brother Bones and His Shadows', 'Memphis Slim and His House Rockers', 'Hal Singer Sextette', 'Pee Wee Crayton and His Guitar', 'Red Miller Trio', 'Do Ray and Me', 'The Ravens', 'Sister Rosetta Tharpe and Marie Knight', 'Camille Howard and Her Trio', 'Little Son Jackson', 'Paul Williams [saxophonist]', 'Dan Grissom', 'Stick McGhee and His Buddies', 'Rudy Render', 'Eddie Williams and His Brown Buddies', 'The X-Rays', 'Jimmy Preston and His Prestonians', 'Freddie Mitchell and His Orchestra', 'Herb Lance and the Classics', 'Joe Thomas [American saxophonist]', 'Mercy Dee', 'The Four Jacks', "Chubby Hip Shakin' Newsom", 'Joe Morris and His Orchestra', 'Ed Wiley and His After Hour Rhythm', 'Doc Sausage and His Mad Lads', 'Herb Fisher and His Trio', 'Roy Byrd and His Blues Jumpers', 'Joe Fritz', 'The Shadows [vocal group]', 'Nellie Lutcher and Nat King Cole', 'Mel Walker', 'Calvin Boze and His All-Stars', 'Archibald and His Orchestra', 'Original Five Blind Boys', 'Johnny Sparrow and His Bows and Arrows', 'Jimmie Nelson and the Peter Rabbit Trio', 'Earl Bostic and His Orchestra', 'James Waynes', 'The Larks [North Carolina band]', 'The Cardinals', 'Jimmy Grissom', 'The Swallows', 'Roscoe Gordon', 'Little Willie Littlefield and Little Lola Wiggins', 'Eddie Wilcox Orch.', 'The Four Blazes', 'Willie Mabon and His Combo', 'John Greer and the Rhythm Rockers', 'Buddy Lucas and His Band of Tomorrow', 'Little Caesar [Harry Caesar]', 'Little Esther and Mel Walker', 'Wini Brown and Her Boyfriends', 'Nat King Cole and Billy May and His Orchestra', 'Willie Mae Big Mama Thornton', 'The Coronets', 'Danny Overbea and His Combo', 'Swinging Sax Kari and Orchestra', 'Emitt Slay Trio', 'Marvin and Johnny', 'Jimmy Wilson and His All-Stars', 'The Chords [1950s american doo wop group]', 'The Charms', 'The Counts', 'Shirley Gunter and the Queens', 'Louis Brooks and His Hi-Toppers', 'Sonny Boy Williamson [2°]', 'Willie Dixon and the Allstars', 'Jack Dupree and Mr. Bear', 'Earl King [Earl Johnson]', 'Earl King [Earl Connelly]', 'Hugo Winterhalter and Eddie Heywood', 'Buchanan and Goodman', 'Jimmie Rodgers [pop singer]', 'Billy Williams', 'Jimmy Newman', 'The Big Bopper', 'The Poni-Tails', 'The Royal Teens', 'The Monotones', 'Art and Dotty Todd', 'John Zacherle the Cool Ghoul', 'Ernie Fields', 'Miss Toni Fisher', 'Rosie', 'Bobby Lewis', 'The Sims Twins', 'The Corsairs', 'The Regents [American doo-wop group]', 'The Spinners [US band]', 'Rochell and the Candles', 'Nathaniel Mayer and the Fabulous Twilights', 'The Angels [US girl group]', 'Martha and the Vandellas', 'The Larks [Los Angeles band]', 'The Reflections [Detroit rock and roll band]', 'Joe Hinton', 'Nancy Wilson', 'The Sapphires', 'Sonny and Cher', 'Jackie Lee [American singer]', 'The Shades of Blue', 'Roy C', 'Bobby Moore [saxophonist]', 'Groove Holmes', 'Ko Ko Taylor', 'Robert Knight', 'The Meters [1°]', 'Dyke and the Blazers', 'The Flaming Ember', 'The Free Movement', 'The Dramatics [1°]', 'Brighter Side of Darkness', 'Holland-Dozier', 'M.F.S.B.', 'Sweet Sensation [english band]', 'Disco-Tex and the Sex-O-Lettes', 'Wing and a Prayer Fife and Drum Corps', 'Bazuka', 'Fatback', 'C.J. & Co.', 'Wilton Place Street Band', 'Silvetti', 'The Imperials', 'Eruption', 'Michael Zager Band', 'Karen Young [US singer]', 'Love and Kisses', 'Switch [funk band]', 'Jimmy Bo Horne', "The McCrary's", 'Linx', 'Fantasy [US soul band]', 'K.I.D.', 'The Quick', 'Junior', 'Chéri', 'Howard Johnson [singer]', 'Dionne Warwick and Johnny Mathis', 'Raw Silk', 'Richard Dimples Fields', 'David Joseph', 'Lydia Murdoch', 'Ollie and Jerry', 'Band of Gold', 'Patti LaBelle and Michael McDonald', 'The Jets [pop band]', 'Regina [Regina Richards]', 'Mel and Kim', 'Déjà [soul duo]', 'Johnny Kemp', "Suave'", 'Rob Base and DJ E-Z Rock', 'Vanessa Williams [Vanessa Lynn Williams]', 'D:Mob', 'The Adventures of Stevie V', 'Enigma [german electronic music project]', 'Partners in Kryme', 'Dave Stewart [David Allan Stewart]', 'Candyman', 'Louie Louie', 'The Chimes [British dance music act]', 'The Geto Boys', 'Jade [American band]', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage', 'Chris Walker', 'Domino [American rapper]', 'Sagat', 'U.N.V.', 'IMx', '4 P.M.', 'B.M.U. (Black Men United)', 'Solo [2°]', 'Damage', "Somethin' for the People", 'Refugee Camp All-Stars', 'Voices of Theory', 'Mo Thugs and Bone Thugs-N-Harmony', 'Profyle', 'D-12', 'The Purple Ribbon All-Stars', 'Cherish', '50 Cent and Olivia', 'Slim [Marvin Scandrick]', 'Rocko', 'Fast Life Yungstaz (F.L.Y.)', 'The Wanted [boy band]', 'Troop 41', 'Future [rapper]', 'Tami LaTrell', 'Chip Chocolate', 'Tyler the Creator', '2Virgins', 'Kevin Ross', 'iAMSU!', 'Round2Crew', 'JR Castro', 'TK n Cash', 'Young Thug and Travis Scott', 'Taylor Girlz', 'DJ Suede the Remix God', 'Goldlink', 'The Reflections [Harlem vocal group]', 'Ramsey Lewis and Earth, Wind & Fire', 'The Moment of Truth', 'The Street People [2°]', 'The Boogie Man Orchestra', 'Ja-Kki', 'Whirlwind', 'John Davis and the Monster Orchestra', 'Bumble Bee Unlimited', 'Impact [1°]', 'The Heart and Soul Orchestra', 'Mike Theodore Orchestra', 'The Lovers [2°]', 'Brainstorm [disco band]', 'Space [french electronic music band]', 'ORS', 'The Philarmonics', 'Arpeggio', 'Macho', 'Eastside Connection', 'Lipstique', 'The Positive Force', 'Pleasure [1°]', 'Trio [German band]', 'Arcadia', 'Doctor and the Medics', 'Taffy', 'Giant Steps', 'Camouflage [German dance group]', 'Lil Louis', 'Sonia [Sonia Evans]', 'Sweet Sensation [American band]', 'Betty Boo and the Beatmasters', 'Soho', 'Nomad [house music act]', 'George Lamond', 'Nirvana [American band]', 'Brothers in Rhythm', 'B.G. the Prince of Rap', 'Altern-8', 'Staxx', 'DJ Duke', 'Loveland vs Darlene Lewis', 'Black Machine', 'The Fog [house project]', 'Meechie', 'E.V.E. (Ebony Vibe Everlasting)', 'Da Mighty Dub Katz', 'Sphinx [electronic music duo]', 'Artie the 1 Man Party', 'The Shades of Love', 'Loveland', 'The Spice Girls', "Stretch 'N' Vern Present Maddogg", 'The Lisa Marie Experience', 'C. J. Bolland', 'Pulse [2°]', 'Jon of the Pleased Wimmin', 'Grace [1°]', 'Praxis', '3rd Party', 'Laguna', 'Stardust [electronic music trio]', 'Déjà Vu [dance music act]', 'Pianoheadz', 'The Don [Nathaniel Pierre Jones]', 'Charlotte [Charlotte Kelly]', 'Mount Rushmore', 'Angel Clivillés', 'Dynamix', 'Rui da Silva', 'The Ones [2°]', 'The Goodfellas', "Mad'House", 'Who Da Funk?', 'KMC [Italian dance act]', 'India [Linda Viera Caballero]', 'Angel City [electronic music group]', 'Sun [Sun Ho]', 'Robbie Rivera and Axwell', 'Industry [2°]', 'Rosko', 'Jennifer Green', 'Tony Moran', 'Altar [dance music band]', 'September [Swedish dance singer]', 'Fireball', 'Charo', 'Twisted Dee', 'Luvndreams', 'Agnes [Agnes Carlsson]', 'Dianne Wesley', 'Dave Matthias', 'Richard Vission and Static Revenger', 'Hannah [British singer]', 'Florence and the Machine', 'Chris Cox and DJ Frankie', 'David Morales and Janice Robinson', 'D.O.N.S. and Terri B!', 'JX Riders and Skylar Stecker', 'Robin S. and DJ Escape', 'Dillon Francis and Kygo', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Karel Ullner', 'Dirty Pop', 'Dana Ordway', 'Tim Myers', 'Jeff Morgan', 'Cliq', 'Dirty Werk', 'DJs from Mars', 'His Orchestra', 'the Four Lads', 'Mary Ford', "Brown's Blues Blowers", 'Grant Jones', 'Marvin', 'His Comets', 'Eunice', 'The Juniors', 'Johnnie', 'Mickey', 'the Hearts', 'Joe Bennett', 'the Sparkletones', 'Jan', 'Billy', 'Lillie', 'Johnny', 'the Hurricanes', 'Edward Byrnes', 'Ferrante', 'Teicher', 'the Zodiacs', 'Skip', 'Flip', 'The Limelites', 'the Romans', 'Paul', 'Paula', 'the Crypt-Kickers', 'B singer]', 'Joe Henderson [American R', 'the Blue Jeans', 'the Bluebelles', 'the Soul Rockers', 'Les Cooper', 'the Romantics', 'The Rainbows', 'the Sunliners', 'Sunny', 'Patty', 'the Emblems', 'The Pharaohs', 'Jr. Walker', 'the All-Stars', 'the Techniques', 'Brenda', 'The Tabulations', 'the Supremes and the Temptations', 'the Drells', 'Tim', 'Mel', 'the Watts 103rd Street Rhythm Band', 'The Emotions [female R', 'B trio]', 'Electric', 'Pacific Gas', 'Cornelius Brothers', 'Sister Rose', 'Paul Humphrey', 'His Cool Aid Chemists', "the Family Cookin'", 'Limmie', 'Cheech', 'Chong', 'Ecstasy', 'Whatnauts', 'Sharon Paige and Harold Melvin', 'the Blue Notes', 'His Cast of Idiots', 'Hall', 'Bell', 'the Soul Searchers', 'the Soul Partners', 'Alton McClain', 'Destiny', 'Sheila', 'Starsound', 'Q. Brooklyn', 'B.B.', 'Queens', 'the GB Experience', 'the Seen', 'Ta Mara', 'B quartet]', 'The Boys [US R', 'the Melody Makers', 'the Funky Bunch', 'Mike Hitman Wilson', 'Sweat', 'LSG Levert', 'the Bizz', 'Divine [R', 'B girl group]', 'Ideal [R', 'Nokio', 'Mýa and Pink', 'Mark Curry', 'the INC.', 'Cashis', 'Dr. Dre and 50 Cent', 'Kid Laroi', 'Bill', 'Mike', 'the Ramblers', 'Ron Butler', 'Jimmy James', 'the Vagabonds', 'Hodges', 'James and Smith', 'Sunshine', 'Martha', 'the Muffins', 'Company', 'the Banshees', 'Siouxsie', 'the Waves', 'Godley', 'Jessie Saunders', 'Shirlie', 'Pepsi', 'Scarlett', 'the Rebel MC', 'the FBI', 'the Big Family', 'JT', 'the On-U Sound System', 'Cause', 'Effect', 'Jennifer Saunders', 'Absolutely Fabulous Pet Shop Boys', 'Farley Project', 'Eniac', 'Coco', 'the Rhythm Bangers', 'the Police', 'DJ Vicious', "Marc O'Tool Tall Paul vs INXS", 'Pet Shop Boys = The Collaboration', 'Andrea Brown', 'Kelly', 'Jorio', 'Joi', 'Blaze', 'the Egg', 'Randy Jackson', 'DJ Bill Bennett', 'Tim Letteer', 'Fawn', 'Angello', 'Crazibiza and Dave Audé', 'Hybrid Heights and Crystal Waters', 'Jess Glynne and DJ Cassidy', 'DirtyFreqs', 'Like Mike and David Guetta', 'Zach Adam', 'Extasia', 'Fenix', 'Barona', 'Bouvier', 'Metro', 'Like Mike and Era Istrefi', 'Fancy Inc.', 'RetroVision'])

print('setN = ',len(setN))
print('setN & setP = ',len(setN.intersection(setP)))
print('setN - setP = ',len(setN.difference(setP)))
print('setN - setP = ',len(setN - setP))
toget=list(setN.difference(setP))
print(toget)

In [ ]:
addAllMusic(mdbmaps, "Spotify", "d267166f6f192e4626b58ae273eb6313", "")   ### [Jamie Fine]
addDeezer(mdbmaps, "Spotify", "d267166f6f192e4626b58ae273eb6313", "")    ### [Jamie Fine]
### Ain't Easy - THE LAUNCH  ,  Catchin Feels
### Ain't Easy - THE LAUNCH  ---> [['Elijah Woods x Jamie Fine']]
### Catchin Feels  ---> [['Elijah Woods x Jamie Fine']]


addAllMusic(mdbmaps, "Spotify", "26e4c8109b6b2a3bb8c9f5898b93187f", "")   ### [Elijah Woods]
addDeezer(mdbmaps, "Spotify", "26e4c8109b6b2a3bb8c9f5898b93187f", "")    ### [Elijah Woods]
### Ain't Easy - THE LAUNCH  ,  Catchin Feels
### Ain't Easy - THE LAUNCH  ---> [['Elijah Woods x Jamie Fine']]
### Catchin Feels  ---> [['Elijah Woods x Jamie Fine']]


addAllMusic(mdbmaps, "Spotify", "7f47f9bcd04a663dd4c5b6cc23ce2fc4", "")   ### [Sueco]
addDeezer(mdbmaps, "Spotify", "7f47f9bcd04a663dd4c5b6cc23ce2fc4", "")    ### [Sueco]
### fast (Remix) [feat. Offset & A Boogie Wit da Hoodie]  ,  fast
### fast (Remix) [feat. Offset & A Boogie Wit da Hoodie]  ---> [['Sueco']]
### fast  ---> [['Sueco']]


addAllMusic(mdbmaps, "Spotify", "d1b09e1ea18d6a92e651530bbc3755ea", "")   ### [Alicia Moffet]
addDeezer(mdbmaps, "Spotify", "d1b09e1ea18d6a92e651530bbc3755ea", "")    ### [Alicia Moffet]
### Beautiful Scar  ,  Take Control
### Beautiful Scar  ---> [['Alicia Moffet']]
### Take Control  ---> [['Alicia Moffet']]


addAllMusic(mdbmaps, "Spotify", "920c63b4fcad01b381d646d6b8b365bc", "")   ### [NEW CITY]
addDeezer(mdbmaps, "Spotify", "920c63b4fcad01b381d646d6b8b365bc", "")    ### [NEW CITY]
### Getting Closer
### Getting Closer  ---> [['NEW CITY']]


addAllMusic(mdbmaps, "Spotify", "9ab59de7434b95bb68e6b9f0c5cff97e", "")   ### [Vanic]
addDeezer(mdbmaps, "Spotify", "9ab59de7434b95bb68e6b9f0c5cff97e", "")    ### [Vanic]
### Too Soon (feat. Maty Noyes)
### Too Soon (feat. Maty Noyes)  ---> [['Vanic']]


addAllMusic(mdbmaps, "Spotify", "dc6f8bc1cb6d39457937f5ed10418334", "")   ### [Christian Hudson]
addDeezer(mdbmaps, "Spotify", "dc6f8bc1cb6d39457937f5ed10418334", "")    ### [Christian Hudson]
### Four Leaf Clover - Klowds Remix
### Four Leaf Clover - Klowds Remix  ---> [['Christian Hudson']]


addAllMusic(mdbmaps, "Spotify", "ce5bb3d8056d6d971672242255a6d3b0", "")   ### [Bob Pressner]
addDeezer(mdbmaps, "Spotify", "ce5bb3d8056d6d971672242255a6d3b0", "")    ### [Bob Pressner]
### American Dream
### American Dream  ---> [['Bob Pressner']]


addAllMusic(mdbmaps, "Spotify", "26154d0d5295a4f04f64f0b7f1c0e067", "")   ### [DiRTY RADiO]
addDeezer(mdbmaps, "Spotify", "26154d0d5295a4f04f64f0b7f1c0e067", "")    ### [DiRTY RADiO]
### True Love
### True Love  ---> [['DiRTY RADiO']]


addAllMusic(mdbmaps, "Spotify", "180bff77810e1d08186cf125b5d7d353", "")   ### [Cassow]
addDeezer(mdbmaps, "Spotify", "180bff77810e1d08186cf125b5d7d353", "")    ### [Cassow]
### La Brea
### La Brea  ---> [['Cassow']]


addAllMusic(mdbmaps, "Spotify", "c12c928aa06f2f10507af4b3a2c9a934", "")   ### [Madalen Duke]
addDeezer(mdbmaps, "Spotify", "c12c928aa06f2f10507af4b3a2c9a934", "")    ### [Madalen Duke]
### Gucci Store
### Gucci Store  ---> [['Madalen Duke']]


addAllMusic(mdbmaps, "Spotify", "88759173674025cf8f480850e3ba4b0b", "")   ### [Joshua Bassett]
addDeezer(mdbmaps, "Spotify", "88759173674025cf8f480850e3ba4b0b", "")    ### [Joshua Bassett]
### Lie Lie Lie
### Lie Lie Lie  ---> [['Joshua Bassett']]


addAllMusic(mdbmaps, "Spotify", "2c7704262fe420489299bef485e21f10", "")   ### [VIC MENSA]
addDeezer(mdbmaps, "Spotify", "2c7704262fe420489299bef485e21f10", "")    ### [VIC MENSA]
### Reverse (Ft. G-Eazy)
### Reverse (Ft. G-Eazy)  ---> [['VIC MENSA']]


addAllMusic(mdbmaps, "Spotify", "2e7143f95646c1272401f710104716dd", "")   ### [DJ Scheme]
addDeezer(mdbmaps, "Spotify", "2e7143f95646c1272401f710104716dd", "")    ### [DJ Scheme]
### Buck 50
### Buck 50  ---> [['DJ Scheme']]


addAllMusic(mdbmaps, "Spotify", "f7e293bc32c1201e5a5a66fa535bf823", "")   ### [Leoni Jut]
addDeezer(mdbmaps, "Spotify", "f7e293bc32c1201e5a5a66fa535bf823", "")    ### [Leoni Jut]
### I'LL BE THERE
### I'LL BE THERE  ---> [['Leoni Jut']]


addAllMusic(mdbmaps, "Spotify", "835daa3012317c49c26a830e14b1e32b", "")   ### [Josh Jips]
addDeezer(mdbmaps, "Spotify", "835daa3012317c49c26a830e14b1e32b", "")    ### [Josh Jips]
### Read
### Read  ---> [['Josh Jips']]


addAllMusic(mdbmaps, "Spotify", "09d950789fd2315b5051dea856a6184f", "")   ### [Skye]
addDeezer(mdbmaps, "Spotify", "09d950789fd2315b5051dea856a6184f", "")    ### [Skye]
### VOICES (feat. XXXTENTACION)
### VOICES (feat. XXXTENTACION)  ---> [['Skye']]


addAllMusic(mdbmaps, "Spotify", "7de9d986161f1cce5810ffdb61615c31", "")   ### [BRELAND]
addDeezer(mdbmaps, "Spotify", "7de9d986161f1cce5810ffdb61615c31", "")    ### [BRELAND]
### My Truck
### My Truck  ---> [['BRELAND']]


addAllMusic(mdbmaps, "Spotify", "7d186492c45514c715a28e500a2f7735", "")   ### [16yrold]
addDeezer(mdbmaps, "Spotify", "7d186492c45514c715a28e500a2f7735", "")    ### [16yrold]
### No Romance
### No Romance  ---> [['16yrold']]


addAllMusic(mdbmaps, "Spotify", "016d228fa16eed569e6887bb0eaa996d", "")   ### [Pedro CapÃ³]
addDeezer(mdbmaps, "Spotify", "016d228fa16eed569e6887bb0eaa996d", "")    ### [Pedro CapÃ³]
### Calma - Remix
### Calma - Remix  ---> [['Pedro Capó', 'Pedro CapÃ³']]


addAllMusic(mdbmaps, "Spotify", "4dc90ffe02b116303dd4830d3f07a656", "")   ### [Rube Claws]
addDeezer(mdbmaps, "Spotify", "4dc90ffe02b116303dd4830d3f07a656", "")    ### [Rube Claws]
### DANG
### DANG  ---> [['Rube Claws']]


mdbmaps["Spotify"].save()

In [ ]:
#https://www.hungama.com/albums/lang/russian/popular/
#https://www.kkbox.com/hk/en/artist/FGNJ6VipM2tycJ0F0kas508K-index-1.html
#https://kworb.net/itunes/extended.html
#http://istanbulmusic.blogspot.com/2010/04/best-musicians-from-ecuador.html
manualAppends(toMatch, chartType, minAlbums=0, add=True)

# Extra Artists

## Top40

In [ ]:
#https://www.allmusic.com/artist/boa-mn0000322754
addAllMusic(mdbmaps, "Top40", "6dfb791d61d3dd8a44fa5b5c1c8dbde0", "mn0000322754")   ### [Boa]
addDeezer(mdbmaps, "Top40", "6dfb791d61d3dd8a44fa5b5c1c8dbde0", "792")    ### [Boa]
addDeezer(mdbmaps, "Top40", "688f28c172f877020462ae0dcd158f7d", "5337130")    ### [Antytila]
addDeezer(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "4251519")    ### [V. Meladze]
#https://www.discogs.com/artist/1027379-The-Gospellers
addDiscogs(mdbmaps, "Top40", "2874db4bf2285180da2878da30719d98", "1027379")    ### [Gospellers]
addDeezer(mdbmaps, "Top40", "2874db4bf2285180da2878da30719d98", "7722644")    ### [Gospellers]




addDeezer(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "10470285")    ### [Z. E.]
addDeezer(mdbmaps, "Top40", "73ea300db5aa1c2154adfe42186a887e", "5418")    ### [Ark]
addDeezer(mdbmaps, "Top40", "c29dd858b42c5e8d6caecd0302c47dfe", "9060182")    ### [Hov 1]
addDeezer(mdbmaps, "Top40", "5127ec6e19c0a6008623adfad8b778c8", "72144882")    ### [Moe]
addDeezer(mdbmaps, "Top40", "2bc0fe9eb315a876af7ff1b91988daf3", "4523116")    ### [E.m.m.a.]
addDeezer(mdbmaps, "Top40", "919603bf283bab261d6607bea9234ce3", "75100")    ### [Crashdiet]
addDeezer(mdbmaps, "Top40", "25c0271edadaa16ad11b4016508ad57c", "10540205")    ### [Sebastian]
addDiscogs(mdbmaps, "Top40", "89946a817a31189393f24f92b32be466", "60136")   ### [Catch]
addDeezer(mdbmaps, "Top40", "0b8ff2909d235838583a92b539b640ef", "1164412")    ### [Floodgate]
addDeezer(mdbmaps, "Top40", "680c0282e232b1f64a3ef9d56da97275", "251483")    ### [Wallstones]
addDiscogs(mdbmaps, "Top40", "cb4133f2381241be2ae73d3365e746e8", "3143651")    ### [Honey]
addDeezer(mdbmaps, "Top40", "b59c32c628ceae2c2c6b59aacce5c3fe", "66502")    ### [Nic]
addDeezer(mdbmaps, "Top40", "6269dc570069e12a96dddc0a674e4ff2", "75178")    ### [Charlotte]
addDeezer(mdbmaps, "Top40", "b0e8915fa1946cafcbb743ec3f4a6ea9", "122866")    ### [Charizma]
addDeezer(mdbmaps, "Top40", "afadfe17376050c150bb0f9cb78e4181", "64411")    ### [Fame]
addDeezer(mdbmaps, "Top40", "5984fbaf75a5530e9a3319136ee014cf", "76721")    ### [Dj Roine]
addDiscogs(mdbmaps, "Top40", "214906164a747a8ce969529a640b9eec", "346187")   ### [Jeanette]
addDeezer(mdbmaps, "Top40", "214906164a747a8ce969529a640b9eec", "450752")    ### [Jeanette]
addDeezer(mdbmaps, "Top40", "c652ef0598d18f7655b92b18d87b8bcd", "56449262")    ### [Cuz]
addAllMusic(mdbmaps, "Top40", "89caf52a1eb268d2335ef2f09c39deb1", "mn0002185241")   ### [H.e.a.t.]
addDiscogs(mdbmaps, "Top40", "89caf52a1eb268d2335ef2f09c39deb1", "1890182")    ### [H.e.a.t.]
addDeezer(mdbmaps, "Top40", "336f4261abc48ea6247320a9b25885b0", "13720517")    ### [Chris Klafford]
addDeezer(mdbmaps, "Top40", "5ae548889336487db92e7746dea26566", "513525")    ### [Jay Smith]
addDeezer(mdbmaps, "Top40", "f412cf34b7f042c6e70017898a977805", "428632")    ### [Jazzattacks]
addDeezer(mdbmaps, "Top40", "71bbc65fd1a58cfe3b7eab94bc666fe4", "4588048")    ### [Ria Hall]
addDiscogs(mdbmaps, "Top40", "b1b3b97ec0478725400c7c8729607be9", "3037345")    ### [Carol Anthony]
addAllMusic(mdbmaps, "Top40", "29c175308260a766645f88a48fda8cb5", "mn0002141576")   ### [Sasha]
addDeezer(mdbmaps, "Top40", "ace5274eac9ac83ecc8be584f2f89678", "389681")    ### [Jezabels]
addAllMusic(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "mn0001488122")   ### [J.williams]
addDeezer(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "1226315")    ### [J.williams]
addDeezer(mdbmaps, "Top40", "3b51465a51f53de3f9406c8c4084b9eb", "4771547")    ### [Hannah]
addDeezer(mdbmaps, "Top40", "4660f8fef24f1de99d82c58cb6d558ae", "11373")    ### [Solitaire]
addDeezer(mdbmaps, "Top40", "bad689c3aa55f468721a0f0e31e00490", "862972")    ### [Knocks]
addAllMusic(mdbmaps, "Top40", "df7a414766c07b227b947fe19d735cca", "mn0000527296")   ### [P.I.]
addDeezer(mdbmaps, "Top40", "ee1825b598532d857e97ba0c03a6c10b", "9049402")    ### [They]
addDeezer(mdbmaps, "Top40", "bdf6a38ef1a824d2fbc71eb0866ec945", "5817516")    ### [Kucka]
addDeezer(mdbmaps, "Top40", "4e61bdac79b1a4b86d16d9a0121b1a77", "1655320")    ### [Tobtok]
addDeezer(mdbmaps, "Top40", "66eaf372536540197d6edb36afe6a9ce", "595")    ### [Destiny`s Child]
addDeezer(mdbmaps, "Top40", "8b3f077c9506a80b91565e354f5939ec", "6756461")    ### [Tony Ansell]
addDeezer(mdbmaps, "Top40", "4dbc5b8f4dabea79e3ae50712cb6a994", "2830461")    ### [Yebba]
addDeezer(mdbmaps, "Top40", "6e9d556d993ea0981f932c68de4c1b77", "9852222")    ### [Kelly Kiara]
addDeezer(mdbmaps, "Top40", "b80ed1fc4530c8fcc8e780deccf26b26", "5358213")    ### [Tom Thum]
addDeezer(mdbmaps, "Top40", "96db126f7d9fb560896964cc9019cfc7", "5922055")    ### [Montaigne]
addDeezer(mdbmaps, "Top40", "ddb7f9b0e8260c58c6f6829d149920a2", "5680005")    ### [James Chatburn]
addDeezer(mdbmaps, "Top40", "7b58a1b910d5311157faf0d7ba943ffa", "6800529")    ### [Benjamin Joseph]
addDeezer(mdbmaps, "Top40", "110294f1b077f24129fc995ebe23fbe8", "6514885")    ### [Beau Ryan]
addDeezer(mdbmaps, "Top40", "620e410c22308e859ec128e1d31cd98f", "323884")    ### [Pez]
addDeezer(mdbmaps, "Top40", "208619acbc541f424ed93ef7efd3f3cd", "6756463")    ### [Peter Wall]
addDeezer(mdbmaps, "Top40", "6b56ba9a8034ebbfd087f70a7f702a1a", "4079487")    ### [Ceekay Jones]
addDeezer(mdbmaps, "Top40", "04d84fa03fde087f3259c8bdc33752bf", "80523")    ### [Tv Rock]
addDeezer(mdbmaps, "Top40", "1be93f066d18ad80f8bfd2e70f363c0b", "1697944")    ### [Mt Raskil Preservation]
addDeezer(mdbmaps, "Top40", "ce801880919906527590fa4ddaa32c45", "1503955")    ### [Musicians Of The Sydney Symphony]
addDiscogs(mdbmaps, "Top40", "f90e3b3a282edb1c217d6e0fcf997d6c", "2931966")   ### [Prophet]
addDeezer(mdbmaps, "Top40", "f31c59cc71f57ef742aa0db40beb9b12", "280294")    ### [B.o.b]
addDeezer(mdbmaps, "Top40", "5f5e50d6614f04aaefd4995dca22f6b0", "443623")    ### [Jes Hudak]
addDeezer(mdbmaps, "Top40", "2176f53302606c207715827ebaa28e2a", "5523244")    ### [George Krikes]
addAllMusic(mdbmaps, "Top40", "1ea871f975022d52aae6eab3e9f86e5e", "mn0000999409")   ### [Geoffrey Gurrumul Yunupingu]
addDeezer(mdbmaps, "Top40", "5b4514a1943a8b0ecfc49c1feb0b95d7", "173576")    ### [R Kelly]
addDeezer(mdbmaps, "Top40", "95f4e2183d8f5105c6f825eae0181a52", "11645017")    ### [Mzwetwo]
addDeezer(mdbmaps, "Top40", "caf2a08a94f016a97e44a10130f3e3cf", "7374704")    ### [G-Easy]
addDeezer(mdbmaps, "Top40", "897216c66805bf28461c18040b04b67b", "10173710")    ### [Esther Elliott]
addAllMusic(mdbmaps, "Top40", "2f31f75ae3cb6d34dd619a1f3287b9ab", "mn0002135283")   ### [Lz7]
addDeezer(mdbmaps, "Top40", "2f31f75ae3cb6d34dd619a1f3287b9ab", "532751")    ### [Lz7]
addDeezer(mdbmaps, "Top40", "9993a84fddfdfd82804bbfed434d9885", "1825991")    ### [Ruby Frost]
addDeezer(mdbmaps, "Top40", "b242e187ccc3073a56e38ae2ee55e650", "243446")    ### [Drew]
addDeezer(mdbmaps, "Top40", "8e3afdb94e098cd079c307fb95c9218e", "4379877")    ### [Spawnbreezie]
addDeezer(mdbmaps, "Top40", "c8f6f45297ea4a12ea03624a875a212e", "406155")    ### [The Auckland Philharmonia Orchestra]
addDiscogs(mdbmaps, "Top40", "520556e00e6d980c7a428f88a6658c68", "5925200")   ### [Jade Louise]
addDeezer(mdbmaps, "Top40", "2f953cbb100b34dcc4f2d90a8000eacc", "4877604")    ### [Jessie G]
addDeezer(mdbmaps, "Top40", "7b1225826f059f0b7da7a22e128f37fb", "106014482")    ### [Mike Puru]
addDeezer(mdbmaps, "Top40", "79024f1f81e52227802f50f57ad5523d", "1405492")    ### [Seth Haapu]
addDeezer(mdbmaps, "Top40", "ec743dea831270125f51df1df5da8e42", "1182264")    ### [Frisko]
addDiscogs(mdbmaps, "Top40", "d5df77c4abe71b7e73ced4bbc963a71f", "6231462")   ### [Timesthree]
addDeezer(mdbmaps, "Top40", "d5df77c4abe71b7e73ced4bbc963a71f", "1658696")    ### [Timesthree]
addDeezer(mdbmaps, "Top40", "eac8f9698979b290371bd2a392c661fc", "1226318")    ### [Tiki]
addDeezer(mdbmaps, "Top40", "49230df907f220c6fc887f8b142492be", "85563822")    ### [Starboy]
addDeezer(mdbmaps, "Top40", "fca25c35f21674eabb0671a206c03930", "208194")    ### [Kin]
addDeezer(mdbmaps, "Top40", "4b4fd591656c8499d7d511fff78579b7", "11732531")    ### [Pipe Down Project]
addDeezer(mdbmaps, "Top40", "ed0eefd23b87ef67f3d0dd9eadf6226b", "62745")    ### [Arise]
addDeezer(mdbmaps, "Top40", "0e92ee4147f408112ff68c135d5202df", "8915798")    ### [#kiwiscurebatten]
addDeezer(mdbmaps, "Top40", "7f25cab06931e5c754eb74229144ca3a", "1031587")    ### [L. A. B.]
addDiscogs(mdbmaps, "Top40", "26c097ad9f7f55270b85ae02a22bb359", "266816")   ### [Jack L]
addDeezer(mdbmaps, "Top40", "26c097ad9f7f55270b85ae02a22bb359", "1438219")    ### [Jack L]
addDeezer(mdbmaps, "Top40", "dabd78a618b6ed0505657303ac766f81", "6788749")    ### [Broken Hill]
addDeezer(mdbmaps, "Top40", "5a73b17da495da0dc6ac4fff5054f0b8", "292468")    ### [Smash Proof]
addAllMusic(mdbmaps, "Top40", "00d5d59798eeffef591f272260b0bbcc", "mn0000124572")   ### [Chuck-N-Blood]
addAllMusic(mdbmaps, "Top40", "21da687c5162d957e0e8aaa39eff1e80", "mn0003750925")   ### [The Carters]
addAllMusic(mdbmaps, "Top40", "c00c9a5d2e3053716de9b79898cb5691", "mn0001225951")   ### [Mustard]
addAllMusic(mdbmaps, "Top40", "78f1feb9204bce3173d42798bc9bcb91", "mn0001808325")   ### [Kobe Bryant]
addAllMusic(mdbmaps, "Top40", "5edcd28a4505dc29e4edebd67052e614", "mn0000795894")   ### [Dolla]
addAllMusic(mdbmaps, "Top40", "5a888aa5d5b0835533b2a73dd895b461", "mn0002528230")   ### [DCUP]
addAllMusic(mdbmaps, "Top40", "7bcb752a835787affb80072d61387b22", "mn0001411030")   ### [Hollis]
addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addAllMusic(mdbmaps, "Top40", "1f9b34897f6440e2484f1ac5e00953f7", "mn0003462822")   ### [Gnash]
addAllMusic(mdbmaps, "Top40", "9d337e01e83fc2b9d9fcbd6930bfb107", "mn0003715465")   ### [Casper Magico]
addAllMusic(mdbmaps, "Top40", "7670c11b0a85e757c3cd25e1cb4f0d2b", "mn0003496227")   ### [Dakota]
addAllMusic(mdbmaps, "Top40", "29c175308260a766645f88a48fda8cb5", "mn0002141576")   ### [Sasha]
addAllMusic(mdbmaps, "Top40", "20cf770061e851a2768a67afad49a089", "mn0001884165")   ### [Chris Malinchak]
addAllMusic(mdbmaps, "Top40", "80891387fa9995ead4c3a41b8eb58cac", "mn0003165846")   ### [Islove]
addAllMusic(mdbmaps, "Top40", "927d313b814225911b0cc45f6d101384", "mn0003525593")   ### [Neales]
addDiscogs(mdbmaps, "Top40", "000a8119d7146d68b89c0a1190ec7e11", "7474592")    ### [Young T & Bugsey]
addDiscogs(mdbmaps, "Top40", "8869a02f414bf8aa44ffa2491e057c18", "5690424")    ### [Sile Seoige]
addAllMusic(mdbmaps, "Top40", "23766e9564639a78dd2aa4469c656e33", "mn0000186917")   ### [Gabriella]
addAllMusic(mdbmaps, "Top40", "ee06f338805166cc652a5ffa29bfcaa4", "mn0000792831")   ### [Shane Macgowan]
addDiscogs(mdbmaps, "Top40", "338160050dd8760f6e21fd314da0c262", "282163")    ### [Glen Keating]
addDiscogs(mdbmaps, "Top40", "8f7bba2ad693b7bd7a54e9f5561df5fd", "670431")    ### [Redone]
addAllMusic(mdbmaps, "Top40", "61e0cad6ba67b77cf03b21bdcc855000", "mn0001072817")   ### [Ndubz]
addAllMusic(mdbmaps, "Top40", "8c59920da3b9afafbe8064702b8ae417", "mn0000866772")   ### [Levada Louca]
addAllMusic(mdbmaps, "Top40", "f9902337667b007c4cb0bd56853d6f77", "mn0001499382")   ### [Don Mescall]
addDiscogs(mdbmaps, "Top40", "3a78a66700126f076bc9880499d46098", "6754053")    ### [Theresa Rex]
addAllMusic(mdbmaps, "Top40", "45ec0da62cf46f95d1d19d6477e15223", "mn0003492788")   ### [Lewisham & Greenwich NHS Choir]
addAllMusic(mdbmaps, "Top40", "b3a0765a3d6a1f9b5f0218cc1a78167d", "mn0001585547")   ### [DJ Champion]
addAllMusic(mdbmaps, "Top40", "4101fb8a2e7d70d355cb78bb670cafb2", "mn0000615641")   ### [N Trance]
addAllMusic(mdbmaps, "Top40", "d4afc32b214e23fb75edc998fa24f102", "mn0000116049")   ### [D'side]
addAllMusic(mdbmaps, "Top40", "0edfaa56ae9a7023ef89e6dc038b27e7", "mn0000556741")   ### [Marshals]
addDiscogs(mdbmaps, "Top40", "ef5682c651f7c5295bc121ad58f18381", "1627595")    ### [St. Julien]
addAllMusic(mdbmaps, "Top40", "50eab1669ad5105c900ba050d23f7b37", "mn0002085581")   ### [Ruth-anne]
addAllMusic(mdbmaps, "Top40", "123b17da9b4f0bc068b6211be4ba6eeb", "mn0002334760")   ### [Erick]
addDiscogs(mdbmaps, "Top40", "0f4330d2f262dae9ff367c6c07b07ebd", "1551192")    ### [Temper Trap]
addAllMusic(mdbmaps, "Top40", "08de644904d4cf3503bdb90e1095eb0a", "mn0003330733")   ### [High Hopes Choir]
addAllMusic(mdbmaps, "Top40", "1b7dbb15e26894ee42a4088cd348af5c", "mn0002840885")   ### [Frankie Sandford]
addAllMusic(mdbmaps, "Top40", "7eac99ffb3a009020ae826491a7328a2", "mn0002300128")   ### [Calvin Goldspink]
addAllMusic(mdbmaps, "Top40", "a937fec3aae407f22faf4d34080e3c53", "mn0001093965")   ### [Frank Skinner]
addAllMusic(mdbmaps, "Top40", "3772bea79ec33ccc235bd5eb74ec4188", "mn0000662225")   ### [H2O]
addAllMusic(mdbmaps, "Top40", "2bb54463a4b2619f4cddb21626fd43f3", "mn0000443133")   ### [Donaeo]
addDiscogs(mdbmaps, "Top40", "5a56911227b3087c65fa0c64cfc22858", "7175745")    ### [Rani]
addAllMusic(mdbmaps, "Top40", "b44c3d5c561812173cb1fadc6bbadad0", "mn0000663567")   ### [Node]
addAllMusic(mdbmaps, "Top40", "94a91dcb424b476f725b107e3d561dd2", "mn0003085199")   ### [A Typisk]
addDiscogs(mdbmaps, "Top40", "244b104a4ecf5d29056a30135da49164", "5558809")    ### [Amanda]
addDiscogs(mdbmaps, "Top40", "9547f484677db99f07f0d2ce639fbcdb", "2644206")    ### [KESI]
addDiscogs(mdbmaps, "Top40", "cf2ec7a5779606dc225eb1ff52a125b5", "610327")    ### [Six]
addDiscogs(mdbmaps, "Top40", "682ff892fde0720742da7539be5cbfd4", "3769696")    ### [Mellemfingamuzik]
addDiscogs(mdbmaps, "Top40", "964b8471bb5b3317ff80c658789fb5c6", "2178850")    ### [Nadja Ga]
addDiscogs(mdbmaps, "Top40", "5702f798cedf95408e7998cae7160f75", "5094759")    ### [Adhd]
addDiscogs(mdbmaps, "Top40", "5f6a0d2ee2e29501ee8668e87c668653", "8290907")    ### [David O'connor]
addDiscogs(mdbmaps, "Top40", "be31dd84b00112c792101e2ed6bd66b3", "1086751")    ### [Donna & Joe]
addDiscogs(mdbmaps, "Top40", "cec35a4d1e39f4804f0329f8a891b635", "606524")    ### [Johnson]
addDiscogs(mdbmaps, "Top40", "00d39e303b291175cb7dc306b9316607", "1003686")    ### [Kyla]
addAllMusic(mdbmaps, "Top40", "c2a35b01b931ff0ea34c41bb944ed7ad", "mn0000302025")   ### [Rune Rk]
addDiscogs(mdbmaps, "Top40", "906b28e54bbb4b4cf8c91f488d978719", "5024518")    ### [Emile Kruse]
addDiscogs(mdbmaps, "Top40", "b4ac9bf7c517d486d0dfbb31267e4d3f", "6257928")    ### [Pattersutter]
addAllMusic(mdbmaps, "Top40", "ff33e0cca2bc86ca1c8aca2edf6ad878", "mn0000175401")   ### [M.V.P.]
addDiscogs(mdbmaps, "Top40", "c3eb573746b90008c43fb9ed843e01c4", "396225")    ### [U$o]
addDiscogs(mdbmaps, "Top40", "a5e6e308d31db00836b85dec9a28fac6", "120009")    ### [lum]
addDiscogs(mdbmaps, "Top40", "9cc336f08daf6b60fdc7df51c4951cd7", "771237")    ### [Soren Huss]
addDiscogs(mdbmaps, "Top40", "2f8ef80aa737b08ccbf4a4f597272dd4", "585244")    ### [Jimmy Jorgensen]
addDiscogs(mdbmaps, "Top40", "9361a0dd987d763afb7ebda59733dd4b", "164681")    ### [Michael Andr]
addDiscogs(mdbmaps, "Top40", "8c4759f7357435c07209f893bd031627", "500375")    ### [Goleo Vi]
addDiscogs(mdbmaps, "Top40", "df5925386d5e772c54fab9c8ee4afc1a", "2449670")    ### [Bjornskov]
addDiscogs(mdbmaps, "Top40", "ea6d8ca839c5321096bf52075b5913a3", "1091530")    ### [Blaes Bukki]
addAllMusic(mdbmaps, "Top40", "dff7682d6b7f9e652467f1d387940420", "mn0002116244")   ### [Michael Caro]
addDiscogs(mdbmaps, "Top40", "8293b2b928ccdcfda3a5b282235ed4ba", "489717")    ### [L.o.c.]
addAllMusic(mdbmaps, "Top40", "f101734989e2fb6b8dd4f0192a44424e", "mn0001546498")   ### [Sha]
addDiscogs(mdbmaps, "Top40", "c88659abef342c45aaeee57a7208f24b", "3871612")    ### [Jimillian]
addDiscogs(mdbmaps, "Top40", "cde97ecc04c4fb0ab06efcceb6ba1766", "393367")    ### [Monday]
addDiscogs(mdbmaps, "Top40", "f7a9e3bb639bf960c442d3854cc8e7f7", "709641")    ### [Herrelandsholdet]
addDiscogs(mdbmaps, "Top40", "9c0415d6fbfcac5c210c5cb9cd921ffd", "4181123")    ### [Fruhstuck Bei Stefanie]
addAllMusic(mdbmaps, "Top40", "56fb2c6c60bd0165fcec8d4622f92b80", "mn0002139381")   ### [Buergermeister]
addDiscogs(mdbmaps, "Top40", "bb4ac57918a08f80cb48be770d9e9979", "5134105")    ### [Cinco De Mayo]
addDiscogs(mdbmaps, "Top40", "853397c4ed3d2cd920a67cc9c17067f5", "1883733")    ### [M0]
addAllMusic(mdbmaps, "Top40", "31c4d92d280c3a99ad3e1a44e9adaf5a", "mn0003793253")   ### [Mathias Og Henriette]
addAllMusic(mdbmaps, "Top40", "0d437db44c23416f96cf5975535f221f", "mn0001963267")   ### [B.a.n.g.e.r.s]
addAllMusic(mdbmaps, "Top40", "2c045db8ead27ec90b26be684dee48a8", "mn0002329134")   ### [Blaojne]
addDiscogs(mdbmaps, "Top40", "2c045db8ead27ec90b26be684dee48a8", "100681")    ### [Blaojne]
addDiscogs(mdbmaps, "Top40", "ce330beec86bfeb671dc2a2d08562025", "1943966")    ### [Fresh]
addDiscogs(mdbmaps, "Top40", "396a9170c68a28370c6a415e5cb0acdd", "6119064")    ### [LOTTE]
addDiscogs(mdbmaps, "Top40", "112cbf6ca3a878116b385274aa05438d", "1482382")    ### [RAF]
addDiscogs(mdbmaps, "Top40", "a07c6ac1b552021b88cfd2e02e78ade3", "5072342")    ### [Laura Van Den Elzen]
addDiscogs(mdbmaps, "Top40", "dce6aefe16db0f22d8509a37392fbda0", "34669")    ### [Mia.]
addDiscogs(mdbmaps, "Top40", "a5d240665753e6ec5149a4cfec868fe1", "518132")    ### [Black Fooss]
addDiscogs(mdbmaps, "Top40", "5016655838213c82095f3cd9941ab8a8", "86482")    ### [J-luv]
addDiscogs(mdbmaps, "Top40", "014056a5b370a16da25725557f53497b", "1399728")    ### [Zipfelbuben]
addDiscogs(mdbmaps, "Top40", "2cfa11ab68fa15392a28ba3473bfafd5", "1383823")    ### [Dschungel-allstars 2009]
addDiscogs(mdbmaps, "Top40", "a9d6b7af886dba4eaa94ee0e6a0400f9", "264574")    ### [The Frank and Walters]
addAllMusic(mdbmaps, "Top40", "c74211123c6662f96a81ef1dd607613b", "mn0003687505")   ### [The Greatest Showman Ensemble]
addDiscogs(mdbmaps, "Top40", "977d879a9ef599db0179842a90eb0829", "2986568")    ### [Nik & Ras]
addAllMusic(mdbmaps, "Top40", "27b4e2613c000e26afc0dcbf6ab8137a", "mn0001560738")   ### [Milk & Honey]
addDiscogs(mdbmaps, "Top40", "27b4e2613c000e26afc0dcbf6ab8137a", "506822")    ### [Milk & Honey]
addDiscogs(mdbmaps, "Top40", "4fb5ad8251f32ebc71a4caa19a79671c", "1404551")    ### [Rald Schmitz]
addDiscogs(mdbmaps, "Top40", "35a5e92b985eb2530625c792eb8794fd", "2261153")    ### [Ell & Nikki]
addDiscogs(mdbmaps, "Top40", "108e43f1076de906cdd8ba014d4b16ff", "2068922")    ### [Bonez Mc]
addDiscogs(mdbmaps, "Top40", "225ac6dbdf0fffd4a9e113da51ca214d", "6081758")    ### [Brudi30]
addDiscogs(mdbmaps, "Top40", "446d29ccb5a2a3de8ac79412defdc3b0", "4255795")    ### [YouNotUs]
addDiscogs(mdbmaps, "Top40", "c3516450970181d37943c2cd92289bfd", "4367398")    ### [Anthony]
addDiscogs(mdbmaps, "Top40", "df31d4c938e84e9804579b48368955ba", "4453381")    ### [Eicka Jane]
addDiscogs(mdbmaps, "Top40", "5921f9df84764cd6c69a4f92a205f390", "7671579")    ### [Lolo]
addAllMusic(mdbmaps, "Top40", "9be0a055a02f9facc0f0fb7fc693c793", "mn0000753347")   ### [Snap!]
addAllMusic(mdbmaps, "Top40", "a0c07f2c1e5e8b60724c92fb83410552", "mn0003380067")   ### [Coleman Hell]
addDeezer(mdbmaps, "Top40", "768af9dd3025f4e218099084284a501b", "54647592")    ### [Mero]
addDeezer(mdbmaps, "Top40", "52752c575988eb5edea512020cec4858", "2211")    ### [Nena]
addDeezer(mdbmaps, "Top40", "e309c2e3fc905eae304b71e10e82eb99", "75530")    ### [Dream]
addDeezer(mdbmaps, "Top40", "8df03772e7cf5b3390e10574a0a90db2", "1407")    ### [Frames]
addDeezer(mdbmaps, "Top40", "40841ab93e4bc05dbd354f23e76fa1ae", "537158")    ### [Sean Og]
addDeezer(mdbmaps, "Top40", "1f50464766f3a9ce94f1754b0463ec8f", "182603")    ### [Sarah]
addDeezer(mdbmaps, "Top40", "c3e16affa11432c13e8be7cc6c8d736b", "261212")    ### [XX]
addDeezer(mdbmaps, "Top40", "fbf1e430c96f43c82045103a7c1407fd", "253597")    ### [George Murphy]
addDeezer(mdbmaps, "Top40", "03ece6e9a07c0a5915cfe676a05567a2", "543560")    ### [Kaka]
addAllMusic(mdbmaps, "Top40", "cdcec0b0558677aa394a85b105a3699c", "mn0000084053")   ### [The Jam]
addDeezer(mdbmaps, "Top40", "cdcec0b0558677aa394a85b105a3699c", "3944")    ### [The Jam]
addDeezer(mdbmaps, "Top40", "410637b6027dc82a2ce4a93308e58dca", "90401")    ### [Aura]
addAllMusic(mdbmaps, "Top40", "91f266ac5ec2839f80f07b9c5385c524", "mn0000057556")   ### [Bill & Gloria Gaither]
addDeezer(mdbmaps, "Top40", "856bd94164974e377d780028096bc128", "70715")    ### [Martin]
addDeezer(mdbmaps, "Top40", "f9793851d6cb8247b6b003a729ce9dc6", "264387")    ### [Nash]
addDeezer(mdbmaps, "Top40", "6f21b97f1010287aa78ee44d6046702f", "77424")    ### [Karen]
addDeezer(mdbmaps, "Top40", "50201d87ecab39af8f2f0aabc1cffc24", "166116")    ### [Jasmin]
addDiscogs(mdbmaps, "Top40", "f7c1455d4147cbf4dc094aa9c2a39356", "3594438")    ### [Zoo]
addAllMusic(mdbmaps, "Top40", "35d8d65ac040e8756273c2f76884e5f5", "mn0003842449")   ### [Beast Coast]
addDeezer(mdbmaps, "Top40", "c4d67d31024c80e72927ac50751161f7", "16891")    ### [Nicole]
addDiscogs(mdbmaps, "Top40", "b1b3b97ec0478725400c7c8729607be9", "3037345")    ### [Carol Anthony]
addDeezer(mdbmaps, "Top40", "154e9916e9dc4d52c6dcc9ea8c5934f7", "211032")    ### [Departure]
addDeezer(mdbmaps, "Top40", "a52581e5c20a50979faeecde53d01381", "247134")    ### [Yasmin]
addDeezer(mdbmaps, "Top40", "86058aed9a9a3acf13a9a98a36b7e7b1", "16065")    ### [Louise]
addDeezer(mdbmaps, "Top40", "f4223a2e29ad572bdd62408309749e55", "4068228")    ### [M.A.D.]
addDeezer(mdbmaps, "Top40", "dd0c42047d7063ec53285cda72c0610d", "3244121")    ### [Tough Love]
addDeezer(mdbmaps, "Top40", "e29a4723271638f34847fc292ffa48c3", "271117")    ### [James Fox]
addDeezer(mdbmaps, "Top40", "95b9824e26389cf67674f50cb242ef70", "205804")    ### [Unk]
addAllMusic(mdbmaps, "Top40", "914aff29bbb33d9bd8742b9b8d7bbb7c", "mn0001453952")   ### [Chima]
addDeezer(mdbmaps, "Top40", "914aff29bbb33d9bd8742b9b8d7bbb7c", "283635")    ### [Chima]
addDiscogs(mdbmaps, "Top40", "9545ffea8117bf350955621bae797917", "39162")   ### [Clever]
addDeezer(mdbmaps, "Top40", "9545ffea8117bf350955621bae797917", "1261918")    ### [Clever]
addDeezer(mdbmaps, "Top40", "a0eaef106d85a20c4f40697e6be08ade", "1219224")    ### [Rory & The Island]
addDeezer(mdbmaps, "Top40", "7b9aa21b6614e8381a912da660081515", "4791900")    ### [Jordan Egan]
addDeezer(mdbmaps, "Top40", "6ce57bb13155b6a976267ae7fea24a81", "5061058")    ### [The Suspects and Guests]
addDiscogs(mdbmaps, "Top40", "dc9afe97092e87a01ec4127299268b14", "97648")    ### [Tone]
addDeezer(mdbmaps, "Top40", "331b04fbaf10fec210a976adb4b1d562", "62971")    ### [Heavy]
addDeezer(mdbmaps, "Top40", "4534407ec1d2e60bd79803fcac70ddab", "5613402")    ### [Jimmy Magee]
addDeezer(mdbmaps, "Top40", "93e2f319eb80a4d47ed7830e34472de8", "164956192")    ### [Steffany Gretzinger]
addDeezer(mdbmaps, "Top40", "d027967af998dd89ee301fbfadc2a9c6", "7274")    ### [Down]
addDeezer(mdbmaps, "Top40", "aaa8882482fb1c672c7048f6ad980017", "72652862")    ### [Poppy Fields]
addDeezer(mdbmaps, "Top40", "84ea1e6425431af3e3c9132c3ef0dff2", "5359329")    ### [Presentation Secondary School Clonmel]
addDeezer(mdbmaps, "Top40", "23a4949a6ef2a59b0a1992d55a5e7119", "100307")    ### [Versatile]
addDeezer(mdbmaps, "Top40", "387f8b95bf312f4e1f06c3fa8a2e702d", "7204178")    ### [The Zog Chorus]
addDeezer(mdbmaps, "Top40", "d571e0c4adaf9473e43a55048b524222", "248033")    ### [Chuckie]
addDeezer(mdbmaps, "Top40", "83e36a7fbb4e47d90df2c7a5f41ceca5", "5283366")    ### [Avener]
addDiscogs(mdbmaps, "Top40", "d8af7b562fb5b6896e9caf208a7631fe", "2500803")    ### [Daisy Dares You]
addDeezer(mdbmaps, "Top40", "3771c6784980f14a97a79960df44683f", "7416924")    ### [Emzee A]
addDeezer(mdbmaps, "Top40", "ea418f0376767c59aea193fb88f2fbed", "11807729")    ### [Audrey Trainor]
addDeezer(mdbmaps, "Top40", "f39917252c23494a2bffc34608080e34", "4566327")    ### [Edmee]
addDeezer(mdbmaps, "Top40", "13568fea713fb70f4a72987863820910", "67704502")    ### [Paradise Hotel 2019]
addDeezer(mdbmaps, "Top40", "a552723762e26b8bf2ff3cf00abb742e", "76729")    ### [Vera]
addDeezer(mdbmaps, "Top40", "e20047c96a03c801a7a761d639303baa", "105146")    ### [Christina]
addDeezer(mdbmaps, "Top40", "747e53fad10278c0be9f12e5064e1a8a", "9435020")    ### [Ung Cezar]
addDeezer(mdbmaps, "Top40", "59755826fe21e0d913917f38e519bd97", "4037981")    ### [Trinidad James]
addDiscogs(mdbmaps, "Top40", "b091868e0d9e66c4f3cfc3d57c0e15c1", "6754464")    ### [Pauline]
addDiscogs(mdbmaps, "Top40", "fdef672eb79aa1e42d68559231f9e91a", "2891027")    ### [Contiez]
addDiscogs(mdbmaps, "Top40", "376424d25e7c667172ea571a9bd5ecf7", "34061")   ### [DT8]
addDeezer(mdbmaps, "Top40", "376424d25e7c667172ea571a9bd5ecf7", "16653")    ### [DT8]
addDeezer(mdbmaps, "Top40", "298dc80023436fd7d1327755dc0f3aea", "5406531")    ### [Virginia To Vegas]
addAllMusic(mdbmaps, "Top40", "df7a414766c07b227b947fe19d735cca", "mn0000527296")   ### [P.I.]
addDiscogs(mdbmaps, "Top40", "9ffc319f3524de0eb5857b3a041e89d8", "4115550")   ### [Rin]
addDiscogs(mdbmaps, "Top40", "6c757d349b981f59ce15f8050351c33b", "286345")   ### [Androids]
addDiscogs(mdbmaps, "Top40", "2580311e731b9bca913825f23a494d96", "4511617")    ### [Alina]
addDeezer(mdbmaps, "Top40", "f8c6ee3fdec7c8e8d68666db7bd07a5b", "1217914")    ### [Zion & Lennox]
addDeezer(mdbmaps, "Top40", "04b0ea321844b974fa623332ec9e81a8", "1234739")    ### [Nadine]
addDiscogs(mdbmaps, "Top40", "5b813121e727842a70d17d40279c8060", "3915565")    ### [Tieks]
addDeezer(mdbmaps, "Top40", "e6852c19581c5c2842941d6585011de8", "507529")    ### [New Sunset Hotel]
addDeezer(mdbmaps, "Top40", "1ee6a7f79efe291ad8bc9ba50f14ee39", "110592")    ### [Tarja]
addDeezer(mdbmaps, "Top40", "f70f7318448d90d9e2648fa729d0ccd9", "7569860")    ### [Mendoza]
addDeezer(mdbmaps, "Top40", "87019c83d1d9002251d5e0123d73de2f", "4158299")    ### [Ladywell Primary School]
addDeezer(mdbmaps, "Top40", "70fd12ba96ac516bed74bb1918828f96", "7215388")    ### [Wealdstone Raider]
addDeezer(mdbmaps, "Top40", "18a4ef47c6e65a8cd0686f17dcb5a2d6", "458443")    ### [Mike Sheridan]
addDeezer(mdbmaps, "Top40", "b6f0eeb7241d52315a563b41a83f03ab", "12544804")    ### [LIV'n'G]
addDeezer(mdbmaps, "Top40", "3203329df817eab4fd383d2515def0c7", "61557012")    ### [RSPB]
addDeezer(mdbmaps, "Top40", "3de5a67ac001eda9da009b4269f64d72", "84950802")    ### [Tobi & Manny]
addDeezer(mdbmaps, "Top40", "8476f77796a006e9d4cbafa3404aebac", "55142")    ### [Virgo]
addDeezer(mdbmaps, "Top40", "7befb4ecd12fe5e883dc1785be298dab", "134048")    ### [Fabric Bent]
addDiscogs(mdbmaps, "Top40", "9d224a59b3df753402c55e6974c203b5", "1335584")   ### [Eddy Bar]
addDeezer(mdbmaps, "Top40", "9d224a59b3df753402c55e6974c203b5", "152363")    ### [Eddy Bar]
addDeezer(mdbmaps, "Top40", "c039dcafb38ef291cd3576f6f2d96498", "14492701")    ### [Ransom]
addDeezer(mdbmaps, "Top40", "538d641f7c607731ce3c97de84888ddc", "10904")    ### [Trooper]
addAllMusic(mdbmaps, "Top40", "576086654e3dac2a5660989c51366659", "mn0000043629")   ### [Bushido Produziert Sonny Black]
addDeezer(mdbmaps, "Top40", "576086654e3dac2a5660989c51366659", "143975")    ### [Bushido Produziert Sonny Black]
addDeezer(mdbmaps, "Top40", "d4e420120a039e03455ee8536740a0a0", "2128")    ### [Goldfinger]
addDeezer(mdbmaps, "Top40", "d9de59336d2d408662aa03ad71e870b1", "4036827")    ### [Andreas Kummert]
addDeezer(mdbmaps, "Top40", "5edc174fb5fefd278c05b53dbd1bd2e0", "14740207")    ### [Prince Karma]
addDeezer(mdbmaps, "Top40", "603304c35dea9d2995f5ea307fc3e437", "5361924")    ### [Judith Van Hel]
addDeezer(mdbmaps, "Top40", "18e08ec319bb5434ce1be10ede35b88d", "15224255")    ### [KMN Gang]
addDeezer(mdbmaps, "Top40", "043c614068c79f134a62512dd92dd205", "4747977")    ### [Meltem Acikgoz]
addDeezer(mdbmaps, "Top40", "f97c6800d97bc9b645de82758b1b616e", "4844456")    ### [Minds Of 99]
addDeezer(mdbmaps, "Top40", "1f84610afa18b37896d9cb8c3ffb929a", "9356090")    ### [POL1Z1STENS0HN]
addAllMusic(mdbmaps, "Top40", "7dbc379090f7843f537ea0df634b1aad", "mn0000748795")   ### [Shifty]
addDeezer(mdbmaps, "Top40", "7dbc379090f7843f537ea0df634b1aad", "6677")    ### [Shifty]
addDeezer(mdbmaps, "Top40", "a17bf82b0c99a5d6a9fdb61e1cba475b", "567150")    ### [Leanne Moore]
addDeezer(mdbmaps, "Top40", "3a9adb2ba05512379660af30c4499d9e", "255246")    ### [Seneca]
addDeezer(mdbmaps, "Top40", "18091df6740eb23606268e1fe70a832e", "262682")    ### [Dustin The Turkey]
addDeezer(mdbmaps, "Top40", "ab0121020ea6f52f5dcc2b0ad8417684", "1569593")    ### [Kanyu Tree]
addDeezer(mdbmaps, "Top40", "1feb0b498fd6b76e794cd4994499bd66", "471918")    ### [Scuba Dice]
addDeezer(mdbmaps, "Top40", "3531e20a4eaab8cd5b085fc9605f6fba", "261093")    ### [Martin Staunton Band]
addDeezer(mdbmaps, "Top40", "8d229bc5952f811627abcecf1a1d9f6d", "404641")    ### [Soul Control]
addAllMusic(mdbmaps, "Top40", "c87cad8f7c42f2e678a0c18782869407", "mn0001951902")   ### [Conway Sisters]
addDeezer(mdbmaps, "Top40", "c87cad8f7c42f2e678a0c18782869407", "5441329")    ### [Conway Sisters]
addDeezer(mdbmaps, "Top40", "bf39144a5e6eb48401d5732ec4edddf4", "15811")    ### [Delirious]
addDeezer(mdbmaps, "Top40", "d1029792b9078894873b1965b343540c", "262762")    ### [Walls]
addDiscogs(mdbmaps, "Top40", "009bc268f4161c893c3469d0f475fa03", "393367")    ### [The Revs]
addDeezer(mdbmaps, "Top40", "0166cabfe7594a261af1250ad91c5ebe", "3039")    ### [Di-rect]
addDeezer(mdbmaps, "Top40", "79947bd82f85385cf579852761e3bcca", "5432668")    ### [Grafa]
addAllMusic(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "mn0003729938")   ### [Z. E.]
addDeezer(mdbmaps, "Top40", "73ea300db5aa1c2154adfe42186a887e", "5418")    ### [Ark]
addDeezer(mdbmaps, "Top40", "c0e4ba1ede3532ccfde5b29e8aacc50b", "789194")    ### [Skryabin]
addDeezer(mdbmaps, "Top40", "12a8a35953b0633264cd439f7dc723ad", "13244969")    ### [Elias]
addDiscogs(mdbmaps, "Top40", "60044c2592b77d0e4a7d4df0d5fdcbba", "908078")    ### [Julia Savicheva]
addDiscogs(mdbmaps, "Top40", "c8c1ade4a56b82b143856c2c590e3fc1", "369368")    ### [Gola]
addDiscogs(mdbmaps, "Top40", "8422de1228cdf9aa9ee5888a858a98b1", "8707189")   ### [Mihaela Fileva]
addDeezer(mdbmaps, "Top40", "8422de1228cdf9aa9ee5888a858a98b1", "5471912")    ### [Mihaela Fileva]
addDeezer(mdbmaps, "Top40", "8e0a54291a8ae6add5c89ae6bd744591", "2077221")    ### [Pavell]
addDeezer(mdbmaps, "Top40", "5793bffe6cf2b42fbcf5ecc4cdf66911", "4796386")    ### [Venci Venc']
addDeezer(mdbmaps, "Top40", "e4a1ed07636cc650fcd92f2ee99a44fe", "1288678")    ### [Lazza]
addDeezer(mdbmaps, "Top40", "c29dd858b42c5e8d6caecd0302c47dfe", "9060182")    ### [Hov 1]
addDeezer(mdbmaps, "Top40", "981871e41d004689023b3867cca2003c", "4239461")    ### [Maan]
addDeezer(mdbmaps, "Top40", "6eb8df7aacfb80b8a5aefbb8155dc13a", "247284")    ### [George Sampanis]
addDeezer(mdbmaps, "Top40", "daf03202751046b2d8cc380e7535cc39", "529")    ### [Raphael]
addDeezer(mdbmaps, "Top40", "8a65b04059e971e76dc736fcf1ab4dd2", "13440379")    ### [Nino]
addDeezer(mdbmaps, "Top40", "1533082f4491090f90aa6f086be1b2b6", "4018571")    ### [Arif]
addDeezer(mdbmaps, "Top40", "3e5169154b281b19469ce1fb806a3919", "795146")    ### [Machete]
addDeezer(mdbmaps, "Top40", "cd5004d3b03507b93a5154c8f703ffb1", "1358419")    ### [KataStrofe]
addDeezer(mdbmaps, "Top40", "f4c09b9cc9b7184e363a22d1a7565ef9", "176639")    ### [Nitro]
addDeezer(mdbmaps, "Top40", "a2f3f9edf03e4bf85df289edf3084780", "58916452")    ### [Dnoteondabeat]
addDeezer(mdbmaps, "Top40", "8bbd146262d01da925cec72df725f3cf", "11937599")    ### [Blockparty]
addDeezer(mdbmaps, "Top40", "0d866c18af54941ec8f33ce3df4be902", "3848")    ### [Elize]
addDeezer(mdbmaps, "Top40", "0c6423fdddb18f976061565d288baded", "1005065")    ### [Como]
addDiscogs(mdbmaps, "Top40", "0f17d781876da4732563e43039aaaa8b", "448654")   ### [L5 (Popstars)]
addDeezer(mdbmaps, "Top40", "ce3fbaee597ec5a524a308b812541f56", "214934")    ### [Rebecca]
addDeezer(mdbmaps, "Top40", "ff69c3ee307619b22c2ee2969aea521d", "11328")    ### [Dj Jose]
addDiscogs(mdbmaps, "Top40", "8f7b584aa940b5b8d46129956fee0c52", "2250857")   ### [Vremia I Steklo]
addDeezer(mdbmaps, "Top40", "8f7b584aa940b5b8d46129956fee0c52", "4878214")    ### [Vremia I Steklo]
addDeezer(mdbmaps, "Top40", "1af733e9abe9956b5d2ef9fdbb1ad764", "537741")    ### [Grigoriy Leps]
addDeezer(mdbmaps, "Top40", "a2ee02d0ff400d2ec0d77d2aae27d098", "164423")    ### [Stan]
addDeezer(mdbmaps, "Top40", "cb1a4cdd6178852f36aba64d12602d1a", "281011")    ### [India Martinez]
addDeezer(mdbmaps, "Top40", "2fbb9be1f137d28d5d01a88b2a61c3a6", "1366660")    ### [Taisiya Povaliy]
addDeezer(mdbmaps, "Top40", "43a4a6af3a411e6c9a24170f30c2815b", "13612387")    ### [Maneskin]
addDeezer(mdbmaps, "Top40", "9318518ff509296fc43d312a9d65ff56", "101471")    ### [Pegasus]
addDeezer(mdbmaps, "Top40", "0abaf01f580932db3646af80a6023868", "1710654")    ### [Potap & Nastia]
addAllMusic(mdbmaps, "Top40", "e3b7acbc2078d13eafa2a765ce38115a", "mn0002080697")   ### [Luttenberger & Klug]
addDeezer(mdbmaps, "Top40", "e3b7acbc2078d13eafa2a765ce38115a", "1495367")    ### [Luttenberger & Klug]
addAllMusic(mdbmaps, "Top40", "760ffac9c884a2e8fabc13d4eb617e75", "mn0002744178")   ### [Kevin]
addDeezer(mdbmaps, "Top40", "760ffac9c884a2e8fabc13d4eb617e75", "70615702")    ### [Kevin]
addAllMusic(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "mn0003060329")   ### [V. Meladze]
addDeezer(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "4251519")    ### [V. Meladze]
addDiscogs(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "753179")
addDeezer(mdbmaps, "Top40", "299683e0e48d3508d34c59fd60d1c969", "6138354")    ### [Lora Karadzhova]
addAllMusic(mdbmaps, "Top40", "9277fef69257e9c368980e44addab30d", "mn0002598754")   ### [Tove Ostman Styrke]
addDiscogs(mdbmaps, "Top40", "9277fef69257e9c368980e44addab30d", "2023759")    ### [Tove Ostman Styrke]
addAllMusic(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "mn0002764822")
addDiscogs(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "2241934")   ### [Maksim Barskih]
addDeezer(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "1365309")    ### [Maksim Barskih]
addDeezer(mdbmaps, "Top40", "f5cfbe3d9410cd26c9f5db2f5d00e2af", "64976")    ### [Yasin]
addAllMusic(mdbmaps, "Top40", "2da7b3559029e16c501f19f2ca9e1f3d", "mn0001509405")   ### [Valeria]
addDiscogs(mdbmaps, "Top40", "2da7b3559029e16c501f19f2ca9e1f3d", "907332")    ### [Valeria]
addDeezer(mdbmaps, "Top40", "71495276459b2c8d9319a4f23736b1fe", "4188612")    ### [NeAngely]
addDiscogs(mdbmaps, "Top40", "df1a1cf249844e537d5f8e301999f050", "508430")   ### [Romeo]
addDeezer(mdbmaps, "Top40", "df1a1cf249844e537d5f8e301999f050", "66510")    ### [Romeo]
addDiscogs(mdbmaps, "Top40", "ddcae8e624e066c57c1a76f7a50c3be5", "5163802")   ### [Pianoboy]
addDeezer(mdbmaps, "Top40", "ddcae8e624e066c57c1a76f7a50c3be5", "4444748")    ### [Pianoboy]
addDeezer(mdbmaps, "Top40", "57dd82cd58ebc1ee03495e7111f68837", "69824902")    ### [Adel]
addDeezer(mdbmaps, "Top40", "9790bfbf128c234ff3dfe9734a29112d", "3097771")   ### [S. Tarabarova]
addDiscogs(mdbmaps, "Top40", "9790bfbf128c234ff3dfe9734a29112d", "3180922")    ### [S. Tarabarova]
addAllMusic(mdbmaps, "Top40", "e87ca03c7907697d6b985a36efec96b3", "mn0000907539")   ### [Dolcenera]
addDeezer(mdbmaps, "Top40", "e87ca03c7907697d6b985a36efec96b3", "77036")    ### [Dolcenera]
addDiscogs(mdbmaps, "Top40", "1a1c4cd2bf4d0da1c3ac37eed6c5e6dd", "349539")   ### [Seer]
addDiscogs(mdbmaps, "Top40", "479632c7dad0bc6491b1dc428c2f384b", "323925")   ### [Carola]
addDeezer(mdbmaps, "Top40", "479632c7dad0bc6491b1dc428c2f384b", "9621")    ### [Carola]
addDeezer(mdbmaps, "Top40", "1018fb9997d689d2cfbeb250cba1e0fc", "8731")    ### [Erika]
addAllMusic(mdbmaps, "Top40", "74d19220f1e9035dc9be17ab0fa7d496", "mn0003059139")   ### [Gradysi]
addDeezer(mdbmaps, "Top40", "5eb7994fa8fc5d454eb9975e8d0982b3", "8399")    ### [Wigwam]
addDeezer(mdbmaps, "Top40", "ec7bcd247bd91b161fd3e236faf60113", "8359")    ### [Laura]
addDeezer(mdbmaps, "Top40", "8cc666d82558e9a3801428ce9377e0cc", "243610")    ### [Eri Esittajia]
addDeezer(mdbmaps, "Top40", "14035f38e39640c7b8a126848f15a17f", "6329")    ### [Belinda]
addDeezer(mdbmaps, "Top40", "ace5274eac9ac83ecc8be584f2f89678", "389681")    ### [Jezabels]
addDeezer(mdbmaps, "Top40", "f808aec316082282ad04c543ea7293fc", "1532710")    ### [Gossling]
addDeezer(mdbmaps, "Top40", "8248a0d3d76dd9b6e989ed30758e1fdb", "7727636")    ### [Young Tapz]
addDeezer(mdbmaps, "Top40", "9abfa0a95115150fa9854d05888b30ad", "323")    ### [T-pain]
addDeezer(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "1226315")    ### [J.williams]
addDeezer(mdbmaps, "Top40", "d8efa30039a62c84a5cd7aae07baaed4", "9388984")    ### [Jess & Matt]
addDeezer(mdbmaps, "Top40", "3b51465a51f53de3f9406c8c4084b9eb", "4771547")    ### [Hannah]
addDeezer(mdbmaps, "Top40", "ff33e0cca2bc86ca1c8aca2edf6ad878", "146348")    ### [M.V.P.]
addDeezer(mdbmaps, "Top40", "7670c11b0a85e757c3cd25e1cb4f0d2b", "121457")    ### [Dakota]
addDeezer(mdbmaps, "Top40", "aca525c6189a7e1746b237ac3c8e75f1", "241381")    ### [OpShop]
addDeezer(mdbmaps, "Top40", "75addee6118eeea8b15dfa7d27a7bc56", "132842")    ### [Gin]
addDeezer(mdbmaps, "Top40", "40841ab93e4bc05dbd354f23e76fa1ae", "537158")    ### [Sean Og]
addDeezer(mdbmaps, "Top40", "8a8901e987f198d467979ba02675bf2a", "480089")    ### [360]
addDeezer(mdbmaps, "Top40", "8b8ee2fa941154779a27093f715c82c1", "2476")    ### [Whitlams]
addDeezer(mdbmaps, "Top40", "f1e6489254cab6f1fcbe17478f66812d", "9164")    ### [Presets]
addDeezer(mdbmaps, "Top40", "4cbe8a7a881e6bbb1b2ad97de6caa28c", "76000")    ### [Stellar]
addDeezer(mdbmaps, "Top40", "33db69043854dc72c7ae28f79e66b295", "134078")    ### [Bleeders]
addDeezer(mdbmaps, "Top40", "5fbe502afaeea5cd3894fbbacc76d0f0", "72186")    ### [Frontline]
addAllMusic(mdbmaps, "Top40", "8776b0955cc02e70c84df62f5d08783c", "mn0000288296")   ### [k-os]
addDeezer(mdbmaps, "Top40", "8776b0955cc02e70c84df62f5d08783c", "1666")    ### [k-os]
addDeezer(mdbmaps, "Top40", "34110bca84cc003cb7f9663d441b97f3", "404975")    ### [Swiss]
addDeezer(mdbmaps, "Top40", "cf724ae1b389235411e9848ab2492b86", "1263309")    ### [Jason Kerrison]
addDeezer(mdbmaps, "Top40", "92b769e55de140d27e26b4f52382dc3c", "4344203")    ### [JGeek]
addDeezer(mdbmaps, "Top40", "fe6ff35dd5203e256dddc02fa796c5f6", "5611287")    ### [Arise Church]
addDeezer(mdbmaps, "Top40", "b877108ea31ee744be61d4cf33ddf382", "4423398")    ### [Lavina Williams]
addDeezer(mdbmaps, "Top40", "fd41667c368acecbf8d672b7afbed008", "476858")    ### [Aftermath]
addDeezer(mdbmaps, "Top40", "3682bbc007fa90e18375fa5edb0c37b3", "76360")    ### [Toya]
addDeezer(mdbmaps, "Top40", "4157b21887eedb61b04d15ae42183eae", "4757049")    ### [Dolla & Skeet]
addDeezer(mdbmaps, "Top40", "a663ace511580671d69a6b3f916fe919", "1116771")    ### [Catfish & The Bottlemen]
addDiscogs(mdbmaps, "Top40", "dc9afe97092e87a01ec4127299268b14", "97648")    ### [Tone]
addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addDeezer(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "7196728")    ### [Jennifer Lawrence]

mdbmaps["Top40"].save()

## BillboardYE

In [ ]:
addDeezer(mdbmaps, "BillboardYE", "52a0965785c941aa958b64dc50f3d972", "176485")    ### [Fuego]
addDeezer(mdbmaps, "BillboardYE", "ccc783c2cd738ace20a5315e7d43e547", "3069")    ### [Rakim]
addDeezer(mdbmaps, "BillboardYE", "edac53ed142896746b3bd4c681c12f64", "347454")    ### [Tim / Berg]
addDeezer(mdbmaps, "BillboardYE", "a7fa9eedf990ab73ba0d7e60db65c95d", "9081")    ### [Eric Prydz Vs. Floyd]
addDeezer(mdbmaps, "BillboardYE", "fbe56cc641e9d04e1b4fcdd924a9c8cc", "318079")    ### [Carlos y Alejandra]

addDeezer(mdbmaps, "BillboardYE", "37f8c0772c31a769f56239563ad7b458", "6500875")    ### [Anuel]
addDeezer(mdbmaps, "BillboardYE", "1c014b1998bb4fab0322f211eec0c611", "445119")    ### [La Adictiva]
addDeezer(mdbmaps, "BillboardYE", "17d5bdd02ad4dee110445a03f919b8ac", "57843")    ### [Marlon]
addDeezer(mdbmaps, "BillboardYE", "33ab8be62d65b7d75df716432b86336f", "240427")    ### [Sky]
addDeezer(mdbmaps, "BillboardYE", "c133daae0111978ba09001d2e7de72f1", "5611")    ### [Willie Colon]
addDeezer(mdbmaps, "BillboardYE", "6510784503b99747dfee77f5a079476d", "50683152")    ### [Banda MS de Sergio Lizarraga
addDeezer(mdbmaps, "BillboardYE", "3f5ee815ed36b113db281ce7ed332da4", "71922")    ### [Dino]
addDeezer(mdbmaps, "BillboardYE", "8046ee19ecf0576f13f6520114447c52", "325722")    ### [James Lloyd]
addDeezer(mdbmaps, "BillboardYE", "73a58c444f3a947331cc0f62fff0cfd5", "628982")    ### [Bastien Laval]
addDeezer(mdbmaps, "BillboardYE", "e28b088f9d859934e4e4cbb39c431199", "4948050")    ### [Il Pomo d'Oro]
addDeezer(mdbmaps, "BillboardYE", "ef95f8a3355a4bbe43fceb8f72be7f9f", "145155")    ### [Divino]
addDeezer(mdbmaps, "BillboardYE", "adc972062945092b4343017780b6c242", "4065944")    ### [Hnos. Vega Jr.]
addDeezer(mdbmaps, "BillboardYE", "8e9f75d573db0b3fd1c5e8b48e3301cc", "4330924")    ### [Codigo FN]

addAllMusic(mdbmaps, "BillboardYE", "0daf21abe59aac57d10e0fe76f304077", "mn0002686289")   ### [Ben Williams]
addDeezer(mdbmaps, "BillboardYE", "0daf21abe59aac57d10e0fe76f304077", "485014")    ### [Ben Williams]
addDeezer(mdbmaps, "BillboardYE", "093b82ff928d96dddbc505cd2f1ddb1f", "469114")    ### [AB Quintanilla]
addDeezer(mdbmaps, "BillboardYE", "d98278206c2f7c2e6232599ee9cb6bc0", "145593")    ### [Adam Holzman]
addDeezer(mdbmaps, "BillboardYE", "dd88591508a27fecc4b05c47f7d9698f", "108626102")    ### [James F. Twyman]

mdbmaps["BillboardYE"].save()

## Billboard

In [ ]:
addDeezer(mdbmaps, "Billboard", "4f617f7ae1994485f0c6fb1e739faa0f", "11064346")    ### [Anibal de Gracia y Sus Invitados]

addDeezer(mdbmaps, "Billboard", "246d14ae520c0bd8adc50cb10538805a", "93555")    ### [Lalo Rodriguez]
addDeezer(mdbmaps, "Billboard", "1f591df9ecc72cdbecc5ca37c953b722", "58203022")    ### [Joe Sambo]
addAllMusic(mdbmaps, "Billboard", "0ac87a2703773df3ee90f35051976d6e", "mn0003205412")   ### [Yuan Yawei]
addDeezer(mdbmaps, "Billboard", "5614aee754da6707437a67aba21ae97b", "9846490")    ### [Gabo Parisi]
addDeezer(mdbmaps, "Billboard", "c751a299bc9b7926f25dd12c5c16c40b", "6459251")    ### [Los de La Nazza]
addDeezer(mdbmaps, "Billboard", "f0ea2c0f6ed64d3d6bc31d1f381092b0", "5138573")    ### [Cantor Yitzchak Meir Helfgot]
addDeezer(mdbmaps, "Billboard", "3c2a14f3f9126d17f7f2fdbfd3e04295", "5162473")    ### [Joe Vasconcelos]
addDeezer(mdbmaps, "Billboard", "fa0c3060dc1ed49905865587788668b7", "4277041")    ### [Hector]
addDeezer(mdbmaps, "Billboard", "d03d6af3ef8c0c4d1e104fe65d94cfb0", "111522992")    ### [Matsuri Nine.]
addDiscogs(mdbmaps, "Billboard", "75d13abe72774129842a4beff75739ea", "7964912")    ### [The Elovaters]
addDeezer(mdbmaps, "Billboard", "75d13abe72774129842a4beff75739ea", "11723379")    ### [The Elovaters]
addDeezer(mdbmaps, "Billboard", "ae1cfe75bb6f91da7d223452168a3d59", "65634")    ### [Vicious]
addAllMusic(mdbmaps, "Billboard", "1f6381c06850170b54c95e6a596ffb28", "mn0003782313")   ### [Ken Tackey]
addDeezer(mdbmaps, "Billboard", "6b3628493b920913e8d550fd3eb75703", "10908080")    ### [Hoo Leeger]
addDeezer(mdbmaps, "Billboard", "bee91a30c653a9d5146bdce553318cf4", "11975779")    ### [Hwang ChiYeul]
addAllMusic(mdbmaps, "Billboard", "7270cf68e89cab0f5febe2edaed0b7d6", "mn0003487021")   ### [Subaru Shibuya]
addAllMusic(mdbmaps, "Billboard", "42bb5146be764807cc62ea7cd44bff02", "mn0002503541")   ### [CNBLUE]
addDeezer(mdbmaps, "Billboard", "42bb5146be764807cc62ea7cd44bff02", "1492698")    ### [CNBLUE]
addDeezer(mdbmaps, "Billboard", "24d5cd50291e2b5c4c2eb9f3cddce277", "65336")    ### [Fiji]
addDeezer(mdbmaps, "Billboard", "725c8f573f9f00fd36ea148172338bbc", "133611")    ### [L'arc En Ciel]
addAllMusic(mdbmaps, "Billboard", "1d5e1f09e81138a1f8efc590f757fc8e", "mn0003255680")   ### [Kakigoori]
addDiscogs(mdbmaps, "Billboard", "a626013386ca6bc138efacdf05aa7a96", "380314")    ### [Mickey]

addDeezer(mdbmaps, "Billboard", "8cf125b4398ec24a60757a1ae0dbde6a", "100731")    ### [Yuri]
addDeezer(mdbmaps, "Billboard", "1a437e23ad6fb8d3077e86589b52a1a3", "9029")    ### [Lucero]
addDeezer(mdbmaps, "Billboard", "a1afb34f24048e019a601fbb42ae6180", "16319")    ### [Banda El Recodo]
addDeezer(mdbmaps, "Billboard", "4b8310b8cd692ffb4a081e8bb4f69ed9", "16599")    ### [Pandora]
addDeezer(mdbmaps, "Billboard", "e75095d73c04898447d4658e618263f3", "1641733")    ### [Jessie Morales: El Original De La Sierra]
addDeezer(mdbmaps, "Billboard", "bff236756e7bf0dd73d59584e3e1bfe7", "4962433")    ### [Rogelio Martinez "EL RM"]
addDeezer(mdbmaps, "Billboard", "c9e0e91d959d73ff5edfb706e6ffa684", "149")    ### [iiO]
addDeezer(mdbmaps, "Billboard", "bb927e3aee770897812deb1e8ab31eae", "264210")    ### [Anaís]
addAllMusic(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "mn0000392577")   ### [R. Ayala]
addDiscogs(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "5845005")    ### [R. Ayala]
addDeezer(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "115142")    ### [R. Ayala]

addDeezer(mdbmaps, "Billboard", "878d720ee17c0c0a4ef8f071950f88f6", "17053")    ### [Voyage]
addDeezer(mdbmaps, "Billboard", "d2122c257dd5b8a3c4c0ac06b2abd0e2", "78111")    ### [The Chimes]
addDeezer(mdbmaps, "Billboard", "9ab14919e655dc9669042291d6771569", "7298")    ### [Andy Bell]
addDeezer(mdbmaps, "Billboard", "6c9e35fa812bd11f2496be3aa1076cd2", "1436738")    ### [Kimberly Davis]
addDeezer(mdbmaps, "Billboard", "a4952315ff2618cc4b878de65a4dfe6a", "494171")    ### [Fever]
addDeezer(mdbmaps, "Billboard", "d79329a688889810aa7ee43aef41c59b", "975")    ### [Lime]
addDeezer(mdbmaps, "Billboard", "e8b586570af23bcd228f54f4af667358", "10716495")    ### [Slingshot]
addDeezer(mdbmaps, "Billboard", "37f4a4ef2c5a3db1176b0d514fc5ebf1", "264546")    ### [Bingo Boys]
addDeezer(mdbmaps, "Billboard", "cafd8f2f1480ab5c07e5a677f94539a2", "149")    ### [iio]
addDeezer(mdbmaps, "Billboard", "c858d2c428f54d66c563a498860fbf56", "184317")    ### [Dany]
addDeezer(mdbmaps, "Billboard", "261a823373b1c2d3538e401b824a0705", "120034")    ### [Gia]
addDeezer(mdbmaps, "Billboard", "52aac3cde8dbb07e987225096f7d3df7", "9710684")    ### [GOODBOYS]

addDeezer(mdbmaps, "Billboard", "dd9ca9f1eec3ec7324764098928fa90a", "62490")    ### [Nils]
addDeezer(mdbmaps, "Billboard", "8126b866ffdc150d799057c736ff708e", "577628")    ### [Steve Oliver]
addDeezer(mdbmaps, "Billboard", "4eb35c41efdc8495ebca5e64cb723217", "4518873")    ### [Jeff Ryan]
addDeezer(mdbmaps, "Billboard", "96aa7eddeddf7079189c5f9c2feb0378", "1594391")    ### [Kim Scott]

addDeezer(mdbmaps, "Billboard", "a0ec3ffe08120ccf0c9b5345714a5c51", "2743")    ### [LTD]
addDiscogs(mdbmaps, "Billboard", "fa00044058206494d4335094632fe4eb", "83751")    ### [Boss]
addDeezer(mdbmaps, "Billboard", "9a0fdd63c4aa97fc9716413a6111c187", "148184")    ### [Enchantment]
addDeezer(mdbmaps, "Billboard", "ef9d8e82a6937e87668db5702cd43b2f", "1816361")    ### [UMC's]
addDeezer(mdbmaps, "Billboard", "71e904485de1ca65917eb99f886f9c4d", "69608322")    ### [Marty]
addDeezer(mdbmaps, "Billboard", "7a85a9d8804e19413bab514f5f51dfb9", "8322164")    ### [Donnie Trumpet & The Social Experiment]
addDeezer(mdbmaps, "Billboard", "4fdc8908f2ee7ad25e35f2ad9531c681", "15080705")    ### [Dennis Reed & Gap]
addDeezer(mdbmaps, "Billboard", "6c999baddd68939e501aa0561d3e0eda", "4052173")    ### [King Bach]
addDeezer(mdbmaps, "Billboard", "ba6cb719ca53bf4f57ecdcd6bd342bf5", "59927")    ### [Magoo]
addDeezer(mdbmaps, "Billboard", "55be138b7b3faad5764a62dc13c9c402", "1236346")    ### [John Reilly]
addDeezer(mdbmaps, "Billboard", "4ba36e510dd7bbd2009fb8ff90aa48c7", "1575744")    ### [Francis & The Lights]
addDeezer(mdbmaps, "Billboard", "07c0deba90ed751e5c2ec92d0d60d7dd", "59218332")    ### [Brooke Staten]

addDeezer(mdbmaps, "Billboard", "66dca5283eac29a7b8c9714ed919bb78", "442981")    ### [Red Sun Rising]
addDeezer(mdbmaps, "Billboard", "7087547bca580c898106d8f7eb761f58", "251366")    ### [Michael Peterson]
addDeezer(mdbmaps, "Billboard", "b470feb6f3d72bfae87a83e16dd83886", "76640")    ### [Sons of the Desert]

addDeezer(mdbmaps, "Billboard", "a6fbf98a79a048834d50ce68d67ad442", "546")    ### [Mya]
addDeezer(mdbmaps, "Billboard", "3bb7b26b6dee914d5d35651092ebbd38", "123941")    ### [Questian Mark & The Mysterians]
addDeezer(mdbmaps, "Billboard", "c5281bc610aaf5fcb1be3ea9b5aef4a7", "1078918")    ### [Bo Donaldson And The Heywoods]
addDeezer(mdbmaps, "Billboard", "ee476cb24bebb0cbe4d7061b0d33a707", "1533511")    ### [Rock Star Supernova]
addDeezer(mdbmaps, "Billboard", "31aa5ad81b2a6342572b6bd54425deae", "390649")    ### [Iration]
addDeezer(mdbmaps, "Billboard", "808c8b3a9909c7d7a3e5753792d1b995", "1225601")    ### [LV]

mdbmaps["Billboard"].save()

In [ ]:
updateManDB(2)

In [ ]:
toMatchDF = DataFrame({item[0]: {"ArtistName": item[1]["ArtistName"], "ArtistAlbums": item[1]["ArtistAlbums"]} for item in toMatch[chartType]}).T
toMatchDF["NumAlbums"] = toMatchDF["ArtistAlbums"].apply(len)
toMatchDF = toMatchDF.sort_values(by="NumAlbums", ascending=False)

for idx,row in toMatchDF.iterrows():
    artistName = row["ArtistName"]
    albumNames = row["ArtistAlbums"]
    nAlbums    = row["NumAlbums"]
    if nAlbums >= 1:
        print(artistName,'\t',nAlbums)
    else:
        break
#https://kworb.net/itunes/extended.html



# Renames

In [ ]:
cad.getArtistData("Dream")

In [23]:
toget, togetID, dbRenames = analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=1.0)
analyzeRenames(manDB, dbRenames)

Artist ID [1102536] is not a member of artistIDToName.
Artist ID [0000108905] is not a member of artistIDToName.
Artist ID [459440] is not a member of artistIDToName.
Artist ID [0000252794] is not a member of artistIDToName.
Artist ID [344672] is not a member of artistIDToName.
Artist ID [0000028948] is not a member of artistIDToName.
Artist ID [82656] is not a member of artistIDToName.
Artist ID [0000032614] is not a member of artistIDToName.
Artist ID [0000773745] is not a member of artistIDToName.
Artist ID [1309989] is not a member of artistIDToName.
Artist ID [0000855695] is not a member of artistIDToName.
Artist ID [428029] is not a member of artistIDToName.
Artist ID [0000145904] is not a member of artistIDToName.
Artist ID [159370] is not a member of artistIDToName.
Artist ID [0001712608] is not a member of artistIDToName.
Artist ID [915906] is not a member of artistIDToName.
Artist ID [1534354] is not a member of artistIDToName.
Artist ID [6269664] is not a member of artistIDT

In [26]:
updateManDB(4)
#updateMultiDB(2)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 8003 artist keys.
  There are currently 9338 renamed artist keys.
Trying to add 7 renamed artist keys
There are currently 8003 artist keys.
There are currently 9338 renamed artist keys.
There are currently 8009 artist keys.
There are currently 9345 renamed artist keys.
Saving 8009 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  --> This file is 211.8kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  --> This file is 211.8kB.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 8009 artist keys.
  There are currently 93

In [27]:
removeDBRenames(mdbmaps, chartType, dbRenames)
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)
maindb     = loadMainDB(True)

Could not remove PrimaryKey: 1b2f12cc6e446ba540bcbe50ad313bcc
Could not remove PrimaryKey: efcad52eba2a1c9e8c3c9c59193f7cfa
Could not remove PrimaryKey: ec235e7844eca64b580a61ce259a85ae
Could not remove PrimaryKey: b07991c4ff7a4049bf0e0c24638a11f7
Could not remove PrimaryKey: 72b2b4eeaacd656fc4b98c2c43a96137
Could not remove PrimaryKey: b6b762730d1ce14aee5ddb63ecfe2320
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbMusicVFMap.p
  --> This file is 712.9kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbMusicVFMap.p
  --> This file is 712.9kB.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 8009 artist keys.
  There are currently 9345 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  Ther

  There are currently 8009 artist keys.
  There are currently 9345 renamed artist keys.
Adding 1 known multi delim artists.
Current Time is Tue Feb 23, 2021 09:45:55 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 15511 Artists
	Shape --> (15511, 1)
	Shape --> (15511, 2)
  Finding Real Artist Name
	Shape --> (15511, 4)
  Removing None Artist
	Shape --> (15511, 4)
  Finding Disc Artist Name
	Shape --> (15511, 5)
DataFrame Shape is (15511, 7)
Current Time is Tue Feb 23, 2021 09:45:55 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 377.0 millseconds.
Saving Master Artist DB File: /Users/tgadfort/Music/Discog/db-deezer/MasterSlimArtistDB.p
  --> This file is 2.0MB.
========================= masterArtistNameDB ==========

Summary Statistics For DB: Deezer
    Using Known Artists: False
    Found 15511 ID -> Name entries
    Found 15326 Name -> ID entries
    Found 0 Albums
=========================   albumoftheyear   =========================
Summary Statistics For DB: AlbumOfTheYear
    Using Known Artists: False
    Found 8328 ID -> Name entries
    Found 8230 Name -> ID entries
    Found 0 Albums
=========================   genius   =========================
Summary Statistics For DB: Genius
    Using Known Artists: False
    Found 371 ID -> Name entries
    Found 367 Name -> ID entries
    Found 1153 Albums
=========================   kworbspotify   =========================
Summary Statistics For DB: KWorbSpotify
    Using Known Artists: False
    Found 7122 ID -> Name entries
    Found 7035 Name -> ID entries
    Found 7122 Albums
=========================   kworbyoutube   =========================
Summary Statistics For DB: KWorbYouTube
    Using Known Artists: False
    Found 1472 ID -> Name ent

# Extra

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsDiscogs import dbArtistsDiscogs
dbAP = dbArtistsAllMusic()




artistURLs=["https://www.allmusic.com/artist/heat-mn0002185241"]
for artistURL in artistURLs:
    artistID=dbAP.dutils.getArtistID(artistURL)
    savename=dbAP.dutils.getArtistSavename(artistID)
    dbAP.dutils.downloadArtistURL(artistURL, savename)
    
dbAP = dbArtistsDiscogs()
artistURLs=['https://www.discogs.com/artist/60136-Catch', 'https://www.discogs.com/artist/3143651-Honey-31', 'https://www.discogs.com/artist/346187-Jeanette-Biedermann']
for artistURL in artistURLs:
    artistID=dbAP.dutils.getArtistID(artistURL)
    savename=dbAP.dutils.getArtistSavename(artistID)
    dbAP.dutils.downloadArtistURL(artistURL, savename)

#Artist ID [0003255680] is not a member of artistIDToName.
#Artist ID [0002503541] is not a member of artistIDToName.
#Artist ID [0003487021] is not a member of artistIDToName.
#Artist ID [7964912] is not a member of artistIDToName.
#Artist ID [0003205412] is not a member of artistIDToName.

In [ ]:
from dbArtistsDeezer import dbArtistsDeezer
from time import sleep
dbAP = dbArtistsDeezer()
print(togetID.keys())
for db in togetID.keys():
    if db != "Deezer":
        continue
    for dbID,artistName in togetID["Deezer"].items():
        artistURL = dbAP.getArtistURL(dbID)
        artistSavename = dbAP.dutils.getArtistSavename(dbID)
        try:
            dbAP.dutils.downloadArtistURL(artistURL, artistSavename, force=False, sleeptime=2)
        except:
            sleep(2)

In [ ]:
updateManDB(4)

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
toget=['Vanessa Jenkins and Bryn West', 'Garcia', 'Dana [Dana Rosemary Scallon]', 'the Bizz', 'the Master of Ceremonies', 'Baby D', 'H Two O', 'Rui da Silva', 'Peters', 'Fedde le Grand', 'LMC vs U2', 'Barry De Vorzon and Perry Botkin Jr.', 'Renato', 'Kelly', 'Disco-Tex and the Sex-O-Lettes', 'Buckner', 'Michael Zager Band', 'Nicole Kidman', 'Stardust', 'Nicole', 'The Blues Magoos', 'Mike Will Made-It', 'Wing and a Prayer Fife and Drum Corps', 'The Manchester United Football Squad']
toget=['Greg Medoro', 'The Rua', 'Matt Hartke', 'Jim Brickman and Adam Crossley', 'Carnie and Wendy Wilson', 'Wilton Place Street Band', 'Daniel DeBourg', 'Grooveatech Orchestra', 'Corbette Jackson', 'Chris Emerson', "Jump 'N the Saddle", 'Dionne Warwick and Johnny Mathis', 'Groove Holmes', 'The Jones Gang', 'The Romeos', 'William Prince', 'The Castells', 'MacNeal', 'Leslie Cours Mather', 'Cnote', 'Dave Barnes', 'Charleene Closshey', 'Neil', 'Band of Gold', 'Allison Park', 'The Villains', 'Scarlett', 'Kaz Bielinski', 'Lareau', 'James Taylor and J.D. Souther', 'Carr', 'Regie Hamm', 'LeBlanc', 'Sasha', 'Sasha Alexander', 'Ryan Chernin', 'Jeff Timmons', 'Il Divo and Céline Dion', 'Nancy Wilson', 'Sunny', 'Margo Rey', 'Robert Maxwell and His Orchestra', 'Mouth', 'Deborah Gibson', 'The Gunter Kallmann Chorus', 'Greg London', 'Lebo M.', 'Sam Mizell', 'Viennie V', 'Paul Stookey', 'Danny Griego', 'TrueHeart', 'the Sunliners', 'Chris Walker', 'The Alternate Routes', 'Giant Steps', 'Jade', 'Burke Roney', 'Desol', 'Whitney Wolanin', 'Sandy Stewart', 'Jane Lynch', 'Johnny Mann Singers', 'the Allis', 'Guys', 'Sylvia', 'Sylvia Kirby', 'Sylvia Hutton', 'Kanisha K', 'Dolls', 'The Sanford', 'John Splithoff', 'Josh Krajcik', 'Randy Jackson', 'Jon', 'Glenn Mohr', 'Father Ted Berndt', 'Townsend Band', 'The Baja Marimba Band', 'John Schneider', 'Senator Everett McKinley Dirksen', 'Richard Kincaid', 'Kathryn Dean', 'King Calaway']
toget=['Janus', 'Siouxsie', 'Hoosier Hot Shots and Two Ton Baker', 'Squirrely', 'Boyd Heath', 'Walter Shrum', 'Zach Seabaugh', 'Johnny Wright', 'Baillie', 'Dave Landers', 'Jerry Irby and His Texas Ranchers', 'Heartland [2°]', 'Marilyn', 'Melissa Lawson', 'Cashman', 'The Used and My Chemical Romance', 'Billy Thunderkloud', 'Johnny Hardy', 'Leon Seago', 'Mel Tillis and Sherry Bryce', 'Carl Butler and Pearl', 'the RubberBand', 'Song Trust', 'Lisa Hartman Black', 'Marion Worth and George Morgan', 'Ray Godfrey', 'Mark Willis', 'the Ride', 'New Medicine', 'Bradley Gaskin', 'Rehab', 'Jimmy Smart', 'Coverdale', 'Page and Plant', 'K. T. Oslin', 'the On-U Sound System', 'Bill Boyd and His Cowboy Ramblers', 'TRUSTcompany', 'Greg Bates', 'Camouflage', 'Angels and Airwaves', 'Kevin Sharp', 'The Buchanan Brothers', 'Pete Scobell Band', 'Holy White Hounds', 'Jake Worthington', 'Mono', 'Gene Woods', 'Bill Anderson and Mary Lou Turner', 'Course of Nature', 'The Raybon Brothers', 'Jeffrey Steele', 'Frankie Miller', 'Jimmie Dean', 'Mike Reid', 'Norma Jean and Liz Anderson', 'COIN', 'Zac Brown', 'Young Guns', 'the Melody Makers', 'Dandy', 'Kirk Jay', 'Phil Stacey', 'Rory', 'the Doolittle Band', 'Earshot', 'Texas Jim Lewis', 'Bill Leatherwood', 'The JaneDear Girls', "Coleman O'Neal", 'The Highwaymen', 'the Chieftones', 'Susan Anton', 'Burl Ives and Captain Stubby and the Buccaneers', 'The Glaser Brothers', 'Arthur Guitar Boogie Smith', 'Karen Brooks', 'Schuyler', "the Citizen's Band", 'The Firm', 'Connie Britton', 'Ruby Wright', 'the Banshees', 'Laverne Downs', 'Andy Gibson', 'Reef', 'Trio', 'Cindy Walker', 'Sam Palladio', 'Jim Carroll Band', 'Burl Ives and Grady Martin and His Slew Foot Five', 'Gary Buck', 'Eddie Dean', 'Børns', 'Waylon Jennings and Anita Carter', 'Tompall', 'Silvertide', 'Charlie Worsham', 'Buck Owens and Buddy Alan', 'Troy Olsen', 'Jimmy Newman', 'the Wilburn Brothers', 'Terry McBride', 'LaCosta Tucker', 'Bill Anderson and Jan Howard', 'The Carlisles Brothers', 'Jimmy Edwards', 'Rainbow', 'the Boys', 'Foy Willing and His Riders of the Purple Sage', 'Elton Britt and Rosalie Allen', 'Edens Edge', 'Adam Gregory', 'The Mission', 'Rockie Lynne', 'Victoria Shaw and Bryan White', 'The Henningsens', 'Operator', 'Vernon Stewart', 'Bobby Bare and Skeeter Davis', 'Bill Nettles', 'The Jenkins', 'Ryan Shupe', 'The Farm Inc.', 'Corey Kent White', 'Jonie Mosby', 'The Andrews Sisters and Ernest Tubb and the Texas Troubadors', 'Sound', 'Charles Mitchell', 'Vince Gill and Patty Loveless', 'Luke Gordon', 'Overstreet', 'Ted Self', 'Josh Gallagher', 'Billy Gray', 'MISSIO', 'Shorty Long', 'Spensha Baker', 'Harlow Wilcox and the Oakies', 'Keith Urban and Miranda Lambert', 'Bobby Lewis', 'Chris Kroeze', 'Reno', 'the Lonesome', 'The Outlaws', 'Knobloch', 'Wiley and Gene', 'Diesel [1°]', 'Alan Riddle', 'Copilot Music', 'Doug McKenzie', 'Device', 'Doug', 'J Roddy Walston', 'Pendulum', 'Soho', 'James Wesley Prosser', 'Lajon', 'Cole Deggs', 'Bobby Edwards', 'Cody Jameson', 'The Primitives', 'Dust for Life', 'John', 'Jack Rivers', 'Esmereldy', 'Rod Hart', 'Shelby Brown']
toget=toget=['Johnny Sparrow and His Bows and Arrows', 'Whatnauts', 'Damage', 'Mercy Dee', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Little Son Jackson', 'Ideal [R', 'Mark Curry', 'Brothers in Rhythm', 'DJs from Mars', 'Eddie Williams and His Brown Buddies', 'John Greer and the Rhythm Rockers', 'DirtyFreqs', 'Nomad', 'The Shades of Love', 'The Tabulations', 'the Ramblers', 'Rocko', 'Dirty Werk', 'Company', 'Joe Bennett', 'Annie Laurie and Paul Gayten and His Orchestra', 'Regina [Regina Richards]', 'Tim Myers', 'Richard Vission and Static Revenger', 'Ja-Kki', 'James and Smith', 'Mo Thugs and Bone Thugs-N-Harmony', 'Nokio', 'the Seen', 'Dynamix', "Mad'House", 'iAMSU!', 'The Street People [2°]', "the Family Cookin'", 'John Davis and the Monster Orchestra', 'The Boogie Man Orchestra', 'The Don [Nathaniel Pierre Jones]', 'Fantasy', 'Danny Overbea and His Combo', 'The X-Rays', 'Hadda Brooks Trio', 'the FBI', 'Pulse [2°]', 'Louie Louie', 'Do Ray and Me', 'Hannah', 'Chris Cox and DJ Frankie', 'Brighter Side of Darkness', 'Howard Johnson', 'Pleasure [1°]', 'Roy C', 'Like Mike and David Guetta', 'Bill', 'B.M.U. (Black Men United)', 'Tim Letteer', 'The Moment of Truth', 'Nathaniel Mayer and the Fabulous Twilights', 'Loveland vs Darlene Lewis', 'JT', 'Whirlwind', 'Dirty Pop', 'Eddie Vinson and His Orchestra', 'Robert Knight', 'Shirley Gunter and the Queens', 'Emitt Slay Trio', 'Calvin Boze and His All-Stars', 'Grant Jones', 'the Bluebelles', 'Tami LaTrell', 'India [Linda Viera Caballero]', 'Hybrid Heights and Crystal Waters', 'TK n Cash', 'Loveland', 'Willie Mae Big Mama Thornton', 'Tim', 'Louis Brooks and His Hi-Toppers', 'The Four Blazes', 'Macho', 'Profyle', 'Andrea Brown', 'The Counts', 'the Rebel MC', 'Young Thug and Travis Scott', 'Mike', 'Bazuka', 'Bobby Moore', 'Gene Ammons and His Sextet', 'the Muffins', "The McCrary's", 'ORS', 'The Chimes', 'The Reflections', 'Metro', 'Robin S. and DJ Escape', 'James Waynes', 'The Quick', 'The Four Jacks', 'Love and Kisses', 'Red Miller Trio', 'Grace [1°]', 'The Heart and Soul Orchestra', 'Marvin and Johnny', 'Willie Dixon and the Allstars', 'Little Esther and Mel Walker', 'Roy Byrd and His Blues Jumpers', 'JR Castro', 'B.G. the Prince of Rap', 'B.B.', 'Joe Thomas', 'Rob Base and DJ E-Z Rock', 'the Soul Rockers', 'Paul Humphrey', 'D.O.N.S. and Terri B!', 'Jimmy James', 'Jack Dupree and Mr. Bear', 'Sonny Boy Williamson [2°]', 'Charo', 'Buddy Lucas and His Band of Tomorrow', 'Jorio', 'Sister Rosetta Tharpe and Marie Knight', 'David Morales and Janice Robinson', 'Chong', '2Virgins', 'the Soul Partners', 'Little Caesar [Harry Caesar]', 'Holland-Dozier', 'Joe Hinton', 'Zach Adam', 'Praxis', 'Fawn', 'Rochell and the Candles', 'Farley Project', 'Dianne Wesley', 'the Rhythm Bangers', 'Dan Grissom', 'Impact [1°]', 'Sagat', 'Electric', 'Mike Hitman Wilson', 'Rosko', 'Archibald and His Orchestra', 'Fancy Inc.', 'The Lisa Marie Experience', 'Crazibiza and Dave Audé', 'Roscoe Gordon', 'Angel City', 'Cliq', 'Like Mike and Era Istrefi', 'Cause', 'the Sparkletones', 'Dana Ordway', 'The Blues Woman', 'Joe Fritz', 'Jimmy Bo Horne', 'The Sapphires', 'The Swallows', 'Queens', 'Wini Brown and Her Boyfriends', "Chubby Hip Shakin' Newsom", 'Switch', 'Taffy', 'the Blue Notes', 'Sun [Sun Ho]', 'Little Willie Littlefield and Little Lola Wiggins', 'The Regents', 'Martha', 'Floyd Hunt Quartette', 'Sharon Paige and Harold Melvin', 'David Joseph', 'Limmie', 'DJ Suede the Remix God', 'Ta Mara', 'The Boys [US R', 'Earl Bostic and His Orchestra', 'Déjà Vu', 'Sphinx', 'RetroVision', 'Pee Wee Crayton and His Guitar', 'DJ Bill Bennett', 'Silvetti', 'Jennifer Saunders', 'Lil Louis', 'Earl King [Earl Johnson]', 'Hal Singer Sextette', 'Bouvier', 'C. J. Bolland', "Marc O'Tool Tall Paul vs INXS", 'K.I.D.', 'Effect', 'Doc Sausage and His Mad Lads', 'The Lovers [2°]', 'the Big Family', 'Memphis Slim and His House Rockers', 'Rudy Render', 'Charlotte [Charlotte Kelly]', 'Cheech', 'Ramsey Lewis and Earth, Wind & Fire', 'Les Cooper', 'Angello', 'Blaze', 'B quartet]', 'Jimmie Nelson and the Peter Rabbit Trio', 'Jackie Lee', 'Joe Morris and His Orchestra', 'Chéri', 'Tony Moran', 'Mel', 'Luvndreams', 'the GB Experience', 'Da Mighty Dub Katz', 'Nellie Lutcher and Nat King Cole', 'Ko Ko Taylor', 'Taylor Girlz', 'Jess Glynne and DJ Cassidy', 'Karel Ullner', 'Q. Brooklyn', 'JX Riders and Skylar Stecker', 'The Larks', 'Jennifer Green', 'Space', 'Paul Williams', 'E.V.E. (Ebony Vibe Everlasting)', 'Fireball', 'Slim [Marvin Scandrick]', 'Staxx', 'The Reflections', 'Jeff Morgan', 'Brenda', 'Hodges', 'Extasia', 'Goldlink', 'Absolutely Fabulous Pet Shop Boys', 'Raw Silk', 'the Emblems', 'George Lamond', 'Stick McGhee and His Buddies', 'The Rainbows', 'Laguna', 'Pet Shop Boys = The Collaboration', 'The Flaming Ember', 'Kevin Ross', 'Artie the 1 Man Party', 'Flip', 'Freddie Mitchell and His Orchestra', 'Troop 41', 'Joi', 'the Police', 'Twisted Dee', 'Pianoheadz', 'The Shades of Blue', 'Lipstique', 'the Soul Searchers', 'The Philarmonics', 'Cashis', 'Eastside Connection', 'Bell', 'The Coronets', 'Fast Life Yungstaz (F.L.Y.)', 'Bumble Bee Unlimited', 'Sheila', 'Fenix', 'Patty', 'Betty Boo and the Beatmasters', 'U.N.V.', "Suave'", 'The Fog', 'Original Five Blind Boys', 'Lennie Lewis and His Orchestra', 'Earl King [Earl Connelly]', 'His Cool Aid Chemists', 'Camille Howard and Her Trio', 'Linx', 'DJ Duke', 'Dave Matthias', 'Herb Lance and the Classics', 'The Goodfellas', "Brown's Blues Blowers", 'Dyke and the Blazers', 'Skip', 'Eniac', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage', 'Jimmy Grissom', 'Trio', 'C.J. & Co.', 'The Positive Force', 'Jon of the Pleased Wimmin', 'Angel Clivillés', 'Arpeggio', 'Brainstorm', 'Destiny', 'the Egg', 'Industry [2°]', 'Ed Wiley and His After Hour Rhythm', 'Black Machine', 'Herb Fisher and His Trio', 'Barona', 'Jessie Saunders', 'the Vagabonds', 'Dillon Francis and Kygo', 'The Geto Boys', '50 Cent and Olivia', 'Swinging Sax Kari and Orchestra', 'KMC', 'Déjà', 'DJ Vicious', 'Alton McClain', 'Meechie', 'Mel Walker', 'Lydia Murdoch', 'Fatback', 'The Ones [2°]', 'Solo [2°]', 'Who Da Funk?', '3rd Party', 'Jimmy Preston and His Prestonians', 'Chip Chocolate', 'Ecstasy', 'The Shadows', 'The Purple Ribbon All-Stars', "The Cats 'N Jammer Three", "Stretch 'N' Vern Present Maddogg", 'Marvin', 'Eunice', 'Richard Dimples Fields', 'Robbie Rivera and Axwell', 'Ron Butler', 'Karen Young', 'Round2Crew', 'The Cardinals', 'Refugee Camp All-Stars', 'Mount Rushmore', 'Sonny Boy Williamson [1°]', 'Altar', 'Pacific Gas', 'Jimmy Wilson and His All-Stars', 'Willie Mabon and His Combo', 'The Corsairs', 'Mike Theodore Orchestra', 'Sunshine']

toget=['Vanessa Jenkins and Bryn West', 'Garcia', 'Dana [Dana Rosemary Scallon]', 'the Bizz', 'the Master of Ceremonies', 'Baby D [dance group]', 'H Two O', 'Rui da Silva', 'Peters', 'Fedde le Grand', 'LMC vs U2', 'Barry De Vorzon and Perry Botkin Jr.', 'Renato', 'Kelly', 'Disco-Tex and the Sex-O-Lettes', 'Buckner', 'Michael Zager Band', 'Nicole Kidman', 'Stardust [electronic music trio]', 'Nicole [German singer]', 'The Blues Magoos', 'Mike Will Made-It', 'Wing and a Prayer Fife and Drum Corps', 'The Manchester United Football Squad']
toget+=['Greg Medoro', 'The Rua', 'Matt Hartke', 'Jim Brickman and Adam Crossley', 'Carnie and Wendy Wilson', 'Wilton Place Street Band', 'Daniel DeBourg', 'Grooveatech Orchestra', 'Corbette Jackson', 'Chris Emerson', "Jump 'N the Saddle", 'Dionne Warwick and Johnny Mathis', 'Groove Holmes', 'The Jones Gang', 'The Romeos', 'William Prince', 'The Castells', 'MacNeal', 'Leslie Cours Mather', 'Cnote', 'Dave Barnes', 'Charleene Closshey', 'Neil', 'Band of Gold', 'Allison Park', 'The Villains', 'Scarlett', 'Kaz Bielinski', 'Lareau', 'James Taylor and J.D. Souther', 'Carr', 'Regie Hamm', 'LeBlanc', 'Sasha [Sasha Alexander]', 'Ryan Chernin', 'Jeff Timmons', 'Il Divo and Céline Dion', 'Nancy Wilson', 'Sunny', 'Margo Rey', 'Robert Maxwell and His Orchestra', 'Mouth', 'Deborah Gibson', 'The Gunter Kallmann Chorus', 'Greg London', 'Lebo M.', 'Sam Mizell', 'Viennie V', 'Paul Stookey', 'Danny Griego', 'TrueHeart', 'the Sunliners', 'Chris Walker', 'The Alternate Routes', 'Giant Steps', 'Jade [Canadian singer]', 'Burke Roney', 'Desol', 'Whitney Wolanin', 'Sandy Stewart [singer]', 'Jane Lynch', 'Johnny Mann Singers', 'the Allis', 'Guys', 'Sylvia [Sylvia Kirby aka Sylvia Hutton]', 'Kanisha K', 'Dolls', 'The Sanford', 'John Splithoff', 'Josh Krajcik', 'Randy Jackson', 'Jon', 'Glenn Mohr', 'Father Ted Berndt', 'Townsend Band', 'The Baja Marimba Band', 'John Schneider', 'Senator Everett McKinley Dirksen', 'Richard Kincaid', 'Kathryn Dean', 'King Calaway']
toget+=['Janus', 'Siouxsie', 'Hoosier Hot Shots and Two Ton Baker', 'Squirrely', 'Boyd Heath', 'Walter Shrum', 'Zach Seabaugh', 'Johnny Wright', 'Baillie', 'Dave Landers', 'Jerry Irby and His Texas Ranchers', 'Heartland [2°]', 'Marilyn', 'Melissa Lawson', 'Cashman', 'The Used and My Chemical Romance', 'Billy Thunderkloud', 'Johnny Hardy', 'Leon Seago', 'Mel Tillis and Sherry Bryce', 'Carl Butler and Pearl', 'the RubberBand', 'Song Trust', 'Lisa Hartman Black', 'Marion Worth and George Morgan', 'Ray Godfrey', 'Mark Willis', 'the Ride', 'New Medicine', 'Bradley Gaskin', 'Rehab', 'Jimmy Smart', 'Coverdale', 'Page and Plant', 'K. T. Oslin', 'the On-U Sound System', 'Bill Boyd and His Cowboy Ramblers', 'TRUSTcompany', 'Greg Bates', 'Camouflage [German dance group]', 'Angels and Airwaves', 'Kevin Sharp', 'The Buchanan Brothers', 'Pete Scobell Band', 'Holy White Hounds', 'Jake Worthington', 'Mono [British pop duo]', 'Gene Woods', 'Bill Anderson and Mary Lou Turner', 'Course of Nature', 'The Raybon Brothers', 'Jeffrey Steele', 'Frankie Miller [country singer]', 'Jimmie Dean', 'Mike Reid', 'Norma Jean and Liz Anderson', 'COIN', 'Zac Brown', 'Young Guns', 'the Melody Makers', 'Dandy', 'Kirk Jay', 'Phil Stacey', 'Rory', 'the Doolittle Band', 'Earshot', 'Texas Jim Lewis', 'Bill Leatherwood', 'The JaneDear Girls', "Coleman O'Neal", 'The Highwaymen [country supergroup]', 'the Chieftones', 'Susan Anton', 'Burl Ives and Captain Stubby and the Buccaneers', 'The Glaser Brothers', 'Arthur Guitar Boogie Smith', 'Karen Brooks', 'Schuyler', "the Citizen's Band", 'The Firm [English rock supergroup]',
       'Connie Britton', 'Ruby Wright [country singer]', 'the Banshees', 'Laverne Downs', 'Andy Gibson [singer]', 'Reef', 'Trio [American country supergroup]', 'Cindy Walker', 'Sam Palladio', 'Jim Carroll Band', 'Burl Ives and Grady Martin and His Slew Foot Five', 'Gary Buck', 'Eddie Dean', 'Børns', 'Waylon Jennings and Anita Carter', 'Tompall', 'Silvertide', 'Charlie Worsham', 'Buck Owens and Buddy Alan', 'Troy Olsen', 'Jimmy Newman', 'the Wilburn Brothers', 'Terry McBride', 'LaCosta Tucker', 'Bill Anderson and Jan Howard', 'The Carlisles Brothers', 'Jimmy Edwards', 'Rainbow [British rock band]', 'the Boys', 'Foy Willing and His Riders of the Purple Sage', 'Elton Britt and Rosalie Allen', 'Edens Edge', 'Adam Gregory', 'The Mission [English band]', 'Rockie Lynne', 'Victoria Shaw and Bryan White', 'The Henningsens', 'Operator', 'Vernon Stewart', 'Bobby Bare and Skeeter Davis', 'Bill Nettles', 'The Jenkins', 'Ryan Shupe', 'The Farm Inc.', 'Corey Kent White', 'Jonie Mosby', 'The Andrews Sisters and Ernest Tubb and the Texas Troubadors', 'Sound', 'Charles Mitchell', 'Vince Gill and Patty Loveless', 'Luke Gordon', 'Overstreet', 'Ted Self', 'Josh Gallagher', 'Billy Gray', 'MISSIO', 'Shorty Long [country singer]', 'Spensha Baker', 'Harlow Wilcox and the Oakies', 'Keith Urban and Miranda Lambert', 'Bobby Lewis [country singer]', 'Chris Kroeze', 'Reno', 'the Lonesome', 'The Outlaws [US band]', 'Knobloch', 'Wiley and Gene', 'Diesel [1°]', 'Alan Riddle', 'Copilot Music', 'Doug McKenzie', 'Device [metal band]', 'Doug', 'J Roddy Walston', 'Pendulum [drum and bass band]', 'Soho', 'James Wesley Prosser', 'Lajon', 'Cole Deggs', 'Bobby Edwards', 'Cody Jameson', 'The Primitives', 'Dust for Life', 'John', 'Jack Rivers', 'Esmereldy', 'Rod Hart', 'Shelby Brown']
toget+=['Johnny Sparrow and His Bows and Arrows', 'Whatnauts', 'Damage', 'Mercy Dee', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Little Son Jackson', 'Ideal [R', 'Mark Curry', 'Brothers in Rhythm', 'DJs from Mars', 'Eddie Williams and His Brown Buddies', 'John Greer and the Rhythm Rockers', 'DirtyFreqs', 'Nomad [house music act]', 'The Shades of Love', 'The Tabulations', 'the Ramblers', 'Rocko', 'Dirty Werk', 'Company', 'Joe Bennett', 'Annie Laurie and Paul Gayten and His Orchestra', 'Regina [Regina Richards]', 'Tim Myers', 'Richard Vission and Static Revenger', 'Ja-Kki', 'James and Smith', 'Mo Thugs and Bone Thugs-N-Harmony', 'Nokio', 'the Seen', 'Dynamix', "Mad'House", 'iAMSU!', 'The Street People [2°]', "the Family Cookin'", 'John Davis and the Monster Orchestra', 'The Boogie Man Orchestra', 'The Don [Nathaniel Pierre Jones]', 'Fantasy [US soul band]', 'Danny Overbea and His Combo', 'The X-Rays', 'Hadda Brooks Trio', 'the FBI', 'Pulse [2°]', 'Louie Louie', 'Do Ray and Me', 'Hannah [British singer]', 'Chris Cox and DJ Frankie', 'Brighter Side of Darkness', 'Howard Johnson [singer]', 'Pleasure [1°]', 'Roy C', 'Like Mike and David Guetta', 'Bill', 'B.M.U. (Black Men United)', 'Tim Letteer', 'The Moment of Truth', 'Nathaniel Mayer and the Fabulous Twilights', 'Loveland vs Darlene Lewis', 'JT', 'Whirlwind', 'Dirty Pop', 'Eddie Vinson and His Orchestra', 'Robert Knight', 'Shirley Gunter and the Queens', 'Emitt Slay Trio', 'Calvin Boze and His All-Stars', 'Grant Jones', 'the Bluebelles', 'Tami LaTrell', 'India [Linda Viera Caballero]', 'Hybrid Heights and Crystal Waters', 'TK n Cash', 'Loveland', 'Willie Mae Big Mama Thornton', 'Tim', 'Louis Brooks and His Hi-Toppers', 'The Four Blazes', 'Macho', 'Profyle', 'Andrea Brown', 'The Counts',
       'the Rebel MC', 'Young Thug and Travis Scott', 'Mike', 'Bazuka', 'Bobby Moore [saxophonist]', 'Gene Ammons and His Sextet', 'the Muffins', "The McCrary's", 'ORS', 'The Chimes [British dance music act]', 'The Reflections [Harlem vocal group]', 'Metro', 'Robin S. and DJ Escape', 'James Waynes', 'The Quick', 'The Four Jacks', 'Love and Kisses', 'Red Miller Trio', 'Grace [1°]', 'The Heart and Soul Orchestra', 'Marvin and Johnny', 'Willie Dixon and the Allstars', 'Little Esther and Mel Walker', 'Roy Byrd and His Blues Jumpers', 'JR Castro', 'B.G. the Prince of Rap', 'B.B.', 'Joe Thomas [American saxophonist]', 'Rob Base and DJ E-Z Rock', 'the Soul Rockers', 'Paul Humphrey', 'D.O.N.S. and Terri B!', 'Jimmy James', 'Jack Dupree and Mr. Bear', 'Sonny Boy Williamson [2°]', 'Charo', 'Buddy Lucas and His Band of Tomorrow', 'Jorio', 'Sister Rosetta Tharpe and Marie Knight', 'David Morales and Janice Robinson', 'Chong', '2Virgins', 'the Soul Partners', 'Little Caesar [Harry Caesar]', 'Holland-Dozier', 'Joe Hinton', 'Zach Adam', 'Praxis', 'Fawn', 'Rochell and the Candles', 'Farley Project', 'Dianne Wesley', 'the Rhythm Bangers', 'Dan Grissom', 'Impact [1°]', 'Sagat', 'Electric', 'Mike Hitman Wilson', 'Rosko', 'Archibald and His Orchestra', 'Fancy Inc.', 'The Lisa Marie Experience', 'Crazibiza and Dave Audé', 'Roscoe Gordon', 'Angel City [electronic music group]', 'Cliq', 'Like Mike and Era Istrefi', 'Cause', 'the Sparkletones', 'Dana Ordway', 'The Blues Woman', 'Joe Fritz', 'Jimmy Bo Horne', 'The Sapphires', 'The Swallows', 'Queens', 'Wini Brown and Her Boyfriends', "Chubby Hip Shakin' Newsom", 'Switch [funk band]', 'Taffy', 'the Blue Notes', 'Sun [Sun Ho]', 'Little Willie Littlefield and Little Lola Wiggins', 'The Regents [American doo-wop group]', 'Martha', 'Floyd Hunt Quartette', 'Sharon Paige and Harold Melvin', 'David Joseph', 'Limmie', 'DJ Suede the Remix God', 'Ta Mara', 'The Boys [US R', 'Earl Bostic and His Orchestra', 'Déjà Vu [dance music act]',
       'Sphinx [electronic music duo]', 'RetroVision', 'Pee Wee Crayton and His Guitar', 'DJ Bill Bennett', 'Silvetti', 'Jennifer Saunders', 'Lil Louis', 'Earl King [Earl Johnson]', 'Hal Singer Sextette', 'Bouvier', 'C. J. Bolland', "Marc O'Tool Tall Paul vs INXS", 'K.I.D.', 'Effect', 'Doc Sausage and His Mad Lads', 'The Lovers [2°]', 'the Big Family', 'Memphis Slim and His House Rockers', 'Rudy Render', 'Charlotte [Charlotte Kelly]', 'Cheech', 'Ramsey Lewis and Earth, Wind & Fire', 'Les Cooper', 'Angello', 'Blaze', 'B quartet]', 'Jimmie Nelson and the Peter Rabbit Trio', 'Jackie Lee [American singer]', 'Joe Morris and His Orchestra', 'Chéri', 'Tony Moran', 'Mel', 'Luvndreams', 'the GB Experience', 'Da Mighty Dub Katz', 'Nellie Lutcher and Nat King Cole', 'Ko Ko Taylor', 'Taylor Girlz', 'Jess Glynne and DJ Cassidy', 'Karel Ullner', 'Q. Brooklyn', 'JX Riders and Skylar Stecker', 'The Larks [North Carolina band]', 'Jennifer Green', 'Space [french electronic music band]', 'Paul Williams [saxophonist]', 'E.V.E. (Ebony Vibe Everlasting)', 'Fireball', 'Slim [Marvin Scandrick]', 'Staxx', 'The Reflections [Detroit rock and roll band]', 'Jeff Morgan', 'Brenda', 'Hodges', 'Extasia', 'Goldlink', 'Absolutely Fabulous Pet Shop Boys', 'Raw Silk', 'the Emblems', 'George Lamond', 'Stick McGhee and His Buddies', 'The Rainbows', 'Laguna', 'Pet Shop Boys = The Collaboration', 'The Flaming Ember', 'Kevin Ross', 'Artie the 1 Man Party', 'Flip', 'Freddie Mitchell and His Orchestra', 'Troop 41', 'Joi', 'the Police', 'Twisted Dee', 'Pianoheadz', 'The Shades of Blue', 'Lipstique', 'the Soul Searchers', 'The Philarmonics', 'Cashis', 'Eastside Connection', 'Bell', 'The Coronets', 'Fast Life Yungstaz (F.L.Y.)', 'Bumble Bee Unlimited', 'Sheila', 'Fenix', 'Patty', 'Betty Boo and the Beatmasters', 'U.N.V.', "Suave'", 'The Fog [house project]', 'Original Five Blind Boys', 'Lennie Lewis and His Orchestra', 'Earl King [Earl Connelly]', 'His Cool Aid Chemists', 'Camille Howard and Her Trio',
       'Linx', 'DJ Duke', 'Dave Matthias', 'Herb Lance and the Classics', 'The Goodfellas', "Brown's Blues Blowers", 'Dyke and the Blazers', 'Skip', 'Eniac', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage',
       'Jimmy Grissom', 'Trio [German band]', 'C.J. & Co.', 'The Positive Force', 'Jon of the Pleased Wimmin', 'Angel Clivillés', 'Arpeggio', 'Brainstorm [disco band]', 'Destiny', 'the Egg', 'Industry [2°]', 'Ed Wiley and His After Hour Rhythm', 'Black Machine', 'Herb Fisher and His Trio', 'Barona', 'Jessie Saunders', 'the Vagabonds', 'Dillon Francis and Kygo', 'The Geto Boys', '50 Cent and Olivia', 'Swinging Sax Kari and Orchestra', 'KMC [Italian dance act]', 'Déjà [soul duo]', 'DJ Vicious', 'Alton McClain', 'Meechie', 'Mel Walker', 'Lydia Murdoch', 'Fatback', 'The Ones [2°]', 'Solo [2°]', 'Who Da Funk?', '3rd Party', 'Jimmy Preston and His Prestonians', 'Chip Chocolate', 'Ecstasy', 'The Shadows [vocal group]', 'The Purple Ribbon All-Stars', "The Cats 'N Jammer Three", "Stretch 'N' Vern Present Maddogg", 'Marvin', 'Eunice', 'Richard Dimples Fields', 'Robbie Rivera and Axwell', 'Ron Butler', 'Karen Young [US singer]', 'Round2Crew', 'The Cardinals', 'Refugee Camp All-Stars', 'Mount Rushmore', 'Sonny Boy Williamson [1°]', 'Altar [dance music band]', 'Pacific Gas', 'Jimmy Wilson and His All-Stars', 'Willie Mabon and His Combo', 'The Corsairs', 'Mike Theodore Orchestra', 'Sunshine']
from collections import Counter
cntr = Counter()
for item in toget:
    if " [" in item:
        if len(item.split(" [")) > 1:
            #print(item)
            continue
        #val="["+item.split(" [")[1]
        #cntr[val] += 1
for item in cntr.most_common():
    print(item[0])
    
    
removes="""[country singer]
[singer]
[saxophonist]
[dance group]
[electronic music trio]
[German singer]
[Canadian singer]
[German dance group]
[British pop duo]
[country supergroup]
[English rock supergroup]
[American country supergroup]
[British rock band]
[English band]
[US band]
[metal band]
[drum and bass band]
[house music act]
[US soul band]
[British singer]
[British dance music act]
[Harlem vocal group]
[American saxophonist]
[electronic music group]
[funk band]
[American doo-wop group]
[dance music act]
[electronic music duo]
[American singer]
[North Carolina band]
[french electronic music band]
[Detroit rock and roll band]
[house project]
[German band]
[disco band]
[Italian dance act]
[soul duo]
[vocal group]
[US singer]
[dance music band]""".split("\n")

toget=['Johnny Sparrow and His Bows and Arrows', 'Whatnauts', 'Damage', 'Mercy Dee', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Little Son Jackson', 'Ideal [R', 'Mark Curry', 'Brothers in Rhythm', 'DJs from Mars', 'Eddie Williams and His Brown Buddies', 'John Greer and the Rhythm Rockers', 'DirtyFreqs', 'Nomad [house music act]', 'The Shades of Love', 'The Tabulations', 'the Ramblers', 'Rocko', 'Dirty Werk', 'Company', 'Joe Bennett', 'Annie Laurie and Paul Gayten and His Orchestra', 'Regina [Regina Richards]', 'Tim Myers', 'Richard Vission and Static Revenger', 'Ja-Kki', 'James and Smith', 'Mo Thugs and Bone Thugs-N-Harmony', 'Nokio', 'the Seen', 'Dynamix', "Mad'House", 'iAMSU!', 'The Street People [2°]', "the Family Cookin'", 'John Davis and the Monster Orchestra', 'The Boogie Man Orchestra', 'The Don [Nathaniel Pierre Jones]', 'Fantasy [US soul band]', 'Danny Overbea and His Combo', 'The X-Rays', 'Hadda Brooks Trio', 'the FBI', 'Pulse [2°]', 'Louie Louie', 'Do Ray and Me', 'Hannah [British singer]', 'Chris Cox and DJ Frankie', 'Brighter Side of Darkness', 'Howard Johnson [singer]', 'Pleasure [1°]', 'Roy C', 'Like Mike and David Guetta', 'Bill', 'B.M.U. (Black Men United)', 'Tim Letteer', 'The Moment of Truth', 'Nathaniel Mayer and the Fabulous Twilights', 'Loveland vs Darlene Lewis', 'JT', 'Whirlwind', 'Dirty Pop', 'Eddie Vinson and His Orchestra', 'Robert Knight', 'Shirley Gunter and the Queens', 'Emitt Slay Trio', 'Calvin Boze and His All-Stars', 'Grant Jones', 'the Bluebelles', 'Tami LaTrell', 'India [Linda Viera Caballero]', 'Hybrid Heights and Crystal Waters', 'TK n Cash', 'Loveland', 'Willie Mae Big Mama Thornton', 'Tim', 'Louis Brooks and His Hi-Toppers', 'The Four Blazes', 'Macho', 'Profyle', 'Andrea Brown', 'The Counts',
       'the Rebel MC', 'Young Thug and Travis Scott', 'Mike', 'Bazuka', 'Bobby Moore [saxophonist]', 'Gene Ammons and His Sextet', 'the Muffins', "The McCrary's", 'ORS', 'The Chimes [British dance music act]', 'The Reflections [Harlem vocal group]', 'Metro', 'Robin S. and DJ Escape', 'James Waynes', 'The Quick', 'The Four Jacks', 'Love and Kisses', 'Red Miller Trio', 'Grace [1°]', 'The Heart and Soul Orchestra', 'Marvin and Johnny', 'Willie Dixon and the Allstars', 'Little Esther and Mel Walker', 'Roy Byrd and His Blues Jumpers', 'JR Castro', 'B.G. the Prince of Rap', 'B.B.', 'Joe Thomas [American saxophonist]', 'Rob Base and DJ E-Z Rock', 'the Soul Rockers', 'Paul Humphrey', 'D.O.N.S. and Terri B!', 'Jimmy James', 'Jack Dupree and Mr. Bear', 'Sonny Boy Williamson [2°]', 'Charo', 'Buddy Lucas and His Band of Tomorrow', 'Jorio', 'Sister Rosetta Tharpe and Marie Knight', 'David Morales and Janice Robinson', 'Chong', '2Virgins', 'the Soul Partners', 'Little Caesar [Harry Caesar]', 'Holland-Dozier', 'Joe Hinton', 'Zach Adam', 'Praxis', 'Fawn', 'Rochell and the Candles', 'Farley Project', 'Dianne Wesley', 'the Rhythm Bangers', 'Dan Grissom', 'Impact [1°]', 'Sagat', 'Electric', 'Mike Hitman Wilson', 'Rosko', 'Archibald and His Orchestra', 'Fancy Inc.', 'The Lisa Marie Experience', 'Crazibiza and Dave Audé', 'Roscoe Gordon', 'Angel City [electronic music group]', 'Cliq', 'Like Mike and Era Istrefi', 'Cause', 'the Sparkletones', 'Dana Ordway', 'The Blues Woman', 'Joe Fritz', 'Jimmy Bo Horne', 'The Sapphires', 'The Swallows', 'Queens', 'Wini Brown and Her Boyfriends', "Chubby Hip Shakin' Newsom", 'Switch [funk band]', 'Taffy', 'the Blue Notes', 'Sun [Sun Ho]', 'Little Willie Littlefield and Little Lola Wiggins', 'The Regents [American doo-wop group]', 'Martha', 'Floyd Hunt Quartette', 'Sharon Paige and Harold Melvin', 'David Joseph', 'Limmie', 'DJ Suede the Remix God', 'Ta Mara', 'The Boys [US R', 'Earl Bostic and His Orchestra', 'Déjà Vu [dance music act]',
       'Sphinx [electronic music duo]', 'RetroVision', 'Pee Wee Crayton and His Guitar', 'DJ Bill Bennett', 'Silvetti', 'Jennifer Saunders', 'Lil Louis', 'Earl King [Earl Johnson]', 'Hal Singer Sextette', 'Bouvier', 'C. J. Bolland', "Marc O'Tool Tall Paul vs INXS", 'K.I.D.', 'Effect', 'Doc Sausage and His Mad Lads', 'The Lovers [2°]', 'the Big Family', 'Memphis Slim and His House Rockers', 'Rudy Render', 'Charlotte [Charlotte Kelly]', 'Cheech', 'Ramsey Lewis and Earth, Wind & Fire', 'Les Cooper', 'Angello', 'Blaze', 'B quartet]', 'Jimmie Nelson and the Peter Rabbit Trio', 'Jackie Lee [American singer]', 'Joe Morris and His Orchestra', 'Chéri', 'Tony Moran', 'Mel', 'Luvndreams', 'the GB Experience', 'Da Mighty Dub Katz', 'Nellie Lutcher and Nat King Cole', 'Ko Ko Taylor', 'Taylor Girlz', 'Jess Glynne and DJ Cassidy', 'Karel Ullner', 'Q. Brooklyn', 'JX Riders and Skylar Stecker', 'The Larks [North Carolina band]', 'Jennifer Green', 'Space [french electronic music band]', 'Paul Williams [saxophonist]', 'E.V.E. (Ebony Vibe Everlasting)', 'Fireball', 'Slim [Marvin Scandrick]', 'Staxx', 'The Reflections [Detroit rock and roll band]', 'Jeff Morgan', 'Brenda', 'Hodges', 'Extasia', 'Goldlink', 'Absolutely Fabulous Pet Shop Boys', 'Raw Silk', 'the Emblems', 'George Lamond', 'Stick McGhee and His Buddies', 'The Rainbows', 'Laguna', 'Pet Shop Boys = The Collaboration', 'The Flaming Ember', 'Kevin Ross', 'Artie the 1 Man Party', 'Flip', 'Freddie Mitchell and His Orchestra', 'Troop 41', 'Joi', 'the Police', 'Twisted Dee', 'Pianoheadz', 'The Shades of Blue', 'Lipstique', 'the Soul Searchers', 'The Philarmonics', 'Cashis', 'Eastside Connection', 'Bell', 'The Coronets', 'Fast Life Yungstaz (F.L.Y.)', 'Bumble Bee Unlimited', 'Sheila', 'Fenix', 'Patty', 'Betty Boo and the Beatmasters', 'U.N.V.', "Suave'", 'The Fog [house project]', 'Original Five Blind Boys', 'Lennie Lewis and His Orchestra', 'Earl King [Earl Connelly]', 'His Cool Aid Chemists', 'Camille Howard and Her Trio',
       'Linx', 'DJ Duke', 'Dave Matthias', 'Herb Lance and the Classics', 'The Goodfellas', "Brown's Blues Blowers", 'Dyke and the Blazers', 'Skip', 'Eniac', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage',
       'Jimmy Grissom', 'Trio [German band]', 'C.J. & Co.', 'The Positive Force', 'Jon of the Pleased Wimmin', 'Angel Clivillés', 'Arpeggio', 'Brainstorm [disco band]', 'Destiny', 'the Egg', 'Industry [2°]', 'Ed Wiley and His After Hour Rhythm', 'Black Machine', 'Herb Fisher and His Trio', 'Barona', 'Jessie Saunders', 'the Vagabonds', 'Dillon Francis and Kygo', 'The Geto Boys', '50 Cent and Olivia', 'Swinging Sax Kari and Orchestra', 'KMC [Italian dance act]', 'Déjà [soul duo]', 'DJ Vicious', 'Alton McClain', 'Meechie', 'Mel Walker', 'Lydia Murdoch', 'Fatback', 'The Ones [2°]', 'Solo [2°]', 'Who Da Funk?', '3rd Party', 'Jimmy Preston and His Prestonians', 'Chip Chocolate', 'Ecstasy', 'The Shadows [vocal group]', 'The Purple Ribbon All-Stars', "The Cats 'N Jammer Three", "Stretch 'N' Vern Present Maddogg", 'Marvin', 'Eunice', 'Richard Dimples Fields', 'Robbie Rivera and Axwell', 'Ron Butler', 'Karen Young [US singer]', 'Round2Crew', 'The Cardinals', 'Refugee Camp All-Stars', 'Mount Rushmore', 'Sonny Boy Williamson [1°]', 'Altar [dance music band]', 'Pacific Gas', 'Jimmy Wilson and His All-Stars', 'Willie Mabon and His Combo', 'The Corsairs', 'Mike Theodore Orchestra', 'Sunshine']
togetX=[]
print(len(toget))
for item in toget:
    val = item
    for rm in removes:
        if rm in val:
            val = val.replace(rm, "").strip()
    togetX.append(val)
print(len(togetX))
print(togetX)


toget=['Johnny Sparrow and His Bows and Arrows', 'Whatnauts', 'Damage', 'Mercy Dee', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Little Son Jackson', 'Ideal [R', 'Mark Curry', 'Brothers in Rhythm', 'DJs from Mars', 'Eddie Williams and His Brown Buddies', 'John Greer and the Rhythm Rockers', 'DirtyFreqs', 'Nomad', 'The Shades of Love', 'The Tabulations', 'the Ramblers', 'Rocko', 'Dirty Werk', 'Company', 'Joe Bennett', 'Annie Laurie and Paul Gayten and His Orchestra', 'Regina [Regina Richards]', 'Tim Myers', 'Richard Vission and Static Revenger', 'Ja-Kki', 'James and Smith', 'Mo Thugs and Bone Thugs-N-Harmony', 'Nokio', 'the Seen', 'Dynamix', "Mad'House", 'iAMSU!', 'The Street People [2°]', "the Family Cookin'", 'John Davis and the Monster Orchestra', 'The Boogie Man Orchestra', 'The Don [Nathaniel Pierre Jones]', 'Fantasy', 'Danny Overbea and His Combo', 'The X-Rays', 'Hadda Brooks Trio', 'the FBI', 'Pulse [2°]', 'Louie Louie', 'Do Ray and Me', 'Hannah', 'Chris Cox and DJ Frankie', 'Brighter Side of Darkness', 'Howard Johnson', 'Pleasure [1°]', 'Roy C', 'Like Mike and David Guetta', 'Bill', 'B.M.U. (Black Men United)', 'Tim Letteer', 'The Moment of Truth', 'Nathaniel Mayer and the Fabulous Twilights', 'Loveland vs Darlene Lewis', 'JT', 'Whirlwind', 'Dirty Pop', 'Eddie Vinson and His Orchestra', 'Robert Knight', 'Shirley Gunter and the Queens', 'Emitt Slay Trio', 'Calvin Boze and His All-Stars', 'Grant Jones', 'the Bluebelles', 'Tami LaTrell', 'India [Linda Viera Caballero]', 'Hybrid Heights and Crystal Waters', 'TK n Cash', 'Loveland', 'Willie Mae Big Mama Thornton', 'Tim', 'Louis Brooks and His Hi-Toppers', 'The Four Blazes', 'Macho', 'Profyle', 'Andrea Brown', 'The Counts', 'the Rebel MC', 'Young Thug and Travis Scott', 'Mike', 'Bazuka', 'Bobby Moore', 'Gene Ammons and His Sextet', 'the Muffins', "The McCrary's", 'ORS', 'The Chimes', 'The Reflections', 'Metro', 'Robin S. and DJ Escape', 'James Waynes', 'The Quick', 'The Four Jacks', 'Love and Kisses', 'Red Miller Trio', 'Grace [1°]', 'The Heart and Soul Orchestra', 'Marvin and Johnny', 'Willie Dixon and the Allstars', 'Little Esther and Mel Walker', 'Roy Byrd and His Blues Jumpers', 'JR Castro', 'B.G. the Prince of Rap', 'B.B.', 'Joe Thomas', 'Rob Base and DJ E-Z Rock', 'the Soul Rockers', 'Paul Humphrey', 'D.O.N.S. and Terri B!', 'Jimmy James', 'Jack Dupree and Mr. Bear', 'Sonny Boy Williamson [2°]', 'Charo', 'Buddy Lucas and His Band of Tomorrow', 'Jorio', 'Sister Rosetta Tharpe and Marie Knight', 'David Morales and Janice Robinson', 'Chong', '2Virgins', 'the Soul Partners', 'Little Caesar [Harry Caesar]', 'Holland-Dozier', 'Joe Hinton', 'Zach Adam', 'Praxis', 'Fawn', 'Rochell and the Candles', 'Farley Project', 'Dianne Wesley', 'the Rhythm Bangers', 'Dan Grissom', 'Impact [1°]', 'Sagat', 'Electric', 'Mike Hitman Wilson', 'Rosko', 'Archibald and His Orchestra', 'Fancy Inc.', 'The Lisa Marie Experience', 'Crazibiza and Dave Audé', 'Roscoe Gordon', 'Angel City', 'Cliq', 'Like Mike and Era Istrefi', 'Cause', 'the Sparkletones', 'Dana Ordway', 'The Blues Woman', 'Joe Fritz', 'Jimmy Bo Horne', 'The Sapphires', 'The Swallows', 'Queens', 'Wini Brown and Her Boyfriends', "Chubby Hip Shakin' Newsom", 'Switch', 'Taffy', 'the Blue Notes', 'Sun [Sun Ho]', 'Little Willie Littlefield and Little Lola Wiggins', 'The Regents', 'Martha', 'Floyd Hunt Quartette', 'Sharon Paige and Harold Melvin', 'David Joseph', 'Limmie', 'DJ Suede the Remix God', 'Ta Mara', 'The Boys [US R', 'Earl Bostic and His Orchestra', 'Déjà Vu', 'Sphinx', 'RetroVision', 'Pee Wee Crayton and His Guitar', 'DJ Bill Bennett', 'Silvetti', 'Jennifer Saunders', 'Lil Louis', 'Earl King [Earl Johnson]', 'Hal Singer Sextette', 'Bouvier', 'C. J. Bolland', "Marc O'Tool Tall Paul vs INXS", 'K.I.D.', 'Effect', 'Doc Sausage and His Mad Lads', 'The Lovers [2°]', 'the Big Family', 'Memphis Slim and His House Rockers', 'Rudy Render', 'Charlotte [Charlotte Kelly]', 'Cheech', 'Ramsey Lewis and Earth, Wind & Fire', 'Les Cooper', 'Angello', 'Blaze', 'B quartet]', 'Jimmie Nelson and the Peter Rabbit Trio', 'Jackie Lee', 'Joe Morris and His Orchestra', 'Chéri', 'Tony Moran', 'Mel', 'Luvndreams', 'the GB Experience', 'Da Mighty Dub Katz', 'Nellie Lutcher and Nat King Cole', 'Ko Ko Taylor', 'Taylor Girlz', 'Jess Glynne and DJ Cassidy', 'Karel Ullner', 'Q. Brooklyn', 'JX Riders and Skylar Stecker', 'The Larks', 'Jennifer Green', 'Space', 'Paul Williams', 'E.V.E. (Ebony Vibe Everlasting)', 'Fireball', 'Slim [Marvin Scandrick]', 'Staxx', 'The Reflections', 'Jeff Morgan', 'Brenda', 'Hodges', 'Extasia', 'Goldlink', 'Absolutely Fabulous Pet Shop Boys', 'Raw Silk', 'the Emblems', 'George Lamond', 'Stick McGhee and His Buddies', 'The Rainbows', 'Laguna', 'Pet Shop Boys = The Collaboration', 'The Flaming Ember', 'Kevin Ross', 'Artie the 1 Man Party', 'Flip', 'Freddie Mitchell and His Orchestra', 'Troop 41', 'Joi', 'the Police', 'Twisted Dee', 'Pianoheadz', 'The Shades of Blue', 'Lipstique', 'the Soul Searchers', 'The Philarmonics', 'Cashis', 'Eastside Connection', 'Bell', 'The Coronets', 'Fast Life Yungstaz (F.L.Y.)', 'Bumble Bee Unlimited', 'Sheila', 'Fenix', 'Patty', 'Betty Boo and the Beatmasters', 'U.N.V.', "Suave'", 'The Fog', 'Original Five Blind Boys', 'Lennie Lewis and His Orchestra', 'Earl King [Earl Connelly]', 'His Cool Aid Chemists', 'Camille Howard and Her Trio', 'Linx', 'DJ Duke', 'Dave Matthias', 'Herb Lance and the Classics', 'The Goodfellas', "Brown's Blues Blowers", 'Dyke and the Blazers', 'Skip', 'Eniac', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage', 'Jimmy Grissom', 'Trio', 'C.J. & Co.', 'The Positive Force', 'Jon of the Pleased Wimmin', 'Angel Clivillés', 'Arpeggio', 'Brainstorm', 'Destiny', 'the Egg', 'Industry [2°]', 'Ed Wiley and His After Hour Rhythm', 'Black Machine', 'Herb Fisher and His Trio', 'Barona', 'Jessie Saunders', 'the Vagabonds', 'Dillon Francis and Kygo', 'The Geto Boys', '50 Cent and Olivia', 'Swinging Sax Kari and Orchestra', 'KMC', 'Déjà', 'DJ Vicious', 'Alton McClain', 'Meechie', 'Mel Walker', 'Lydia Murdoch', 'Fatback', 'The Ones [2°]', 'Solo [2°]', 'Who Da Funk?', '3rd Party', 'Jimmy Preston and His Prestonians', 'Chip Chocolate', 'Ecstasy', 'The Shadows', 'The Purple Ribbon All-Stars', "The Cats 'N Jammer Three", "Stretch 'N' Vern Present Maddogg", 'Marvin', 'Eunice', 'Richard Dimples Fields', 'Robbie Rivera and Axwell', 'Ron Butler', 'Karen Young', 'Round2Crew', 'The Cardinals', 'Refugee Camp All-Stars', 'Mount Rushmore', 'Sonny Boy Williamson [1°]', 'Altar', 'Pacific Gas', 'Jimmy Wilson and His All-Stars', 'Willie Mabon and His Combo', 'The Corsairs', 'Mike Theodore Orchestra', 'Sunshine']

# Suffix

In [ ]:


mdbmaps["BillboardYE"].save()

In [ ]:
cad.getAlbumsData("George Frideric Handel: Messiah")

In [ ]:
updateManDB(2)

In [ ]:
updateManDB(2)

In [ ]:


mdbmaps["BillboardYE"].save()

In [ ]:
updateManDB(2)

In [ ]:
cad.getAlbumsData("Common Ground")

In [ ]:
updateManDB(2)

# Analyze Matches

In [ ]:
1/0
addAllMusic(mdbmaps, "Top40", "b67c4a39d846b0e14fcda8299193701a", "mn0000417717")   ### [Raybion Bros.]
addDiscogs(mdbmaps, "Top40", "b67c4a39d846b0e14fcda8299193701a", "2188194")    ### [Raybion Bros.]
### Butterfly Kisses

addAllMusic(mdbmaps, "Top40", "00d5d59798eeffef591f272260b0bbcc", "mn0000124572")   ### [Chuck-N-Blood]
### My Dogs

addAllMusic(mdbmaps, "Top40", "866a64f5a5e684f61aa573af7f63bc57", "mn0000233423")   ### [Magoo]
### Up Jumps Da Boogie

addAllMusic(mdbmaps, "Top40", "78f1feb9204bce3173d42798bc9bcb91", "mn0001808325")   ### [Kobe Bryant]
### Hold Me

addAllMusic(mdbmaps, "Top40", "d92e471232aa6a0bc4a03a49f16b0997", "mn0000186401")   ### [Karen O.]
addDiscogs(mdbmaps, "Top40", "d92e471232aa6a0bc4a03a49f16b0997", "245778")    ### [Karen O.]
### Hello Tomorrow

addAllMusic(mdbmaps, "Top40", "5edcd28a4505dc29e4edebd67052e614", "mn0000795894")   ### [Dolla]

addAllMusic(mdbmaps, "Top40", "7bcb752a835787affb80072d61387b22", "mn0001411030")   ### [Hollis]
### White Walls

addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addDiscogs(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "4210193")    ### [Jennifer Lawrence]
### The Hanging Tree

addAllMusic(mdbmaps, "Top40", "083b67cc423fe5482eaf1412aebb65b3", "mn0003323773")   ### [Lookas]
addDiscogs(mdbmaps, "Top40", "083b67cc423fe5482eaf1412aebb65b3", "3619379")    ### [Lookas]
### GDFR

addAllMusic(mdbmaps, "Top40", "f37bd04269e23d0c4c10c435a4f1dd7f", "mn0002943184")   ### [Mnek]
### Never Forget You  ,  So Good

addAllMusic(mdbmaps, "Top40", "1f9b34897f6440e2484f1ac5e00953f7", "mn0003462822")   ### [Gnash]
### I Hate U, I Love U  ,  Lights Down Low

addAllMusic(mdbmaps, "Top40", "f731f61bae9f9216f33f0cf76d403fee", "mn0000351749")   ### [Royce Da 5'9]
addDiscogs(mdbmaps, "Top40", "f731f61bae9f9216f33f0cf76d403fee", "40792")    ### [Royce Da 5'9]
### Not Alike

addAllMusic(mdbmaps, "Top40", "296504ec4590a1522dcf6afb838fd6f4", "mn0003332851")   ### [London On Da Track]
### Numbers

mdbmaps["Top40"].save()

In [ ]:
updateManDB(2)

In [ ]:
updateManDB(4)

In [ ]:
print([item[1]["ArtistName"] for item in toMatch["Top40"]])

In [ ]:
saveData = []
for fullName in sorted(list(cad.manyArtists.keys())):
    appendName = " & ".join(["[{0}]".format(manDB.renamed(indivName)) for indivName in sorted(list(cad.manyArtists[fullName]))])
    saveData.append([fullName, {"Guess": appendName, "Truth": ''}]) # = list(cad.manyArtists[fullName].keys())
saveFile(idata=saveData, ifile="tmp.yaml")

In [ ]:
fixedData = getFile("tmp.yaml")

In [ ]:
multis = []
multirenames = {}
for item in fixedData:
    if len(item[1]["Truth"]) > 0:
        name = item[1]["Truth"]
        if len(name.split("] & [")) == 1:
            multis.append(item[1]["Truth"][1:-1])
        else:
            rename = name[1:-1].split("] & [")
            multirenames[item[0]] = " ::: ".join(rename)
        
print("Found {0} renames".format(len(multirenames)))
print("Found {0} multi-artists".format(len(multis)))

#### Save MultiArtists (if found)

In [ ]:
knownFilename = "../multiartist/knownMultiArtists.yaml"
knownMultis = getFile(knownFilename)
print(len(multis))
print(len(knownMultis))
knownMultis += multis
print(len(knownMultis))
knownMultis = sorted(list(set(knownMultis)))
print(len(knownMultis))
saveFile(idata=knownMultis, ifile=knownFilename)

#### Save multi renames (if needed)

In [ ]:
multiManDB = masterArtistNameDB("multi", init=False)
#multiManDB.forceReload(getFile("main.yaml"))
multiManDB.addRenames(multirenames)

In [ ]:
multiManDB = masterArtistNameDB("multi", init=False)
multiManDB.addRenames(multirenames)
multiManDB.save()
saveFile(idata=multiManDB.getRenames(), ifile="multi.yaml")
multiManDB = masterArtistNameDB("multi", init=True)
multiManDB.forceReload(getFile("multi.yaml"))
multiManDB.checkForRecursives()
multiManDB = masterArtistNameDB("multi", init=False)

In [ ]:
#### Edit main.yaml (if needed)

In [ ]:
mdbmaps["Billboard"].getDF()

In [ ]:
mdf = mdbmaps["BillboardYE"].getDF()
mdf.shape
#mdf[mdf["DBMatches"] == 0]

# Multi Match

In [ ]:
cad.getTypeArtistAlbumData("Joe Rene")

In [ ]:
toMatch["Billboard"]

In [ ]:
manualAppends(cad, mType, toMatch, chartType, minAlbums=0, add=True)

In [ ]:
mdbmc.getMasterDF("Billboard")

In [ ]:
matchItAll(mdbmaps, getThresholdsWithoutAlbums(0.85), toMatch=toMatch, useAlbums=False)
reMatch()

In [ ]:
#singleArtistAlbumData["Billboard"]['Ernest']
saveFile(idata=[x[1]["ArtistName"] for x in toMatch["Top40"]], ifile="toget.p")

# Find Near Artist Name Matches

In [ ]:
toget, dbRenames = analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=1.0)

In [ ]:
if len(dbRenames) > 0:
    analyzeRenames(manDB, dbRenames)
    # Test for overap with manDB

    dbRenames = getFile(ifile="relDBRenames2.yaml")
    redos = {}
    dels  = []
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys(): # and bestname not in manDB.artistNameDB.keys():
            #print("Older=[{0}] \t Best=[{1}]".format(oldername,bestname))

            redos[bestname] = oldername
            dels.append(oldername)
            #print('PROBLEM ({0} in master DB): \t{1}  -->  {2}'.format(oldername, oldername, bestname))
            try:
                matchOlderData = mdbmaps[chartType].getArtistDataByName(oldername).getDict()
            except:
                matchOlderData = {}

            try:
                matchBestData  = mdbmaps[chartType].getArtistDataByName(bestname).getDict()
            except:
                matchBestData = {}

            #print("Older: {0}".format(matchOlderData))
            #print("Best:  {0}".format(matchBestData))
            #print("\n")
        else:
            print("{0}: {1}".format(oldername, bestname))

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()
manDB.save()
manDB = masterArtistNameDB("main", init=False)
saveFile(idata=manDB.dbRenames, ifile="main.yaml")
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
vals="""
[Sander Van Doom                0.9                Sander Van Doorn] 	 --> (Top40) 27813b44b4282de50171e13adb4ebcf9 / 183267 (Discogs)
"""

lines=vals.split("\n")
lines2=[x.split("({0}) ".format(chartType))[1] for x in lines if len(x) > 0]
names2=[x.split("({0}) ".format(chartType))[0] for x in lines if len(x) > 0]
lines3={x.split(" / ")[0]: x[x.find("(")+1:-1] for x in lines2}
names3=[x.split("    ")[0].strip()[1:] for x in names2]
if len(names3) != len(lines3):
    raise ValueError("Err")

for i,(k,v) in enumerate(lines3.items()):
    artistName = names3[i]
    #print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', \'{2}\')".format(k,v,))
    print("### ----> {0}".format(artistName))
    print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', None)".format(k,v))    
    albums = singleArtistAlbumData[chartType][artistName]
    print("### {0}".format("  ,  ".join(albums)))
    print("### ignores.append(\"{0}\")".format(artistName))
    print("\n")
    
    
#mdbmaps[chartType].addArtistDataByID("e309c2e3fc905eae304b71e10e82eb99", "AllMusic", '0000803469')
print("")
print("mdbmaps[chartType].save()")
print("saveFile(idata=list(set(ignores)), ifile=\"chartIgnores.yaml\")")

In [ ]:
### ----> Sander Van Doom
mdbmaps[chartType].addArtistDataByID('27813b44b4282de50171e13adb4ebcf9', 'Discogs', None)
### Grasshopper
### ignores.append("Sander Van Doom")



mdbmaps[chartType].save()
saveFile(idata=list(set(ignores)), ifile="chartIgnores.yaml")

In [ ]:
toget

****
****
****
****

# Multis

In [ ]:
ignores.append('Daryl Hall & John Oates')

In [ ]:
from multiArtist import multiartist
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
knownMultiArtists = getFile("../multiartist/knownMultiArtists.yaml")
print(len(knownMultiArtists))
print(len(mdbmaps[chartType].getArtists()))
for idx,artistName in mdbmaps[chartType].getArtists().items():
    N = mularts.getArtistNames(artistName)
    if len(N) > 1:
        result = {subName: mdbmaps[chartType].isKnownByName(subName) for subName in N}
        if not any(result.values()):
            knownMultiArtists.append(artistName)
            #print(result)
        #print("ignores.append(\'{0}\')".format(artistName))

print(len(knownMultiArtists))
knownMultiArtists = list(set(knownMultiArtists))
print(len(knownMultiArtists))
        
#saveFile(idata = sorted(knownMultiArtists), ifile="../multiartist/knownMultiArtists.yaml")

In [ ]:
ignores = getFile("chartIgnores.yaml")
dels = []
for idx,artistName in mdbmaps[chartType].getArtists().items():
    if artistName in ignores:
        dels.append([idx,artistName])
        
print(len(dels))
for idx,artistName in dels:
    mdbmaps[chartType].removeArtistByID(idx)
    
mdbmaps[chartType].save()

In [ ]:
len(dels)

# Functions

In [ ]:
class matchthresholds:
    def __init__(self):
        thresholds = {}
        thresholds[1000] = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 10.0}
        thresholds[500]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 5.0}
        thresholds[200]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 2.5}
        thresholds[100]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(100/8), 'score': 1.5}
        thresholds[50]   = {'numArtistName': 2, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(50/8), 'score': 1.5}
        thresholds[20]   = {'numArtistName': 2, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 3, 'score': 1.5}
        thresholds[10]   = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[5]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[3]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[2]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 1, 'score': 2.0}
        thresholds[1]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 1, 'score': 0.9}
        self.thresholds = thresholds

        iterItems = {20: {"Max": 50, "Vals": 100}}
        iterItems.update({10: {"Max": 20, "Vals": 200}})
        iterItems.update({50: {"Max": 10000, "Vals": 50}})
        iterItems.update({5: {"Max": 10, "Vals": 250}})
        iterItems.update({3: {"Max": 5, "Vals": 500}})
        iterItems.update({2: {"Max": 3, "Vals": 500}})
        iterItems.update({1: {"Max": 2, "Vals": 500}})
        self.iterItems = iterItems

        self.thresholdsNoAlbums = {'numArtistName': 5, 'artistNameCutoff': 0.9, 'artistAlbumCutoff': None, 'numArtistAlbums': None, 'score': None}
    
    def getThresholdsWithAlbums(self, minAlbums):
        if self.thresholds.get(minAlbums) is not None:
            return self.thresholds[minAlbums]
        
    def getThresholdsWithoutAlbums(self, cutoff):
        self.thresholdsNoAlbums["artistNameCutoff"] = cutoff
        return self.thresholdsNoAlbums
    
    def getIterItems(self):
        return self.iterItems
        

In [ ]:
from functools import partial
from matchDBArtist import matchDBArtist

class multimatch:
    def __init__(self, maindb, mdbmc, debug=False):
        self.debug  = debug
        self.maindb = maindb
        self.mdbmc  = mdbmc
        
        self.mt = matchthresholds()
        
        self.toMatch = None
        self.dbName  = None
        self.maxMatches = None
        
        self.matchFunc = {True: self.matchDBArtistWithAlbums, False: self.matchDBArtistWithoutAlbums}
            
        
    def setMDBMap(self, dbName, mdbmap):
        self.mdbmap = mdbmap
        self.dbName = dbName
    
    
    def setDataToMatch(self, toMatch):
        if toMatch is not None:
            self.toMatch = toMatch
            
            
    def setMaxMatches(self, maxMatches):
        self.maxMatches = maxMatches
        
        
    ############################################################################################################
    ## Set Data To Be Matched
    ############################################################################################################
    def setData(self, useAlbums=True, minAlbums=None, maxValues=None, cutoff=None):
        self.useAlbums = useAlbums
        print("HI")
        if useAlbums is True:
            if minAlbums is not None:
                self.thresholds = self.mt.getThresholdsWithAlbums(minAlbums)
                iterItems = self.mt.getIterItems()[minAlbums]
                maxAlbums = iterItems["Max"]
                if maxValues is None:
                    maxValues = thresholds["Vals"]

                self.toMatch   = mdbmc.getDataToMatch(self.dbName, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=[])
                print("Found {0} Artists To Match With Albums and Using Thresholds: {1}".format(len(self.toMatch[self.dbName]), self.thresholds))
            else:
                raise ValueError("Must supply a minAlbums value")
        else:
            self.thresholds = self.mt.getThresholdsWithoutAlbums(cutoff)
            if maxValues is None:
                maxValues = 100000
            self.toMatch   = mdbmc.getDataToMatch(self.dbName, maxValues=maxValues, maxAlbums=100000, minAlbums=0, ignores=[])
            print("Found {0} Artists To Match Without Albums and Without Using Thresholds".format(len(self.toMatch[self.dbName])))
            
    
    


    def matchItAll(self):
        num_processes = 3
        func       = self.matchFunc[self.useAlbums]
        thresholds = self.thresholds
        pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
        argument_list = self.toMatch[self.dbName]
        print("Args: ",argument_list)
        
        if len(argument_list) == 0:
            return
        print("Using {0} Matching Function".format(pfunc))
        print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
        result_list = self.multiProc(func=pfunc, argument_list=argument_list,
                                               num_processes=num_processes)
        return result_list
        

    def matchDBArtistWithoutAlbums(self, item, *args, **kwargs):
        return 1

        print("Item ==>",item)
        1/0
        primaryKey   = item[0]
        artistData   = item[1]
        artistName   = artistData["ArtistName"]
        artistID     = primaryKey
        artistAlbums = None
        mdbMatcher = matchDBArtist(self.maindb)
        mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
        mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'],
                                 matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                                 matchScore=kwargs['score']),
        mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
        retval = [primaryKey,artistName,artistID,mcs]
        return retval


    def matchDBArtistWithAlbums(self, item, *args, **kwargs):    
        #time.sleep(0.0025)

        return 1
        print("Item ==>",item)
        1/0
        

        primaryKey   = item[0]
        artistData   = item[1]
        artistName   = artistData["ArtistName"]
        artistID     = primaryKey
        artistAlbums = artistData["ArtistAlbums"]

        mdbMatcher = matchDBArtist(self.maindb)
        mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
        mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                                 matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                                 matchScore=kwargs['score'])
        mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
        retval = [primaryKey,artistName,artistID,mcs]
        return retval


    def multiProc(self, func, argument_list, num_processes):
        pool = Pool(processes=num_processes)
        result_list_tqdm = []
        for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
            result_list_tqdm.append(result)
        return result_list_tqdm


In [ ]:
mm = multimatch(maindb, mdbmc)
mm.setMDBMap("Billboard", mdbmaps["Billboard"])

In [ ]:
mm.setData(useAlbums=True, minAlbums=10, maxValues=1)

In [ ]:
mm.matchItAll()

In [ ]:
ignoresList = ["chartIgnores.yaml"]
for minAlbums,minAlbumsData in getIterItems().items():
    print("="*10,minAlbums)
    maxAlbums = minAlbumsData["Max"]
    maxValues = minAlbumsData["Vals"]
    for i in range(10):
        toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=ignoresList)
        if len(toMatch[chartType]) == 0:
            break
        matchItAll(mdbmaps, getThresholds(minAlbums), toMatch=toMatch)
        
reMatch()
print("Done.\n\n")

In [ ]:
from billboardCharts import billboardCharts
from fileUtils import getBasename, getDirname, getBaseFilename
from timeUtils import clock, elapsed
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from listUtils import getFlatList
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
from os.path import join
from searchUtils import findExt
import urllib
from time import sleep
from collections import Counter
from artistIgnores import getArtistIgnores

from billboardCharts import billboardCharts
from top40Charts import top40Charts

class billboardFiles:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Billoard"

    def findFiles(self):
        savedir = join(self.basedir, "data", "billboard", "results")
        self.files   = findExt(savedir, ext='.p')
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
        for ifile in self.files:
            fdata = getFile(ifile)
            for chartName, cnameResults in fdata.items():
                if chartName not in self.charts:
                    continue
    
    
class billboardFullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Billboard"
        self.charts  = charts
        self.tfiles  = billboardFiles(self.basedir)
        self.files   = self.tfiles.findFiles()
        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))


class top40files:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Top40"

    def findFiles(self):
        savedir = join(self.basedir, "data", "top40")
        self.files   = findExt(savedir, ext='.p')         
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
class top40FullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Top40"
        self.charts  = charts
        self.tfiles  = top40files(self.basedir)
        self.files   = self.tfiles.findFiles()
        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))


class top40files:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Top40"

    def findFiles(self):
        savedir = join(self.basedir, "data", "top40")
        self.files   = findExt(savedir, ext='.p')         
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
class top40FullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Top40"
        self.charts  = charts
        self.tfiles  = top40files(self.basedir)
        self.files   = self.tfiles.findFiles()        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))
        
    
        
    def setFullChartData(self):
        fullChartData = {}
        renameStats   = Counter()
        
        self.findFiles()
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        
        for chartName, ifile in self.files.items():
            if chartName not in self.charts:
                continue
            print("==> {0: <40}".format(chartName), end="\t")
            #t40chart = top40chart(chartID, chartName, chartURL)
            chartResults = getFile(ifile)

            for date, values in chartResults.items():
                if self.minYear is not None:
                    if getDateTime(date).year < int(self.minYear):
                        continue
                if self.maxYear is not None:
                    if getDateTime(date).year > int(self.maxYear):
                        continue

                        
                for i,item in enumerate(values):
                    artist = item["Artist"]
                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist
                    

                    artist = artist.replace("\r", "")                    
                    
                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue
                    
                    album  = item["Album"]
                    if album in ["Soundtrack"]:
                        continue

                    if fullChartData.get(artist) is None:
                        fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if fullChartData[artist][key].get(album) is None:
                        fullChartData[artist][key][album] = {}
                    if fullChartData[artist][key][album].get(chartName) is None:
                        fullChartData[artist][key][album][chartName] = {}
                    fullChartData[artist][key][album][chartName][date] = i
            print(len(fullChartData))
        self.fullChartData = fullChartData
        
        if self.artistRenames is not None:
            print("Renamed {0} artists".format(len(renameStats)))
            print("Most Common Artists:")
            for item in renameStats.most_common(5):
                print(item)


class chartData:
    def __init__(self, source, minYear=None, maxYear=None, country=None, debug=False):
        debug=False    
        self.source = source
        
        self.basedir  = "/Volumes/Piggy/Charts/"
        self.basename = source
        

        try:
            self.cts    = {"Billboard": billboardCharts(), "Top40": top40Charts()}[source]
            self.cFiles = {"Billboard": billboard40files(self.basedir), "Top40": top40files(self.basedir)}[source]
            self.files  = self.cFiles.findFiles()
        except:
            raise ValueError("Could not create charts data for {0}".format(source))

                        
        self.charts = []
            
        self.minYear   = minYear
        self.maxYear   = maxYear
        
        self.artistRenames   = {}
        self.dbRenames       = {}

        self.chartData       = {}
        self.fullChartData   = {}
        self.artistAlbumData = {}
        
        
    #####################################################################################################################################
    ## Artist Data
    #####################################################################################################################################
    def getArtists(self):
        return list(self.artistAlbumData.keys())
        
        
    
    #####################################################################################################################################
    ## Full Chart Data
    #####################################################################################################################################
    def getFullChartDataFilename(self):
        ifile="current{0}FullChartArtistAlbumData.p".format(self.basename)
        return ifile

    def getFullChartData(self):
        return getFile(self.getFullChartDataFilename())
        
    def saveFullChartData(self):
        print("Saving {0} Full Artist Data".format(len(self.fullChartData)))
        saveFile(idata=self.fullChartData, ifile=self.getFullChartDataFilename(), debug=True)        
        
        
    
    #####################################################################################################################################
    ## Artist Album Data
    #####################################################################################################################################
    def getArtistAlbumDataFilename(self):
        ifile="current{0}ArtistAlbumData.p".format(self.basename)
        return ifile
    
    def setArtistAlbumData(self):
        self.artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in self.fullChartData.items()}
        
    
    def getArtistAlbumData(self):
        return getFile(self.getArtistAlbumDataFilename())
        
    def saveArtistAlbumData(self):
        print("Saving {0} Artist Album Data to {1}".format(len(self.artistAlbumData), self.getArtistAlbumDataFilename()))
        saveFile(idata=self.artistAlbumData, ifile=self.getArtistAlbumDataFilename(), debug=True)  
        
        
    
    #####################################################################################################################################
    ## Rename Data
    #####################################################################################################################################        
    def setRenames(self, artistRenames):
        self.artistRenames = artistRenames
        
    def setDBRenames(self, dbRenames):
        self.dbRenames = dbRenames
        
        
    
    #####################################################################################################################################
    ## Find Files
    #####################################################################################################################################
    def setChartUsage(self, name=None, rank=None):
        if rank is not None:
            if isinstance(rank, list):
                for item in rank:
                    self.charts += self.cts.getChartsByRank(item)
            elif isinstance(rank, int):
                self.charts += self.cts.getChartsByRank(rank)
        elif name is not None:
            self.charts += self.cts.getCharts(name)
        else:
            self.charts = self.cts.getCharts(None)
        if name is None:
            name = "None"
        print("  Using Charts ({0}): {1}".format(name, self.charts))
        
        
        
                
    #####################################################################################################################################
    ## Set Artist Album Data
    #####################################################################################################################################
    def setArtistAlbumData(self):
        self.artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in self.fullChartData.items()}
        print("There are {0} unique artist entries".format(len(self.artistAlbumData)))
        

    #####################################################################################################################################
    ## Set Full Chart Data
    #####################################################################################################################################
    def setFullChartData(self):
        {"Billboard": self.setFullChartDataBillboard(), "Top40": self.setFullChartDataTop40()}[self.source]
    
    def setFullChartDataBillboard(self):        
        renameStats  = Counter()
        chartCounter = Counter()
        
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        
        for ifile in self.files:
            fdata = getFile(ifile)
            for chartName, cnameResults in fdata.items():
                if chartName not in self.charts:
                    continue
                
                for date, dResults in cnameResults.items():
                    if self.minYear is not None:
                        if getDateTime(date).year < int(self.minYear):
                            continue
                    if self.maxYear is not None:
                        if getDateTime(date).year > int(self.maxYear):
                            continue
                    stryear = getDateTime(date).year

                    artist = dResults["Artist"]

                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist 
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist  

                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue


                    chartCounter[chartName] += 1

                    album  = dResults["Name"]

                    if self.chartData.get(artist) is None:
                        self.chartData[artist] = Counter()
                    self.chartData[artist][album] += 1
                    
                    if self.fullChartData.get(artist) is None:
                        self.fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if self.fullChartData[artist][key].get(album) is None:
                        self.fullChartData[artist][key][album] = {}
                    if self.fullChartData[artist][key][album].get(chartName) is None:
                        self.fullChartData[artist][key][album][chartName] = {}
                    self.fullChartData[artist][key][album][chartName][date] = 0
                #print("{0: <40}{1}".format("{0}-{1}".format(chartName,stryear),len(self.fullChartData)))
                
        
    def setFullChartDataTop40(self):
        fullChartData = {}
        renameStats   = Counter()
        
        #self.findFiles()
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        
        for chartName, ifile in self.files.items():
            if chartName not in self.charts:
                continue
            print("==> {0: <40}".format(chartName), end="\t")
            #t40chart = top40chart(chartID, chartName, chartURL)
            chartResults = getFile(ifile)

            for date, values in chartResults.items():
                if self.minYear is not None:
                    if getDateTime(date).year < int(self.minYear):
                        continue
                if self.maxYear is not None:
                    if getDateTime(date).year > int(self.maxYear):
                        continue

                        
                for i,item in enumerate(values):
                    artist = item["Artist"]
                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist
                    

                    artist = artist.replace("\r", "")                    
                    
                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue
                    
                    album  = item["Album"]
                    if album in ["Soundtrack"]:
                        continue

                    if fullChartData.get(artist) is None:
                        fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if fullChartData[artist][key].get(album) is None:
                        fullChartData[artist][key][album] = {}
                    if fullChartData[artist][key][album].get(chartName) is None:
                        fullChartData[artist][key][album][chartName] = {}
                    fullChartData[artist][key][album][chartName][date] = i
            print(len(fullChartData))
        self.fullChartData = fullChartData
        
        if self.artistRenames is not None:
            print("Renamed {0} artists".format(len(renameStats)))
            print("Most Common Artists:")
            for item in renameStats.most_common(5):
                print(item)